<a href="https://colab.research.google.com/github/sanujsriv/_topic_models/blob/WTM/Embedding_Space_Vis_Model_Experiments(Read_Only).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**0. Just Run these..**

In [ ]:
import os
if 'grad16' in os.getcwd(): local=True
else: local = False
if local == True: os.chdir('/home/grad16/sakumar/')
down_count=0

In [ ]:
## Full data
if local == True:
  model_name = 'Embedding_Vis_Model'
  cur_dir='/home/grad16/sakumar/CIKM_Experiments_2021/'+model_name
  data_dir = '/'

  os.chdir(cur_dir+data_dir)
  # os.mkdir(model)
  # os.chdir(model_name)
  # os.mkdir(d_data) ## uncomment this if data is new
  # os.chdir(d_data)

In [ ]:
!nvidia-smi

Fri Jun 11 08:56:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P2000        Off  | 00000000:01:00.0 Off |                  N/A |
| 44%   28C    P8     4W /  75W |    379MiB /  5055MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pwd

'/home/grad16/sakumar/CIKM_Experiments_2021/Embedding_Vis_Model'

In [ ]:
# @title Imports
# from sympy.stats import RaisedCosine, density
if local==False:
  !pip3 install pickle5
import pickle5
import torch
import os
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
# %matplotlib inline
import pandas as pd
from torch import Tensor
from torch import nn
import re
from time import time
import numpy as np
import collections
import gc
from sklearn.feature_extraction.text import CountVectorizer
import torch.nn
# cos_sim = nn.CosineSimilarity(dim=-1,eps=1e-08)
# from nltk.corpus import stopwords

In [ ]:
#@title torch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import Parameter
import math

In [ ]:
#@title function : load / save pickle_obj
import pickle

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

def load_obj_pkl5(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle5.load(f)

# **Download Data & Helper Functions(Documents)**

In [ ]:
#@title OLDER_Download Data Function

# def download_data(data,dtype):
#   data = data.lower()
#   dtype = dtype.lower()

#   if data=='wos':
#     if dtype == 'full':
#       # os.system('wget -N https://www.dropbox.com/s/krx3am9pfsiidzx/wos_full.zip')
#       # os.system('unzip wos_full.zip')
#       os.system('wget -N https://www.dropbox.com/s/768lhd76l49nsa4/wos_full.zip')
#       os.system('unzip wos_full.zip')

#     if dtype == 'short':
#       os.system('wget -N https://www.dropbox.com/s/aauu04pahwptexz/wos_short.zip')
#       os.system('unzip wos_short.zip')

#   elif data=='bbc':
#     if dtype == 'full':
#       os.system('wget -N https://www.dropbox.com/s/5n7khg6ht1lglb6/bbc_full.zip')
#       os.system('unzip bbc_full.zip')

#     if dtype == 'short':
#       os.system('wget -N https://www.dropbox.com/s/p5pb0w27cp41qeb/bbc_short.zip')
#       os.system('unzip bbc_short.zip')

#   elif data=='searchsnippet':
#     if dtype == 'short':
#        os.system('wget -N https://www.dropbox.com/s/yp0b4wl6w51af47/searchsnippet_short.zip')
#        os.system('unzip searchsnippet_short.zip')

#   else: print('either "data" or "dtype" is entered incorrectly ')


In [ ]:
# @title Old Download Data Function

# def download_data(data,dtype):
#   data = data.lower()
#   dtype = dtype.lower()

#   if data=='wos':
#     if dtype == 'full':
#       os.system('wget -N https://www.dropbox.com/s/w7ryozpj6qq1snx/wos_full4000.zip')
#       os.system('unzip wos_full4000.zip')

#     if dtype == 'short':
#       os.system('wget -N https://www.dropbox.com/s/eqkvxl6z0jwp44u/wos_short4000.zip')
#       os.system('unzip wos_short4000.zip')

#   elif data=='bbc':
#     if dtype == 'full':
#       # os.system('wget -N https://www.dropbox.com/s/a628h7e7lnlasc7/bbc_full2000.zip')
#       # os.system('unzip bbc_full2000.zip')
#       os.system('wget -N https://www.dropbox.com/s/0gbgvvqk4xoqfk6/bbc_full2000_20.zip')
#       os.system('unzip bbc_full2000_20.zip')

#     if dtype == 'short':
#       os.system('wget -N https://www.dropbox.com/s/4ly63afqc7ugz69/bbc_short2000.zip')
#       os.system('unzip bbc_short2000.zip')
#       # os.system('wget -N https://www.dropbox.com/s/l1aargdbd8e3bjm/bbc_short2000_20_4.zip')
#       # os.system('unzip bbc_short2000_20_4.zip')

#   elif data=='searchsnippet':
#     if dtype == 'short':
#        os.system('wget -N https://www.dropbox.com/s/8emmtg3qdqq7vpg/searchsnippet_short3000.zip')
#        os.system('unzip searchsnippet_short3000.zip')

#   else: print('either "data" or "dtype" is entered incorrectly ')


In [ ]:
#@title OLD Data Loading functions..
# def load_data(data,dtype,dtype2):
#   data = data.lower()
#   dtype = dtype.lower()
#   dtype2 = dtype2.lower()
#   d,l,e,fn = load_d_data(data,dtype,dtype2)
#   return d,l,e,fn
# sample_size = 50
# def load_data(data,dtype,dtype2='small'):
#   if (dtype == 'full' or dtype == 'short') and dtype2 == 'normal':
#     data_preprocessed = load_obj('data_preprocessed')
#     data_preprocessed_labels = load_obj("data_preprocessed_labels")
#     embeddings = load_obj('embeddings')
#   elif (dtype == 'full' or dtype == 'short') and dtype2 == 'small':
#     data_preprocessed = load_obj('docs_sampled_small_'+str(sample_size)+data+dtype)
#     data_preprocessed_labels = load_obj("labels_sampled_small_"+str(sample_size)+data+dtype)
#     embeddings = load_obj('embeddings')

#   os.system("rm -r *.pkl")
#   return data_preprocessed,data_preprocessed_labels,embeddings,d_data


In [ ]:
#@title Download Data Function

def download_data(data):
  data = data.lower()
  if data=='wos':
    os.system('wget -N https://www.dropbox.com/s/f3qw75jmsdlyk8j/wos_4000_20.zip')
    os.system('unzip wos_4000_20.zip')

  elif data=='bbc':
     os.system('wget -N https://www.dropbox.com/s/4mmiaed6rg5lpb1/bbc_2000_20.zip')
     os.system('unzip bbc_2000_20.zip')

  elif data=='searchsnippet':
       os.system('wget -N https://www.dropbox.com/s/tq31csl55b3oc09/searchsnippet_3000_20.zip')
       os.system('unzip searchsnippet_3000_20.zip')

  else: print('"data" is entered incorrectly ')

In [ ]:
#@title Data Loading Functions

sample_size = 20
small_data_no_of_ids = 5

def load_data(data,dtype,dtype2):
  data = data.lower()
  dtype = dtype.lower()
  dtype2 = dtype2.lower()
  d,l,e,fn = load_d_data(data,dtype,dtype2)
  return d,l,e,fn

def load_data(data,dtype,dtype2):
  if local==True:
    dir ='/content/data_'+data+'/'+dtype
    os.chdir(cur_dir+dir)
  if local==False:
    dir ='/content/content/data_'+data+'/'+dtype
    os.chdir(dir)


  if dtype2=='normal':
      data_preprocessed=load_obj("data_preprocessed_"+data+"_"+dtype)
      data_preprocessed_labels=load_obj("data_preprocessed_labels_"+data+"_"+dtype)
      embeddings=load_obj("embeddings_"+data+"_"+dtype)

      if local==True:
        os.chdir('/home/grad16/sakumar/CIKM_Experiments_2021/PLSV-VAE')
      if local==False:
        os.chdir('/content/')
      return data_preprocessed,data_preprocessed_labels,embeddings,data

  elif dtype2=='small':
    data_preprocessed=[]
    data_preprocessed_labels=[]
    if (dtype == 'full' or dtype == 'short') and dtype2 == 'small':
      for i in range(small_data_no_of_ids):
        id = str(i+1)
        data_preprocessed.append(load_obj(id+'_docs_sampled_'+dtype2+'_'+str(sample_size)+d_data+dtype))
        data_preprocessed_labels.append(load_obj(id+'_labels_sampled_'+dtype2+'_'+str(sample_size)+d_data+dtype))
        embeddings = load_obj('embeddings'+'_'+data+'_'+dtype)
    if local==True:
      os.chdir('/home/grad16/sakumar/CIKM_Experiments_2021/'+model_name)
    if local==False:
      os.chdir('/content/')
    # os.system("rm -r *")
    return data_preprocessed,data_preprocessed_labels,embeddings,d_data


# **1. Preparing Data**

In [ ]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')
def doc_remove_extra_stopwords(doc):
  extra_stopwords = ['la','wa','will','fa','ha','pa','co','v','said']
  doc = word_tokenize(doc)
  doc = filter(lambda x:x not in extra_stopwords, doc)
  doc = ' '.join(e for e in doc)
  return doc

[nltk_data] Downloading package punkt to
[nltk_data]     /home/grad16/sakumar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# cd /home/grad16/sakumar/CIKM_Experiments_2021/PLSV-VAE
!pwd

/home/grad16/sakumar/CIKM_Experiments_2021/Embedding_Vis_Model


In [ ]:
# !wget https://www.dropbox.com/s/w1aum2lfvx4swaq/stackoverflow_4000_20.zip
# !unzip stackoverflow_4000_20.zip
# !wget https://www.dropbox.com/s/er4nnqu38jou2mn/stackoverflow_4000_20.zip
# !unzip stackoverflow_4000_20.zip
# !wget https://www.dropbox.com/s/p7b2msz5fzqvc1q/agnews120k_4000_.zip
# !unzip agnews120k_4000_.zip

In [ ]:
# !wget https://www.dropbox.com/s/0qqv3srqru9xii0/newscategory_4000_20.zip
# !unzip newscategory_4000_20.zip

In [ ]:
#### Data Downloading ####
d_data= 'agnews120k' # wos,bbc,searchsnippet,stackoverflow,agnews120k
# min_df = 0 # we already applied min_df =3 in preprocessing
dtype='short' # full, short
dtype2 = 'normal' # small , normal

# download_data(d_data) ###

# ##### Data loading #####

loaded_data = load_data(d_data,dtype,dtype2)
# loaded_data = load_short_small_data_(d_data,dtype)
# loaded_data = load_small_data(d_data)
##########################

data_preprocessed , data_preprocessed_labels , embeddings, name = loaded_data
# for d in range(len(data_preprocessed)):
  # data_preprocessed[d] = doc_remove_extra_stopwords(data_preprocessed[d])
print(name,len(data_preprocessed_labels),len(data_preprocessed),len(embeddings),dtype,dtype2)

agnews120k 119822 119822 4000 short normal


In [ ]:
len_docs = [len(d.split(" ")) for d in data_preprocessed]
print(np.min(len_docs),np.mean(len_docs).round(2),np.max(len_docs))

3 14.04 67


In [ ]:
# for data in data_preprocessed:
#   len_docs = [len(d.split(" ")) for d in data]
#   print(np.min(len_docs),np.mean(len_docs).round(2),np.max(len_docs))

In [ ]:
# if local:
  # os.chdir(colab_dir+model_name+'/plsv_vae')
def get_data_label_vocab_normal(data,lables):
  min_df=0
  preprossed_data = data
  train_label = lables
  vectorizer = CountVectorizer(min_df=min_df)
  train_vec = vectorizer.fit_transform(preprossed_data).toarray()
  vocab = vectorizer.vocabulary_
  id_vocab = dict(map(reversed, vocab.items()))
  # nonzeros_indexes = np.where(train_vec.any(1))[0]
  # train_vec_non_zeros = [train_vec[i] for i in nonzeros_indexes]
  # preprossed_data_non_zeros = [preprossed_data[i] for i in nonzeros_indexes]
  # train_label = [data_preprocessed_labels[i] for i in nonzeros_indexes]
  print(len(train_vec),len(vocab))
  print(vocab)

  tensor_train_w = (torch.from_numpy(np.array(train_vec)).float())
  train_label = np.asarray(train_label)

  return tensor_train_w,train_label,id_vocab

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_coordinate = 2
bs = 256
en1_units_x = 100
en2_units_x = 100
variance_x = 1.0
learning_rate = 0.002
beta1 = 0.99
beta2 = 0.999
drop_rate = 0.0
num_topic = 10

# **Cosine Scores**

In [ ]:
cos_sim = nn.CosineSimilarity(dim=-1, eps=1e-8)
def flatten_list(user_list): return [item for sublist in user_list for item in sublist]
def get_embedding_tensor(word_list,embeddings): return torch.tensor([embeddings[w] for w in word_list])
def get_doc_word_embeddings(id_vocab,embeddings):
  sorted_id_word_vocab = sorted(id_vocab.items(), key=lambda x: x[1]) ### alphabetically sorted
  word_list = [s[1] for s in sorted_id_word_vocab]
  words_tensor = get_embedding_tensor(word_list,embeddings)
  embedding_tensor_sorted_alp = get_embedding_tensor(word_list,embeddings)
  emb_size = embedding_tensor_sorted_alp.shape[1]
  return embedding_tensor_sorted_alp,emb_size

# **Functions (Plot_loss, get_topwords, get_Z, get_Contour, print_Topics)** - *Run Once*

In [ ]:
#@title get_all_tensor_size
def getall_tensor_size():
  for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size(),get_mem_size(obj))
    except:
        pass

In [ ]:
#@title Testing_&_Extra_Fns

# embdding_input_NV300 = embedding_tensor_sorted_alp.unsqueeze(0).expand(tensor_train_w.shape[0],embedding_tensor_sorted_alp.shape[0],embedding_tensor_sorted_alp.shape[1])
# tensor_train_w_input = tensor_train_w
# y = torch.ones(tensor_train_w.size())
# w_input = torch.where(tensor_train_w_input >= 1, y, tensor_train_w_input)
# docWord_embedding_input = w_input.unsqueeze(-1).expand(w_input.shape[0],w_input.shape[1],300) * embdding_input_NV300

# def Log_NormDist_wC(x,mu,sigma=sig0,sigma_ir=sigc,d=emb_size):
#   x_Q = torch.narrow(x,1,0,x.shape[1]-1) # First Q (KxQx300)
#   mu_Q = torch.narrow(mu,1,0,mu.shape[1]-1) # First (KxQx300)
#   x_C_ir = torch.narrow(x,1,x.shape[1]-1,1) # Last C (Kx1x300)
#   mu_C_ir = torch.narrow(mu,1,mu.shape[1]-1,1) # Last C (Kx1x300)

#   First_Q = d/2.0 * torch.log(torch.tensor(sigma/(2.0*pi)))
#   Second_Q = -(sigma/2.0) * (torch.norm(x_Q-mu_Q,dim=-1)**2)

#   First_C_ir = d/2.0 * torch.log(torch.tensor(sigma_ir/(2.0*pi)))
#   Second_C_ir = -(sigma_ir/2.0) * (torch.norm(x_C_ir-mu_C_ir,dim=-1)**2)

#   First = First_Q+First_C_ir
#   Second = torch.cat((Second_Q,Second_C_ir),dim=1)
#   ret = First + Second

#   return ret

def min_max_normalization(ip):
  vmax,_ = torch.max(ip,-1)
  vmin,_ = torch.min(ip,-1)
  vmax = vmax.unsqueeze(-1).expand(ip.shape[0],ip.shape[1])
  vmin= vmin.unsqueeze(-1).expand(ip.shape[0],ip.shape[1])
  ip = (ip - vmin) / (vmax-vmin+smoothen)
  return ip

def NewBeta_dist(x,mu,phi_beta,smoothen):
  First = torch_lgamma(phi_beta+smoothen) - (torch_lgamma(phi_beta * mu+smoothen) + torch_lgamma(phi_beta * (1-mu)+smoothen ))
  Second_1 = (mu*phi_beta-1) * torch.log(x+smoothen)
  Second_2 = ( (1-mu)*phi_beta-1 )* torch.log(1-x+smoothen)
  Second = Second_1 + Second_2
  return First + Second

def LogL_NewBeta(x,relevant_scores):
  x_norm = torch.norm(x,dim=-1)
  mu = torch.exp(-0.5 * (torch.pow(x_norm,2))/mu_div)
  # LogL = torch.log(NewBeta_dist(relevant_scores,mu,sig,smoothen)+smoothen)
  LogL = NewBeta_dist(relevant_scores,mu,100.0,smoothen)
  return LogL

# wd_idx = torch.where(tensor_train_w[0]>0.0)
# tensor_train_w[0][wd_idx],wd_idx
# print(data_preprocessed[0])
# print(len(data_preprocessed[0].split(' ')))
# for w in wd_idx[0]:
#   print(id_vocab[w.item()],)
# print(wd_idx[0].shape)

# id_word_input = []
# count = 0
# dum_vocab = []
# for data in data_preprocessed:
#   doc = []
#   for w in data.split(' '):
#     if w in vocab and w not in dum_vocab:
#       dum_vocab.append(w)
#     if(w in vocab): doc.append(dum_vocab.index(w))
#   id_word_input.append(doc)
#   # print(data.split(' '))
#   # id_word_input.append(sorted([vocab[p] for p in data.split(' ') if p in vocab]))
#   #id_word_input.append([vocab[p] for p in data.split(' ') if p in vocab])
#   count = count + 1


############################
        # Testing #
############################
# a = torch.ones(12,10) * 4
# b = torch.ones(2,1)
# torch.norm(b,dim=0)
###
# for i in range(12):
#   a[i] = i
# a.unsqueeze(1).expand(12,5,10)
###
# x = torch.rand(5,4,3)
# a=torch.narrow(x,1,0,x.shape[1]-1) # First
# b=torch.narrow(x,1,x.shape[1]-1,1) # Last
# torch.cat((a,b),dim=1)
##
# a_muz = torch.rand(10,300) * 2.0
# b_wnm = torch.rand(20,300) * 4.0
# a = a_muz.unsqueeze(1).expand(10,20,300)
# b = b_wnm.unsqueeze(0).expand(10,20,300)
# c = torch.norm(a-b,dim=-1)**2

In [ ]:
import plotly.graph_objects as go
def plot_loss(y,name):
  figure = go.Figure()
  figure.add_trace(go.Scatter(x=[i for i in range(1,epochs+1)], y=y,mode='lines',name=name))
  figure.show(renderer='colab')

In [ ]:
def get_topwords(beta, id_vocab):
    topic_indx = 0
    no_of_topwords = 10
    topwords_topic = []
    for i in range(len(beta)):
        topwords_topic.append(str(topic_indx)+": "+ " ".join([id_vocab[j] for j in beta[i].argsort()[:-no_of_topwords - 1:-1]]))
        topic_indx+=1
    return topwords_topic

In [ ]:
def get_Z(X,Y):
  norms = torch.zeros(X.shape)
  Z = torch.zeros(X.shape)

  for i in range(X.shape[0]):
    for j in range(X.shape[0]):
      v = torch.tensor([X[i][j],Y[i][j]])

      norms[i][j] = torch.norm(v)
      Z[i][j]=torch.exp(-0.5 * (torch.pow(norms[i][j],2))/mu_div)

      # norms[i][j] = (v-query_center.T).pow(2).sum(-1)
      # Z[i][j]=torch.exp(-0.5 * norms[i][j]/mu_div)
  return Z

In [ ]:
def get_Contour(ax,x_list,lim):
  xlist = np.linspace(-lim, lim, 200)
  ylist = np.linspace(-lim, lim, 200)
  X, Y = np.meshgrid(xlist, ylist)
  Z = get_Z(X,Y)
  levels = np.arange(0,1,0.1)
  cp = ax.contour(X, Y, Z, levels=levels)
  ax.clabel(cp, inline=1, fontsize=10)

In [ ]:
import seaborn as sb

def plot_fig(zx, labels_list, zphi,lim,contour='No'):
    labels = []
    for i in range(len(labels_list)):
        labels.append('C'+str(labels_list[i]))
    fig, ax = plt.subplots( figsize=(20, 20))
    if contour=='yes':
       get_Contour(ax,zx,lim)

    sb.scatterplot(ax=ax,x=zx[:,0],y=zx[:,1],hue=labels_list,alpha=0.8,palette='deep')
    ax.set(ylim=(-lim,lim))
    ax.set(xlim=(-lim,lim))
    ax.text(0,0, 'X' ,c='black')
    ax.scatter(zphi[:, 0], zphi[:, 1], alpha=1.0,  edgecolors='black', facecolors='none', s=30)

    for indx, topic in enumerate(zphi):
        ax.text(zphi[indx, 0], zphi[indx, 1], 'topic'+str(indx))

In [ ]:
def print_Topics(beta):
  print("---"*10)
  topword_topics = get_topwords(beta, id_vocab)
  topword_topics_list=[]
  for topwords in topword_topics:
      topword_topics_list.append(topwords.split())
      print(topwords)
  print("---"*10)

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
def cal_knn(coordinate, label):
    output = []
    for n_neighbors in [10, 20, 30, 40, 50]:
        neigh = KNeighborsClassifier(n_neighbors=n_neighbors, n_jobs=-1)
        neigh.fit(coordinate, label)
        output.append(neigh.score(coordinate, label))
    return output

# **Current Working Model**

In [ ]:
torch.cuda.empty_cache()
en1_units_x = 100
en2_units_x = 100
learning_rate = 0.001
beta1 = 0.99
beta2 = 0.999
drop_rate = 0.0
num_topic = 10
emb_size = 300
smoothen = 1e-30
variance_x = 1.0

# sig_phi = 1.0

pi = torch.acos(torch.zeros(1)).item() * 2

def Log_NormDist(x,mu,sigma,d=emb_size):
  # sigma = sigma + smoothen
  First = d/2.0 * torch.log(torch.tensor(1.0/(2.0*pi*sigma)))
  Second = -(1.0/(2.0*sigma)) * (torch.norm(x-mu,dim=-1))
  # Second1 = -(1.0/(2.0*sigma))
  # Second2 = ((x-mu)**2).sum(-1)
  ret = First + Second
  return ret

#phi
def gaussian(alpha): return -0.5*alpha
#def inverse_multi_quadric(alpha): return -0.5*torch.log(torch.ones_like(alpha) + alpha)
def inverse_quadratic(alpha): return -torch.log(torch.ones_like(alpha) + alpha)

class Model(nn.Module):
    def __init__(self, num_input, en1_units_x, en2_units_x, num_coordinate, num_topic, drop_rate, variance_x, bs,
                 embedding_words, word_emb_size, distance="gaussian"):

        super(Model, self).__init__()
        self.num_input, self.num_coordinate, self.num_topic, self.variance_x, self.bs \
            = num_input, num_coordinate, num_topic, variance_x, bs

        self.embedding_words = embedding_words
        self.emb_size = word_emb_size

        # encoder
        self.en1_fc     = nn.Linear(num_input, en1_units_x)
        self.en2_fc     = nn.Linear(en1_units_x, en2_units_x)
        self.en2_drop   = nn.Dropout(drop_rate)
        self.mean_fc    = nn.Linear(en2_units_x, num_coordinate)
        self.logvar_fc  = nn.Linear(en2_units_x, num_coordinate)

        self.mean_bn    = nn.BatchNorm1d(num_coordinate)
        self.logvar_bn  = nn.BatchNorm1d(num_coordinate)
        self.decoder_x_bn = nn.BatchNorm1d(num_coordinate)
        self.decoder_phi_bn = nn.BatchNorm1d(num_coordinate)
        self.decoder_bn = nn.BatchNorm1d(self.num_topic)

        self.en1_fc_e     = nn.Linear(300, en1_units_x)
        self.en2_fc_e     = nn.Linear(en1_units_x, en2_units_x)
        self.en2_drop_e   = nn.Dropout(drop_rate)

        # RBF
        self.in_features = self.num_coordinate
        self.out_features = self.num_topic
        self.centres = nn.Parameter(torch.Tensor(self.out_features, self.in_features)) # K x 2
        self.mu_z = nn.Parameter(torch.Tensor(self.out_features,self.emb_size))# K x 300
        # self.mu_z =nn.Embedding(self.out_features,self.emb_size)# K x 300
        # mu_z_MultiNormal = torch.normal(mean=self.embedding_words.mean(0).unsqueeze(0).expand(self.out_features,self.emb_size),std=0.00001) # Kx300
        # self.mu_z = nn.Parameter(mu_z_MultiNormal)

        if distance=="gaussian": self.basis_func = gaussian
        if distance=="inverse_quadratic": self.basis_func = inverse_quadratic
        self.init_parameters()

        # prior mean and variance as constant buffers
        prior_mean   = torch.Tensor(1, num_coordinate).fill_(0)
        prior_var    = torch.Tensor(1, num_coordinate).fill_(variance_x)
        self.prior_mean = nn.Parameter(prior_mean, requires_grad=False)
        self.prior_var  = nn.Parameter(prior_var, requires_grad=False)
        self.prior_logvar = nn.Parameter(prior_var.log(), requires_grad=False)

    def init_parameters(self):
        nn.init.normal_(self.centres, 0, 0.01)
        nn.init.normal_(self.mu_z, 0, 0.01)

    def encode(self, input_,normalized_input_):
        N, *_ = normalized_input_.size()
        en1 = F.softplus(self.en1_fc(normalized_input_))                           # en1_fc   output
        en2 = F.softplus(self.en2_fc(en1))                              # encoder2 output
        # en3 = F.softplus(self.en3_fc(en2))
        en2 = self.en2_drop(en2)
        posterior_mean   = self.mean_bn(self.mean_fc(en2))          # posterior mean
        posterior_logvar = self.logvar_bn(self.logvar_fc(en2))          # posterior log variance
        # posterior_mean   = (self.mean_fc(en2))          # posterior mean
        # posterior_logvar = (self.logvar_fc(en2))          # posterior log variance
        posterior_var    = posterior_logvar.exp()

        return en2, posterior_mean, posterior_logvar, posterior_var

    def take_sample(self, input_, posterior_mean, posterior_var, prior_var):
        eps = input_.data.new().resize_as_(posterior_mean.data).normal_(std=1) # noise
        z = posterior_mean + posterior_var.sqrt() * eps                   # reparameterization
        return z

    def get_beta(self):
      # mu_z_unit = self.mu_z/(torch.norm(self.mu_z,dim=-1).unsqueeze(1))
      # return F.softmax(torch.mm(self.mu_z,self.embedding_words.T),dim=-1)
      return F.softmax(self.decoder_bn( torch.mm(self.mu_z,self.embedding_words.T).T).T,dim=-1)

    def decode(self, z):

        ## Theta - P(z|x,phi)
        N, *_ = z.size()
        size = (N, self.out_features, self.in_features) # N,K,2
        zx =  self.decoder_x_bn(z) # Nx2
        # zx = z
        x = zx.view(N, 1, self.num_coordinate).expand(size) # Nx1x2

        # zc = self.centres
        zc = self.decoder_phi_bn(self.centres)

        c = zc.view(1, self.num_topic, self.num_coordinate).expand(size)
        d = (x-c).pow(2).sum(-1)
        distances = self.basis_func(d)
        zx_phi = torch.exp(distances - torch.logsumexp(distances, dim=-1, keepdim=True)) # N x K

        beta = self.get_beta()
        recon_v = torch.mm(zx_phi,beta)

        return recon_v, zx, zx_phi, d,zc

    def forward(self, input_,normalized_input_, compute_loss=False):
        en2, posterior_mean, posterior_logvar, posterior_var = self.encode(input_,normalized_input_)
        z = self.take_sample(input_, posterior_mean, posterior_var, self.variance_x)
        recon_v, zx, zx_phi,d,zc= self.decode(z)

        if compute_loss:
            return recon_v, zx,self.loss(input_, recon_v, zx_phi, posterior_mean, posterior_logvar, posterior_var, d,zx)
        else: return z, recon_v, zx,zc, zx_phi

    def KLD(self, posterior_mean,posterior_logvar,posterior_var):
        N = posterior_mean.shape[0]
        prior_mean   = self.prior_mean.expand_as(posterior_mean)
        prior_var    = self.prior_var.expand_as(posterior_mean)
        prior_logvar = self.prior_logvar.expand_as(posterior_mean)

        var_division    = posterior_var  / prior_var
        diff            = posterior_mean - prior_mean
        diff_term       = diff * diff / prior_var
        logvar_division = prior_logvar - posterior_logvar

        xKLD = 0.5 * ((var_division + diff_term + logvar_division).sum(-1) - self.num_coordinate)
        return xKLD

    def loss(self, input_, recon_v, zx_phi, posterior_mean, posterior_logvar, posterior_var, d, zx,avg=True):
        N = posterior_mean.shape[0]

        NL = - (input_ * (recon_v+smoothen).log()).sum(-1) # (Word_Count_input_loss)
        # NL = - (input_ * recon_v).sum(-1) # (Word_Count_input_loss)
        NL= NL.mean(0)
        KLD = self.KLD(posterior_mean,posterior_logvar,posterior_var).mean(0)
        loss = NL+KLD
        return loss,NL,KLD

In [ ]:
def train(tensor_train_w,train_label,num_input,num_topic,embedding_tensor_sorted_alp):
  # norm_emb_alp = embedding_tensor_sorted_alp/(torch.norm(embedding_tensor_sorted_alp,dim=-1).unsqueeze(1))  ## alphabetically sorted Vocab
  embedding_tensor_words = embedding_tensor_sorted_alp # norm_emb_alp
  embedding_tensor_words_d = embedding_tensor_words.to(device)

  # embedding_0_center_words = embedding_tensor_words - embedding_tensor_words.mean(0)
  # embedding_tensor_words_d = embedding_0_center_words.to(device)

  kld_arr,recon_arr = [],[]
  all_indices = torch.randperm(tensor_train_w.size(0)).split(bs)

  model = Model(num_input, en1_units_x, en2_units_x, num_coordinate, num_topic, drop_rate, variance_x, bs,
                embedding_tensor_words_d,emb_size, "gaussian")
  model.to(device)

  optimizer = optim.Adam(model.parameters(), learning_rate, betas=(beta1, beta2)) # ,weight_decay=1.2 * 1e-6
  epochs = 1000

  for epoch in range(epochs):

      loss_u_epoch = 0.0 ## NL loss
      loss_KLD = 0.0  ## KL loss
      loss_epoch = 0.0 ## Loss per batch #

      model.train()
      zx_l = []
      label_l = []
      for batch_ndx in all_indices:
      # batch_ndx = all_indices[0]
        input_w = tensor_train_w[batch_ndx].to(device)
        normalized_inputw = input_w/(input_w.sum(1).unsqueeze(1))
        labels = train_label[batch_ndx]
        label_l.extend(labels)
        recon_v, zx,(loss, loss_u, xkl_loss) = model(input_w,normalized_inputw,compute_loss=True)
        zx_l.extend(zx.data.detach().cpu().numpy())
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()             # backpror.step()
        # loss_epoch += loss.item()
        # loss_u_epoch += loss_u.item()
        # loss_KLD += xkl_loss.item()
        current_model = model
      # kld_arr.append(loss_KLD)
      # recon_arr.append(loss_u_epoch)

      if epoch % 10 == 0:
           print('Epoch -> {}'.format(epoch))
          # print('Epoch -> {} , loss -> {}'.format(epoch,loss_epoch))
          # print('recon_loss==> {} || KLD==> {}'.format(loss_u_epoch, loss_KLD))
          # plot_fig(np.array(zx_l),label_l,model.decoder_phi_bn(model.centres).data.cpu().numpy(),10.0,'No')
  return current_model,all_indices

In [ ]:
def test(model,all_indices,tensor_train_w,train_label,id_vocab):
  model.eval()
  x_list = []
  labels_list = []
  doc_ids = []
  zx_phi_list=[]
  with torch.no_grad():
      for batch_ndx in all_indices:
          input_w = tensor_train_w[batch_ndx].to(device)
          normalized_inputw = input_w/(input_w.sum(1).unsqueeze(1))
          labels = train_label[batch_ndx]
          labels_list.extend(labels)

          z, recon_v, zx,zphi, zx_phi = model(input_w,normalized_inputw,compute_loss=False)
          zx = zx.data.detach().cpu().numpy()
          zphi = zphi.data.detach().cpu().numpy()
          zx_phi = zx_phi.view(-1, num_topic).data.detach().cpu().numpy()
          zx_phi_list.extend(zx_phi)
          x_list.extend(zx)
          doc_ids.extend(batch_ndx)

      x_list = np.array(x_list)
      beta = model.get_beta().data.cpu().numpy()
      zphi = model.decoder_phi_bn(model.centres).data.cpu().numpy()
      # zphi = model.centres.data.cpu().numpy()
  print_Topics(beta)
  print('KNN:- ',cal_knn(x_list,labels_list),np.mean(cal_knn(x_list,labels_list)))
  return x_list,labels_list,zphi,doc_ids,beta

# **Run**

In [ ]:
# plot_loss(neg_log_rscore_arr,"map")

In [ ]:
# plot_loss(recon_arr,"Recon")

In [ ]:
tensor_train_w,train_label,id_vocab = get_data_label_vocab_normal(data_preprocessed,data_preprocessed_labels)
embedding_tensor_sorted_alp,emb_size = get_doc_word_embeddings(id_vocab,embeddings)

119822 4000
{'reuters': 2983, 'short': 3224, 'seller': 3142, 'wall': 3880, 'street': 3435, 'band': 292, 'green': 1537, 'private': 2712, 'investment': 1823, 'firm': 1377, 'group': 1548, 'reputation': 2943, 'making': 2115, 'controversial': 800, 'play': 2606, 'defense': 949, 'industry': 1758, 'quietly': 2804, 'bet': 360, 'market': 2150, 'soaring': 3294, 'crude': 871, 'price': 2697, 'worry': 3968, 'economy': 1139, 'outlook': 2469, 'earnings': 1124, 'expected': 1257, 'stock': 3417, 'week': 3913, 'summer': 3491, 'authority': 266, 'oil': 2415, 'export': 1274, 'flow': 1397, 'main': 2109, 'pipeline': 2586, 'southern': 3319, 'iraq': 1833, 'intelligence': 1795, 'rebel': 2858, 'militia': 2228, 'strike': 3440, 'infrastructure': 1765, 'official': 2409, 'saturday': 3090, 'afp': 70, 'record': 2868, 'economic': 1137, 'barely': 305, 'three': 3632, 'month': 2268, 'presidential': 2689, 'election': 1155, 'ended': 1187, 'higher': 1630, 'friday': 1452, 'stayed': 3398, 'low': 2084, 'year': 3990, 'surged': 351

In [ ]:
start = time()
num_input = tensor_train_w[0].shape[0]
model,all_indices = train(tensor_train_w,train_label,num_input,num_topic,embedding_tensor_sorted_alp)
stop = time()

In [ ]:
x_list,labels_list,zphi,doc_ids,beta = test(model,all_indices,tensor_train_w,train_label,id_vocab)

In [ ]:
plot_fig(x_list, labels_list,zphi,lim =10,contour='no')

# Save all results

In [ ]:
# os.chdir('/content')
# !rm -r *

In [ ]:
# http://localhost:8083/?token=167a26051381d93793eed0f73bf5161db4b6292f3f42a793
# http://localhost:8081/?token=0ce7448e70368335577888aceaacfa5c1f5ab97e5d934e86

In [ ]:
# %time
# d_data= 'searchsnippet' # wos,bbc,searchsnippet
# min_df = 0 # we already applied min_df =3 in preprocessing
# dtype='short' # full, short
# dtype2 = 'small' # small , normal

import time,sys

num_of_KNN_ks=5
num_of_runs = 3
model_name = 'Embedding_Vis_Model'
os.chdir('/home/grad16/sakumar/CIKM_Experiments_2021/'+model_name+'/content')

# all_data = ['bbc','searchsnippet','wos']
# dtypes = ['full','short']
# dtype2 = 'normal'

num_topics = [50]
all_data = ['agnews120k']
dtypes = ['short']
dtype2 = 'normal'

for d_data in all_data:
  print(d_data+'\n\n')
  for dtype in dtypes:
    print(dtype+'\n\n')
    if d_data=='searchsnippet' and dtype=='full':
      continue

    all_results_ = {
    # '__id__':[],
    # 'sample_id':[],
    'all_indices':[],
    'KNN_Scores':[],
    'vocabs':[], ## save once
    'runtimes':[],
    # 'num_topics':[],
                    }

    save_results_dir = "/content/SavedResults/"+dtype+"/"+dtype2
    model_signature=d_data+'_'+dtype+'_'+dtype2+'_model_'+model_name
    os.makedirs(cur_dir+save_results_dir, exist_ok=True)

    loaded_data = load_data(d_data,dtype,dtype2)
    data_preprocessed , data_preprocessed_labels , embeddings, name = loaded_data

    tensor_train_w,train_label,id_vocab = get_data_label_vocab_normal(data_preprocessed,data_preprocessed_labels)
    embedding_tensor_sorted_alp,emb_size = get_doc_word_embeddings(id_vocab,embeddings)

    os.chdir(cur_dir+save_results_dir)
    KNN_scores_list = []
    for num_topic in num_topics:
      with open(str(num_topic)+"_topics_KNN_"+model_signature+".txt", "w") as f:
        runtimes_list_mean = []
        # id_list=[]
        all_indices_list=[]
        KNN_sample_list = []
        runtime_list=[]
        for k in range(num_of_runs):
          # id=1
          num_input = tensor_train_w[0].shape[0]
          start = time.time()
          model,all_indices = train(tensor_train_w,train_label,num_input,num_topic,embedding_tensor_sorted_alp)
          runtime= time.time()-start
          runtime_list.append(runtime)

          f.write("\n | Run - "+str(k+1)+"| data - "+d_data+dtype+dtype2+"| runtime:  "+str(runtime)+"s"+"\n")
          x_list,labels_list,zphi,doc_ids,beta = test(model,all_indices,tensor_train_w,train_label,id_vocab)
          topword_topics = get_topwords(beta, id_vocab)
          f.write("---"*10+"\n")
          for topwords in topword_topics:
            f.write(topwords+"\n")
          f.write("---"*10+"\n")
          f.write('KNN: '+str(cal_knn(x_list,labels_list))+', KNN mean score: '+str(np.mean(cal_knn(x_list,labels_list))))
          f.write("\n")
          f.flush()
          os.fsync(f.fileno())
          KNN_sample_list = np.append(KNN_sample_list,[cal_knn(x_list,labels_list)])
        KNN_scores_list = KNN_sample_list.reshape(num_of_runs,num_of_KNN_ks)
        all_results_['KNN_Scores'].append(KNN_scores_list)
        all_results_['runtimes'].append(runtime_list)
        torch.save(model.state_dict(), model_signature+'topics_'+str(num_topic)+'_run_'+str(k+1)+'.pt')

        ail=[t.numpy() for t in all_indices]
        all_indices_list.append(ail)
      all_results_['all_indices'].append(all_indices_list)

    save_obj(all_results_,'all_results_'+model_signature)
  # f.close()

agnews120k


short


119822 4000
{'reuters': 2983, 'short': 3224, 'seller': 3142, 'wall': 3880, 'street': 3435, 'band': 292, 'green': 1537, 'private': 2712, 'investment': 1823, 'firm': 1377, 'group': 1548, 'reputation': 2943, 'making': 2115, 'controversial': 800, 'play': 2606, 'defense': 949, 'industry': 1758, 'quietly': 2804, 'bet': 360, 'market': 2150, 'soaring': 3294, 'crude': 871, 'price': 2697, 'worry': 3968, 'economy': 1139, 'outlook': 2469, 'earnings': 1124, 'expected': 1257, 'stock': 3417, 'week': 3913, 'summer': 3491, 'authority': 266, 'oil': 2415, 'export': 1274, 'flow': 1397, 'main': 2109, 'pipeline': 2586, 'southern': 3319, 'iraq': 1833, 'intelligence': 1795, 'rebel': 2858, 'militia': 2228, 'strike': 3440, 'infrastructure': 1765, 'official': 2409, 'saturday': 3090, 'afp': 70, 'record': 2868, 'economic': 1137, 'barely': 305, 'three': 3632, 'month': 2268, 'presidential': 2689, 'election': 1155, 'ended': 1187, 'higher': 1630, 'friday': 1452, 'stayed': 3398, 'low': 2084, 'year'

In [ ]:
# plot_fig(x_list, labels_list,zphi,lim =10,contour='no')

In [ ]:
# # %time
# # d_data= 'searchsnippet' # wos,bbc,searchsnippet
# # min_df = 0 # we already applied min_df =3 in preprocessing
# # dtype='short' # full, short
# # dtype2 = 'small' # small , normal

# import time,sys

# num_of_KNN_ks=5
# # num_topics = [10,20,30,40,50]
# num_topics = [20]
# num_of_runs = 5
# model_name = 'Embedding_Vis_Model'
# os.chdir('/home/grad16/sakumar/CIKM_Experiments_2021/'+model_name+'/content')

# # all_data = ['bbc','searchsnippet','wos']
# # dtypes = ['full','short']
# # dtype2 = 'normal'

# all_data = ['searchsnippet']
# dtypes = ['short']
# dtype2 = 'normal'

# for d_data in all_data:
#   print(d_data+'\n\n')
#   for dtype in dtypes:
#     print(dtype+'\n\n')
#     if d_data=='searchsnippet' and dtype=='full':
#       continue

#     all_results_ = {
#     # '__id__':[],
#     # 'sample_id':[],
#     'all_indices':[],
#     'KNN_Scores':[],
#     'vocabs':[], ## save once
#     'runtimes':[],
#     # 'num_topics':[],
#                     }

#     save_results_dir = "/content/SavedResults/"+dtype+"/"+dtype2
#     model_signature=d_data+'_'+dtype+'_'+dtype2+'_model_'+model_name
#     os.makedirs(cur_dir+save_results_dir, exist_ok=True)

#     loaded_data = load_data(d_data,dtype,dtype2)
#     data_preprocessed , data_preprocessed_labels , embeddings, name = loaded_data

#     tensor_train_w,train_label,id_vocab = get_data_label_vocab_normal(data_preprocessed,data_preprocessed_labels)
#     embedding_tensor_sorted_alp,emb_size = get_doc_word_embeddings(id_vocab,embeddings)

#     os.chdir(cur_dir+save_results_dir)
#     with open("topics_KNN_"+model_signature+".txt", "w") as f:
#       KNN_scores_list = []
#       for num_topic in num_topics:
#         runtimes_list_mean = []
#         # id_list=[]
#         all_indices_list=[]
#         KNN_sample_list = []
#         runtime_list=[]
#         for k in range(num_of_runs):
#           # id=1
#           num_input = tensor_train_w[0].shape[0]
#           start = time.time()
#           model,all_indices = train(tensor_train_w,train_label,num_input,num_topic,embedding_tensor_sorted_alp)
#           runtime= time.time()-start
#           runtime_list.append(runtime)

#           f.write("\n | Run - "+str(k+1)+"| data - "+d_data+dtype+dtype2+"| runtime:  "+str(runtime)+"s"+"\n")
#           x_list,labels_list,zphi,doc_ids,beta = test(model,all_indices,tensor_train_w,train_label,id_vocab)
#           topword_topics = get_topwords(beta, id_vocab)
#           f.write("---"*10+"\n")
#           for topwords in topword_topics:
#             f.write(topwords+"\n")
#           f.write("---"*10+"\n")
#           f.write('KNN: '+str(cal_knn(x_list,labels_list))+', KNN mean score: '+str(np.mean(cal_knn(x_list,labels_list))))
#           f.write("\n")
#           f.flush()
#           os.fsync(f.fileno())
#           KNN_sample_list = np.append(KNN_sample_list,[cal_knn(x_list,labels_list)])

#         KNN_sample_list = KNN_sample_list.reshape(num_of_runs,num_of_KNN_ks).mean(0)
#         KNN_scores_list = np.append(KNN_scores_list,KNN_sample_list)
#         # runtimes_list_mean.append(np.mean(runtime_list))
#         all_results_['runtimes'].append(np.mean(runtime_list))
#         # torch.save(model.state_dict(), model_signature+'topics_'+str(num_topic)+'_sample_'+str(id)+'.pt')

#         # plot_fig(x_list, labels_list, zphi,query_center=0.0)

#         # id_list.append(id)
#         ail=[t.numpy() for t in all_indices]
#         all_indices_list.append(ail)
#         # id =id+1

#       all_results_['KNN_Scores'].append(KNN_scores_list.reshape(len(num_topics),num_of_KNN_ks))
#       all_results_['runtimes'].append(runtimes_list_mean)
#       # all_results_small['sample_id'].append(id_list)
#       all_results_['all_indices'].append(all_indices_list)

#     all_results_['vocabs'].append(id_vocab)
#     save_obj(all_results_,'all_results_'+model_signature)
#   # f.close()

In [ ]:
# os.chdir('/content')
# zipped_pickle_filename = str(num_of_runs)+'_runs_'+d_data+'_ep1000_'+'lr_'+str(learning_rate)+model_name+'_topics_10_50'
# os.system('zip -r '+zipped_pickle_filename+'.zip /content/SavedResults/'+dtype+"/"+dtype2)
# files.download(zipped_pickle_filename+'.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# plot_fig(x_list, labels_list,zphi,lim =10,contour='no')

# ____ STOP____

In [ ]:
STOP!!!!

In [ ]:
STOP!!!!

In [ ]:
STOP!!!!

# <<**Small Text Dataset** >>

#**0. Just Run these..**

In [ ]:
import os
if 'grad16' in os.getcwd(): local=True
else: local = False
if local == True: os.chdir('/home/grad16/sakumar/')
down_count=0

In [ ]:
## Full data
if local == True:
  model_name = 'Embedding_Vis_Model'
  cur_dir='/home/grad16/sakumar/CIKM_Experiments_2021/'+model_name
  data_dir = '/'

  os.chdir(cur_dir+data_dir)
  # os.mkdir(model)
  # os.chdir(model_name)
  # os.mkdir(d_data) ## uncomment this if data is new
  # os.chdir(d_data)

In [ ]:
!nvidia-smi

Fri Jun  4 15:32:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P2000        Off  | 00000000:01:00.0 Off |                  N/A |
| 44%   29C    P8     4W /  75W |   1092MiB /  5055MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pwd

'/home/grad16/sakumar/CIKM_Experiments_2021/Embedding_Vis_Model'

In [ ]:
# @title Imports
# from sympy.stats import RaisedCosine, density
if local==False:
  !pip3 install pickle5
import pickle5
import torch
import os
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
# %matplotlib inline
import pandas as pd
from torch import Tensor
from torch import nn
import re
from time import time
import numpy as np
import collections
import gc
from sklearn.feature_extraction.text import CountVectorizer
import torch.nn
# cos_sim = nn.CosineSimilarity(dim=-1,eps=1e-08)
# from nltk.corpus import stopwords

In [ ]:
#@title torch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import Parameter
import math

In [ ]:
#@title function : load / save pickle_obj
import pickle

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

def load_obj_pkl5(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle5.load(f)

# **Download Data & Helper Functions(Documents)**

In [ ]:
#@title OLDER_Download Data Function

# def download_data(data,dtype):
#   data = data.lower()
#   dtype = dtype.lower()

#   if data=='wos':
#     if dtype == 'full':
#       # os.system('wget -N https://www.dropbox.com/s/krx3am9pfsiidzx/wos_full.zip')
#       # os.system('unzip wos_full.zip')
#       os.system('wget -N https://www.dropbox.com/s/768lhd76l49nsa4/wos_full.zip')
#       os.system('unzip wos_full.zip')

#     if dtype == 'short':
#       os.system('wget -N https://www.dropbox.com/s/aauu04pahwptexz/wos_short.zip')
#       os.system('unzip wos_short.zip')

#   elif data=='bbc':
#     if dtype == 'full':
#       os.system('wget -N https://www.dropbox.com/s/5n7khg6ht1lglb6/bbc_full.zip')
#       os.system('unzip bbc_full.zip')

#     if dtype == 'short':
#       os.system('wget -N https://www.dropbox.com/s/p5pb0w27cp41qeb/bbc_short.zip')
#       os.system('unzip bbc_short.zip')

#   elif data=='searchsnippet':
#     if dtype == 'short':
#        os.system('wget -N https://www.dropbox.com/s/yp0b4wl6w51af47/searchsnippet_short.zip')
#        os.system('unzip searchsnippet_short.zip')

#   else: print('either "data" or "dtype" is entered incorrectly ')


In [ ]:
# @title Old Download Data Function

# def download_data(data,dtype):
#   data = data.lower()
#   dtype = dtype.lower()

#   if data=='wos':
#     if dtype == 'full':
#       os.system('wget -N https://www.dropbox.com/s/w7ryozpj6qq1snx/wos_full4000.zip')
#       os.system('unzip wos_full4000.zip')

#     if dtype == 'short':
#       os.system('wget -N https://www.dropbox.com/s/eqkvxl6z0jwp44u/wos_short4000.zip')
#       os.system('unzip wos_short4000.zip')

#   elif data=='bbc':
#     if dtype == 'full':
#       # os.system('wget -N https://www.dropbox.com/s/a628h7e7lnlasc7/bbc_full2000.zip')
#       # os.system('unzip bbc_full2000.zip')
#       os.system('wget -N https://www.dropbox.com/s/0gbgvvqk4xoqfk6/bbc_full2000_20.zip')
#       os.system('unzip bbc_full2000_20.zip')

#     if dtype == 'short':
#       os.system('wget -N https://www.dropbox.com/s/4ly63afqc7ugz69/bbc_short2000.zip')
#       os.system('unzip bbc_short2000.zip')
#       # os.system('wget -N https://www.dropbox.com/s/l1aargdbd8e3bjm/bbc_short2000_20_4.zip')
#       # os.system('unzip bbc_short2000_20_4.zip')

#   elif data=='searchsnippet':
#     if dtype == 'short':
#        os.system('wget -N https://www.dropbox.com/s/8emmtg3qdqq7vpg/searchsnippet_short3000.zip')
#        os.system('unzip searchsnippet_short3000.zip')

#   else: print('either "data" or "dtype" is entered incorrectly ')


In [ ]:
#@title OLD Data Loading functions..
# def load_data(data,dtype,dtype2):
#   data = data.lower()
#   dtype = dtype.lower()
#   dtype2 = dtype2.lower()
#   d,l,e,fn = load_d_data(data,dtype,dtype2)
#   return d,l,e,fn
# sample_size = 50
# def load_data(data,dtype,dtype2='small'):
#   if (dtype == 'full' or dtype == 'short') and dtype2 == 'normal':
#     data_preprocessed = load_obj('data_preprocessed')
#     data_preprocessed_labels = load_obj("data_preprocessed_labels")
#     embeddings = load_obj('embeddings')
#   elif (dtype == 'full' or dtype == 'short') and dtype2 == 'small':
#     data_preprocessed = load_obj('docs_sampled_small_'+str(sample_size)+data+dtype)
#     data_preprocessed_labels = load_obj("labels_sampled_small_"+str(sample_size)+data+dtype)
#     embeddings = load_obj('embeddings')

#   os.system("rm -r *.pkl")
#   return data_preprocessed,data_preprocessed_labels,embeddings,d_data


In [ ]:
#@title Download Data Function

def download_data(data):
  data = data.lower()
  if data=='wos':
    os.system('wget -N https://www.dropbox.com/s/f3qw75jmsdlyk8j/wos_4000_20.zip')
    os.system('unzip wos_4000_20.zip')

  elif data=='bbc':
     os.system('wget -N https://www.dropbox.com/s/4mmiaed6rg5lpb1/bbc_2000_20.zip')
     os.system('unzip bbc_2000_20.zip')

  elif data=='searchsnippet':
       os.system('wget -N https://www.dropbox.com/s/tq31csl55b3oc09/searchsnippet_3000_20.zip')
       os.system('unzip searchsnippet_3000_20.zip')

  else: print('"data" is entered incorrectly ')

In [ ]:
#@title Data Loading Functions

sample_size = 20
small_data_no_of_ids = 5

def load_data(data,dtype,dtype2):
  data = data.lower()
  dtype = dtype.lower()
  dtype2 = dtype2.lower()
  d,l,e,fn = load_d_data(data,dtype,dtype2)
  return d,l,e,fn

def load_data(data,dtype,dtype2='small'):
  if local==True:
    dir ='/content/data_'+data+'/'+dtype
  if local==False:
    dir ='/content/content/data_'+data+'/'+dtype

  data_preprocessed=[]
  data_preprocessed_labels=[]
  os.chdir(cur_dir+dir)
  if (dtype == 'full' or dtype == 'short') and dtype2 == 'small':
    for i in range(small_data_no_of_ids):
      id = str(i+1)
      data_preprocessed.append(load_obj(id+'_docs_sampled_'+dtype2+'_'+str(sample_size)+d_data+dtype))
      data_preprocessed_labels.append(load_obj(id+'_labels_sampled_'+dtype2+'_'+str(sample_size)+d_data+dtype))
      embeddings = load_obj('embeddings'+'_'+data+'_'+dtype)
  if local==True:
    os.chdir('/home/grad16/sakumar/CIKM_Experiments_2021/'+model_name)
  if local==False:
    os.chdir('/content/')
  # os.system("rm -r *")
  return data_preprocessed,data_preprocessed_labels,embeddings,d_data


# **1. Preparing Data**

In [ ]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')
def doc_remove_extra_stopwords(doc):
  extra_stopwords = ['la','wa','will','fa','ha','pa','co','v','said']
  doc = word_tokenize(doc)
  doc = filter(lambda x:x not in extra_stopwords, doc)
  doc = ' '.join(e for e in doc)
  return doc

[nltk_data] Downloading package punkt to
[nltk_data]     /home/grad16/sakumar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#### Data Downloading ####
d_data= 'bbc' # wos,bbc,searchsnippet
# min_df = 0 # we already applied min_df =3 in preprocessing
dtype='full' # full, short
dtype2 = 'small' # small , normal

# download_data(d_data) ###

# ##### Data loading #####

loaded_data = load_data(d_data,dtype,dtype2)
# loaded_data = load_short_small_data_(d_data,dtype)
# loaded_data = load_small_data(d_data)
##########################

data_preprocessed , data_preprocessed_labels , embeddings, name = loaded_data
# for d in range(len(data_preprocessed)):
  # data_preprocessed[d] = doc_remove_extra_stopwords(data_preprocessed[d])
print(name,len(data_preprocessed_labels),len(data_preprocessed[0]),len(embeddings),dtype,dtype2)

bbc 5 100 2000 full small


In [ ]:
for data in data_preprocessed:
  len_docs = [len(d.split(" ")) for d in data]
  print(np.min(len_docs),np.mean(len_docs).round(2),np.max(len_docs))

35 122.06 516
45 123.7 376
36 123.21 526
30 116.38 256
49 122.87 373


In [ ]:
# if local:
  # os.chdir(colab_dir+model_name+'/plsv_vae')
def get_data_label_vocab(data_preprocessed,data_preprocessed_labels):
  min_df=0
  tensor_train_ws = []
  train_labels = []
  id_vocabs =[]

  for data,lables in zip(data_preprocessed,data_preprocessed_labels):
    preprossed_data = data
    train_label = lables
    vectorizer = CountVectorizer(min_df=min_df)
    train_vec = vectorizer.fit_transform(preprossed_data).toarray()
    vocab = vectorizer.vocabulary_
    id_vocab = dict(map(reversed, vocab.items()))
    # nonzeros_indexes = np.where(train_vec.any(1))[0]
    # train_vec_non_zeros = [train_vec[i] for i in nonzeros_indexes]
    # preprossed_data_non_zeros = [preprossed_data[i] for i in nonzeros_indexes]
    # train_label = [data_preprocessed_labels[i] for i in nonzeros_indexes]
    print(len(train_vec),len(vocab))
    print(vocab)

    tensor_train_ws.append(torch.from_numpy(np.array(train_vec)).float())
    train_labels.append(np.asarray(train_label))
    id_vocabs.append(id_vocab)
    return tensor_train_ws,train_labels,id_vocabs

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_coordinate = 2
bs = 256
en1_units_x = 100
en2_units_x = 100
variance_x = 1.0
learning_rate = 0.002
beta1 = 0.99
beta2 = 0.999
drop_rate = 0.0
# num_topic = 10

# **Cosine Scores**

In [ ]:
cos_sim = nn.CosineSimilarity(dim=-1, eps=1e-8)
def flatten_list(user_list): return [item for sublist in user_list for item in sublist]
def get_embedding_tensor(word_list,embeddings): return torch.tensor([embeddings[w] for w in word_list])
def get_all_cosines(id_vocabs,embeddings):
  all_word_embedding_tensor = []
  for id_vocab in id_vocabs:
    sorted_id_word_vocab = sorted(id_vocab.items(), key=lambda x: x[1]) ### alphabetically sorted
    word_list = [s[1] for s in sorted_id_word_vocab]
    # words_tensor = get_embedding_tensor(word_list,embeddings)
    all_word_embedding_tensor.append(get_embedding_tensor(word_list,embeddings))
    emb_size = get_embedding_tensor(word_list,embeddings).shape[1]
    return all_word_embedding_tensor,emb_size

# **Functions (Plot_loss, get_topwords, get_Z, get_Contour, print_Topics)** - *Run Once*

In [ ]:
#@title get_all_tensor_size
def getall_tensor_size():
  for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size(),get_mem_size(obj))
    except:
        pass

In [ ]:
#@title Testing_&_Extra_Fns

# embdding_input_NV300 = embedding_tensor_sorted_alp.unsqueeze(0).expand(tensor_train_w.shape[0],embedding_tensor_sorted_alp.shape[0],embedding_tensor_sorted_alp.shape[1])
# tensor_train_w_input = tensor_train_w
# y = torch.ones(tensor_train_w.size())
# w_input = torch.where(tensor_train_w_input >= 1, y, tensor_train_w_input)
# docWord_embedding_input = w_input.unsqueeze(-1).expand(w_input.shape[0],w_input.shape[1],300) * embdding_input_NV300

# def Log_NormDist_wC(x,mu,sigma=sig0,sigma_ir=sigc,d=emb_size):
#   x_Q = torch.narrow(x,1,0,x.shape[1]-1) # First Q (KxQx300)
#   mu_Q = torch.narrow(mu,1,0,mu.shape[1]-1) # First (KxQx300)
#   x_C_ir = torch.narrow(x,1,x.shape[1]-1,1) # Last C (Kx1x300)
#   mu_C_ir = torch.narrow(mu,1,mu.shape[1]-1,1) # Last C (Kx1x300)

#   First_Q = d/2.0 * torch.log(torch.tensor(sigma/(2.0*pi)))
#   Second_Q = -(sigma/2.0) * (torch.norm(x_Q-mu_Q,dim=-1)**2)

#   First_C_ir = d/2.0 * torch.log(torch.tensor(sigma_ir/(2.0*pi)))
#   Second_C_ir = -(sigma_ir/2.0) * (torch.norm(x_C_ir-mu_C_ir,dim=-1)**2)

#   First = First_Q+First_C_ir
#   Second = torch.cat((Second_Q,Second_C_ir),dim=1)
#   ret = First + Second

#   return ret

def min_max_normalization(ip):
  vmax,_ = torch.max(ip,-1)
  vmin,_ = torch.min(ip,-1)
  vmax = vmax.unsqueeze(-1).expand(ip.shape[0],ip.shape[1])
  vmin= vmin.unsqueeze(-1).expand(ip.shape[0],ip.shape[1])
  ip = (ip - vmin) / (vmax-vmin+smoothen)
  return ip

def NewBeta_dist(x,mu,phi_beta,smoothen):
  First = torch_lgamma(phi_beta+smoothen) - (torch_lgamma(phi_beta * mu+smoothen) + torch_lgamma(phi_beta * (1-mu)+smoothen ))
  Second_1 = (mu*phi_beta-1) * torch.log(x+smoothen)
  Second_2 = ( (1-mu)*phi_beta-1 )* torch.log(1-x+smoothen)
  Second = Second_1 + Second_2
  return First + Second

def LogL_NewBeta(x,relevant_scores):
  x_norm = torch.norm(x,dim=-1)
  mu = torch.exp(-0.5 * (torch.pow(x_norm,2))/mu_div)
  # LogL = torch.log(NewBeta_dist(relevant_scores,mu,sig,smoothen)+smoothen)
  LogL = NewBeta_dist(relevant_scores,mu,100.0,smoothen)
  return LogL

# wd_idx = torch.where(tensor_train_w[0]>0.0)
# tensor_train_w[0][wd_idx],wd_idx
# print(data_preprocessed[0])
# print(len(data_preprocessed[0].split(' ')))
# for w in wd_idx[0]:
#   print(id_vocab[w.item()],)
# print(wd_idx[0].shape)

# id_word_input = []
# count = 0
# dum_vocab = []
# for data in data_preprocessed:
#   doc = []
#   for w in data.split(' '):
#     if w in vocab and w not in dum_vocab:
#       dum_vocab.append(w)
#     if(w in vocab): doc.append(dum_vocab.index(w))
#   id_word_input.append(doc)
#   # print(data.split(' '))
#   # id_word_input.append(sorted([vocab[p] for p in data.split(' ') if p in vocab]))
#   #id_word_input.append([vocab[p] for p in data.split(' ') if p in vocab])
#   count = count + 1


############################
        # Testing #
############################
# a = torch.ones(12,10) * 4
# b = torch.ones(2,1)
# torch.norm(b,dim=0)
###
# for i in range(12):
#   a[i] = i
# a.unsqueeze(1).expand(12,5,10)
###
# x = torch.rand(5,4,3)
# a=torch.narrow(x,1,0,x.shape[1]-1) # First
# b=torch.narrow(x,1,x.shape[1]-1,1) # Last
# torch.cat((a,b),dim=1)
##
# a_muz = torch.rand(10,300) * 2.0
# b_wnm = torch.rand(20,300) * 4.0
# a = a_muz.unsqueeze(1).expand(10,20,300)
# b = b_wnm.unsqueeze(0).expand(10,20,300)
# c = torch.norm(a-b,dim=-1)**2

In [ ]:
import plotly.graph_objects as go
def plot_loss(y,name):
  figure = go.Figure()
  figure.add_trace(go.Scatter(x=[i for i in range(1,epochs+1)], y=y,mode='lines',name=name))
  figure.show(renderer='colab')

In [ ]:
def get_topwords(beta, id_vocab):
    topic_indx = 0
    no_of_topwords = 10
    topwords_topic = []
    for i in range(len(beta)):
        topwords_topic.append(str(topic_indx)+": "+ " ".join([id_vocab[j] for j in beta[i].argsort()[:-no_of_topwords - 1:-1]]))
        topic_indx+=1
    return topwords_topic

In [ ]:
def get_Z(X,Y):
  norms = torch.zeros(X.shape)
  Z = torch.zeros(X.shape)

  for i in range(X.shape[0]):
    for j in range(X.shape[0]):
      v = torch.tensor([X[i][j],Y[i][j]])

      norms[i][j] = torch.norm(v)
      Z[i][j]=torch.exp(-0.5 * (torch.pow(norms[i][j],2))/mu_div)

      # norms[i][j] = (v-query_center.T).pow(2).sum(-1)
      # Z[i][j]=torch.exp(-0.5 * norms[i][j]/mu_div)
  return Z

In [ ]:
def get_Contour(ax,x_list,lim):
  xlist = np.linspace(-lim, lim, 200)
  ylist = np.linspace(-lim, lim, 200)
  X, Y = np.meshgrid(xlist, ylist)
  Z = get_Z(X,Y)
  levels = np.arange(0,1,0.1)
  cp = ax.contour(X, Y, Z, levels=levels)
  ax.clabel(cp, inline=1, fontsize=10)

In [ ]:
import seaborn as sb

def plot_fig(zx, labels_list, zphi,lim,contour='No'):
    labels = []
    for i in range(len(labels_list)):
        labels.append('C'+str(labels_list[i]))
    fig, ax = plt.subplots( figsize=(20, 20))
    if contour=='yes':
       get_Contour(ax,zx,lim)
    a
    sb.scatterplot(ax=ax,x=zx[:,0],y=zx[:,1],hue=labels_list,alpha=0.8,palette='deep')
    ax.set(ylim=(-lim,lim))
    ax.set(xlim=(-lim,lim))
    ax.text(0,0, 'X' ,c='black')
    ax.scatter(zphi[:, 0], zphi[:, 1], alpha=1.0,  edgecolors='black', facecolors='none', s=30)

    for indx, topic in enumerate(zphi):
        ax.text(zphi[indx, 0], zphi[indx, 1], 'topic'+str(indx))

In [ ]:
def print_Topics(beta):
  print("---"*10)
  topword_topics = get_topwords(beta, id_vocab)
  topword_topics_list=[]
  for topwords in topword_topics:
      topword_topics_list.append(topwords.split())
      print(topwords)
  print("---"*10)

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
def cal_knn(coordinate, label):
    output = []
    for n_neighbors in [10, 20, 30, 40, 50]:
        neigh = KNeighborsClassifier(n_neighbors=n_neighbors, n_jobs=-1)
        neigh.fit(coordinate, label)
        output.append(neigh.score(coordinate, label))
    return output

# **Current Working Model**

In [ ]:
torch.cuda.empty_cache()
en1_units_x = 100
en2_units_x = 100
learning_rate = 0.001
beta1 = 0.99
beta2 = 0.999
drop_rate = 0.0
num_topic = 10
emb_size = 300
smoothen=1e-30
variance_x = 1.0

# sig_phi = 1.0

pi = torch.acos(torch.zeros(1)).item() * 2

def Log_NormDist(x,mu,sigma,d=emb_size):
  # sigma = sigma + smoothen
  First = d/2.0 * torch.log(torch.tensor(1.0/(2.0*pi*sigma)))
  Second = -(1.0/(2.0*sigma)) * (torch.norm(x-mu,dim=-1))
  # Second1 = -(1.0/(2.0*sigma))
  # Second2 = ((x-mu)**2).sum(-1)
  ret = First + Second
  return ret

#phi
def gaussian(alpha): return -0.5*alpha
#def inverse_multi_quadric(alpha): return -0.5*torch.log(torch.ones_like(alpha) + alpha)
def inverse_quadratic(alpha): return -torch.log(torch.ones_like(alpha) + alpha)

class Model(nn.Module):
    def __init__(self, num_input, en1_units_x, en2_units_x, num_coordinate, num_topic, drop_rate, variance_x, bs,
                 embedding_words, word_emb_size, distance="gaussian"):

        super(Model, self).__init__()
        self.num_input, self.num_coordinate, self.num_topic, self.variance_x, self.bs \
            = num_input, num_coordinate, num_topic, variance_x, bs

        self.embedding_words = embedding_words
        self.emb_size = word_emb_size

        # encoder
        self.en1_fc     = nn.Linear(num_input, en1_units_x)
        self.en2_fc     = nn.Linear(en1_units_x, en2_units_x)
        self.en2_drop   = nn.Dropout(drop_rate)
        self.mean_fc    = nn.Linear(en2_units_x, num_coordinate)
        self.logvar_fc  = nn.Linear(en2_units_x, num_coordinate)

        self.mean_bn    = nn.BatchNorm1d(num_coordinate)
        self.logvar_bn  = nn.BatchNorm1d(num_coordinate)
        self.decoder_x_bn = nn.BatchNorm1d(num_coordinate)
        self.decoder_phi_bn = nn.BatchNorm1d(num_coordinate)
        self.decoder_bn = nn.BatchNorm1d(self.num_topic)

        self.en1_fc_e     = nn.Linear(300, en1_units_x)
        self.en2_fc_e     = nn.Linear(en1_units_x, en2_units_x)
        self.en2_drop_e   = nn.Dropout(drop_rate)

        # RBF
        self.in_features = self.num_coordinate
        self.out_features = self.num_topic
        self.centres = nn.Parameter(torch.Tensor(self.out_features, self.in_features)) # K x 2
        self.mu_z = nn.Parameter(torch.Tensor(self.out_features,self.emb_size))# K x 300
        # self.mu_z =nn.Embedding(self.out_features,self.emb_size)# K x 300
        # mu_z_MultiNormal = torch.normal(mean=self.embedding_words.mean(0).unsqueeze(0).expand(self.out_features,self.emb_size),std=0.00001) # Kx300
        # self.mu_z = nn.Parameter(mu_z_MultiNormal)

        if distance=="gaussian": self.basis_func = gaussian
        if distance=="inverse_quadratic": self.basis_func = inverse_quadratic
        self.init_parameters()

        # prior mean and variance as constant buffers
        prior_mean   = torch.Tensor(1, num_coordinate).fill_(0)
        prior_var    = torch.Tensor(1, num_coordinate).fill_(variance_x)
        self.prior_mean = nn.Parameter(prior_mean, requires_grad=False)
        self.prior_var  = nn.Parameter(prior_var, requires_grad=False)
        self.prior_logvar = nn.Parameter(prior_var.log(), requires_grad=False)

    def init_parameters(self):
        nn.init.normal_(self.centres, 0, 0.01)
        nn.init.normal_(self.mu_z, 0, 0.01)

    def encode(self, input_):
        N, *_ = input_.size()
        en1 = F.softplus(self.en1_fc(input_))                           # en1_fc   output
        en2 = F.softplus(self.en2_fc(en1))                              # encoder2 output
        en2 = self.en2_drop(en2)
        posterior_mean   = self.mean_bn(self.mean_fc(en2))          # posterior mean
        posterior_logvar = self.logvar_bn(self.logvar_fc(en2))          # posterior log variance
        posterior_var    = posterior_logvar.exp()

        return en2, posterior_mean, posterior_logvar, posterior_var

    def take_sample(self, input_, posterior_mean, posterior_var, prior_var):
        eps = input_.data.new().resize_as_(posterior_mean.data).normal_(std=1) # noise
        z = posterior_mean + posterior_var.sqrt() * eps                   # reparameterization
        return z

    def get_beta(self):
      # mu_z_unit = self.mu_z/(torch.norm(self.mu_z,dim=-1).unsqueeze(1))
      # return F.softmax(torch.mm(self.mu_z,self.embedding_words.T),dim=-1)
      return F.softmax(self.decoder_bn( torch.mm(self.mu_z,self.embedding_words.T).T).T,dim=-1)

    def decode(self, z):

        ## Theta - P(z|x,phi)
        N, *_ = z.size()
        size = (N, self.out_features, self.in_features) # N,K,2
        zx =  self.decoder_x_bn(z) # Nx2
        # zx = z
        x = zx.view(N, 1, self.num_coordinate).expand(size) # Nx1x2

        # zc = self.centres
        zc = self.decoder_phi_bn(self.centres)

        c = zc.view(1, self.num_topic, self.num_coordinate).expand(size)
        d = (x-c).pow(2).sum(-1)
        distances = self.basis_func(d)
        zx_phi = torch.exp(distances - torch.logsumexp(distances, dim=-1, keepdim=True)) # N x K

        beta = self.get_beta()
        recon_v = torch.mm(zx_phi,beta)

        return recon_v, zx, zx_phi, d,zc

    def forward(self, input_, compute_loss=False):
        en2, posterior_mean, posterior_logvar, posterior_var = self.encode(input_)
        z = self.take_sample(input_, posterior_mean, posterior_var, self.variance_x)
        recon_v, zx, zx_phi,d,zc= self.decode(z)

        if compute_loss:
            return recon_v, zx,self.loss(input_, recon_v, zx_phi, posterior_mean, posterior_logvar, posterior_var, d,zx)
        else: return z, recon_v, zx,zc, zx_phi

    def KLD(self, posterior_mean,posterior_logvar,posterior_var):
        N = posterior_mean.shape[0]
        prior_mean   = self.prior_mean.expand_as(posterior_mean)
        prior_var    = self.prior_var.expand_as(posterior_mean)
        prior_logvar = self.prior_logvar.expand_as(posterior_mean)

        var_division    = posterior_var  / prior_var
        diff            = posterior_mean - prior_mean
        diff_term       = diff * diff / prior_var
        logvar_division = prior_logvar - posterior_logvar

        xKLD = 0.5 * ((var_division + diff_term + logvar_division).sum(-1) - self.num_coordinate)
        return xKLD

    def loss(self, input_, recon_v, zx_phi, posterior_mean, posterior_logvar, posterior_var, d, zx,avg=True):
        N = posterior_mean.shape[0]

        NL = - (input_ * (recon_v+smoothen).log()).sum(-1) # (Word_Count_input_loss)
        # NL = - (input_ * recon_v).sum(-1) # (Word_Count_input_loss)
        NL= NL.mean(0)
        KLD = self.KLD(posterior_mean,posterior_logvar,posterior_var).mean(0)
        loss = NL+KLD
        return loss,NL,KLD

In [ ]:
def train(tensor_train_w,train_label,num_input,num_topic,embedding_tensor_sorted_alp):
  # norm_emb_alp = embedding_tensor_sorted_alp/(torch.norm(embedding_tensor_sorted_alp,dim=-1).unsqueeze(1))  ## alphabetically sorted Vocab
  embedding_tensor_words = embedding_tensor_sorted_alp # norm_emb_alp
  embedding_tensor_words_d = embedding_tensor_words.to(device)

  # embedding_0_center_words = embedding_tensor_words - embedding_tensor_words.mean(0)
  # embedding_tensor_words_d = embedding_0_center_words.to(device)

  kld_arr,recon_arr = [],[]
  all_indices = torch.randperm(tensor_train_w.size(0)).split(bs)

  model = Model(num_input, en1_units_x, en2_units_x, num_coordinate, num_topic, drop_rate, variance_x, bs,
                embedding_tensor_words_d,emb_size, "gaussian")
  model.to(device)

  optimizer = optim.Adam(model.parameters(), learning_rate, betas=(beta1, beta2))
  epochs = 1000

  for epoch in range(epochs):

      loss_u_epoch = 0.0 ## NL loss
      loss_KLD = 0.0  ## KL loss
      loss_epoch = 0.0 ## Loss per batch #

      model.train()
      zx_l = []
      label_l = []
      for batch_ndx in all_indices:
      # batch_ndx = all_indices[0]
        input_w = tensor_train_w[batch_ndx].to(device)
        labels = train_label[batch_ndx]
        label_l.extend(labels)
        recon_v, zx,(loss, loss_u, xkl_loss) = model(input_w,compute_loss=True)
        zx_l.extend(zx.data.detach().cpu().numpy())
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()             # backpror.step()
      #   loss_epoch += loss.item()
      #   loss_u_epoch += loss_u.item()
      #   loss_KLD += xkl_loss.item()
        current_model = model
      # kld_arr.append(loss_KLD)
      # recon_arr.append(loss_u_epoch)

      # if epoch % 10 == 0:
      #     print('Epoch -> {} , loss -> {}'.format(epoch,loss_epoch))
      #     print('recon_loss==> {} || KLD==> {}'.format(loss_u_epoch, loss_KLD))
      #     # plot_fig(np.array(zx_l),label_l,model.decoder_phi_bn(model.centres).data.cpu().numpy(),10.0,'No')
  return current_model,all_indices

In [ ]:
def test(model,all_indices,tensor_train_w,train_label,num_input,id_vocab):
  model.eval()
  x_list = []
  labels_list = []
  doc_ids = []
  zx_phi_list=[]
  with torch.no_grad():
      for batch_ndx in all_indices:
          input_w = tensor_train_w[batch_ndx].to(device)
          labels = train_label[batch_ndx]
          labels_list.extend(labels)

          z, recon_v, zx,zphi, zx_phi = model(input_w,compute_loss=False)
          zx = zx.data.detach().cpu().numpy()
          zphi = zphi.data.detach().cpu().numpy()
          zx_phi = zx_phi.view(-1, num_topic).data.detach().cpu().numpy()
          zx_phi_list.extend(zx_phi)
          x_list.extend(zx)
          doc_ids.extend(batch_ndx)

      x_list = np.array(x_list)
      beta = model.get_beta().data.cpu().numpy()
      zphi = model.decoder_phi_bn(model.centres).data.cpu().numpy()
      # zphi = model.centres.data.cpu().numpy()
  print_Topics(beta)
  print('KNN:- ',cal_knn(x_list,labels_list),np.mean(cal_knn(x_list,labels_list)))
  return x_list,labels_list,zphi,doc_ids,beta

# **Run**

In [ ]:
# os.chdir('/content')
# !rm -r *

In [ ]:
# %time
# d_data= 'searchsnippet' # wos,bbc,searchsnippet
# min_df = 0 # we already applied min_df =3 in preprocessing
# dtype='short' # full, short
# dtype2 = 'small' # small , normal

import time,sys

num_topics = [10,20,30,40,50]
num_of_runs = 5
model_name = 'Embedding_Vis_Model'
all_results_small = {
'id':[],
'all_indices':[],
'KNN_Scores':[],
'vocabs':[], ## save once
'runtimes':[],
'num_topics':[],
}
os.chdir('/home/grad16/sakumar/CIKM_Experiments_2021/'+model_name+'/content')


all_data = ['bbc','searchsnippet','wos']
dtypes = ['full','short']
# all_data = ['bbc']
# dtypes = ['full']
dtype2 = 'small' #
for d_data in all_data:
  for dtype in dtypes:
    if d_data=='searchsnippet' and dtype=='full':
      continue
    # all_results_small['dtype'].append(dtype)
    # all_results_small['dtype2'] =
    save_results_dir = "/content/SavedResults/"+dtype+"/"+dtype2
    model_signature=d_data+'_'+dtype+'_'+dtype2+'_model_'+model_name
    os.makedirs(cur_dir+save_results_dir, exist_ok=True)
    # os.chdir(cur_dir+save_results_dir)

    loaded_data = load_data(d_data,dtype,dtype2)
    data_preprocessed , data_preprocessed_labels , embeddings, name = loaded_data
    tensor_train_ws,train_labels,id_vocabs = get_data_label_vocab(data_preprocessed,data_preprocessed_labels)
    all_word_embedding_tensor,emb_size = get_all_cosines(id_vocabs,embeddings)

    os.chdir(cur_dir+save_results_dir)

    with open("topics_KNN_"+model_signature+".txt", "a") as f:
      for num_topic in num_topics:
        id=1
        KNN_scores_list = []
        runtimes_list_mean = []
        id_list=[]
        all_indices_list=[]
        for tensor_train_w,train_label,id_vocab,input_embeddings in zip(tensor_train_ws,train_labels,id_vocabs,all_word_embedding_tensor):
          KNN_sample_list = []
          runtime_list=[]
          for k in range(num_of_runs):
            num_input = tensor_train_w[0].shape[0]
            start = time.time()
            model,all_indices = train(tensor_train_w,train_label,num_input,num_topic,input_embeddings)
            runtime= time.time()-start
            runtime_list.append(runtime)
            f.write("\n Sample - "+str(id)+"| Run - "+str(k)+"| data - "+d_data+dtype+dtype2+"| runtime:  "+str(runtime)+"s"+"\n")
            x_list,labels_list,zphi,doc_ids,beta = test(model,all_indices,tensor_train_w,train_label,num_input,id_vocab)
            topword_topics = get_topwords(beta, id_vocab)
            f.write("---"*10+"\n")
            for topwords in topword_topics:
              f.write(topwords+"\n")
            f.write("---"*10+"\n")
            f.write('KNN: '+str(cal_knn(x_list,labels_list))+', KNN mean score: '+str(np.mean(cal_knn(x_list,labels_list))))
            f.write("\n")
            KNN_sample_list = np.append(KNN_sample_list,[cal_knn(x_list,labels_list)])
          KNN_scores_list = np.append(KNN_scores_list,KNN_sample_list.reshape(5,5).mean(0))
          runtimes_list_mean.append(np.mean(runtime_list))
          torch.save(model.state_dict(), model_signature+'topics_'+str(num_topic)+'_sample_'+str(id)+'.pt')

          # plot_fig(x_list, labels_list, zphi,query_center=0.0)

          id_list.append(id)

          ail=[t.numpy() for t in all_indices]
          all_indices_list.append(ail)

          id =id+1

        all_results_small['id'].append(id_list)
        all_results_small['KNN_Scores'].append(KNN_scores_list)
        all_results_small['all_indices'].append(all_indices_list)
        all_results_small['runtimes'].append(runtimes_list_mean)

      all_results_small['vocabs'].append(id_vocabs)
      save_obj(all_results_small,'all_results_'+model_signature)
    f.close()

100 1767
{'photo': 1158, 'challenge': 259, 'pro': 1224, 'picture': 1162, 'cheaper': 275, 'higher': 757, 'quality': 1268, 'high': 756, 'street': 1545, 'developer': 459, 'test': 1609, 'survey': 1577, 'carried': 245, 'magazine': 973, 'looked': 958, 'offered': 1084, 'deal': 416, 'lab': 902, 'image': 794, 'top': 1633, 'longer': 957, 'produced': 1229, 'wrong': 1758, 'month': 1035, 'warned': 1704, 'group': 722, 'cost': 375, 'create': 383, 'device': 462, 'compared': 328, 'online': 1093, 'revealed': 1352, 'finished': 641, 'water': 1712, 'damage': 407, 'ink': 825, 'produce': 1228, 'print': 1219, 'send': 1421, 'store': 1540, 'definitely': 435, 'worth': 1752, 'choose': 284, 'ross': 1373, 'generation': 696, 'professional': 1234, 'shop': 1448, 'developing': 460, 'size': 1470, 'producing': 1231, 'paper': 1126, 'considering': 353, 'sun': 1568, 'simpson': 1462, 'marketing': 993, 'manager': 985, 'belief': 160, 'continue': 361, 'customer': 403, 'people': 1147, 'thing': 1613, 'computer': 337, 'three': 162

In [ ]:
for tensor_train_w,train_label,id_vocab,input_embeddings in zip(tensor_train_ws,train_labels,id_vocabs,all_word_embedding_tensor):
      KNN_sample_list = []
      runtime_list=[]
      for k in range(num_of_runs):
        num_input = tensor_train_w[0].shape[0]
        start = time.time()
        model,all_indices = train(tensor_train_w,train_label,num_input,num_topic,input_embeddings)
        runtime_list.append(time.time()-start)

In [ ]:
# from google.colab import files
# os.chdir('/content')
# zipped_pickle_filename = str(num_of_runs)+'_runs_'+d_data+'_ep1000_'+'lr_'+str(learning_rate)+model_name+'_topics_10_50'
# os.system('zip -r '+zipped_pickle_filename+'.zip /content/SavedResults/'+dtype+"/"+dtype2)
# files.download(zipped_pickle_filename+'.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Saving Model Results
> Parameters Saved : Learning rate, Dropout, en1_units_x, [KNN Scores  (x10 times)]

In [ ]:
##### Run once -->
all_results = {
'model':[],
'learning_rate':[],
'en1_units_x':[],
'drop_rate':[],
'dtype':[],
'KNN_Scores':[],
'doc_ids_list_all':[],
}

In [ ]:
KNN_acc_list2 = []
doc_ids_list_all=[]
labels_list_all=[]
iter=10
for i in range(iter):
  model,all_indices = train()
  x_list,labels_list,zphi,doc_ids = test(model,all_indices)
  plot_fig(x_list, labels_list,zphi,lim =10,contour='no')
  KNN_acc_list2 = np.append(KNN_acc_list2,cal_knn(x_list,labels_list))
  # labels_list_all.append(labels_list)
  doc_ids_list_all.append(doc_ids.item())

In [ ]:
all_results['model'].append('Embedding_Space_Vis_Model')
all_results['learning_rate'].append(learning_rate)
all_results['en1_units_x'].append(en1_units_x)
all_results['drop_rate'].append(drop_rate)
all_results['dtype'].append(dtype)
all_results['doc_ids_list_all'].append(doc_ids_list_all)
all_results['KNN_Scores'].append(np.reshape(KNN_acc_list2,(iter,5)))

In [ ]:
save_obj(all_results,'EMB_Vis_results_'+d_data+'_mindf'+str(vectorizer.min_df)+'_'+dtype+'_iter'+str(iter))

In [ ]:
all_results

In [ ]:
# from google.colab import files
# files.download("EMB_Vis_results_bbc_mindf0_iter10.pkl")

In [ ]:
##BBC
        ## Short

        # array([[0.80622348, 0.79726544, 0.79396511, 0.79726544, 0.79490806],
        #  [0.8038661 , 0.79113626, 0.79349364, 0.79255068, 0.78689298],
        #  [0.77369165, 0.77227723, 0.76944837, 0.76709099, 0.76661952],
        #  [0.77840641, 0.77887789, 0.77369165, 0.77699198, 0.77652051],
        #  [0.78595002, 0.77746346, 0.77746346, 0.77369165, 0.77133428],
        #  [0.76049033, 0.75577558, 0.75907591, 0.7534182 , 0.7515323 ],
        #  [0.7241867 , 0.71522867, 0.70815653, 0.70532768, 0.70815653],
        #  [0.82649694, 0.81612447, 0.81706742, 0.81518152, 0.8175389 ],
        #  [0.82602546, 0.81612447, 0.80999529, 0.80858086, 0.80858086],
        #  [0.8000943 , 0.79679397, 0.79585101, 0.79349364, 0.79255068]])

        ## Small

        # array([[0.468, 0.408, 0.426, 0.414, 0.404],
        #        [0.636, 0.618, 0.602, 0.596, 0.58 ],
        #        [0.548, 0.52 , 0.502, 0.504, 0.49 ],
        #        [0.592, 0.56 , 0.542, 0.54 , 0.55 ],
        #        [0.554, 0.5  , 0.484, 0.48 , 0.478],
        #        [0.68 , 0.664, 0.662, 0.658, 0.656],
        #        [0.534, 0.484, 0.472, 0.468, 0.466],
        #        [0.46 , 0.402, 0.406, 0.384, 0.366],
        #        [0.482, 0.466, 0.446, 0.458, 0.436],
        #        [0.632, 0.634, 0.63 , 0.622, 0.63 ]])


##Searchsnippet
        ## Short

        # array([[0.69106438, 0.67866654, 0.67232987, 0.66874828, 0.66626871],
        #        [0.65616677, 0.64037102, 0.63440169, 0.63256497, 0.62962623],
        #        [0.64523831, 0.62806502, 0.62154468, 0.62273854, 0.62007531],
        #        [0.65662595, 0.64312609, 0.64193223, 0.64367711, 0.64266691],
        #        [0.70943154, 0.70539076, 0.69887042, 0.69648269, 0.6967582 ],
        #        [0.72485995, 0.71310497, 0.71246212, 0.71218661, 0.71255395],
        #        [0.68077877, 0.66140141, 0.65901368, 0.65478924, 0.65221783],
        #        [0.72219671, 0.71282946, 0.71034989, 0.70667646, 0.70502342],
        #        [0.69712554, 0.68509505, 0.68344201, 0.6818808 , 0.68013592],
        #        [0.7219212 , 0.71448251, 0.71310497, 0.71218661, 0.71227845]])

        ## Small
        #  array([[0.5875 , 0.57   , 0.54625, 0.5575 , 0.545  ],
        #  [0.54125, 0.52875, 0.52125, 0.515  , 0.51   ],
        #  [0.61   , 0.585  , 0.56375, 0.565  , 0.565  ],
        #  [0.63875, 0.625  , 0.61125, 0.595  , 0.59   ],
        #  [0.6225 , 0.5975 , 0.6025 , 0.5975 , 0.58875],
        #  [0.62875, 0.61875, 0.6175 , 0.615  , 0.61375],
        #  [0.59875, 0.58625, 0.575  , 0.57   , 0.555  ],
        #  [0.58375, 0.5775 , 0.5825 , 0.57375, 0.565  ],
        #  [0.59375, 0.58875, 0.58625, 0.5725 , 0.565  ],
        #  [0.55   , 0.53125, 0.52875, 0.5275 , 0.51875]])


In [ ]:
all_results

# Gaussian SV (Results - BBC)



In [ ]:
#@title new_50
gaussian_sv_x = '''
0.361397628528422	-1.38175079195553
-0.0891302243152068	-0.871218961907398
0.87301305164188	-0.581123838868132
0.469390632696861	-0.980961200444093
0.875612143600963	-1.1517396463105
0.533715238419066	-1.17070904500729
0.223170644620359	-1.22664506074125
-0.0372272502936482	-0.526888998391416
0.00427679806900373	-1.44142257129484
0.257383492106126	-0.794672130016377
-0.0885573080353299	-1.33975171848539
0.639376429171229	-1.01350260034659
0.233768772478758	-1.32361711544602
-0.00175054057852161	-1.23017475854131
0.702134795233367	-1.11234937304913
0.443124516794812	-1.22740614542583
0.736977092525436	-1.17825048184485
1.37140686223957	-0.581837673761687
0.0973553856076558	-1.12137833546742
0.469340922844887	-1.13365008604577
0.424957604748832	-1.21064414050479
0.358813973029223	-1.09866129544682
-0.246062205254504	-1.50851681577454
0.341603128331043	-1.24829733742009
0.442011493294137	-0.840820135303577
0.438315382598239	-0.526763309792627
0.844107350169052	-1.09260298588441
0.565958554235357	-1.10093316030078
0.432585181334789	-1.31194608649566
0.391745795740835	-1.31761159034657
0.428244650481822	-1.20385938405072
0.423036232518825	-1.61054384273748
0.966263388585209	-1.08499999408046
0.815539554022855	-1.05971421463121
1.26100757043044	-0.38253680636201
0.262590717490476	-1.39626278403411
0.779911030965657	-0.697088573352074
0.435223480711796	-1.34068886124545
-0.328753808862933	-1.00819126751562
0.431097561465948	-1.27905113632922
0.926337331504901	-0.500687961095993
0.704138115345127	-1.09899490427688
0.349221727889406	-0.928731304068409
0.121287331356959	-1.40604293506695
-0.139636329279336	-1.55052998769387
0.997198296619262	-0.623197590010157
0.469340922844887	-1.13365008604577
-0.609810096621254	-1.05720865799413
0.956555330366251	-1.2017762473957
0.452252647900719	-1.02252831188463
0.315593416741949	-0.329464964020205
0.948431130911241	-0.82293496106561
0.705154285892749	-1.27877485264127
-0.138859992787989	-1.22324556601258
-0.0936774578921802	-1.37059013992407
1.31850543796417	-0.915476583594607
0.709926962529556	-1.14600430710144
0.297557916124582	-1.09293555286154
0.979128319376366	-0.918501153993189
1.25385487355596	-0.711425737725394
0.133937719539128	-1.13286789121268
0.0868263965611398	-0.94512390404393
0.169289735839877	-1.37958524852245
0.821405681072654	-0.939114813561663
1.13928920939324	-0.689970061986564
0.544625862870505	-1.0680134226436
-0.0102588097772618	-1.36901282718735
-0.0498106276046492	-1.13950542005999
1.14164283946771	-0.851003411111111
0.277469429347485	-1.0631754794958
1.2008720587917	-0.757749050849323
0.401912394949476	-1.09386966848125
0.137423217132657	-1.04773222657874
-0.241263505530541	-0.976047025876271
0.425536470000499	-1.22766322012392
1.12941808925288	-0.961333794209081
1.39341102174743	-1.03183756498909
0.363784849302992	-0.572461941262113
0.369245122687687	-1.19612631067071
0.576690093509518	-0.277915567038907
0.617885586543897	-1.04721474743063
0.843118385199186	-1.10805423438577
0.430459677746257	-1.01904588248793
0.401038340461203	-1.19685485723797
0.386406009692719	-1.12324429161867
0.540815694310088	-1.15090780260547
1.17975149701321	-0.801675834884017
-0.251359168271273	-1.23787346096205
1.02052021359594	-0.610951885441673
1.01345187676345	0.0458973843225529
0.42566386319399	-1.23734848405407
0.497179983973124	-1.26808066948593
1.17670625875624	-0.221040368254515
0.244018330067558	-1.15086272444924
0.313778603847067	-1.40984800480899
-0.650671577089658	-1.45198330744878
0.0753453260791014	-0.879926791669675
0.476352856855549	-1.23541471585845
0.559121066619112	-1.30359924199015
1.00566614736394	-1.03349056537581
0.658846850968126	-1.11973723643141
0.831784220218004	-1.11441802839812
0.22040969595706	-1.25729766843637
0.785237813878473	-1.22996246851586
0.188578037414299	-1.22929522788919
0.796536001677894	-0.9118305470202
-0.032492345561012	-1.21725308572821
-0.349703406856258	-1.08070854220564
0.52253348720387	-1.07474613999462
0.713975293328212	-1.13317609235929
1.32091901099961	-0.941217208939316
0.801896577936476	-1.13102042211532
0.695661101730285	-1.17649698855896
-0.262325842686743	-1.2472076684207
1.26846172275791	-0.260264706647969
0.602618883620872	-1.23511228331118
0.322483746985425	-1.1587575809601
0.946474798804382	-1.02530069479965
0.397294486671186	-0.998075155616071
0.446815504324737	-1.27281793168563
0.340960875042749	-1.2181324941874
0.72270128367861	-0.994828701077831
0.432924589975794	-0.974312080143578
0.116273834704423	-0.0704274578633616
0.96452466140564	-0.655789362701836
1.15930734247175	-0.862680872250561
0.0897022555977243	-0.771508929335683
0.481640538206021	-1.2937910676919
0.967562617835353	-0.675021617654806
0.476291920976069	-1.13238932463211
0.0786992662412765	-1.11131604871741
0.670312052990785	-0.261278978842908
0.875448187475319	-0.834389177544947
1.26025948373613	-0.346988594027718
1.36715241168132	-0.814810595813936
0.342697108226091	-0.757530914871956
0.645447889771223	-0.844546814209244
0.751008402435201	-0.738401290169886
0.191403564134744	-1.42039735725678
0.823443824384524	-0.913206181858059
0.816535326591008	-1.13470198003561
-0.464029332434402	-1.45987594090008
1.01496060345957	-0.447799735246139
0.247170179430669	-1.26450090729496
-0.530708424775297	-0.917720410466641
0.485280147875418	-1.036105724306
0.0950934929365367	-1.30877127326396
0.988120095407264	-0.876117712425397
0.409144061626964	-1.28796468600726
0.070418212655096	-1.20265456304119
1.11872721875292	-0.703361796701652
0.135808961639222	-1.05500602353794
0.364648642029105	-1.07803876575001
0.975501378545747	-1.26158858197254
0.729256475531438	-0.616324610836465
0.659016079644789	-1.3057315734546
1.48565340119801	-0.76687060054557
0.620810700604619	-1.062526600945
0.846190761108924	-1.16457269799982
1.45528584393575	-0.464579291076814
0.328594001965645	-1.3752741120196
1.27084241992895	-1.10603592357331
-0.283013421676264	-1.45492661106105
1.04660300959604	-0.761196042702949
0.186810432649746	-0.9419800012755
1.11966174506868	-0.642848283676323
0.652389917365797	-1.22941283510365
0.0117666174029366	-1.31956427464348
1.2204184851264	-0.320716346711532
0.413806504190283	-1.06045177154071
0.756047263273916	-0.343128628071429
1.05470033313684	-0.866672795634159
0.0227261030400382	-1.06978537247596
0.0674403744972565	-1.2151486467108
0.0239306810776658	-1.32590528469152
0.529133383316149	-1.0667545238627
-0.251625039645855	-1.2152491094403
-0.0098290243670117	-1.23553201935669
0.748694248147274	-0.526750584846053
0.490945147507038	-1.04973291286308
0.398258137690305	-1.03380989152765
0.316633336310314	-1.25046570924632
0.811780857057456	-0.980880467327531
0.832397543701355	-1.24629271707309
-0.113060825485635	-1.09686117005264
0.070786323830169	-1.04571472118378
1.58850319661306	-0.793313959196125
0.0508168259832586	-0.471980123596956
0.0646698454375168	-1.48104180766432
1.15277609706998	-1.01906189823677
0.428854124602621	-0.918425945743362
0.203365459784863	-0.795489341730285
1.44470747299125	-0.588473015557234
0.635464637756806	-1.16153345509018
1.00435706241831	-0.968412711336975
0.155393404808521	-1.123668577166
0.34174014941642	-1.18214310978328
0.0420321709722491	-1.36555470469196
0.50706874014216	-1.29760353699619
0.164833428457059	-1.39556380290496
0.111744399865842	-1.14627660734688
0.222629724596908	-1.40591812310795
0.177706184080463	-1.29674852238796
1.06241700400155	-0.502123741695315
0.54440507583433	-1.4258926187561
0.78699491849046	-1.04400402405002
-0.0806731112936397	-1.2144925185224
0.564241579226739	-1.01008162383144
0.588359390433219	-0.695421118500955
-0.365051972121271	-0.354227730638103
0.768048518575716	-1.09211134720429
-0.0189841214270075	-1.26027497510647
0.208167183045139	-1.03498846668299
0.367414518518493	-1.38916362385486
0.277484901634336	-0.959327980076841
0.430674525986251	-1.09487945869598
1.04704686086641	-0.916522635436053
0.244679877763381	-1.17904481418162
0.758388218801021	-0.862656551775608
0.196914606405904	-1.17562906648994
0.523787323903234	-1.2573139536175
0.820465260412568	-0.969583807682279
-0.269449665604964	-1.12968550576083
0.0645641925490026	-0.487108447195551
0.484287271712108	-0.977061180900861
0.543336280948725	-1.234445203842
0.649801184662287	-1.23447149325914
0.294570786302548	-0.984951362944632
1.13886308780641	-0.550560198477571
0.88431189055478	-0.888645369144017
0.499304471588534	-1.01104656630204
0.926089709155677	-1.03522295318125
0.342321535261669	-1.31086864286394
0.458672848296195	-1.24315263799052
-0.021119157662819	-0.606102283284149
0.389280325040982	-0.9990969738606
0.0482152134931009	-1.3361305375438
0.855489461857495	-1.07705214793576
1.63204820012951	-0.685745605646107
0.14188898935884	-1.32251053788364
0.229350632760407	-1.3410094022861
0.656708202967447	-1.13115929520222
1.04859596803986	-1.10469424629249
-0.515415660618125	-0.923606312189943
0.662489287274158	-1.02826918615413
0.903457739321268	-0.927961709246369
0.859151341172977	-0.692063410800883
-0.0746661810224464	-1.1729540889331
0.0129484512479585	-1.26277456340317
-0.117611468026932	-1.1652199949733'''

gaussian_sv_phi = '''-0.615366812747926	1.48165527027071
    -0.615367265661325	1.48165572639089
    -0.756867987121743	-0.846932412800338
    -0.615367078706819	1.48165553811322
    -0.615366797843711	1.48165525526089
    -0.615367404687421	1.48165586640045
    0.679577574726369	-0.741535467490471
    -0.615366805249734	1.48165526271939
    -0.615366867843867	1.48165532575699
    -0.615367740878551	1.48165620496803'''

In [ ]:
#@title gaussianSV Old
# ##-------Xai-------
# gaussian_sv_x = '''-1.54676556694973e-13	8.3672643944915e-13
# -9.30178670178395e-13	1.66402729022304e-12
# -2.82892291745705e-12	5.37058315326918e-12
# -5.005175156056e-12	7.70625587014964e-12
# 3.05613714125399e-13	4.8876264205399e-13
# -5.04302157685964e-12	7.3212867866008e-12
# -7.96257807882651e-13	-6.29741088981042e-13
# 1.23605507195233e-12	2.49283140603109e-13
# -1.01656110255686e-11	1.41249658555156e-11
# 1.35461842045957e-12	5.83494536282428e-13
# -1.10615899676403e-12	3.23821394050308e-12
# -3.01860620578997e-12	5.64053538304014e-12
# 3.73527195196815e-12	-1.81994392137739e-12
# -2.18368852618417e-12	4.05391781550738e-12
# -3.52059458694332e-12	6.35164550504305e-12
# -1.70953750457194e-12	8.21324582302056e-13
# -6.1116464272863e-12	9.06723097521059e-12
# 2.44087010722456e-12	1.01687535595056e-12
# -6.23829489918241e-12	8.38184224603225e-12
# 2.78158048767948e-12	2.06637988116215e-12
# -4.65233316006651e-12	6.97053409095149e-12
# 1.63206748271087e-13	2.26839986949074e-12
# -9.17078823130042e-12	8.4491318413687e-12
# 3.16462778069433e-12	-9.80099478405959e-13
# -2.76522785695846e-12	5.71232759722448e-12
# -4.22835838467763e-12	7.58572889131647e-12
# -1.87573152036081e-12	4.55163491517269e-12
# -1.0131203659375e-13	1.1600995556952e-12
# -7.97447559597784e-14	1.84867160671393e-12
# -1.94192798217516e-12	3.22107426800472e-12
# 2.94428993648874e-12	-2.05882030416163e-12
# 7.35534495598939e-12	-6.23949556188653e-12
# -1.98267764895944e-12	4.51096480061444e-12
# 4.25608137329158e-13	2.19855283656215e-12
# -3.17660294602624e-12	6.18155960133867e-12
# -2.46321967711661e-14	2.98075457277843e-12
# -1.19428883685286e-12	1.95577673423861e-12
# -2.90928455659873e-12	4.96871261301078e-12
# -3.2333111849856e-12	5.05775541284605e-12
# 2.48532213160616e-12	-2.68345855935459e-13
# -1.27286126290431e-12	3.9794860228616e-12
# 7.742559438954e-13	-1.53222589877154e-12
# 3.50644415716806e-12	-2.42364579680227e-12
# 1.66871237715344e-12	1.57117342279364e-12
# 1.3626625958664e-12	1.19483246881335e-13
# 3.80392570705377e-12	-2.68242026455814e-12
# -1.8884696189112e-12	4.89696766424338e-12
# 4.10668228212475e-12	3.21611092988393e-13
# -4.6480771149913e-12	7.26618257722058e-12
# -4.7898131849057e-12	6.56654391198068e-12
# -5.64990331354434e-13	3.18914550568356e-12
# -3.60761929471701e-12	7.22141364263232e-12
# -6.28444466362866e-13	2.90157072055841e-12
# 2.9100704465556e-12	-5.21795852680768e-13
# 2.97259626214543e-13	2.09814522010456e-12
# -1.70607609817837e-12	3.85645716567493e-12
# -3.34955475535773e-13	3.58089832248643e-12
# 6.92317152097071e-13	1.08660176619778e-12
# 8.88401408003278e-13	7.46997710793454e-13
# -4.79267522380569e-12	4.65241281869913e-12
# 1.22699867823924e-12	1.17542569908806e-12
# -2.92656933014191e-12	4.87021834366049e-12
# 1.24563994453942e-13	3.1262515412312e-12
# -2.09543716773491e-12	4.63965315337584e-12
# -4.94477613127155e-13	2.08449760182793e-12
# 1.0297352819916e-13	3.9145874160124e-12
# 2.66729563999035e-12	-2.07913852159511e-12
# -6.3160506541036e-13	2.6064357381689e-12
# -4.7004602181003e-12	5.98198527236037e-12
# -2.80256203547439e-12	4.43921184855046e-12
# -4.95893600882856e-13	2.74144746749962e-12
# -3.64470755605128e-12	6.87480698933523e-12
# -7.41144529952242e-15	2.14815088280908e-12
# 2.69510413120809e-12	-1.75346629144802e-12
# -1.68819535682558e-12	3.91230676217603e-12
# 4.25092924808568e-12	-1.28989415419372e-12
# -1.28021000365567e-12	3.62113061023864e-12
# 1.60189913065331e-12	-1.24491606774722e-12
# 6.70981379765545e-13	1.61424090448774e-12
# -2.76654475029745e-12	5.02541815208042e-12
# -2.54075646643736e-12	5.24231446733064e-12
# 2.77920859167761e-12	-4.46044073682107e-13
# -2.84286235944677e-13	1.98336058011015e-12
# -1.23265593252656e-12	3.36727922692122e-12
# -1.3536615730816e-12	3.02911216281645e-12
# -2.72321493524449e-12	5.31784959280105e-12
# -4.21503500327292e-12	4.07554561502928e-12
# -2.09000463112177e-12	4.69991980275354e-12
# 3.59329699092803e-12	4.6755085275358e-13
# -1.6252335805029e-12	3.95234519294824e-12
# 3.94068235691287e-13	2.41475391113614e-12
# 2.73571459123237e-12	-1.00035072097423e-12
# 1.09538234101075e-12	1.53798127805835e-12
# -4.10787737446506e-12	6.52922043232544e-12
# -3.46599915858807e-12	4.81217846936967e-12
# 5.59759291586503e-12	-1.91137087821264e-12
# 8.67946296639916e-13	2.7839361169817e-12
# -8.15721252292009e-14	1.46074141284454e-12
# -9.49103356130525e-12	1.24091593265021e-11
# -5.6494794556767e-13	3.53481391199792e-12
# -1.09803467908175e-13	1.51597518100616e-12
# -6.59588619828702e-12	8.30891051144015e-12
# 1.97195112853564e-12	6.50354944861426e-13
# -5.56730053964446e-12	6.70633980745344e-12
# -6.994724183764e-12	9.11039758411124e-12
# -1.03181747423814e-12	2.87391068140135e-12
# 3.30566980053364e-13	-2.46179541961403e-13
# -9.31263608830514e-13	3.38435082963216e-12
# 1.27900610453286e-13	1.93445336737925e-12
# -1.80135576498828e-12	5.02047649777463e-12
# 1.50069658250492e-13	2.35264393873867e-12
# -8.29036374664214e-13	-9.98459550966841e-13
# -8.65846121322034e-13	-2.59795055052152e-13
# -2.40433565868176e-12	5.34919767449146e-12
# -4.15090394842877e-12	7.3410344193651e-12
# 7.39612840077924e-13	1.33177649876355e-12
# -4.0394255922048e-12	7.17246229415347e-12
# -2.03601562073024e-12	3.76298467107274e-12
# -1.24479621973794e-13	1.84229207114226e-12
# -5.1936223304244e-13	6.14148881325654e-12
# 2.08536045604577e-12	8.42054579699826e-13
# 1.93010378743436e-12	1.89514323263046e-13
# -1.5212882022033e-12	6.42610411205923e-13
# -6.36764956894799e-12	8.87627743428175e-12
# -2.25056731928473e-12	6.39783006707425e-12
# -1.128535637758e-12	9.230077041167e-13
# -5.35422874908294e-12	8.91362518640665e-12
# -2.8912676554367e-12	5.8212805111955e-12
# -3.24644425413918e-12	5.66335606965901e-12
# -8.02169904432304e-13	2.97614458539868e-12
# 2.92555902052373e-12	-9.49472524217981e-13
# 2.33872563121497e-13	2.06604860954351e-12
# 2.16040494535422e-12	-1.54554610981391e-12
# -1.01708530862513e-12	4.82735439672819e-12
# -7.99584014373667e-12	1.12895280424206e-11
# -7.84767324297348e-13	3.71180057021103e-12
# 5.63687119369411e-13	1.7868670098816e-12
# -4.31364980030512e-14	2.82515258350707e-12
# -3.7891194055137e-13	2.42269086969241e-12
# 9.76988256592003e-14	5.35341273808097e-13
# 2.31418170430636e-12	6.44934741326219e-13
# 9.96342047783159e-13	1.29921257236163e-12
# -3.60227810929331e-12	7.26451061631874e-12
# -5.57247603277257e-12	7.76710491255348e-12
# -7.9645411426889e-13	2.07742990634694e-12
# -2.51785254132852e-12	5.1785234939349e-12
# -1.20676380391992e-12	4.8820856731737e-12
# -2.92536669917614e-12	6.3893694985447e-12
# 1.04922585465529e-12	1.36159260724187e-12
# 1.94365231115553e-12	1.1899109046151e-13
# 1.84897247401908e-12	1.22563797094245e-12
# 1.18773596543253e-13	2.12682055061492e-12
# -2.63872792685283e-13	1.02093350481646e-12
# 9.9971159981498e-14	-1.29839518259383e-12
# 7.44223417561817e-13	2.02299177864223e-12
# -2.63358762810896e-12	5.31932713985834e-12
# -2.98350526873192e-13	3.46612986264365e-12
# 2.3355501937011e-12	-3.98992358568914e-12
# 6.45116517547699e-14	1.18869945740197e-12
# -2.07926356730808e-12	3.41562447746741e-12
# 1.93726957688451e-12	-2.52941061382866e-13
# -3.6495492545002e-12	3.4846165551808e-12
# -3.48712945880078e-12	5.16560326972618e-12
# -3.647081092872e-12	6.81134047384023e-12
# 2.73490646274427e-12	-3.94796663971728e-12
# -2.66459081996732e-12	5.62363202502408e-12
# -1.41061002715506e-13	-2.80148844433419e-13
# 4.80419230772959e-12	-3.26844460066035e-12
# -4.25432535016133e-14	2.4875440205372e-12
# -1.65726919502707e-12	4.28339150906009e-12
# 1.93499249001512e-12	-1.71007398795847e-12
# 2.22964603592215e-12	9.61489889504437e-13
# -3.0822678878095e-12	5.69843514022396e-12
# -1.70097473594466e-12	4.20047064091653e-12
# -6.29535973310556e-13	3.41405141074024e-12
# -2.60550987098951e-12	3.00213600638941e-12
# 8.26929879982854e-15	4.11261922503196e-13
# -2.55554069875236e-12	5.26324725089928e-12
# 5.17258625257585e-12	-3.00441348674122e-12
# 4.48312414660389e-12	-2.56499114764243e-12
# -6.66422778504474e-13	3.43065616263876e-12
# -1.688689617397e-12	3.88023296416005e-12
# 2.33702963733969e-12	-1.02429595978643e-12
# -4.41324250199452e-12	7.22284458189227e-12
# 3.41585440947697e-13	1.99392057587185e-12
# 3.57476752151026e-12	-1.21530259515123e-12
# -4.74275185242505e-12	7.15878472446714e-12
# -2.00662726966594e-12	4.08566298403307e-12
# -1.45855293347856e-13	2.51980368650543e-12
# 1.58524252301879e-13	6.17736827478628e-12
# 8.91261304155066e-13	2.19684321661375e-13
# 1.47984725514829e-11	1.36649593953627e-11
# 3.51449313182449e-13	1.70077267809312e-12
# -9.48371883304091e-13	7.51794161253005e-13
# 1.71914652199619e-13	3.8848710261368e-12
# 3.41899507598726e-12	-9.50427019183558e-13
# -8.10810088214323e-12	7.55359760970617e-12
# 1.90315717368915e-12	1.13644922406708e-13
# -3.19271666341528e-12	5.79580472184066e-12
# -7.57088440465512e-12	1.14166142270534e-11
# 2.5520479349265e-12	1.36993737107403e-12
# -1.87270641395677e-12	4.29229501860569e-12
# -2.11559185769037e-12	3.06054595551543e-12
# -8.33078146630889e-13	1.0561106708325e-12
# -4.85032739497771e-13	2.47047747968297e-12
# 4.78832307155219e-12	-2.58953664591696e-12
# -1.82434954604095e-12	4.38890828563827e-12
# -1.89688218367807e-13	8.41090149529759e-13
# 4.76675511804499e-12	-4.13456464528016e-12
# 1.09940705166572e-12	-1.69946838175495e-12
# -1.25035481330871e-12	2.9018958998637e-12
# 5.9950275932165e-13	1.39849977747657e-13
# -3.14181327904598e-12	4.76036112662496e-12
# -3.5743383790303e-12	6.20369134292331e-12
# -4.38101822881242e-12	8.31956213872292e-12
# -5.51265003234927e-12	7.34646063364207e-12
# -3.89286577460093e-12	8.65717942560857e-12
# -5.8099025647786e-12	8.09208197473054e-12
# 2.30178615394331e-12	-1.84483849562911e-12
# -3.34364732362339e-13	2.52681608863854e-12
# -8.78182720654024e-13	3.44429975805702e-12
# 1.76747977029921e-12	-1.76794538818823e-12
# 1.98900659986483e-13	3.3513827004988e-12
# -4.90501305776263e-12	7.21728147078212e-12
# -1.42983087274378e-12	2.98337826558735e-12
# -6.95846449293721e-13	3.61985840811915e-12
# -6.73105902295281e-12	8.05119819104414e-12
# -2.55677721001134e-12	5.12499212512863e-12
# -4.62635994784704e-13	5.15857286232622e-12
# 3.21409609427162e-12	-3.12263859826172e-12
# -8.03768618248266e-13	5.8123310189088e-13
# -2.66136125183698e-12	3.15324381227546e-12
# -1.78424155979664e-12	3.8406661256837e-12
# 2.11325276070684e-12	4.22414154252994e-13
# -8.41518764993552e-13	2.95212471908974e-12
# -1.70452431623494e-12	3.23121159988288e-12
# 4.75454898465348e-12	-3.15872185271773e-12
# 7.15521635148049e-14	3.5532295589522e-12
# 2.2396973329222e-12	3.65507822176232e-13
# 1.35036069591294e-13	5.68751166243054e-13
# -2.40740104142451e-12	4.72051401678255e-12
# 4.08410509432937e-13	1.95342202064516e-12
# -5.26443874826422e-12	7.78909350254903e-12
# 2.51833347374686e-12	-2.88739119777311e-13
# 3.00922126076785e-12	-5.53019098499956e-13
# -1.03522125971432e-12	4.01399367762537e-12
# -1.40633445240853e-12	3.88393711373599e-12
# 4.70681457931066e-13	1.52158344222354e-12
# -9.09163403725797e-13	3.6728789304759e-12
# -2.44876891947949e-13	4.01470082452276e-12'''

# gaussian_sv_phi = '''1.24910125560946e-12	6.3101864025539e-13-1.01581032584752e-13	3.09881605550631e-135.78947317153711e-13	-1.16449688127887e-136.33671950844544e-13	3.32387939185879e-141.4440389482393e-12	-1.01910961257819e-123.14779166879013e-13	-1.29575827375502e-12-6.64107532626497e-14	-5.93973530030955e-14-2.132040090058e-16	-2.95975999454438e-137.0285394087754e-13	6.26371744043548e-131.07171889754836e-12	9.43828698657013e-13'''

In [ ]:
#@title original

gaussian_sv_x = '''
0.629585607222542	-0.619075033495543
-0.175261487907166	-1.08242918861122
1.02115107163837	-1.31619263173639
1.10756649554608	-1.13325299043925
1.42865478538112	-1.16270959013127
1.0892567513596	-0.983947518079986
1.07722688974707	-0.95846651255059
0.0687779841965295	-1.54819740281999
1.48352000296858	-0.191021062982553
1.67737594148032	-1.43757446850872
1.07704181572979	-0.876705418032563
0.325219168161445	-1.57950145043761
0.149831847126732	0.0913323589645621
-0.161605770029254	-0.846534812089215
0.982072476478967	-0.780801140414891
-0.290047707730767	-0.941472812260666
-0.662130824017999	-1.20762425437103
0.598019002857851	-0.621186655787529
1.36636415554572	-1.23425793276132
0.644039762576659	-1.35390800117249
0.553090625104783	-1.57638684395228
1.25302542127206	-0.985811927142474
1.50454356850787	-1.49140265555476
0.705485070027432	-0.72090279242498
0.238450635723168	-0.347936434837728
1.12120638763836	-1.21088832142019
0.789284070298762	-0.450756599031469
0.145062709123414	-0.0469843349588447
0.964521707574319	-1.63437112367193
0.945842750394961	-0.839115336843363
0.871566627302086	-0.949277112460342
0.132521965529405	-0.716466453022105
1.4918755855282	-1.67027360432334
0.900403077423603	-1.22904904327512
1.35517497833601	-1.26449162962091
0.161911765374619	-1.3010156009007
-0.193724472964121	-0.97570365479574
0.516303464079322	-0.90743980186433
0.939198369700304	-1.23578400238992
0.712781045202562	-0.904945509533592
0.519822077781468	-1.7940074141087
0.617399920230501	-1.21277227972368
1.27171350587021	-1.49223132623041
-0.144043385608652	-0.959217979543544
0.551357140428979	-0.461546518511338
1.25023484177185	-1.15291650007049
1.08996868316213	-1.18353444511109
0.92446110868225	-0.887036280860669
1.37187703273598	-0.950619289741866
0.774250084322627	-1.02839741224765
-1.30288007909993	1.4912167724209
0.870918486164571	0.347157195527453
-0.243442517615945	1.04675531832067
-0.12172029775	0.59184891693709
-0.120483258605061	0.900590756087001
-0.355437956260826	0.913664952687064
-0.974353238507322	0.0541251616397779
-0.165307525812758	0.913868951947984
-1.24376268312861	0.665995697076313
-1.3058761588295	0.454243087747997
-0.27236759383791	1.25776509439751
-0.63881779592866	1.09508777236056
-0.438666369898989	0.478729723176782
0.569558950484575	0.415595824436782
-0.846573897460312	0.39745085501455
0.310269762231562	0.988344853367592
-0.541457577328352	0.184649919215751
-0.286477475333264	1.40185651555522
0.478491938938888	0.723369317377177
-0.394575873043839	0.429703173603809
-0.366078591833466	0.949961934219893
-1.42653336253489	0.240717721719661
-0.764617365809355	0.837553172604517
-0.61993569682197	0.304428222811318
0.0349230343788728	0.914630047276311
-0.526489141175372	0.782610740140805
-0.628179673669013	0.900762640374357
-0.977003059537927	0.682850844012477
-0.416266734803057	0.516021689033814
-0.497044454939233	0.60563709288091
-1.24838776990751	1.10363950940183
-1.49897833247331	-0.82605059046664
-0.799521055913905	1.05939545234235
0.546933319518282	0.67850379537218
-1.37881202674996	0.834684584652289
-1.38208899849201	0.97470953819261
-0.177165708931897	0.824205409695723
-0.12952021375384	0.72978097567777
-0.161090169617036	1.23662334807289
-0.859627079517342	0.971110354059873
-0.675964286970416	0.289653507984644
-0.586169677444321	1.02810342218029
-0.832132986073149	0.45837404575024
-1.67469010072259	0.613149873798062
-0.0111279182539486	-0.262539899645085
-0.719254667825634	1.1916954372634
-0.34029159898358	0.0242632793528384
-0.946882955789334	0.334951879580349
-0.667523522282486	0.681100370738037
-0.304763384905491	0.034282318443217
0.208717721785175	-1.23074438612237
-1.68702589269021	-1.2938973040285
-0.323731058066196	-1.27066853475493
0.059658159466579	0.649718061621705
-0.040078274061969	-0.531903243789501
-0.385697325716392	-0.801398144165916
-0.0576586633733941	-0.680956347099199
0.341019969691094	-0.225864317904865
-0.760685229143985	-1.14904144532273
0.47346850370901	-0.679444494695553
-0.144953433102296	0.345669855585475
-0.14171490167927	-0.835463039546461
-0.361974060989426	-0.724025282843687
-0.270396996852335	-0.563967781200189
-0.38146536352708	-0.482671102871447
-0.0797321095808082	-0.60783282409487
-0.264193982644698	-1.47456336299698
0.018631517508198	-1.12878943423178
-0.516902974682539	-1.46059695109022
-0.587752016382613	-0.155638439578567
-0.764852975299557	-1.3453468308491
-0.418506631538107	-1.29870896425142
-0.344995010690684	-1.73188883776729
-0.413555139793379	-1.22984219560725
-0.202761286671216	-0.97405966604604
-0.0997927661763049	-0.680614306945173
-1.38253931124571	-1.2678833719944
-0.639638638030746	-0.480382870934293
0.32720177045694	-0.809324617786208
-0.725981675942448	-0.536512018301034
-0.821681042436094	-1.93537628424356
-0.339292427640149	-0.925000081776804
0.29914655864959	-0.362576750553575
-0.00835662055931795	-0.904594703348286
0.428782573508295	-0.881890065193657
-0.211649034231793	-0.265329043127549
-0.476810817370023	-1.08035540629245
-0.768592539548246	-1.31532360678807
-0.181972894364463	-1.20329493517363
-0.309367592093899	-0.405444405091071
0.0321179534781622	-0.827195266633552
-0.0554757283650445	-0.0381480966958037
0.195173746204754	-0.141579356592373
0.0993829764996629	-1.01377118418843
-0.019300996029829	-0.470297052810584
-0.29355853125509	-1.16567053566767
-0.256550145431489	-0.83608900716402
-0.171528793105314	-0.525274925679549
0.116531620644173	-0.828538633519877
-0.344301354946891	-0.55680238451658
-1.11552133906944	-0.0588313154366642
-1.39143123391493	-0.159444359153973
-0.1359440971661	0.189294444121254
-0.443275256714831	0.174728217756362
-0.639866314690068	0.133472099699631
-1.55543734427617	-0.230226695048372
-1.12900624898718	-0.0548848602677562
-0.220019958988166	0.00508987229050529
-0.48584851683442	-0.0174576232563594
-1.1867416003774	0.0245997503789175
-1.24092860276822	0.175758158758085
-1.1087833359325	0.20639397725893
-1.06471939818112	0.401615592076339
-1.62786197127983	0.118865828504955
-0.953422129302374	0.196905370378124
-0.900023003742121	-0.0051436829317062
-0.470515559672183	-0.0916304615878393
-1.81354983731555	0.0668001819557014
-1.14446804321295	0.0327125748022428
-0.82841079128872	-0.234702097001657
-0.0875694536952964	-0.00479470902215297
0.0287890551842431	0.25230485499284
-0.539080157295281	0.229157003783257
-1.28840125027458	-0.0395315551991551
-0.847274574432135	-0.121156805175912
-0.342924730505252	-0.796048324036336
-1.65199315653485	0.088151808052498
-0.501331675853147	-0.399118822125305
-0.839168009542617	0.539945235486652
0.0814298804175472	0.419475892323468
-0.678665177469867	0.0572011890342356
-0.961707178145144	-0.0125169467133501
-0.369512165029656	-0.780518943757449
-1.37495800703923	0.334868885331901
-0.793935237530002	-0.164966229540909
0.562292674079896	0.0693412014703923
-0.893402504183902	-0.0925736384765877
-1.253388978674	0.29468498751153
-0.649696760952102	0.439962729922117
-0.0490561644677574	0.205199097404398
-0.730962075936188	-0.0799955820535316
-1.15314725333488	0.0368857904954003
-0.867470019674691	-0.161947030269868
-0.185824954599942	0.673265395494331
-0.543675950394313	0.0522769294078998
-1.69180921409476	0.051142232711624
-1.39457010891768	0.14983740656335
-0.887425536319813	0.291756748179216
-1.35260762924564	-0.286769237878431
-1.11686286407594	0.00717543047815324
0.450709413142308	0.404738342639009
1.30730085947648	0.397269721465546
1.8116201073711	0.335245533700225
0.758356892900804	-0.69280881280981
0.427858243596997	0.786434858502574
1.019046949622	0.419091223462155
1.73368645179785	0.327974489838628
1.04969027730328	-0.37583390938116
0.726818291986012	0.257567420892381
-0.238323817546117	0.6536861553244
1.44447625603	0.52955265887607
1.28262226446795	0.538752376878719
-0.79928259028512	-0.127047246477065
1.13130817662858	0.0813084981901365
1.21011789627237	0.672643331240436
0.463352037664766	0.145250428928191
1.53776337680455	1.10109526206455
1.25922209964178	0.507136666402929
0.637257284404925	0.445001951845216
1.08369298231768	0.785708695173124
1.07167021177674	0.858454537207251
2.42079547122747	0.629020813622055
1.05561155430067	0.364613703558102
1.44087623275846	0.359644918234859
1.17692843442213	0.718759054093102
1.24645098926668	1.83077699110471
0.336815693396514	0.185521351761503
1.70375659769198	0.558955668329362
0.484480447333523	0.715350447953767
1.45458052595646	0.177801635022196
1.3235690539402	0.348835798829981
1.64690207892387	0.381024787811878
1.24130202238782	-0.324821936465504
1.0736380589226	0.881865585355681
0.617791171856166	-0.179183099158741
0.856538633552287	0.182977222316471
0.614407384712756	0.564736273694816
1.45806515730096	0.579651336006582
1.24494749021641	0.366914080216261
-1.14778139996865	0.20595365508057
0.271414008296047	-0.662001941591431
1.5016635777124	1.15509085425932
1.1300362367143	0.109003031854572
1.42469750431049	0.669926713428974
0.799293626941138	0.101677350721214
0.494289324135513	0.889830301284658
1.16165622124845	0.537052618942638
1.4486700129544	0.938859277892983
0.298576858897177	0.0237241086131946
2.15640070903002	0.191254952241688
'''
gaussian_sv_phi = '''
-0.370437786555724	0.0767985882083361
-1.58160022265142	0.119240966186222
-0.444795086928507	1.69053188061663
0.355840769068723	-0.098261062179189
-0.744915804181061	-1.34478804673204
1.18080675386425	-1.13064223531519
0.162506652800008	-0.0239508764642033
0.107396616582269	-0.301943147259452
-0.351403943849174	1.49793342008488
1.47342735786231	0.949942255361852
'''

In [ ]:
#@title new1_50_label
gaussian_sv_x = '''
1.16242561093739	-0.768957279107564
1.12004760093069	-0.483297589735551
1.16996506863091	-0.940150538731017
0.700914236860862	-1.18850490259338
0.943469658629311	-0.764564142763772
0.896240461018924	-0.630704031176271
1.86719428628966	0.0362782806594964
1.30817454254003	-0.324436955705153
0.427853945723313	0.178143561184925
0.967810051190393	-0.297929005959028
1.04639455544525	-0.688566611814179
0.604446760521424	-1.19743642843206
1.2986077711739	-0.0323869373137246
0.534433781087155	-1.3203158543763
1.33372738497874	-0.775409334939587
0.804842566531027	-0.764159316661325
0.593332060388462	-1.20483929106868
1.27464952401315	-0.131991220749809
1.18024328881629	-0.711060459887162
1.01791412346003	-0.669100203962786
0.988315210207911	-0.491297184307486
0.537083376144646	-0.672855695881894
0.536885650181364	-1.05419975644316
0.28839985845305	-0.327177757873492
0.567415657454915	-0.98985055412464
0.907289654162011	-0.856614965995525
0.979848460455259	-0.708749242058795
0.768593068610712	-1.18753857159555
1.12451106818709	-0.844482667411477
1.1307245297421	-0.988221674217483
0.916016435902844	-0.305561289566877
1.26732370701709	-0.292114029719572
0.993852199377749	-0.730239917299371
0.791233417107748	-0.646611196587394
0.843224812964403	-1.02984729746468
-0.38155843589064	-1.50958657756427
0.900790168083625	-0.850227083599067
0.51677899302865	-1.28239526790302
1.23109405225397	-0.421392616645728
1.64703505903705	-0.454243649171171
-0.0534819030276518	-1.08289889163895
-0.0454196025614505	-0.415958557341902
0.189433524933521	-1.14923346695583
1.68196486878061	0.00383466766853443
0.0571715118146399	-1.07468777557982
0.411357267034912	-0.546163313755249
0.846239368257142	-1.09314948425892
1.06740247716577	-0.783452459656452
0.615992675392682	-0.902709999898926
1.13211927350232	-0.731585482600361
0.707913035188568	-0.964471898740051
0.759459988487433	-1.08180423729095
0.656871974532097	-1.26355013466494
0.888494451669764	-0.988295331833748
0.0911505462689512	-0.93444483805309
0.672668385754894	-1.08000102635389
0.573970617417929	-0.39659543638455
0.215245672524887	-1.45457474205996
0.718048789165798	-1.14419223458955
0.553488267645882	-1.15033463897164
1.33677437465214	-0.652841600687871
1.43849972113041	-0.357714869162306
0.522475307001365	-1.29446219448766
0.93840350790651	-0.963016142574282
1.56402836989744	-0.382755365887904
1.08525767354565	-0.834293066379097
0.444252571223761	-0.941000917386096
1.1872751627534	-0.631181617968229
1.0387921659042	-0.603409835995682
0.524368082333144	-0.238294220511036
1.26882074756208	-0.397706545202799
0.947477796710266	-0.229154079991319
0.93247200812292	-0.636118161810541
0.953697195722956	-0.81692182804609
1.24168858008039	0.271711564148481
1.21503977745238	-0.414637923115904
0.636022717255352	-0.839979149430772
0.837310814962433	0.552020450665659
1.24173551832128	-0.70803647004461
0.36649137239859	-1.0050211002435
0.484180268135508	-1.46383482485148
-0.0409459398102299	-1.04612379226278
-0.339563312969086	-0.841022746439311
1.31790677494398	-0.630467534090444
0.410858986437862	-1.17291947926077
0.752966246764132	-0.295842376123056
0.588334104343658	-0.782660614065402
0.593309943498233	-1.08821872221817
0.611699539374919	0.46655058908842
1.25025683893722	-0.68675662756635
1.29250406408449	-0.600733581096053
1.14470055351406	-0.842555954370128
0.75046045784209	-0.352441855683162
0.242371842561421	-1.59362885930928
1.31445799437356	-0.776760849734022
0.242371842561421	-1.59362885930927
0.823731657706337	-0.993687989123371
0.972765307708723	-0.336319625645185
1.39087292257198	-0.474468608911412
1.11609902650931	-0.519216704410665
-0.258435946783166	-0.347868852918328
0.801104011120005	-1.02265489488646
0.785353169882033	-0.503280290265069
0.890971783953313	-0.708062952077767
0.687758543252766	-0.948267642293594
1.67367120436642	-0.339418999651171
1.60812029749371	-0.354724056831071
1.58233512995607	-0.520279060523079
0.953106026237323	-0.0447828433649492
1.13890696834406	-0.187997996235194
1.07963853596546	-0.50746933271633
1.09449721307391	-0.731242815028311
0.865792173497953	-1.08673724650602
0.890224990007428	-0.894237362111964
0.762787425866929	-1.18785498677116
0.820440087838984	-1.1753320420365
1.22855773192712	-0.63813252554153
0.933912542897457	-0.671079867737521
1.13955840602752	-0.726487226915106
0.766561644354371	-1.18306375584409
0.485119574812602	-1.10802584568006
0.761264143991279	-1.0577256507945
0.441944160690562	-1.15840387055587
1.06416444183489	-0.0709017930220592
1.10925729090359	-0.78273641600219
0.237714507292794	-1.12811804459262
0.92891067410693	-0.936371741175296
0.465095338952908	-1.05939744684934
1.02607344487913	-1.02757902418603
0.949106947809674	-0.964841865811352
1.39709969231751	-0.20765486351021
0.978421892796788	-0.72594938849522
0.592714070426847	-1.13311125556386
0.441176799163917	-1.13220517326585
0.740206887259456	-0.387107851109315
1.11616182859403	-0.587766413998246
0.906376540193406	-0.251685892707182
0.720444105331456	0.534856195080042
0.703927862803591	-1.32466264973126
1.2022164903138	-0.764448659163395
0.945640141114301	-0.851684391625613
0.933046752038936	-0.889365595547609
0.913125356401109	-0.79207724029923
1.09455665372882	-0.106324724651565
1.04087258761872	-0.809393262354138
0.584250892331161	-1.24373096263544
1.13007061175999	-0.601105662678607
0.543762420805884	-1.13421503665067
1.41004113164716	-0.665570536410208
0.100956797399635	-0.962507223715891
0.736044960733655	-1.17383855711242
1.36198048039969	-0.481985483216479
1.09463511655627	-0.907652230481149
1.26078222920823	-0.377697478065403
0.759937698443577	-1.18941996052905
1.07088262952354	-0.730296993959379
1.00742123223823	-0.904245809847479
1.72429648352764	0.0606718477179441
0.81851499867918	-1.16356635418542
0.749168056978422	-1.02767027153857
1.0679403849196	-0.907835055872376
0.646002671894694	-0.610334836731739
1.32731311244586	-0.113666712763248
0.81658914820539	-0.910804134741295
0.568400165331391	-1.1454148179904
0.942663617600934	-1.09729719827976
0.539419492633149	-1.00537467599259
0.430636756402737	-0.780224390427836
0.837718122266468	-1.06013889157652
0.667568280005505	-1.30704545816526
0.997242412380751	-0.622070823839041
1.1091122055819	-0.754678064816342
1.61043322105267	-0.324618154208888
0.370585369528077	-1.0915202300616
0.932320023094597	0.194734597696991
0.515275567321374	-0.817232972292285
0.932164502774502	-0.855914409237619
1.15197664386272	-0.672965532306237
0.684258661191511	-1.18713687979331
0.837979871818996	-0.629410897531519
0.841172729689516	-0.199217073975602
1.0245855198761	-0.68608074717206
1.08684953699274	-0.402857092499033
0.944722804295072	-0.392716180154382
0.640029308649384	-1.22062099968603
0.938719263665086	-0.473916100707807
0.733676628080596	-0.944549652195079
0.722466189205755	-0.331515028369758
1.58871066335569	-0.507443742323993
1.52558076661949	-0.405470948544854
1.33717077324674	-0.728014157154225
0.856316458090542	-0.242233147535969
1.27853102799083	-0.329272330666646
0.902168459586305	-1.00550956041792
1.3505873535175	-0.121159872155972
1.36895806462439	-0.348212657483287
-0.0609902571242093	-1.15872848264774
0.560562110241725	-1.06854693995762
0.850392453033626	-0.478816848418132
0.964631597510934	-0.596545837518894
1.00642993376232	-0.0637098702085224
1.37713195612993	-0.378516960113497
0.460525847762226	-1.06316451083541
0.828437885840335	-0.539895574034129
0.613796533982928	-1.11953221209777
1.34367154825773	-0.267581455599514
0.78427254577497	-0.798524611350126
0.0488510849490112	-0.328638529901976
0.998806202595616	-0.783740135360991
0.949664119951364	0.528150419510443
0.735667310675056	-0.213108153727739
1.15435614580465	-0.836631149871367
1.06917183310262	-0.957644051741619
0.548639975947403	-0.907120776291561
0.919115059268877	-0.933603296513501
0.732439535611531	-0.533208221138641
1.28932344567434	-0.405887967582762
0.8597220485692	-0.873864729462131
-0.0190057166364525	-0.401502678778469
1.12819797418661	-0.314397132272841
-0.244745779779869	-0.88677373039385
0.853052961624634	-0.306951705915994
0.309265402343664	-0.203473870574119
0.626770564819321	-0.782553986250134
1.16741282190226	-0.202144584918118
1.48445140111225	-0.607143787875127
0.916796833561328	-1.08820057408278
0.861631513782947	-0.836676093391091
1.30456469498713	-0.617440776910726
1.23900539722427	0.347887754297095
1.30142140707327	-0.110290974237096
1.37370082929912	-0.689672326573762
0.34084285380303	-1.23132421893918
1.09880516177717	-0.507140239723222
0.682134951989838	-1.42169574901768
0.623907161767446	-1.38361411332897
0.973129201530075	-1.10911784981971
0.220259712730675	-0.405290388356829
1.37332456654891	-0.167621173888972
0.98371597436213	-0.433628270615753
0.538451580678746	-1.03256864409703
0.524611777221362	-0.0299206039920237
0.682711179547713	-0.8229682904289
0.672927940689729	-0.429531771295783
0.956721144972259	-0.525024132101987
1.06033312276682	-0.630287525464629
1.00066477092893	-0.348932632153293
0.403531760382458	-0.655163355244455
0.945646968228958	-0.9303981505893
1.07193842792282	-0.720664229787057
'''

gaussian_sv_phi = '''
-1.16353742103107	1.06029305313782
-1.1635374210334	1.06029305313773
-1.16353742103337	1.06029305313774
-1.16353742103292	1.06029305313776
0.935181271599282	-0.37508438323873
-1.16353742103296	1.06029305313776
-0.298915886242005	-1.08287468660265
-1.16353742103121	1.06029305313782
-1.16353742103407	1.06029305313771
-1.16353742103091	1.06029305313783
'''

In [ ]:
#@title new1_50_emb_norm

gaussian_sv_x_norm = '''
-0.264641495446512	0.524721114076226
0.180972027072823	0.018567082970146
-1.19349648080359	0.188841462065231
-0.682276897294779	-0.698164821806768
-0.0368907472349341	1.09243400057548
-0.703317080179435	0.72749180922842
-0.667113110810742	1.50329039600314
-0.333367555569747	-0.464064253002727
-0.366027111556359	2.03644319052823
-0.599598034522538	1.12229433861783
-0.631220153086097	-0.00810655188652585
0.307431179036814	-1.52711639602008
0.0926279032174027	-0.189027922418339
-0.44178871470245	-0.0558538875117619
-0.64411700524531	0.184085345005669
-0.0762041204193851	0.0988257759804012
-0.837881933681787	-0.911718328295104
-0.287659773011085	1.97172560785981
-0.435600625399097	-0.153930703104143
-0.396510297150639	0.311886249216808
0.674496586345405	-1.49092849329789
-0.0544266755041643	0.112944021498735
-0.559639571164715	-0.0739485814872159
1.04320551358359	-1.43268759818118
-0.909424763952913	-0.271468305343479
-0.425999562345426	-0.348984040624315
0.357783640776502	-1.26007153982827
-0.00609952698692937	-1.60439723961852
-0.475994644802505	-0.63332510822834
-0.414023072886423	-0.707178565657295
0.598555372543853	-0.22474565767288
-0.281497051337486	-0.292723117337964
0.122421289823364	-0.0326107738215151
0.128145509769307	1.0217759834911
-0.247034903007714	-0.123337969431974
-0.163575629051278	0.364005903524515
-0.121736604968132	-1.20856016904102
-0.762279920070867	-0.191921425915155
-0.801313605213431	0.967620686394825
-1.3557789782525	-0.0602020406125782
0.00843521005807941	-0.632608008366179
0.575023011668244	-1.21867301373524
-0.0771977558618878	-0.464009390916055
-0.969024159314769	0.193942448861456
-0.641284884413801	0.0217219467700913
0.663455641219704	-0.301819739138147
-0.489524180825942	-1.13804974920278
-0.967766574686459	-0.995123442493817
0.174871240123882	-1.14379670095921
-0.519694524840852	0.377791363529258
0.42320138796014	1.93580543662682
-0.469760677382288	-0.0540656829276881
-1.08427592965119	0.288451506859954
-0.0565250882585064	-0.167605870591275
0.0636690480293846	-0.635337429433563
-0.775833291058369	-0.319321009548282
0.103556850964165	-0.970273588702084
-0.0735437072503636	-0.273605531702218
-0.186132270240179	-0.0273791467632617
-0.460786662771216	-0.738973600799067
-0.146117037145425	-1.24636730288907
-0.49434972003601	0.455608847787962
-0.786480420218768	-0.524107319884611
-1.00132743800248	0.458668665670277
0.818239185466196	-0.954632575182264
-0.0664872241302702	-0.0835574448453806
-0.477753035537901	-0.0724155717750158
0.069074286022213	-0.362665844777994
-0.547132981459533	0.646068090118203
-0.00158611732341022	0.599073998857656
-1.23697598309918	0.805997337755445
0.883176452798711	-0.667364455170451
-0.281625669931496	-0.763801972972159
-0.311801447533705	-0.572066817327491
-0.0188986600405214	-0.388106430818842
-0.940433533433813	1.47649373601932
-0.168117054558793	0.47732245610073
0.943751791754054	-0.341868344034714
-0.799958096245814	0.350001891188892
-0.15664978372896	-0.642727251665399
-0.729568478291259	-0.140313379716215
-0.0948718020446128	0.0939764833541103
0.471244971067678	-0.667202706076887
-0.900671685451965	-0.961183895568824
0.0360380903715231	0.354810544594811
-0.112630636228591	0.539885432850371
-0.204693627720783	-0.0453301576677237
0.443038809693047	-0.980384825137879
-0.440236995986946	1.55287487975437
-0.330449330413075	-0.128396383175727
0.332812255682181	-0.147144019846579
-1.39887729237896	-0.082772261819849
0.0416269341806878	0.574942999660865
-0.324631023614271	-0.526591919993547
-1.70019014385586	0.0977451713613594
-0.324631023614271	-0.526591919993547
-0.277732263251746	-0.479043970435029
-0.124703371919684	0.0841002031121991
-0.43976382861967	0.371094414224417
0.514889068194432	-1.15857338353915
-0.140534841168401	-0.179799342003884
-0.57002876339511	-1.37137287341518
-0.0269376715939832	-1.63461068945983
0.290634601178046	-0.199487150689833
0.258428363555691	-0.451801531800218
-1.35494125142211	0.203236396920724
-1.669623814993	-0.374956643946032
-1.4561737911242	0.481152679977446
-0.168763914614654	1.80351980795506
0.326851117139289	1.11792621965266
0.118085046848046	0.509088186666656
-1.06116787525237	-0.0588469760809489
0.154758222263057	-1.8361006747536
-0.527988521034671	0.820013458317668
-0.533553599292062	-0.69673136111978
-0.442797000930439	0.242799573458347
-0.624902116910892	0.0854291068774858
0.634122953873324	-0.571645497834144
-0.670029140488824	-0.508071496721296
0.204900071428352	0.973136013727006
-0.62466845035759	-0.881797775341949
-0.347103872935258	0.357814351229498
-0.206701581105751	-0.492097266037598
0.37442993917208	-1.0174802930756
-0.397035342835753	0.418701498049341
0.433824474793061	-0.248062369826805
-0.225087729012269	-0.750667360340452
0.0160268073032062	0.106518155140163
-0.438030804443183	-0.401329910398268
-0.567556395083048	-0.402904171484182
-0.247731489216633	0.560122870890591
-0.264011271624207	-0.153820878876281
-0.679089478305066	-0.112392620328748
-0.00602392482546678	-0.24951069802422
-0.265508728569569	0.0798802069377811
-0.24019691560536	0.711693455532062
0.725714700423245	-1.42260529945913
0.447086254368129	0.834016145549813
-0.146637056050273	-0.220151255545997
-1.67013060354686	0.397108298344139
-1.33522093384798	0.272244266736732
-0.372749188306036	0.268841831921122
-0.632405813129721	0.0813070515205717
0.024849415450492	0.867537112696881
-0.894391653865415	0.0531671903342043
0.250366619159662	-0.411095337700259
0.91039236489878	-0.316182726727277
-0.45416065001066	-0.862450609490113
0.418289138282453	-2.01938197422771
0.0325138723363561	-0.641620857100889
-0.347057071269187	-0.668526727162009
-1.34883835414923	1.09268865950675
-0.875596770223424	0.487650901083298
0.0773442496216951	-0.768950103021362
-0.98562409389615	-0.617184613163548
-1.04046953503689	-0.218524121637818
-0.220180764365246	0.3632121320175
-0.236667309034292	0.401358855834587
-0.514859450450262	0.141396435873458
-0.511746660575935	-0.38934758082921
-0.326405010454745	-0.672088704181858
0.104292056990659	1.19760831648072
0.147093207575573	0.212182107752166
-0.034130652544453	-0.0446517370445653
-0.36150606707385	0.907954829219993
-1.48440194058127	-0.175090246371882
0.199228101468845	-0.721172197378672
0.0403190868378382	0.0361391517624892
-1.3038407938163	0.0855997437692847
-0.7628632557366	-0.197642638633535
-0.877356308583821	0.369174688251409
-0.464056144437644	-0.093672424237941
-0.877204625143071	0.73447857333003
0.0745542982923173	-0.107476383905872
-0.246372414740787	1.95833416888115
0.349977006588507	1.09766462414007
-0.478912863870341	-1.11468218755004
-0.524710794170657	0.35161892130304
0.0335754746304365	0.140366453750489
-0.794745215975267	-1.01540444598485
0.487871382744445	0.726536283807387
0.446444224952717	-1.03292770140232
0.952989663045898	-0.545424107610332
-0.171606319682341	0.705923953893317
-0.207507071155173	0.00475485961849385
0.143361335263791	-0.542222095516497
-0.814346000577288	0.211632219534683
-0.0466535624927047	1.35802973237542
-0.872048881937265	0.229306041812653
-0.411492542591729	-0.154432650045099
-0.111250282309042	0.398950247465846
0.148825271109264	1.43014643064814
0.792989464592367	-1.27327246133594
-1.17190120134896	-0.775530925966374
0.409313757658791	0.783016899182514
-0.0338317346015738	1.48918233343201
0.168455857589771	-0.951261564046852
-0.685811786451772	0.484610060590735
0.00608622136484756	-0.216749624572421
0.688009332286096	-0.587745434748857
0.658822836810513	-1.1379019066549
-1.84411300734233	1.59169962674979
-0.375303804955668	-0.226877165383272
-0.818947713927121	-0.29418210909937
-0.423601813409396	-0.994881750310518
0.0202936723358556	1.63229717194163
-0.702518978661214	0.420796451000738
-0.323995468688307	0.647192088216569
-0.544330945076565	0.549213021065994
0.92230881760242	1.2810614897291
0.760117245061263	-0.117839245959975
-0.904697080696023	0.164346802580357
-1.00108014802525	-0.181246743649508
-0.934281143751081	0.647357404701995
-0.759326249411287	0.0233741942968502
0.715237060973165	-1.60740578176398
-0.985156984097902	0.137546883168872
-0.136603836496774	-0.487888611183437
0.031223226938862	1.17399458488191
0.20135178006163	0.791126122875477
0.343634123185256	-0.249041056074339
0.273434865269246	0.565279625124691
0.276170717829667	-0.0539464973792396
-0.513194964501645	0.295620463316468
-1.25727975035855	0.472487182380661
-1.12553385997081	0.235315075187457
-0.907460016399348	-0.556240192934396
-0.0283787386334933	-0.769464450692109
-0.279591868846226	-0.967167905192819
-0.444722927899337	1.05325041464954
0.264689998708458	-1.03695997103698
-0.855296251349139	0.12406951995911
-0.319140407556874	-0.717365347618325
0.465541530029909	-1.44349351311537
-0.370158037260886	-0.184373859194532
-0.658366566389598	-0.275767279206127
-0.765597190690656	0.109368675315504
0.302150838352779	-0.689717415028654
-0.103558749797522	0.831423811553223
0.124545312968363	-0.780620607187665
0.29744427189179	-0.624845111335326
-0.911107343212228	0.570190029982364
0.403160291950366	-0.358850436012036
-0.32534723135684	1.17499444349864
-0.0165480480951582	-0.0839151950857215
-0.14925281938194	-0.503406441725328
0.124782072086067	0.950882200428353
0.262839215340185	0.0676973331614557
-0.883933834751497	0.524981525902277
-0.250222830743346	-0.30193608944889
'''
gaussian_sv_phi_norm = '''
1.11427573507813	-0.27456271983113
-0.0765688604979312	0.519290980298244
0.992291226906452	0.443408362907209
0.157772440401232	0.31266831912271
0.823312403321226	0.0403628702872646
-1.19114014661268	0.74683173554592
0.686963909832642	1.49859058166393
1.22815072361581	-1.13561571875322
-0.877346319152816	-1.17653258193427
-0.149698501603105	-0.46079763458997
'''

In [ ]:
#@title new2_50_emb_norm
gaussian_sv_x_norm = '''
0.0348975181153951	-0.0465436442898507
-1.14434274066611	0.785551038951523
-1.21001291919518	1.02665044918512
-1.18105743650892	0.441165799891909
0.143018268477016	0.668894639674115
1.90233092331189	-0.0599769269728126
0.000155472698109805	-0.523737856771747
0.467460033058902	0.201511195191007
-0.264057399840132	-0.483518302822082
0.44684313440777	-0.076296050058437
-0.258919274575721	1.1178013232046
-1.37873152112495	0.782757969048673
0.708340718154883	-1.50167829128733
-0.917286947413165	0.177537389547972
0.294720592516083	0.975448958810366
-0.787161209904204	0.86219837225954
1.02257665442109	0.400597426153287
-0.765879267514881	0.6530402756679
-0.127514621753027	0.7350738553626
1.33078392794012	0.40447778299988
-1.81073413663303	-0.723607776554458
-0.815151018046048	0.879591383636174
-0.195073833316435	0.622228963104978
1.38666771960151	-0.145484662942654
-0.415098251063263	0.334422895965763
1.03206893841236	1.02726785032742
-0.754898341841474	0.57390500370796
-0.832515625109812	0.3816655144489
-1.26399307860514	-0.333628253796036
1.22704376746082	0.530924534957398
0.381712773107134	-0.767196185068118
-1.38565617957179	1.3614200121525
0.83440213827764	-1.0647770315039
-0.542632518420401	0.0797283204655607
-0.268433859789316	0.850553313010639
0.378558097605704	1.50789096735151
0.110328470481001	0.932502688622981
0.363660176396647	-0.353346217384021
-0.355648134117142	0.101921960632204
-0.270787563237403	-0.413438170380853
-0.247540063833862	-0.115907248531895
-0.515362632518244	1.26730255045048
0.2470856569034	0.457757051545583
-0.195936815110803	0.200423907564951
1.07185928809423	0.757992770736528
-1.00892837416529	0.718441298854671
0.62907060528856	0.332424948847607
0.833937583287168	0.529311510491026
-1.58815561802574	-0.174343396647531
-0.847300933973175	0.248604116673428
0.664748609363475	0.574329089791628
-1.2730511092451	-0.0565264635849542
0.0104777677347759	1.11698770530155
-0.845758742235307	-0.732463343005791
0.0128836559025718	0.623946224709479
1.22105314461545	0.454659880527783
-0.127814966523316	-0.455766931342351
-1.27703265899856	0.573341691635486
-1.37130815158618	0.00989144904559224
0.225589223479261	0.193283110511428
1.26305853135473	0.782013453878458
0.000749506893795718	0.887318155103353
-1.0775309407405	-0.096408820303683
-0.107419276892189	1.32131454677489
0.266537522484879	0.806545656704941
0.859575274635433	0.526769035348909
1.74075736740339	0.905921922336568
-0.556694316504761	0.0202748516182002
-1.6008344120207	-0.299337364411059
-0.199522883859286	1.15219126533028
0.733691194708117	-1.21191215683352
1.3050138936499	0.261361047291463
-0.713333854544452	0.782506578952836
0.00848338898781193	0.168910709893723
0.532572773231721	0.566815737833414
1.16529620301149	0.56361748608005
-1.29914133901518	0.486158309653378
1.10151134745796	0.632755840303654
-0.323627293619829	0.89180415824659
1.91609655422672	0.235180990379495
-0.575693355972595	1.36465693056455
0.0302225187404182	1.32371328760437
-0.827432692016092	0.757514496000684
0.388501490571641	1.03565680489683
0.362660654527388	-0.859135524517209
-0.789758476443399	-0.364764163918246
-0.152421141992755	0.739861098337693
-1.42447214756567	0.597465325152388
-0.238739117117511	2.04795741896895
0.750605851140723	-0.522167616608813
0.669760302602528	0.355571795397516
-0.187831448161756	-0.928830793714931
0.151870137377099	-0.378685821269213
0.494152233328203	-0.164999218518799
0.823342526045999	0.522940820936244
0.233778262702766	0.674537025172566
-0.474129281094849	0.651930719838435
-1.03967710657374	0.0590206795274312
-0.525222299137726	1.04677590203956
0.386086377289503	-0.359335474618295
0.243907877663047	1.12391215445876
-1.95119636170617	0.560711453464409
1.64151754534906	-0.241675648984096
-0.00895817585452608	-0.475370700033093
-0.197963603438888	-0.719187876315416
-0.713859482873304	1.15424730290124
0.633899312788748	0.263477327729761
0.671421264070432	0.289128843036036
0.127517266174757	-0.594812156327646
-1.19981586959109	0.589394719836231
-0.0544551124443378	-0.419181048604509
-1.05402864698284	0.129156304711624
-0.0711511225648025	0.802770483507072
1.35962789305199	0.701863301546644
-0.82762037743059	0.00802059341577269
-0.270917671269917	-0.95037079291338
0.759001372601315	0.560666443218116
-0.161835129887933	0.732703752228045
-0.336665830929397	1.39882113795057
-0.715642287180859	0.565826047123226
0.312086089285803	-0.991332445360568
-1.2806831488325	0.25597465835474
1.48122797623001	0.343492100365933
0.389520497214186	0.618166959881386
-0.850025456284937	0.193048040343873
-1.08276822227376	-0.0132056756278475
-1.37353865403638	0.155194348753352
0.118033074554068	1.7208771679261
-0.440932723483194	0.53788273968553
0.12935663652413	-1.22802373010737
0.0767756892534773	-0.357662231378264
0.142508900798039	0.378529974677931
0.229929687398383	1.33320371622516
-1.08752858874784	0.713249525949916
-2.03201237698695	0.554749674946623
1.31340912318568	0.112629446324968
1.0816191475599	0.0987903052842055
0.147800284329794	0.440208943811783
-0.941668820410141	0.19766791921302
0.631781636077328	0.356325388095299
-0.741313448950745	0.315868193733297
0.341453811932314	0.382734723190301
0.60014258182233	-0.363053266948358
1.0100026808805	0.683503047066961
-1.09830595627591	0.131349997780449
1.72232728414157	0.390844810685232
-0.39988976371324	-0.31281727016409
-0.99606940204124	0.569290737546452
-1.39090502679857	0.368209055429056
-0.732356514339508	0.347290366655326
0.194706014983826	0.549898788480536
-0.434562716144962	0.471357116136426
0.152359583543897	1.51999772342207
0.27572856934238	0.610353895293209
1.1568396623725	0.698022467417955
-0.657265695105086	1.68743705013968
-0.408548309168447	1.16844889724489
-0.0468871511561448	-0.0175793439056672
-0.191211783663827	0.730967113927855
-0.738824068681178	0.755820189744728
1.06509976756433	0.138898958964231
1.18771331765636	-0.125233741908842
-0.941801839537464	-0.0326164457777768
0.0304502298626908	1.25123997818863
-1.33372362570578	-0.739939557231014
-0.179361477378641	1.1608176982295
-0.777937648611341	-0.105795849353567
-0.637553225601996	0.525447414797567
-1.23341502559703	0.377146289559431
0.633709201567202	-0.079768215166614
1.54362963394615	0.345923312700711
-1.46653122850421	0.0556762098777671
0.101623976314095	0.0846829725823914
1.68831276075291	1.11601560785958
-0.321156642769739	0.537387698303345
-0.165559241869215	1.28173751764011
0.514019604500479	-0.0939763448775804
-0.0573325359649764	-0.273211349046709
-0.270315575273993	1.33895940101156
-0.0964096058096123	-0.694608779478151
-0.139451330714692	0.575283887740531
1.14629130029482	0.893403017270251
1.45021325367165	0.699194757880311
-1.73791196201067	-0.405199847226897
1.78213983866508	0.876429225896926
0.0295918261069212	1.1432977490109
0.910684927267347	-0.395741799582311
-0.57342819191374	0.215885289587198
-1.28548223988336	-0.0147605493340357
0.590005640729827	-0.031291696382008
0.55078171614517	-0.277425173419402
0.30727412265809	1.32299368023391
-0.344067373645412	-0.201657469386251
-1.28385912850872	0.682947684536989
-0.0153721549189056	-0.0345016695645784
0.845992253869773	0.136858584468541
-0.935969462516482	0.452246989833143
-0.872384273718109	0.407139592048947
-1.16487636424693	0.110979197939524
-0.0372095335219883	0.531416395828753
0.0661346161253906	-0.44728537615037
-0.831002322177935	0.544154811359887
0.405542074815172	1.08805106776966
-0.48740906286249	1.47558739508243
0.74164366704862	0.135990873289851
-0.211161327761492	0.0743937603419718
1.49577219594137	0.717226864820543
-0.055074347340311	0.26569729726372
0.0985418877489409	0.614734665029637
-1.21319649900399	0.215431287563018
0.241056755252483	0.743351533130016
-1.63076911900157	-0.983783036688815
-1.23212584374618	0.181984897133822
-1.03375228074348	0.246684774959033
-0.95173561216736	0.708101952725603
1.43493965723218	0.518409550866119
-1.29426319525072	-0.294620696520843
-0.564471547700556	0.229361450385402
-1.14884367046637	1.10700156804687
0.344655181114086	-0.663016055431309
0.269202568513757	0.900532676933513
-0.744057678463058	0.609868251418082
0.0190490732704251	0.949758478827112
0.385118920694255	1.65649031964531
-0.980672002996026	0.675817770878463
0.940802898781693	0.440749333357625
-1.31358535250343	0.448282725502509
0.293478618726683	0.747404214912185
1.63038381165351	0.669079422188741
1.35059911418647	1.06932835735064
1.01273126915471	0.596932352787099
0.775561622645625	0.0706399219643543
-1.34793975834373	-0.139082780573419
-0.990846261213725	0.092214884777898
-1.57834719242413	-0.0816770530639702
-1.2745519612723	0.546835700097823
-0.0518300322554833	0.311193045506641
-1.15220250072243	0.506780016842821
-0.490299530547501	1.6194176568905
-0.60707730158242	0.612604360169856
-0.034682926916897	-1.11775884687819
0.296468660617438	0.602392104580995
-1.27761237819094	0.8221045385729
-0.0180149133689619	1.10589354786671
0.0982914820127656	0.889647805534089
-0.75432399118595	0.341612297626382
1.45211868011075	0.58646581758807
-0.741417620041462	-0.401700834197037
1.11804594964899	0.456375631132525
-0.969031227553451	0.429247619916422 '''

gaussian_sv_phi_norm = '''
-0.271636291901243	-0.641710125695707
-1.64333932098175	-0.00327908612064664
0.540194835541368	0.0715406053816314
0.324069551479562	-1.39179898618986
0.194343132113608	0.646390535387121
0.398452469986531	-0.737888028738694
0.0680309093221357	-1.46825975093139
0.0700823349413125	-1.42832313676943
-0.208909752104063	1.64055961326829
1.55830086256899	-0.378015986301273
'''

In [ ]:
#@title fulldf2
gaussian_sv_x_fulldf2 = '''
-0.885405978769376	-1.79097900596004
-0.0870604544104978	-0.46261480286675
0.99993248069677	-1.28224487947566
0.237058570638301	-0.965381034898871
0.300797227187036	-1.03209971037415
-0.403970106011228	-0.693647822779191
-0.4264785084636	-0.821730750067563
-0.0571550822642129	-0.0865602952055882
-0.474717342175383	-0.81005438124593
1.13233963850759	-0.574723374283209
-0.195007276279046	-1.4881213814313
0.567930444247314	-0.615635036583292
0.132777438720776	-0.554055071902678
1.0132060460987	-0.0738907090183346
0.353546384372446	-0.444021960573204
0.493659015120705	-0.558649305615994
0.631333785400009	-0.841537180350466
-0.267988148465973	-0.608352144715145
0.321350808590027	-1.16117385783516
0.413069639008212	-1.03322802145619
0.395388905735374	-1.10892792178056
-0.12499425567965	-0.614301761976984
0.0407302910416413	-0.304217563085639
-0.629926608821585	-1.35493026673048
0.877995367522111	-0.546560959370127
0.711594256008931	-0.44171930060497
0.213170031011778	-0.189518291547591
0.430306461943569	-1.17342750421466
0.352783055861785	-0.571466539468533
0.475144587022202	-0.984164139162929
-0.393865936242946	-1.42870490831305
0.382776026573959	-0.53759943882454
0.458646651441038	-0.451267926832597
0.326991960103843	-1.07905519379813
0.213513716924857	-1.39843323877037
0.00651610312904863	-1.23645592950492
0.417874553962962	-1.14214878075013
-0.158266257126697	-1.50904792265163
0.366443101748092	-0.600869712102243
-0.304150402289273	-1.06279665865184
0.541737272380296	-0.0145210090368023
-0.211983481021179	-0.755392456474196
0.366899648007754	-0.0368664535628066
-0.126083949212239	-0.347408061163848
-0.353439358677556	-1.15433804419111
0.152889055858253	-0.662471134361961
0.872851654018099	-0.765896188299676
0.43445691511765	-0.94051166011834
0.0291689411560011	-0.776445534793448
0.0749515367242851	-0.995811769465954
0.771968185519149	-1.21866533618307
0.159301525247626	-0.0983799292424132
0.111356608399985	-0.689619364865244
0.614937977699962	-0.466774130434837
0.627436280331472	-0.286779077510962
-0.28024557470452	-1.04058562165129
0.0323554387711851	-0.37562984935775
0.132358088073034	-1.19133514878375
0.858609551045184	-0.342254925113734
1.00184421567602	-0.402775902314364
-0.393360875541665	-0.304930608480089
0.0376779320000153	-0.911131561201015
-0.10491811551418	-1.22484534451476
1.00188731544517	-0.177820880495669
0.781939392667339	-0.423664639995884
-0.18904746660699	-0.767231253173424
1.14662783268493	-0.777154080777572
0.659654465727061	-0.612694665454367
0.505151842241967	-0.773268430126945
0.993858043922282	-1.32632735163171
0.471928403644325	-0.509198223130428
0.390917165618613	-0.595251034638261
-0.514464696994305	-0.72861415495085
0.247187455064368	-1.25212817813705
0.75804548187023	-0.0151604339916622
0.118924958656596	-0.430302462414097
0.45420319790141	-0.673058365265162
0.470420481774031	-0.237640818859701
0.521901012854453	-0.889545196635512
0.0462549607015812	-0.282579181170875
0.512638870632054	-0.49490280544966
-0.117350618792221	-1.44734068756431
1.01625761564627	-0.859801825481355
-0.294878014551305	-0.652169078285968
-0.170401672615046	-0.453194881037724
-0.537605899730895	-1.14020648252193
0.533480847447336	-0.348778124794451
0.722618583990272	-1.34460975172024
0.148197986607973	-0.165760484637362
0.373698875781385	-0.803026132037024
0.47057223905939	-1.00477922368606
0.363475121586684	-1.19789612124787
0.35355068053779	-0.827509180852216
0.208424015590071	-0.685145074255293
-0.148034543455873	-1.36407760383904
0.639321770928393	-0.408651064032712
0.473120069085883	-0.371412146248737
0.0352629353993332	-0.96569462875321
0.118296139850372	-0.873531716193486
0.72733143932622	0.0759847058148305
-0.452450385480756	-0.775399583183104
1.19197633825321	-0.935644081217929
0.431398211462108	-0.796722535006103
1.3199578732945	-0.68136237371674
0.428092761299138	-0.402951832282619
0.499235108419171	0.052059099985827
0.423879794577563	-0.633267287394151
-0.295193959038399	-1.11695711949102
0.115925991749787	-0.787099961977814
-0.572699793746516	-0.861051397739046
0.00123239452963189	-1.49710179473972
0.0678531781436847	-1.2553194530502
0.496791534094426	-0.802224419372158
1.2962337826912	-0.236065911116626
-0.433359998588394	-1.44908762828616
0.374382240179031	-1.17547896341864
0.0892908852596138	-0.955937639870784
-0.0465329278552503	-1.23759585450846
-0.133308008191886	-0.42173556916939
-0.406527811801856	-1.39868812935643
-0.657459418453303	-0.837958452802612
0.702078779839892	-0.962932810903156
0.350115027919956	-0.161811072453612
0.0615360405663724	-1.06660876979214
0.603363105948854	-1.04113714178205
-0.328114880014399	-0.952224477161348
0.876400869505722	-0.478158701052543
0.0684325185089884	-1.00722795380634
0.110978473667952	-0.815024898400059
0.577429534174831	-0.729233742518503
0.0720549134801815	-1.20412344253609
-0.178206684986308	-1.04977063647681
0.780323729930031	-0.837011264081545
-0.0600334134297374	-0.444312266563087
0.54187645523742	-0.310038704660703
0.330206843709329	-0.943038938740594
0.0545474653817624	-0.454611731923957
-0.32426705487076	-0.857267704142024
-0.0808524509342286	-0.176048037073984
-0.0933555527392541	-1.67765169747553
0.72384832667578	-0.599290776972802
0.228972450768779	-0.515562770352091
-0.0111836968339378	-1.04000606763831
-0.257610867802998	-1.05417521945395
0.553809648100144	-0.667854212692324
0.517432148173449	-0.23431333704908
0.532381752409148	-0.670744077552363
0.386966247460397	-0.661973863954503
0.549162106008807	-0.808538589075284
0.481982994129314	-1.0876858608652
-0.0668730786441171	-0.502635047243572
0.027340682992022	-0.733779937647283
0.233810420106704	-1.019067903671
0.508902157590437	-0.474113933116381
0.833414944022173	-1.59229970420204
0.23392885950896	-0.727358295109901
0.927961936228417	-0.0375846158197124
0.458923189480458	-0.667556447420317
0.736431706582675	-0.39933463565602
1.11287118748188	-1.12753810787373
0.678936347217296	0.0549629026206725
0.245640386349267	-1.14766475375056
-0.103045579138055	-1.23078491674165
0.601221259869769	-1.12264039535469
0.203800253399992	-1.08887779702911
0.0204915437465382	-0.881501836666599
0.29509230238121	-1.18387841700166
-0.137501847854263	-1.33592586359364
0.426414859719211	-1.15988101816862
0.377606689553311	-0.508691180465316
0.037949586646544	-1.13101785788915
0.407856549463572	-0.426615706083387
0.158018944391439	-1.29570629516834
-0.0137689230944194	-1.0140925402944
0.361312996779705	-0.837991620281351
0.760519056272128	-0.767882283746423
0.345345604095575	-0.828896042533473
0.154660505035057	-0.220803153643631
-0.0692822578094696	0.142022899320811
0.658046242281731	-0.500221889334438
0.379346688536121	-0.378002807092022
0.604601692935019	-0.539807674140134
0.340442907613359	-0.570631784538704
0.207740478500114	-0.917744259773436
1.06470270839243	-0.696839825050947
0.135596130638343	-0.713100315867062
1.05978660570702	-0.323257908921092
0.0875366818340885	-0.571526132469206
0.317335843982929	-0.387764706501497
-0.244094255607758	-1.02250389963398
0.800878620911779	-0.849380008811226
1.22295989203901	-0.517126283437521
-0.202409052108683	0.183285889633851
-0.039020301116948	-0.857491340130754
0.279667927294828	-0.378484306972615
0.729135750212399	-0.246697980611831
1.075441338519	-0.511940733311292
0.480722748716198	-0.919764747803404
1.33044045788706	-0.985783528652462
1.08226303336358	-0.760666664271432
0.559690141266864	-1.17937927930614
1.06254637979824	-1.3264834368934
0.238235451488942	-0.87293009358932
0.664707157045263	-0.422164744865983
0.115944271706486	-1.03884969166317
0.293516297442144	-0.818253341422469
0.406296308288153	-1.12109115275972
0.263852614347621	-1.54614197564877
0.284348462159249	-0.390129071606152
0.282166757586801	-0.709869333650087
0.444371239812436	-1.69137302527048
-0.203463268081404	-0.501492414417204
0.272920124087339	0.368046190516523
-0.318344076774646	-0.790688604603712
0.670323659236122	-0.814352040999113
0.486532108348339	0.601215160629508
0.54217907783483	-0.716320379556666
0.599337824585125	-0.593797390974101
-0.349578785126501	-0.296437301405238
0.479144964895103	-1.14711163959674
0.317755581303323	-1.23880312276088
0.779725988740121	-1.03250807442275
0.100840538015825	-1.14667719766464
0.415707648310277	0.260334719036636
0.0103165515734762	-0.572572006420124
-0.0241702261594185	-0.711085639451199
0.829407903747407	-1.10741316844041
0.0362759677638363	-0.107041776786729
0.259672632833217	-0.737937438958509
0.797488893524725	-0.773127032312145
0.89965820292321	-0.877213800670287
0.891033486926271	-0.53051487476439
0.360784758804998	-1.41155530643088
0.160099872764113	0.492233736866374
0.676097820877371	-0.365561071509633
1.20438693608085	-1.29850520652906
0.934331009602923	-0.76733716639112
0.14237824848658	-0.952745743818907
-0.14374480535169	-0.709200153196796
-0.318344076774646	-0.790688604603712
0.802114092453433	-0.151977346387344
0.272223999930873	-0.443363388642654
0.100972489789643	-0.508981631301864
-0.0453945905182461	-0.329502019003666
0.970707116409376	-0.63291797883102
0.697524804336045	-0.392610478619018
-0.504009878012178	-0.931832333955219
0.820679777222714	-0.988886410235961
0.464267761785926	-0.308531561896873
-0.374639621895966	-1.35057260612769
0.41122810676869	-1.51681010712924
-0.0876562704849527	-1.33382878722689
-0.4264785084636	-0.821730750067564
-0.153952880710485	-1.03696306814576
0.838854744663436	-0.456288536522629
-0.0229089917702716	-0.703553515001207
0.177950476909995	-0.575559233195532
0.0580323128143101	-0.456253042815498
-0.516049346366968	-0.815795602217007
-0.194913003326825	-0.818234915954467
0.334405224291168	-0.496423031074674
0.202181810713619	-0.723910015265419
0.766627139095895	-0.750927252202672
0.369449206595634	-0.955795052238251
1.41309832482277	-1.38426760342559
0.217403637163221	-0.996952351929163
0.877572086201035	-1.69738335090365
0.848950906027844	-0.834434573585985
0.168747818526211	-0.94086932532758
-0.868569981308873	-1.28685530411217
0.343975353629028	-0.872919500822653
0.172121030257276	-0.978425911680316
0.364387631247053	-0.650362689755741
-0.00517242744162151	-1.30000817350026
-0.39760482162141	-0.358281706591875
-0.346047556540282	-0.909532552111384
0.214254571097173	-0.277311320060347
0.370750636220865	-1.48455640524111
0.302651216378344	-0.219471904198408
-0.0300793783649321	-1.02115734991414
0.72395822041722	-1.02370252847268
0.558240161712101	-0.498149626745642
-0.303013692281191	-0.959193994520821
0.586883701140501	-0.277185818103219
0.00798958368510463	-0.359925922815798
0.0386334997883719	-0.514234809797049
-0.498536207702796	-0.715388193629802
0.455116066528952	-1.00577889468754
0.25049917987169	-0.620183512652996
0.499515462559097	-0.30886264050023
1.49556363269639	-0.805099134186413
0.141697955586164	-0.460070968051213
0.0160592394028608	-0.180024323530917
0.0286052276426029	-0.497981105767369
0.168049903091619	-0.95074012401493
0.714994288620405	-0.361781194380941
0.148523969575747	-0.66931355581569
0.398724211864289	-0.942470373133067
0.463426234605357	-0.626404377791272
0.800330466046212	-1.19478019970726
0.777497446377677	-0.985205207022227
0.345708064282404	-0.598788544605863
0.00167442680350362	0.0840067359955773
0.514946966057651	-0.514434747587608
0.838949538185247	-0.476209540198822
0.0618900194155643	-1.09120544550325
0.650938522920531	-1.03449620930703
0.0785000089387181	-0.0226719295279359
0.285613542946569	-0.598708755828849
-0.0281315608258619	-1.19594431905656
0.0140586216626141	-0.618316081324148
0.652854216682181	-0.507250846071902
0.151952429787306	-0.821031552143247
0.4656859392266	-1.59023674859257
0.10753892068487	-0.839558874070242
0.181591929622773	-1.0876857202431
-0.0520190054877922	-0.825469489016096
0.567750420600831	-0.706344131297348
0.472641081792788	-1.08003462743317
0.0198513419945307	-0.0944245920883543
1.05082030833948	-1.34661713244753
0.020512022240886	-0.602076369204725
0.416624992231048	-1.08217979905822
0.985903876802566	-0.965831279577047
0.640920083727494	-1.44317661260649
0.67823948772542	-0.0312559163274165
0.466942838739242	-0.931660686859576
0.45163925928319	-0.166733566075161
0.163364394070497	-1.06650065430809
-0.3199769998673	-0.379756858945595
0.931635779114441	-0.276007729945722
0.718675217619525	-1.09177523478849
1.41309832482295	-1.38426760342564
0.505556665684106	-0.622834664575889
0.454189210621413	-1.08121788678513
0.733408898430424	-0.425278873675163
0.964942173353057	-0.847416591877678
0.473700104959378	-0.253802257770682
0.508438380346731	-0.962856198567493
0.433484967227856	-0.0580782413279381
0.379679348114217	-1.33319247094922
-0.0591337683131179	-0.0582800155638553
0.667694157105588	-0.662493656723829
0.137541065049921	-0.742582487855321
0.834965765752814	-0.555705271316933
0.547266824848069	-0.616465014575866
-0.145601897301319	-0.613679170799907
0.8894084066117	-0.621803367147616
0.508645182671616	-1.37114450748759
0.00317259796828805	-0.508628575196787
-0.603518924630744	-1.05555102599651
0.453261666452475	-0.983269203703875
0.656463580434542	-0.915596573873204
0.0098093914089264	-0.677706616285867
-0.464616675315156	-0.720504626880318
0.0580323128143101	-0.456253042815497
0.790612619040038	-0.215906196692345
0.757620710768099	-0.338479445008802
0.445403309861154	-0.536165149509584
0.107664995092057	-0.734670663440257
0.124894003814079	-1.03475882833297
0.430029122820166	-0.930860422178437
0.691830891748831	-0.608852429454293
0.766628662468878	-0.94116267509801
0.877572086200452	-1.69738335090359
0.557830851574482	-1.02636267937595
0.255774662779739	-0.505171931284569
1.34202754377711	-0.235720499480168
0.430725955470175	-1.23008409616025
0.251061441840509	-1.66646022182765
-0.0229089917702733	-0.703553515001205
1.00853522196113	-0.23343850065736
0.366380617147228	-1.56321645831105
-0.0625116644188907	-0.649760315638766
0.358296698119161	-0.809171613701293
-0.0652884622842601	-0.431898962787164
0.148082517558627	-0.206091484439005
0.661232049245649	-0.0935795212265566
0.727687880533662	-0.904827298277831
-0.252345717858487	-0.643783632911604
-0.257601744587784	-0.698340711501011
1.07839484249827	-0.192800859865954
-0.186826658572441	-0.655195002662714
-0.447248676819881	-0.990338188047905
0.0162922114395786	0.21977402789278
0.524908128496055	-1.37670389589234
-0.575376654592916	-0.536842966742232
0.330655171750175	-0.395005603815459
0.844199368036829	-0.70097310435393
0.581236397562785	-0.907326315999209
0.556412375982241	-0.551123774571648
0.183144522318981	-0.779697053651824
0.446286633890787	0.260954163559115
0.93533857370346	-0.487106255939806
0.754726425893042	-0.603310983616891
0.274304463626065	-1.08452569495851
0.0898613058171069	-1.16785906795217
0.786243900200348	-0.744237625048738
0.523422153195286	-0.888764065675343
0.630945344244752	-0.458182720911685
1.72181913436471	-0.456987952731823
-0.263139925139195	-0.568919198126164
0.521507649731704	-0.131440836205715
0.515932094381811	-1.52147480760773
0.160801273924897	-0.784968085541813
-0.203932938372297	-1.30473826083047
1.01811832340462	-0.148783218064498
0.341942773209168	-0.659575198675684
0.510818910558598	-0.98110128348605
-0.189241696909496	-0.963710631595717
0.0626547028898862	-1.1454441452686
0.921738808016316	-0.401595655903717
0.2129542742491	0.0335141336160257
0.747606844408569	-0.457247863717642
0.415594391432175	-0.248874335434213
0.141697955586164	-0.460070968051213
0.181538648168108	-0.958625248461529
0.15496452926964	-0.884623537408086
0.067868439583812	-0.623292062302489
0.488277133725702	-0.404386253490927
0.14002948118147	-0.883786258953818
1.29146379312931	-1.48070466881428
1.1600389146266	0.168092332521491
0.69979104493856	-0.209807473278593
0.0457533733577249	-1.7595624161733
0.791264145768285	-0.403531937674056
-0.398947871636692	-1.14624981735458
-0.420070397285249	-1.18797555840111
-0.227386654220767	-1.0052040104313
0.256209168454476	-0.78685772666537
0.0391443804993377	-0.166980883960903
0.0930053157896163	-0.225101065111296
0.481064564151146	-0.620183483162717
0.673892348280864	-0.264706670820807
0.650243666846325	-1.10456200829506
-0.0779246356475787	-0.634477313486573
0.723276327420453	-0.645995410695237
0.317764193097644	-0.225449210111313
-0.25929750607855	-0.800967026372101
-0.375034047166562	-1.15200413881715
-0.287352914886526	-0.946140810724002
1.38246719870499	-0.397940198567045
0.788524982442107	-1.00706167367907
1.22441679642332	-1.09582532813382
0.279807773109337	0.0829971170477361
0.428869225978206	-0.323487709757827
-0.153952880710483	-1.03696306814576
0.620690939500223	-1.1865550171967
0.0781449773122524	-1.18383888646953
-0.384996558411776	-1.27255907165214
1.12572758207563	-1.04179806901621
0.191437106959354	-0.195918409253299
-0.528684199997901	-1.44502410428812
0.359714124178836	-1.35031440942327
0.90435497170088	-0.524293316206019
0.572451698838766	-1.02722605639811
0.729616267671533	0.00554454288920636
0.651577684543546	-0.348819897351456
-0.294476632635532	-0.699470955568392
0.0257894435896438	-0.949600624256684
0.560192071781162	-0.211961357768537
0.393261249292415	-0.529608190946471
0.181692515965111	-0.396915764111099
0.0155657370694888	-1.28368658065273
0.467095303506065	-0.275709852752773
0.179804490959497	-1.04557188020316
-0.273044247999181	-0.702251096274875
0.977820204612181	-0.180934498672185
1.11799933048937	-0.446628171882285
0.566324468887697	-0.89207619575596
0.655430365577379	-0.633378102800932
0.34377721608432	-0.363591910371749
0.726890483015391	-0.547026940315711
-0.113049513994849	-1.04253657139169
0.848309826803308	-1.07278049879992
-0.0902633257713885	-1.13313372191777
0.139210877054699	0.00851267759309961
0.0583101545708675	-0.678565528599528
-0.208754805447518	-0.96009735154257
-0.101995424548255	-1.04630487249631
-0.191052674251171	-0.868448582792166
0.548581680954913	-0.675031028775834
0.817309539220912	-0.93561076270698
-0.372691386391384	-0.464117185570049
0.413197960790698	-1.16431195425667
0.53810501044971	-0.657523507030235
0.375539189937507	-0.0844125433306974
-0.180935316954368	-0.348698119585441
0.798725521605056	-0.529812413491798
-0.53884725017565	-0.550566826941218
0.365028948762922	-0.0240931476643677
1.13439305026371	-0.427043760630046
0.0497093773732074	-0.527126962085104
0.670323659236122	-0.814352040999113
0.99399915464754	-0.567702851539272
0.361472201986158	-0.169384503624988
0.919025706331676	-0.847044250122447
0.371614228979089	-0.41432183927169
1.02219856504528	-0.709396140947485
0.266436670425113	-1.12210852718091
0.377475491407883	-0.7407140949124
0.726145957711802	-0.647054004582873
0.0529507319256456	-1.03099942474975
0.375316478916912	-0.341498736261035
0.0616254854253781	-0.210615218169998
0.333184670466528	0.0884138883500105
-0.265568526126696	-0.443491165656491
0.706676686006065	-1.26604049394721
0.832062225244271	-0.894482915373776
0.371131595121772	-1.26418342268352
0.428421562132507	-0.160115589954801
-0.213145918203569	0.817538846924622
-0.418233275488577	1.08986354177394
-0.39533757275129	0.266750590311801
-0.394236130121818	0.618272294683683
-0.324651578997437	0.79348167706194
-0.873123786755136	0.739114320909604
-0.110054825053498	0.351027337520557
0.288086154125343	0.0935920468388897
-0.274978898580408	-0.265383530824014
-0.119044381560619	-0.564762294104541
-0.0100864386505504	0.891450189884604
-0.658213207648145	0.846553782324592
-0.562766473316714	1.32338420601112
-0.64746914779013	1.51084245850582
0.461931736392743	0.0560684488990982
-0.274646723187751	0.243085521092792
-0.133468792528447	-0.682704517775473
-0.658603380461745	1.0363858129127
0.541978489779932	-0.480130995040009
0.358537828630809	0.0643137351701274
-0.178999297733617	-0.206282543657281
0.0158876240083784	1.34157968139973
-0.780135298987244	1.32803964720948
-0.433969136023488	0.475371487248359
0.281645323103893	-0.220673297071544
0.0254110725825616	0.843860340537472
-0.125312097030058	0.569609078195764
0.10610694560521	0.868332125126916
0.357825071991727	0.596284913822358
0.234177121816099	-0.354317712378424
-0.593745071047336	-0.24810332680034
-0.230031099284224	0.450438242488057
-0.536693702014355	0.836152330189197
-0.381189254507482	0.722469750578655
-0.469611608289226	0.774577772344454
-0.068089767073947	0.819435336385801
-0.167368054995895	0.384677056800914
0.110492922721309	0.763923309978282
-0.0869151947059796	1.17380932948834
0.252527076433677	-0.046956876237256
-0.0314387943475213	1.05982045098583
-0.399331200019467	1.00681694819884
0.201269549771819	0.796671121958912
-0.625650355073357	1.20886997573361
-0.167713228470748	-0.666262721123097
-0.33160981549042	0.746238783764068
0.0286697925855586	0.443617481370154
-0.122765524670393	0.497956301592938
-0.45050629254971	0.594819874334189
-0.467350877593675	0.79956704403347
-0.17649920117768	0.00793004112304552
0.122148210861429	0.579633811601718
0.602155893539848	0.376539279786337
0.0722160665512256	-0.753448237696735
-0.391242948507348	0.548710767057429
0.179022661987545	0.770470332350989
-0.57941282657788	0.755808906062125
-0.41575665091898	-0.244584951762468
-0.111534651874802	0.283590611097461
0.28453386740219	0.985904473920098
-0.0146761174606413	0.501131219665154
-0.318615566409466	0.763208207097473
-0.472893516607956	0.186427292297334
-0.301481417573714	-0.0505741583252021
-0.543106164146354	-0.924511572961111
-0.352700153662733	0.718782376154002
-0.492180744082239	1.20480067299272
-0.386670155607582	0.896937677852683
-0.256989672863732	0.730478783015011
-0.634874780757893	0.999587893820218
0.134439399863899	0.148407547767062
-0.195513744887551	-0.256794551647345
-0.298868959988671	0.204238310905484
-0.20945149114067	1.03661321427688
-0.393846331738617	-0.109468488931737
0.172938560353478	0.240245413578364
-0.108873460896297	0.399632678853483
-0.585710993794202	1.28459513248853
-0.176011721327078	0.128843246130985
-0.847916247839353	0.240236110053124
0.110492922721311	0.763923309978283
-0.537948258942128	-0.0876349457787294
-0.331653708338285	0.298750293115921
0.206949463437521	0.400726949095204
-0.49713484657602	0.715111695277548
-0.106624865434939	-0.116578208064297
-0.298868959988671	0.204238310905484
-0.091601599359772	0.825141999721168
-0.177786537851075	0.895717003166769
-0.363964806870407	1.14549569825997
-0.114570956999551	0.72435651569129
-0.220690418635383	1.37181212417973
-0.240501522731488	0.996654176298157
-0.245220198454836	0.9383196257535
-0.666634607449063	1.23916976155634
0.0190621965855888	0.586439461770104
-0.372048764993819	0.734589226199413
-0.580473514242376	1.22076063041382
-0.391205800444667	0.127752064964135
-0.490811175371402	1.07927439020911
-0.814600946743081	0.928366040037902
-0.540511292513467	-1.08956145143949
0.0785651621591439	0.467890154641223
-0.486642033803295	0.363569738761128
-0.480122722210247	0.591025678151713
-0.00344122458294906	0.289511317948388
0.261342169488509	-0.0297634428985288
-0.353576384436823	-0.0936442813090795
0.250360305990982	0.698896489125352
-0.0871838768230112	0.465299989868273
-0.681419445340253	-0.0673802779177538
0.202568904649976	-0.574446425114346
-0.0178146889532939	-0.0767260486315823
-0.0470371972669423	0.722405706358665
-0.966200231504331	0.944215505367242
-0.272537365284588	0.363586174042882
-0.352122678059926	1.0023528035626
0.567525874603411	0.101526633288496
0.188671783176388	0.465243360503582
-0.593406067457099	0.174546451574576
0.0128165667862184	1.0275359219936
-0.114380096774361	0.934260991356913
0.0748889751096445	0.507483732292841
-0.189444604846618	0.602649432497427
-0.309271787867697	0.649058197816537
-0.540511292513468	-1.08956145143949
-0.622806958469682	0.70943586017264
-0.286713334300497	0.436443474497896
0.296011254285592	-0.00809853793056256
0.419736555427799	0.421631971496233
-0.95345789842269	0.888983592130836
-0.717904867072786	0.350305744388244
0.118371590248977	1.04251073310599
-0.0115623644492304	-0.00374356552015864
-0.322722532172072	0.329887279713329
-0.251222062346509	0.0761763886952869
-0.408823984215276	0.720098893553827
-0.762608771576541	1.312793291401
-0.329795335386194	-0.173008208123376
0.393798178994785	0.354536328000722
-0.169206080255994	0.644977704594645
0.155354320069864	0.0757053552845492
-0.60756200922121	0.144790870091561
-0.717904867072786	0.350305744388244
-0.609420667711054	1.07720023011899
-0.592155941629144	0.655133430302399
0.743625409212174	-0.69931287392925
0.569859849047135	0.489524745358631
-0.223531826089492	-0.00961610191094247
-0.567824409627805	0.996593269563297
0.330584757354006	-0.952546822971219
-0.882504848609804	0.792630951783788
-0.0386352314539765	0.407959171171844
-0.167664754436095	-0.678841130067866
-0.302885617917947	0.216275534304039
0.806327215600594	-0.0289226361446625
-0.408823984215276	0.720098893553827
-0.194828501099816	0.511641512571092
0.205637867811937	0.0685964017294631
-0.38234390804099	0.358388917751795
-0.603705522829333	0.678009683737299
-0.397171827848633	-0.205953703935536
-0.231884436251358	-0.10291358969834
-0.25742885814795	0.869748946493863
0.335758919837393	-0.384555036871346
-0.136611895248843	1.02789452136032
0.479299174635153	0.887940436478709
0.0163320481896074	0.334945577570216
0.23515807786344	-0.566536498951735
0.361183799428843	-0.353416999148752
-1.05191804800143	0.551712019465553
0.446124402958544	-0.54873380967159
-0.64860620514202	-0.369706962677011
-0.765728902421751	0.34268829377472
-0.754973541344279	1.18568008396387
0.34526683346697	0.640611324164439
0.0950807696600971	0.268973842449206
-0.195397369742753	-0.654574955704679
0.113288449472412	0.501763052784159
0.175859859995701	-0.52603525120978
-0.158683666561382	0.0473590272667037
-0.397109682938133	0.619455369745569
-0.440904776085512	0.618326347427124
-0.271400582126405	0.261979838800294
-0.00530751299956223	0.34323836757178
0.0559723150739191	-0.208121236746402
-0.61103984882549	1.03180523941918
-0.602719501525235	0.517173804647815
0.510137790047736	1.2635210843713
-0.451559805339769	0.701853861494851
-0.174140801772403	0.782716257374502
0.187967258262414	0.00990652563209587
0.420987176326606	-0.409241789388758
0.202534181410205	0.00402981423757122
-0.347178478635212	1.08979339846132
-0.268180791133446	0.020099600726797
0.277378401970208	0.483972855666564
0.613105377047348	-0.888053755944417
-0.0776760760416992	1.08200231337115
-0.450231759870978	0.613492091957561
0.573951765540444	-0.20011234509081
0.298507087943648	0.177763006015995
0.323085432611977	0.134020724273841
0.841290044258981	-0.3690443338964
-0.161234008271886	1.20015787743605
-0.47370460985548	-0.0499626094424935
-0.316704176734276	0.345917381055065
0.098616837233462	-0.320000886080146
-0.0675078417652256	0.504864478868541
0.844215209645911	-0.0752175059178824
-0.250023267205302	0.349839031063549
-0.0844081154135857	-0.353755928935234
-0.444056378517973	0.464079366550656
0.528750699500654	0.5476466315436
-0.195397369742754	-0.654574955704678
-0.504684359351977	0.110848383845887
0.588033458886069	-0.625963928081342
-0.12930187829704	0.0750704024993531
0.151874629230301	-0.8272914543049
0.479239532011265	-0.338183055983029
-0.0201465001326099	0.299569830904418
-0.448551996388741	0.744057495456117
-0.492481152273955	0.816347116750579
0.215034923672836	0.620991764129922
0.54088193945936	-0.127983078211304
-0.381720178448772	0.157535056277255
-0.0742384682142774	1.00639060214619
-0.169206080255994	0.644977704594644
0.670128496589475	-0.339742446323042
-0.596883018403427	-0.383876587149406
-0.891704884894033	1.54997573088336
-0.47183452122037	0.792013377824541
-0.402601729861663	0.0244846375752295
-0.486642033803292	0.363569738761127
-0.435725057446622	0.883586257600389
0.339066582065628	0.482893498398283
-0.697396991567282	0.530124144279206
-0.614080339035042	0.559731692071515
-0.825653871555833	-0.481880524357996
-0.620028981494305	1.25446285398769
-0.504261223569038	0.837404778004487
-0.777000879752784	0.794380015550718
-0.694626238283787	0.437358587243337
-0.0857500040845855	0.800349129407857
0.037978182311417	1.01308641005774
0.473370421409302	-0.491965853880996
0.225213038372728	0.0506247022971637
0.25880128805821	0.654730432067612
-0.603705522829333	0.678009683737291
-0.415754212441201	1.20503061099421
0.0197547744588522	0.56109071913593
-0.526914651211125	0.275171407820926
-0.721427162340412	0.205815829122099
-0.540913831334228	1.1679871000164
-0.306292713155868	0.718088495176637
0.215376849761429	-0.421043387626205
-0.246292151062062	0.833060687021431
-0.63147456821175	0.6540450139526
0.0986574311176511	2.0216729288742
-0.286713334300497	0.436443474497896
-0.535843946036851	0.403088311769965
-0.478600751234902	1.1813215578892
-0.527029788165149	1.42234528169231
-0.676871016974099	1.61592892701574
0.501390197370389	-0.702838395553031
-0.739755948201017	-0.515504121250543
-0.60282027871093	0.495696923457175
0.145695683128504	0.610345135058437
0.312201524854826	-0.698707721719039
0.402358702487238	-0.522086763114083
-0.418233275488577	1.08986354177394
0.0158876240083785	1.34157968139973
-0.239850531432871	0.589069922730697
-0.201674219035053	1.30442292831436
-0.270303278372406	1.45990455099572
-0.567067040988015	0.473279069119012
-0.360162373794135	-0.0639861271387572
0.0707355211759286	1.05633216994872
0.0751824215264566	0.695270120977678
0.50765189119497	0.68237863853841
0.429967858140772	0.253062782942023
-0.561170064281157	0.480844749569314
-0.241523467265068	0.278035224522038
-0.825660622086916	0.335950237089924
-0.553300836658915	0.782077911915205
-0.362952295988148	1.03782447160719
0.277533177999974	0.091916339362143
0.0143872727346593	0.20116281112594
0.121686531493739	0.56614763107859
-0.37203118127939	1.37894335043583
-0.107570529290887	0.197235184279207
-0.0590526227126161	0.779092743144499
-0.197066146143182	0.985038092599341
-0.473232925066172	1.27931678230425
0.550972924081339	0.326859992584618
-0.228346376593715	0.00614723643623474
0.115031799475562	0.0628904935906844
-0.199020986278584	0.99161713102317
-0.679333477537441	0.659608794368051
-0.404952467202282	0.138968225365096
-0.253451525841125	0.769978931491085
-0.371437357523529	0.760752199070925
-0.209355430559776	0.702281344055845
-0.152954845219241	0.60215274894171
-0.437472699275868	0.539830486523032
0.161072632565151	0.848485968738884
-0.492180744082238	1.20480067299272
-0.34861650395983	0.758546744328286
-0.288079986410635	0.291653623014549
-0.11146137941495	0.569468573691567
-0.136353431103016	0.0408584752173791
-0.334038786199077	0.748676549647723
-0.385444911930711	0.105944812328635
-0.337827526911254	0.431516557932566
-0.197518377106909	0.518825665764045
-0.275494995278603	1.06395480199607
-0.249512008083683	1.39403798139245
0.469047459797997	-0.734450375007422
0.617740472726913	-0.00729605307459256
-0.120583448330481	-0.629180137292595
-0.48513505190263	0.0240887635515155
-0.420007063759952	0.309221662424072
-0.0591628322165905	0.83110192297963
-0.402315637142215	1.40630553868836
-0.0658644831728593	0.283974592552555
-0.418152639392681	0.507685728781841
-0.49713484657602	0.715111695277547
0.255678383532033	0.939013268178779
-0.678894575348216	1.31637841739112
-0.585710993794199	1.28459513248853
-0.667040770277262	1.37817625172188
0.0110270628185033	0.33641156216172
-0.666634607449063	1.23916976155634
-0.122765524670393	0.497956301592938
-0.180606354357175	0.623124968064997
-0.734562758218814	1.54406745390759
-0.167713228470748	-0.666262721123098
-0.202752685113435	1.24607116406299
-0.368732265932205	0.103000733517958
-0.814600946743081	0.928366040037902
-0.781551169178298	0.865329423984243
0.167301403622969	-0.0160156211546662
-0.511407337091738	0.811032865674457
0.342512258822417	0.360828248037097
-0.0745064010498876	0.549782773377029
0.17762383451997	0.789192038691779
-0.612294163535189	0.747516281797006
0.150987158118503	0.421469506255751
-0.195513744887551	-0.256794551647345
0.115179243207922	0.142007242099378
-0.740670904702228	0.57367240347326
-0.158744595361557	0.0889566675332814
-0.640915371557933	0.325544620009328
-0.355790172754327	0.986354720809341
-0.399331200019467	1.00681694819884
-0.440768923228321	0.790409779646231
0.00906355231758853	0.561304322285692
-0.225488250501112	-0.173129060521616
-0.235807894404722	0.679514003063695
-0.454059305261346	1.11382331811554
-0.445337351064366	1.37636705399163
-0.453151370662594	-0.205728202059716
-0.269102892023374	-0.156444983819529
-0.200409747105797	1.10392428043016
-0.29546791972463	-0.300880836344172
0.0419780684721801	0.684583453621836
-0.503424004590528	1.12260790388153
-0.766842098998406	0.302220586767959
-0.0401528118809699	0.564437063425578
-0.485546113430944	0.705543093794704
-0.194669866191233	0.141210344410898
-0.667500783370952	0.643489539955977
-0.608657173533377	1.4688317405733
0.302769310239481	0.488808751897238
-0.310114258842721	0.5949531668431
-0.0915571037232098	0.598550548260044
-0.0364449217884385	0.943808183113315
0.00475239995034624	0.773488295977346
-0.162800395057144	0.266357132101967
-0.477183555077965	0.478179816543831
-0.0425121862841958	0.542172322421569
0.0934503314106341	0.808823359197363
-0.148053265714097	-0.113248476076005
-0.299912466816806	0.364272040304735
0.441470329286704	0.7007556698676
0.707579730094685	-0.511397789156429
0.509043138397491	-0.56097529370489
0.410447357711586	-0.302585402777644
-0.0617567470686801	0.0343819452084607
0.870715655374175	-0.172733941620711
0.815001346749289	-0.946738521119644
0.604459159938886	-0.0403338085232586
0.528740489856958	-0.192715491812354
0.369987023641596	-0.408823157751841
0.233058323964224	-0.41893139909009
0.115921493774662	-0.291722693971215
0.426464712628486	-0.090590983662185
0.775606223342419	0.029412420215465
0.705799416234067	-0.150168049802696
-0.0900660199679379	-0.406991811785461
0.523594621753957	-0.340723712599035
0.914663423756933	-0.189251836810355
0.804825305900934	-0.287578032947722
0.900962803121168	0.184466611580583
0.465430671118781	-0.232629405098907
0.466794206378696	-0.181635376026811
0.17004331001211	0.209972544303403
0.943758745416899	-0.542452156151109
-0.192560608729633	-0.0277391541114815
0.77989473175149	-0.376071307467489
0.0137321053506101	0.14918642353995
1.1209199756612	-0.593233627966838
0.0987840117975122	-0.30590189702311
0.219981963622744	-0.241868650141592
0.412035603670563	0.0377435615971593
0.948260917075647	-0.133682430743066
0.340178829428396	-0.480270332718407
0.985730813023644	-0.427746140018906
0.290962586923573	0.0982995950818899
0.35715274242556	0.327256715772119
0.796725152167816	-0.103208003323247
0.223266957175695	-0.209478038051637
0.905122900982472	-0.267091497500312
0.111969597844916	0.293642691946603
0.897341328158126	-0.382948122151408
0.334132494004443	-0.142535266142703
1.04209557383272	-0.217696689064834
0.97854247617758	-0.423239806464847
0.801437776788582	0.281611084128962
0.772214913695097	0.162583506799343
0.0133225610071885	0.306882898788812
0.216589445644103	-0.251709966182778
0.776863808354919	0.0739282305976348
0.137776342739124	0.171363543006762
0.361492252319822	-0.108213944971236
0.789267422084415	-1.1929819423733
0.627159727060555	-0.414280834579269
-0.105469525722343	-0.127371365435529
0.253689966041819	-0.362205207602744
0.325480474122332	-0.0442644177280622
0.862934894648812	-0.114598807015037
0.828578617373535	0.751762430076247
-0.160615688677539	-0.296377483816899
1.08666480574891	-0.368489366467713
0.334031486941194	-0.474802365961203
1.61445666748244	-0.627751882248762
0.675699823336833	0.103820833439999
-0.881826962381427	-0.558691914988911
0.60261813019821	-0.180142186121943
0.529273493893438	-0.14489084073698
1.23835767555548	-0.534070113109794
1.09709009922635	-0.659035652696806
1.02446212577112	-0.158954789500155
0.482896799506816	-0.44729122529318
1.19812239219063	-0.483414045210905
0.390106326577938	-0.785835019520678
0.737857547801495	0.0692840476939557
1.26204075709545	-0.425450513505647
0.415823477516672	0.480651053988939
0.263062968882813	0.215200316962214
0.746766366033451	-0.803447090796367
1.09100322213583	-0.177646123817707
1.08693846097984	-0.577984455177947
1.10516773988392	-0.298717809210895
0.178678351676724	-0.528551340461614
0.364518877659041	-0.210340120712205
0.500769540977726	-0.199415194125046
0.620605729461576	0.274193553758426
1.1567664607488	-0.814077575746164
0.85074172462927	-0.587016776216316
0.397164917496164	-0.0744452110269608
0.424462067773406	-0.115739682766508
0.950977795566021	-0.635063110886551
0.685362516242516	0.00183418543436499
0.537566859393626	-0.361069356629952
0.324166500563293	-0.488911996691162
0.439882879326732	-0.0941831765827066
0.160411767066019	0.147310479982108
0.661349378845462	-0.048551413852089
0.382934985085741	0.0052037356582547
-0.0918033737982743	0.146643170785972
1.1350839203897	-0.367267187646154
0.89137636372997	-0.32028625574162
0.493180598958639	-0.320722035268289
0.314567774877737	-0.280859220305281
0.412086829217535	-0.514810910032017
-0.498889784034814	0.0669936241631969
0.609733832444355	-0.718984268640919
0.572087019891146	-0.258971861225442
0.576570504452074	0.142911779180521
0.50634827589253	-0.436510202568376
0.57613603015791	-0.0788420928640657
0.470483119188451	-0.615784332456893
0.70461804986819	-1.10770721916217
0.202247638551793	0.244126151190926
0.4572643820519	-0.148008390223728
0.475903012489248	-0.27964533212428
0.00229045625986932	-0.205839945794868
1.02656426754392	-0.42449964745124
0.0533466025788774	-0.549926313709112
0.759198881516268	0.200838466809357
0.771133746932258	-0.155099224739437
0.739040041437357	-0.241414140700751
0.99699131409537	0.0973399963375216
0.207389559082557	0.226831123092358
-0.498858897278679	-0.481876012336185
0.742498449182476	-0.236318852751102
0.166341347310774	0.195221635100655
0.55962769183252	-0.0310463186939098
1.44580718523284	-0.355329469248759
0.61973207700929	0.153219500401509
-0.430477726316011	-0.462516439166235
0.62019634853461	-0.494330290182566
0.476601116967075	0.0152202279976345
0.350419347434864	-0.482405925445445
0.486775633769486	-0.585075345872144
0.92883853196387	-0.10351226863382
0.470990280660431	0.051779452014027
0.907075880186179	-0.376274068415344
0.393301649624011	-0.169027802598818
0.456972186380678	-0.490026648046031
0.474765780281117	-0.138266178717379
0.89231545201971	-0.0640200632422553
0.513196682307502	-0.0864884011553013
0.697750427341731	-0.191590905302201
0.231694315049596	-0.13204113339923
0.939707507277615	-0.130646169998869
0.31136599302983	-0.0109902278363996
0.387027209011624	0.100824127573283
0.790088097482447	0.342923270322709
1.24290679576786	-0.530425867642119
0.53063766956355	-0.18017383396995
0.664230202335338	-0.566810827222483
0.635463687692485	-0.0383831542869723
0.196769440666351	0.36515575812986
0.384399649877029	-0.223717921087461
0.850680857552685	0.0299619190293403
0.937368821689331	-0.323778024320017
0.296577721026165	0.357359443915045
0.387027356274355	0.204154800252409
0.151890137958485	-0.173843976367993
0.560876609694335	-0.0273407397820143
0.729375004129031	-0.173019144455206
0.555528593990801	-0.3890793646423
1.29792239573703	-0.0939614180499667
0.903731424300707	-0.281531961998454
1.16857439419736	-0.149436459158003
1.05131650965461	-0.231247668486118
0.115808164389772	-0.120909480787753
0.248859004936977	-0.230094453045393
-0.147201518146462	0.827899775229068
0.677879045783696	-0.249903717144216
0.709041306198009	-0.3935478480829
0.975389699210958	-0.325228071033041
0.0666601596517037	-0.116450719413603
-0.292180075025631	-0.10798812563892
1.4748486309609	-0.670560592736675
0.820177233810069	0.120900516161136
0.791711836043688	0.00879006547271871
0.925135833946312	-0.0725949232170776
0.603098088388549	-0.597202336122417
0.348441412667765	-0.313765046519587
1.29850429987677	-0.600213024216432
-0.0082771268865436	-0.15458649024786
1.52341122671432	-0.0762452187682071
-0.0455862483299842	0.252134695736127
0.716518383933643	-0.293335645801192
0.452286494273824	0.493482893064014
0.482228977732662	-0.120157666751273
0.695718069594236	-0.236041900149418
0.295318316117307	0.323681658600242
0.626872272764455	-0.27122029677472
1.05178284923373	-0.0320586915282329
0.161549990346572	0.0751161792514643
-0.0114801701184992	0.263926148918558
0.678782643896541	-0.230401039400559
-0.44329032548157	-0.271506517688872
0.588577555765563	-0.0262348396031342
1.08949104485792	-0.222903741918161
0.385245372848456	0.0640232865143613
-0.00313378813290416	0.138952437033267
0.314081276310634	-0.118453758883613
1.60323404948229	-0.151529196751353
0.045932061266792	-0.481849346367386
0.296326046117352	-0.275597419719599
0.277032820915454	0.171232371029338
-0.284986074506936	-0.489777469166003
0.514127180602538	0.0302835503467308
0.338984147327802	-0.30536509476194
0.548910580136237	-0.146081987660611
-0.13022053548274	-0.117797776964929
0.365913020000728	-0.101120558010282
0.724389160187689	-0.353232850344924
0.430630067410286	0.327348683791672
0.383882485152743	-0.192873015253163
0.606140261371193	0.352839115548899
0.614423963185122	0.346066165719434
0.702911046438603	0.00884196127093236
0.627726777570448	-0.417969845474631
0.606811960992408	-0.324254402269978
1.21362949082875	-0.520350856609579
0.738775454261749	-0.254142095482457
0.750676809007917	0.031997239642237
0.123594754846074	0.432824861729826
0.387391378293744	0.117278270929042
1.35116077359928	-0.289370259550356
0.803304633602366	-0.231900021995415
0.275920388004937	0.257600782090645
0.757303523694566	-0.2508246306388
0.0119184651583753	-0.148233952281329
0.682264598424712	-0.398400830306359
0.0142150180717646	-0.056153159573801
0.127364826361315	0.108852736173452
0.235531672697602	-0.0931514965952592
1.13040918461248	-0.401772504878916
1.20750259403577	-0.375327291736661
0.552250863414318	-0.0725580394862388
0.568473164201619	-0.355577474143733
0.542536105669604	-0.587580315798631
0.620089983315766	0.504067525840004
0.276007302718916	0.206362520869235
0.629815139832646	-0.180546348848362
1.55861778830889	-0.382160947625541
0.961241553601324	-0.248841123621494
-0.121041689182362	-0.31842085800671
-0.0869016162467236	-0.308946623226833
0.30487987328437	-0.106506305741057
0.100436217095277	-0.534105863358732
0.58317098754101	0.0836219021952547
0.588128707286732	0.155401813610003
0.334013992894999	-0.257313130224966
0.487556196009148	-0.200479959220338
0.507218793459851	0.552710734495094
0.392847521915461	-0.0518873898349454
0.420458814228178	-0.402374313570166
-0.0048421307328494	-0.234953528236542
0.290051479961546	0.21692403581684
0.750676809007917	0.0319972396422371
-0.606073446798614	0.0229897142046614
0.895157344636026	-0.0441695881458296
0.530877394164013	-0.0383159015192786
0.100883221249669	-0.110754327891353
0.467496527504441	-0.0397980181232829
-0.615582239721278	0.668314377146639
0.687293761257181	-0.213995234245435
-0.0406701412562718	-0.187096960298102
0.0281427132653991	0.190662306456174
0.579664602069762	-0.113618434916123
0.258368148053116	-0.368387286133298
0.68226459842471	-0.398400830306359
0.384952958796893	-0.732533271258879
-0.404377384847378	0.47280407154781
0.42360684192383	-0.0345155819144167
1.00927567661279	-0.325604315447273
0.106917402771278	-0.00514729693518005
0.581712015382771	0.0660046769275559
0.557669824858398	-0.336226781796257
0.686752715887301	0.181244282127617
0.0942759724283792	-0.570945434325445
0.372863494539764	-0.0823008242927802
0.607785156425598	-0.0666538064853651
0.568862266362856	0.0436000834002473
0.0722821285954421	-0.241112349288251
1.53671976539698	-0.237074435349016
0.441025372310535	-0.0184633067743725
0.875893598339404	0.0392320787766921
0.642076524494132	-0.0812096847953328
0.790703047550484	0.0445188513347923
0.385705245959614	-0.00317900827120374
0.620605729460529	0.274193553759189
1.4508058759512	-0.369585170802695
0.771133746932259	-0.155099224739436
0.555112307627478	-0.0717834792182467
0.681301019840718	-0.048295655875132
1.26030351085626	-0.295296452170103
0.470483119188451	-0.615784332456893
0.224248088343415	0.115859076197497
0.585054955448057	-0.226762523834861
1.22527999272185	-0.533694096872559
0.210509615227384	-0.01217662198686
0.911031277215082	-0.511558686870109
-0.0604676048916087	0.173889699566975
1.08666480574877	-0.368489366467652
0.657415588029685	-0.311099145171057
0.231694315049596	-0.13204113339923
0.303210583681451	0.0710092165420512
0.0649280058515723	0.125131549995583
0.624382518273609	-0.295754594588305
0.70133118829549	-0.211844682174561
0.632684370383781	-0.0412076667736269
0.415611706493632	-0.00418522520884964
0.669492159105009	0.366347508583255
0.458992364168904	0.375867629435106
0.755484251189423	-0.433068637368453
0.166341347310774	0.195221635100655
0.482896799506817	-0.44729122529318
0.802981637724836	-0.443509083625697
0.978542476177578	-0.423239806464847
0.889104891202519	0.0189207562962216
0.487122720342954	0.124161640937448
0.25129331908292	0.258561474820887
0.599492067695768	-0.487306810947104
-0.25139670232048	-0.118324972229201
0.700271568683465	-0.322395692776577
0.321401023860857	-0.587484759994623
0.738775454261742	-0.254142095482451
0.493018717588506	-0.34822595643843
0.168062704636898	0.0699496575180242
0.335758725010163	-0.13503556696494
0.656584747428542	-0.0130654269841975
0.334679062483374	0.0578135109103178
0.966402403720898	0.110369391876981
0.591686980766755	0.367836001638212
0.414469564780551	0.379977440436099
-0.215818235721946	0.108602455699632
0.426880481033196	0.363252221587909
0.663095329822827	-0.225099351148085
0.446390023965435	-0.00878584526045225
0.745642504407914	-0.4900951444121
0.273520622210108	-0.086844547231026
0.579761852655246	-0.37340102692373
0.950977795566026	-0.635063110886552
0.41579923154071	-0.631691933182672
0.282431633524822	-0.681933489460951
1.12613013868555	-0.515081906400037
0.275920388004938	0.257600782090645
0.57613603015791	-0.0788420928640658
0.392449133075236	0.299239885767161
0.00425558446393307	-0.214647762862143
0.213164001822045	0.606121175138848
1.36310486635403	-0.124170420682854
1.33281556585876	-0.291602773300903
0.568024531038476	-0.0183001927862555
0.906686733826411	-0.0514508729686359
0.322002810869499	-0.197292163581231
0.604459159938885	-0.0403338085232584
1.23927438700415	-0.408711056212118
0.481053710327193	-0.257107510324308
1.09100322213582	-0.177646123817705
0.120948195966068	-0.0452721971381269
0.466794206378695	-0.181635376026811
1.42652890369355	-0.483443217343847
-0.182789326534391	-0.773596864667751
0.870715655374176	-0.172733941620711
0.437290356926911	0.114714775397643
0.524853543148303	-0.336898978427674
1.31759778759414	-0.499059564123123
0.485594314758789	-0.0837290504264076
0.605053732040212	0.0757524328056627
-0.298033963031163	-0.924646324348236
1.32297796664646	-0.679628712042427
0.202247638551793	0.244126151190926
1.03421455220419	-0.203656936396972
1.15012971784756	-0.648531129954926
0.310311661141012	0.14133915399989
0.323796958264638	-0.12540134258784
0.750075580889195	0.112478881242256
0.315600204023771	0.00417622220316899
-0.0290290040990937	-0.190884753985062
0.678647839411737	-0.501289816755742
0.461056584045741	-0.459404476488837
0.528624182376507	-0.210996868938416
1.01304045925647	-0.168432072836541
0.940066355403546	0.154883652641357
0.232023497017154	-0.38079616305375
0.407060963684644	-0.314049474891968
0.635904130305982	-0.107647291864517
0.543640149714159	-0.211199612320441
1.07640395629898	-0.30948640746616
0.306650790895287	-0.270529690069238
0.376190911012397	-0.0481469842496847
0.156702725049037	-0.37553478049166
1.28344378849047	-0.463378096743068
-0.17287896529819	-0.178334741826147
1.12019179308765	-0.669075582401595
0.593491148130396	-0.411608759523393
-0.312877766337246	0.119182144385551
0.962994614130061	-0.472286150104208
1.03918435509637	-0.0912878853051274
0.231868177323912	-0.0203157830679431
0.914925977875774	-0.328347228814719
0.659257819374555	0.277470138464882
0.332465683163302	-0.0518075417029126
0.75666662441194	0.156632216026123
0.622042472610109	0.103207529666592
0.292639944341311	-0.144884642545361
0.965987803938715	-0.0574383317901119
1.29978381681389	-0.424549708724825
0.716635756743632	-0.583726742293374
0.933561264455336	-0.315996065949304
-0.261023793316793	-0.485740399333106
0.883575267305032	0.299565583986535
0.197234229493715	0.169892024012408
0.274366598577168	0.0807231010456759
-0.124277835747643	-0.045440795592037
0.210941120426895	-0.320030699359835
0.485381648482636	-0.0371492448166614
1.72465194196231	-0.0538152349987535
0.487147269582379	-0.140438439009871
0.779045189596205	0.101671296181819
-0.0135701598794625	-0.0123480014786048
0.604643229262446	0.037427557873589
-1.09903349448389	0.147628059568674
-0.446510455075159	0.155319535496811
-1.22091813918272	0.108877254627181
0.240231522238587	0.117049332566589
-0.838596090038137	-0.179089498894832
-0.481906077581266	-0.468045842441137
-0.741847153418975	0.282327684978191
-0.642598290150774	0.00820982201016936
-0.426262610891777	0.139887527455396
-0.804808556331584	0.447476533652725
-0.347347282021523	0.041034398909034
-0.456137085237268	-0.0328453880940878
-0.173998628534297	0.329972226019924
-0.882818340777389	0.181540578238066
-0.253862495761151	-0.192775367071608
-0.416668855712448	0.11460131536333
0.224926726349439	0.24629191315157
-0.703825774408664	-0.511316872807864
-0.806774657038999	0.247063926534213
-0.456137085237267	-0.0328453880940879
-0.875430461964667	0.138006681083486
-0.841912688506941	0.254779912582333
0.226636818354674	-0.230000346969431
-0.258071290235035	-0.167004647563378
-0.584327058019607	-0.213718803260601
-0.0618926084099007	0.0968912003332188
-0.28208634612631	0.19039350861671
-0.508989898986637	0.0126293378609974
-0.219587653444964	0.325591328471781
-1.54150633750065	0.0727455780422388
-0.875172783774177	0.0657836337882585
-0.53400214021815	-0.175983565628393
-0.392343812881221	0.183266494995449
-0.762811012370179	-0.480717429179665
-0.0496154695104653	0.0702764746229937
0.340106871559439	0.0367975520177664
-0.669492702147342	0.488238124803853
-0.885332868761728	0.291890904991452
-0.073813001214158	0.201079716044701
-0.507475918343726	0.00307839310769588
-0.59932398808207	0.776923194058552
-1.24948177719274	0.411119870657603
-0.224240607238038	0.621267200857987
-0.0548680195214338	-0.0995916765488449
0.896988908764274	-0.396234407557707
0.304785279215937	-0.0532601800300739
-0.054542740295305	-0.121281886265245
-0.238344779156136	-0.581280288210949
0.0168144872754363	0.0982171613049335
0.586057333354473	-0.266540017684429
-0.504830432686583	-0.0958709380196652
-0.861280019741491	0.150904406872996
-1.18861711364024	-0.0222288166585928
-0.00633323044080932	-0.236163109447702
-1.12434035846013	0.40526683020703
-0.697147586720708	0.140687029051687
-0.491828135499077	0.0222544126301541
-0.426852091616866	0.0837303837420467
-0.750541475313256	-0.00756846337398522
-1.43501078633243	0.157164764271296
-0.360173593746735	-0.110711481928726
-1.02820491824457	0.32226285595789
-0.496007184624128	0.35042546333746
-0.737615748109197	-0.163153458139216
-1.05867760994971	0.225217130351721
-0.961500020450266	-0.174449965098228
-0.452693145391187	0.0254037392384227
-0.454556693078695	0.037479275966944
-0.689983108339368	0.160159583674543
-0.336404435365685	-0.46192753649487
-0.389025446264963	-0.0632324488440374
-0.239397750142782	0.0842066718509993
-0.85345629692914	0.057492258579473
-0.240908946387781	-0.232616101077339
-0.663517201438948	0.0253465958346825
-0.799126329673654	0.113869407251498
-0.379373809368333	-0.141794777077708
-0.589682809682089	-0.111828429500635
0.940198577756994	-0.289378784393501
-0.930306409926145	0.285794639289903
-0.256655228386076	0.206017367032958
0.112882289049352	0.0727533247694455
-0.634195360233867	0.173534675203631
-0.861280019741491	0.150904406872999
-0.480823391407555	0.136637479684739
-1.12493455816186	0.00938303673014151
-0.667957325511782	0.105819462857247
0.0168144872754363	0.0982171613049335
-0.170507948546435	-0.157675277853882
-0.801314894005235	-0.0176077971932227
0.939350463462034	-0.589036931030225
-0.373947308126077	0.230243203310351
-0.574829940097288	0.241777978946725
-0.550285282401811	0.228039159328818
-0.926205374323528	0.437089278245733
-0.979213093845136	0.454227683912786
-0.654245131407147	0.48612973011688
0.00309316088661405	-0.0699892722626291
0.106870315463468	0.18654136005566
0.302347521251551	0.0377465497192646
-0.430519762910468	0.0483498746668963
-0.916532523412249	0.673237651977313
-0.691207514826381	0.592126647132025
-1.56510486462965	-0.0928390557280223
-1.18213451411767	0.468796588305784
-0.665758930235659	0.21259662500486
-0.913959104299416	0.294910970556206
-0.801527633774164	-0.0322642784389764
-1.24109016479086	0.0968753537082076
-0.439961540536763	0.701630737889477
-0.530023164942063	-0.152164489325333
-0.394775500461463	-0.474427207383044
-0.43524015923672	0.274687746437111
-0.745113816535343	-0.117532495850484
-0.704467153268958	0.121805466331625
-0.179244319562533	-0.285044857997593
-0.207545911032271	-0.149965808205381
-0.0670111628962231	0.357173501617676
-0.456868656518238	-0.187793814465094
-0.897837843112659	0.158906136564898
-0.353260942203101	-0.170981568037016
-0.87002222959991	0.318915873520475
-0.586899945838631	0.0531889012427227
0.409209662788856	-0.0901424005190962
0.0013191324040619	0.379950986919708
0.149535321215533	0.48794475102507
-0.406571491700363	0.10163027330232
-0.398368129878129	0.45847430503896
-0.748226262336184	0.211690300892636
0.0834977413811369	-0.0732998155879977
-0.199811831211661	-0.0318775839634498
-0.0518394730950163	-0.109271034635147
-0.556769295007124	0.26793737988088
-0.763361168537095	0.790366993931225
-1.20830640249408	0.510328506565669
-1.09998653184372	0.641995812526421
-0.0957506385027602	0.192727156570783
-0.697638431574299	0.194244524662503
-0.440700011360504	0.156370725646164
-0.155696109478062	0.284754976038461
-0.514193364699288	0.552814529300876
-0.0394370389994813	0.196481434842791
-0.365300896839442	-0.114307512989793
-0.271805439438883	0.0409186595146682
-1.17625234876782	0.497526018474629
-0.70320535886206	-0.0201474978486257
-0.28737264614621	0.307543520816598
0.168904601357864	0.140221489069917
-0.366198502720662	0.612404738715211
-0.896905763082936	0.495271173533559
-0.727445716192722	-0.105895590045614
-0.120752763446496	0.0392702643274978
0.402486663760322	-0.0216633809025258
0.280918230902004	-0.187053926823856
-0.968771736167604	0.10999171588206
-0.804496944058353	-0.101452589900872
-0.621902812565109	0.282805202169701
-0.258688778153372	-0.138817077461119
-0.176296849332083	0.225436346667884
-0.881196706953511	0.0391742263665637
-1.01679587852216	0.201144768577246
-0.928735377677761	0.0399615183574042
-0.972032474491984	0.22693152168561
-0.443671584340066	0.310778964941505
-0.639828342672217	0.14244918633018
-0.468257576992704	0.0957116129750012
-0.564360797346633	0.487824664756502
0.305353207594284	0.553040124607202
-0.578132635265049	0.37992583298119
-0.371588873847404	0.173894773511857
-0.577655260666968	0.226469779395933
0.0504964573116621	0.232718673408624
-0.724614380169673	0.214878543898814
-1.24053386587894	0.463390900027683
0.0903367627627445	0.00696349324367017
-0.210420264635419	0.195998722426631
-0.186321738517188	0.547460311037585
-0.215413692778903	0.1539957887991
0.1801785503402	0.00391187367926541
-0.0142054399644843	0.0908821230555749
-0.513254280002623	-0.00782638752051887
-0.513254280002611	-0.0078263875205304
-0.741817512036315	0.601124862335473
-0.66851296656118	0.552752075132472
-0.440921897211078	0.18129811762968
-0.545392493413596	0.310371312263842
-0.505368617019072	0.148013451646418
-0.0244080452761193	0.0315521129021741
-0.251306225736735	0.25945360300027
-0.609565392191584	0.134335163360893
0.0153154923168279	0.17275604941319
-0.774390070928368	0.470042403988448
-0.00404706850974074	-0.0752072034532903
-0.344583436841175	0.199890379388634
-0.100209536319768	0.119796731255537
0.180801727023141	0.333154554540422
-0.499857527349573	0.155245133318459
-0.174497618327396	0.316478051933165
-0.00152457919797346	0.0167976749343226
0.0709116463783884	0.261913361333649
-0.00846472326438027	0.234906665521066
0.639161913222014	0.73844514176513
-0.496347937302963	0.238587485289922
-0.876898431244541	0.247126768414606
-0.814378827536056	0.511773167441688
-0.3975934756777	0.187805686287087
-0.218356451396808	0.543360225812325
-0.0465328353715531	0.2295549709601
-0.663988475347338	0.060994863292619
-0.757651635075188	-0.154149248861006
-0.302160024175189	-0.134540540475445
-0.13807406593707	0.318277038926459
-0.817213294379863	-0.268100915409006
-1.02267089796327	0.194453674990812
-0.357595753786129	0.0237042179995172
-0.286674546578059	0.030536348015976
-0.618600962477377	0.252025346399546
-0.411613119392484	0.015207759226115
-1.03538650737757	0.177083855920417
-0.755930468606824	0.772057071678626
-0.330193202388707	-0.168385479471031
-0.622759123649751	0.34513195707832
-0.749060944805414	0.332136312587575
-0.785256110442608	0.0519734798485382
-1.37922243336748	0.455236431598927
0.673479801150348	0.325103476651635
-0.395611235115189	-0.032573981776439
0.266633648515605	0.492467530685334
-1.30163513327201	0.320050295832812
0.110932526265042	0.147568407796417
-0.485853476403031	0.279290815384428
1.01086948630529	0.430282552105057
-0.039121735011249	0.158628495300262
-0.69486931255152	0.375036834949791
0.0341482332580941	0.175727045715827
-1.26072350408457	0.527950003587951
-1.00668584383619	0.331259490301822
-0.522767796961078	0.325388186753512
-0.682106404697902	0.130261596235164
-0.64875196423294	0.102284965936031
-0.977370323807101	1.04685424062875
-0.24270985609583	0.287488990443924
-0.119520143466296	0.0631647492614392
0.494258550087738	0.504288027615175
-0.751266731203118	0.0943800424366159
-0.554648058192649	-0.161301677669365
-1.02302180380807	0.0757499138377906
-0.572202668055928	0.0100608615760637
-0.638315120011066	0.241417015829488
0.0818700399934965	-0.211790180057584
0.295371176577505	-0.513615409032814
-0.482018909430684	-0.108702252512702
-0.286748092902892	-0.468440734478084
-0.128988312761504	0.332652121225662
-0.482018909430684	-0.108702252512702
-0.634422985863961	0.240845184804685
-0.00702427886872491	0.197505495939792
-0.007024278868725	0.197505495939792
-0.223378192219973	0.394671248336756
-0.457106177339657	0.120587742484196
-1.16278482457913	0.328446572016377
-0.162133185700705	-0.0870683165017185
-0.0959457642970974	0.376955660440739
-0.451274585665389	-0.232935022213473
0.329504427390745	0.103058708818109
-0.206337529501742	-0.596054455844522
-0.36403026162804	0.0299617022996871
-0.810450909935161	0.15953531405329
-0.839545403050301	0.0814836404646476
-0.0496544052937312	0.420337785318202
-0.266779834261681	0.436430520406026
-0.658279337214117	0.241524011926156
0.0948366550621818	0.250635893799896
-0.663232062734544	0.515879513541835
-0.361918650070612	0.3709100996879
0.00478595082077632	-0.0626386014527695
0.00586633573432978	-0.0900633439923746
-0.826307788786681	0.205104052499169
-0.279413068194658	0.232977371592093
-0.300545332403981	0.147684320114285
-0.166693817141406	-0.0282036131005033
-0.205597301489894	0.23532146500074
-0.501181417681628	0.0759826453259724
-0.668693661902879	0.0818830326559283
0.0474407801587514	-0.0445425666403124
-1.13339743902029	0.520733736998042
-0.785586847698254	0.518138025124851
0.00536476377119373	-0.0290428749582632
-0.232967379735762	0.16818752261654
-0.865149981448764	0.0916898067532184
-0.173446359309927	0.156440542339302
-0.811169986461218	0.759946823366952
-0.734302488488197	0.717418555286088
-0.264950894092087	-0.163962364636251
-0.433620994150326	0.218271078828743
-0.501242557876578	0.334032005530943
-0.0644998996715585	-0.163633372647683
-0.313062719744978	0.450207724237608
-0.608410765050694	0.121317935895624
-0.410420394390614	0.0251917947806008
-0.706557015590144	0.15171277857128
-0.853034566692836	0.21686023098723
-0.566638495613588	0.0169327428864081
-0.870463938069827	0.258526942468849
-0.500316674109052	-0.0588160708705675
-0.967245022864852	0.280631284031382
-0.460592640832578	0.0589797270513182
-0.507500005223872	-0.457192588896878
-0.719255117020998	0.170263898633351
-0.691778379847616	0.669880654964482
-0.48632547145748	-0.0945201484651114
-1.11051644825696	0.316852388513855
-0.388675115443213	-0.0920799008112861
-0.848279477747111	0.0925593979963893
-0.452280082303097	0.220621139513068
-0.464184548243329	0.626512679559595
-1.07797937238084	0.461772303482384
-0.595396797182348	0.0871465724053857
-0.836251242001701	0.113875764489779
-0.438206211815642	0.171573339880062
-0.190093284010566	-0.236250978569871
-0.389846835202062	0.11008996505223
-0.196797232958571	-0.071290208424302
-0.344266226124217	0.219416203373193
-0.0355556184874086	-0.840305668462479
-0.528151619344265	0.602218800003279
-0.55826868316238	0.199051540014749
-0.621578291813548	0.0397952016029422
-0.630291387124564	-0.0165015253410114
-0.288241499448274	0.350070803898356
-0.557372116950052	0.317039955701623
-0.41314520347757	0.240786158670601
-0.618308213511036	0.237396215852654
-0.194019042285709	0.177396863283571
-0.86412135176463	0.393331659365809
-0.737969104353269	0.275533855025192
-0.348798230750428	-0.433404460056245
-0.250542889189767	0.266217130093767
-0.703881312687439	0.233340007987351
-0.383494730325951	0.201587112392997
-0.957898750071103	0.263005638907728
-0.573671644265786	0.229238568123591
-0.719140950081307	0.394135076222791
-0.413145203477571	0.240786158670601
-0.266137090725246	-0.0166357746577351
-0.267198888976964	-0.253009227264608
-0.653886654801922	0.271528427560004
-0.71094234779408	0.307576579909444
-0.455182641077154	0.123123303395413
-0.326272791859855	0.237499072540027
-0.398378920581308	0.602180530738658
-0.97100716491462	-0.0832520343352829
-0.417551006007096	0.0483616312538737
-0.686984565072157	-0.295499602025212
-0.830780367063266	0.308918968957394
-0.815190015576236	0.236021462942973
-1.07176718243594	0.39953127544981
-0.146845286704933	0.263239100762844
-0.36189948094015	0.762299501444503
-1.01626838145299	0.733136151289874
-0.451457147841365	0.292712551279943
-0.177808265367254	-0.0107198066754044
0.452707576078538	-0.191094531864784
-1.14743416829107	-0.354257479270294
-0.187988142827645	-0.00803491041998198
-0.53633188408153	0.401302672085927
-0.349588140585774	0.409340021478287
-0.245488091324679	-0.161709035922952
-0.456293766319102	0.272438200165029
-0.395582202234944	0.0451780851265019
-1.08741584116329	0.126936876134819
-0.519198570988386	0.111629207400734
-0.573594569147876	0.205743295734159
-0.656907560869211	0.357884071328756
-0.220753802864517	-0.0770243281743508
-1.03431742340214	0.413085591736982
-0.570306964302423	-0.0765948284398013
-0.427151917035092	0.164673610347675
-0.823802698778916	0.299265953016969
-1.02249280058933	0.133106433074937
-1.27798588557448	0.609880515324375
-0.727659078908632	0.0615963729688593
-0.846275424117016	0.418388675656377
-0.875282810015043	0.131123069403941
-1.02349026889262	0.293496629004968
0.302731498560445	-0.384308258103313
-1.13096120664246	0.148527330219431
-0.680438605907728	0.15156156497257
-0.44905783977475	-0.0034381312227048
-0.156083472501949	-0.0351494516032608
-0.410891501933549	0.151703823669013
-0.297802084259639	-0.0510362791757193
0.0612850487898568	0.0256089698751322
-0.0832923600759099	-0.207300261467806
-0.137736832267098	0.532032234448156
-0.532157364324312	-0.148291833624279
-0.409373586373506	0.487858572627209
-0.568914917668884	0.0790734337940146
-0.4277727410998	0.139419191237284
-1.36879416211197	0.326726192785113
-0.697521991846853	0.61908708130658
-0.239955253755758	0.0696594536730194
-0.634042719216967	-0.296198647457922
-0.785784501595567	0.24748740108737
-0.707067996284841	0.122450261799665
-0.290775962307334	0.192517376010657
-0.881683863896788	0.297257814362464
-0.86412135176463	0.393331659365809
-0.209866456785825	-0.213047114860914
-0.411396650369303	0.477651902940223
-0.553687352859178	-0.316079536755872
-0.385382790677419	-0.0610054993275031
-0.108440774812425	-0.00150384634351566
-0.991716077587042	0.249299751548832
-0.752114798594142	0.41069212113583
-0.98259851856877	0.222518955827303
-0.617453060855032	0.382601591530714
-0.711130441060683	0.599711311232892
-0.692257378503941	-0.275949801007008
-0.384760798761976	0.0807090262970524
-1.02911706572278	0.240161584121606
-1.12110798110119	0.0765913724598231
-0.42155885022398	-0.489980114566521
-0.952444967144101	0.399687577202033
-0.742923457708993	0.197505730287117
-1.1277429240272	0.0885424898770539
-0.917026474120342	-0.278550844601745
-0.726000914351407	-0.0433172644635937
-0.600870345855528	-0.139354257682704
-0.837614992420346	0.090285064089314
-0.942532382954338	0.313781023929587
-0.197087592226789	0.0873473839917507
-0.742923457708993	0.197505730287117
-0.885064547142517	-0.0422270621159379
-1.01549614773894	0.0296551766914799
-0.837614992420348	0.0902850640893142
-0.950911478503362	0.432358270334031
-0.197087592226789	0.0873473839917507
-1.35053484290768	0.174097085042224
-0.561038887584883	0.257853529610919
-0.584588572542292	0.0429863533544753
-0.743299360833653	-0.0546347438809505
-0.876495683527788	0.546077033336992
-1.01004272792544	0.135733195230957
0.177377818793395	0.0875109649551315
-0.671100850896467	-0.0460006689763776
-0.556296898651342	0.138869655398065
0.255707389125033	-0.0419840280021766
-0.774041355081706	0.457057864752168
-0.899881431192432	-0.0648370248773572
-0.873392290656912	0.135586526227136
-0.69691630318158	-0.186819285145895
-1.52277164909137	0.616368762455913
-0.670767844573554	0.246515776677657
-1.42937655065636	0.191663191943339
-1.06885480437771	0.542470429233262
-1.21173899170003	0.361207189194614
-0.98744567588076	0.158746506921163
-0.992901834247887	0.163650957034714
0.0191290217905127	0.040826819864215
-0.284487786999179	0.0762921890008963
-0.0856476770877139	0.443409525742073
-1.04673006511948	0.381793746505062
-0.243633042057387	-0.0596860622024626
-1.54062856279565	0.440624769186245
-1.21595292643102	0.281607191265019
-0.305180351116403	0.0991046826128849
-0.971310617983106	0.542656807162494
-1.00399013050265	0.521458467382177
-1.00399013050265	0.521458467382177
-0.338157626712958	0.0162124121274082
-0.843137406564695	-0.167453263601421
-0.745327957722667	0.40488670984274
-1.1765446048753	0.512473226816233
-0.43636010368297	0.15086516078076
-0.770146563406994	-0.184079672194171
-0.611731914780558	-0.104544871021371
-0.224833354106253	0.104695241404245
-0.440910344095414	0.204157205205354
-0.827332451065291	-0.18252849431363
-0.845615406403296	0.195658445624689
-0.572744563085554	0.451466984557213
-0.687130107838236	0.149124317755307
-0.511013994295934	0.226191350206203
0.0181146512774291	-0.24065895205743
-0.936809877582588	0.110666210943265
-0.0335053414195247	0.283902337234124
-0.987100725544632	0.289909119291841
-0.680885785429543	0.234226299353329
-0.707663397734776	0.356151776275401
-0.434170761694186	0.160904441831855
-0.428134437200274	0.463542623477944
-0.6095231100562	-0.204189452051589
-0.636868733163138	0.0841568325504189
-0.759157392431627	-0.0282149230837436
-0.692188050864911	-0.0966454146281792
-1.41387813071606	0.589356695924624
-0.89327385987258	0.249962794058152
-0.701333026241212	0.588604640195654
-0.65650742220246	0.486837023442729
-1.32408031564697	0.550620210507783
-0.0875876611611938	0.114042487450839
-0.451838039493939	-0.161619021488646
-1.1567253779166	0.445543153414233
-1.44307057725858	0.642357078239352
-0.357732735390834	0.0655096318515342
-0.487485190292685	-0.252397059264388
-0.916913702689254	0.0141302887181909
-0.868373554405027	-0.150046210735951
-1.09780492165491	0.101587403034264
-0.95389383180343	0.295058867479748
0.586025872477101	-0.624279096946586
-0.0878528237708281	-0.441053009405927
0.597474501811866	-1.25168756786661
0.418928134939705	-0.835198104623288
-0.0397866482927227	-1.54417004546332
0.37500404311059	-1.19665586787436
0.338086823340389	-0.547533314239215
0.495218779939975	-0.90804921919315
0.0618743753084487	-1.09047795313672
0.555160714180546	-1.02415068211466
-0.21519121390548	-1.00634316569706
-0.468528537538532	-0.776804188254416
-0.209195399052331	-0.806367875045461
0.706870412405505	-0.938619121783911
0.440732821408255	-0.638423298811567
0.644644995496574	-0.763271840966884
0.88493392126479	-1.10620083636176
-0.146229247231964	-0.373630433672491
-0.0928361902049856	-0.585081008842057
0.394555874694253	-1.40899787726676
0.611736078416613	-0.997921379663804
0.240848402180979	-1.14271330390424
0.434663375674444	-1.14452762432295
-0.360342172057984	-0.375653282744127
-0.0685410019067068	-0.858225465396987
0.639841007869984	-1.18074860261198
1.19089515470017	-1.10502294096482
-0.423430231517678	0.054568715102345
0.148749887631791	-0.714120483866941
0.967859402082795	-0.657957019020024
0.764035274807645	-1.19940327626902
-0.0110133842899182	-1.60496933227363
0.573656877029815	-1.10926321387062
0.74816752873271	-1.04100532500502
0.0628802976869348	-0.0368818945457761
0.597474501811866	-1.25168756786661
1.11909551758022	-1.23787013837073
0.102493070045952	-0.645921699216849
0.818087355236491	-0.65326970043389
0.321158071334702	-0.609394173739577
0.18868354031733	-0.970714374934564
0.110571349561339	-1.40398379073635
0.513950588882583	-0.866850153120707
0.890343667359859	-0.595318074836258
-0.416045453228081	-0.934389058431801
-0.46219207086139	-0.613352929562662
-0.279449166638191	-0.545110697406933
0.473621900032889	-0.735537153379756
0.987522294993307	-1.00809674158971
-0.0978143024296788	-2.05650094633081
0.76298478133788	-0.344480177745631
-0.125518661032111	-0.572808955060901
0.254385093263463	-0.356847896246067
0.560647096539349	-0.982011399886886
-0.205504279123445	-1.64999148189505
-1.01476452424867	-0.0646677286425605
-0.330347374931173	0.0130856048720005
0.13285652086945	-1.31003216478181
-0.0568650141189069	-1.11636950201896
0.441007153072221	-0.526538954210547
0.576284958957328	-1.51263727520786
0.627515873997601	-1.62310263287726
0.418928134939705	-0.835198104623287
-0.117529549024873	-1.2698460525472
-0.624324981987674	-1.02439352532308
0.488101923843334	-1.15916121782161
0.230027655473276	-1.53867876758623
0.102755749350027	0.360021024108485
0.236977183677777	-1.15899956734879
0.0940310660650785	-1.17908280748199
-0.114557136320104	-0.386655759124567
0.854147430030802	-1.72714344779218
0.0148175696280892	-1.00471427858402
-0.660609396076935	-0.424329737571269
0.950249453208756	-1.73902111817793
0.449320030253951	-0.844933674154364
0.108416602549976	-1.20537112242169
0.0567069133015981	-1.03075544522696
0.860593537320694	-0.761766788843272
0.515018783362787	-0.744534657190459
0.406643462197161	-1.29713257869124
-0.577754215081642	-0.993045468286098
0.426091046229512	-1.08647275284901
0.559636803503216	-1.11419537281163
0.00083236433412584	0.176400171119791
0.992568058156772	-0.784772774930178
0.475556795028099	-0.458754901145672
0.415454576584677	-1.17779321397352
0.113985514001186	-0.728764406242023
0.0986376003993283	-0.478927952708931
0.513214284439998	-0.216641851227045
0.827590487797982	-0.319615631193995
-0.705340424816985	-1.09755974574886
-0.0449806093898838	-0.615295918013921
-0.356236261190072	-0.926183574487272
0.0866112603382514	-0.890002535112856
1.40841400708425	-0.952066658727778
-0.59255817084425	-1.34189378049551
0.0638108422675136	-1.73482585660201
-0.512771139222881	-0.658417351100279
0.957608450077558	-0.316369017696754
0.984568893104612	-0.558425526422477
-0.196738899672836	-1.30160770836814
-0.323656621562801	-0.28103611008425
1.13547367397276	-1.07692572270077
0.95251050639095	-1.25627731543469
0.364721417808426	-1.39954494839933
-0.0841492348164449	-0.545144480269608
0.401321446919069	-1.30034440922351
-0.439693013415958	-0.196883947621821
-0.140644981818445	-0.0404643008776596
-0.120566522219239	-0.758161735275967
-0.222729929289332	0.288372342865054
-0.276538896636498	-0.810860265603057
0.782847978211442	-1.31659943105502
0.117284375782221	-1.27917951686823
0.364027087752964	-0.976148609140629
0.0328500748345326	-1.45520315568442
0.207464901332315	-0.599037303180429
0.991193679631536	-1.02857663520018
0.829166626490191	-0.876483987748337
1.16771242602873	-1.29900688603886
1.12039265831876	-0.760646762569516
-0.33778292668088	0.45330978618185
0.303815811975954	-0.0288234695809818
-0.429253324412067	0.119805140687256
-0.0333472490893079	-0.440016438818075
0.838749448517843	-1.06269411988904
-0.428045529939214	-0.833968392827376
-0.366649088352917	-0.966676232402598
-0.0139169462609427	-0.990230883191792
0.750183426713198	-0.906560463826614
1.49506650644511	-0.516783534789884
0.396791543829665	-1.03137471431373
-0.491416330240933	-0.555120369246408
0.257804138449757	-1.23702045318342
0.0748139438361609	-1.21793836684898
0.472039651003374	-0.952950199387314
-0.588825617756459	0.241756915681144
-0.805215939457405	-0.200882467921574
-0.122306636840765	-1.043186646312
0.934963894047588	-1.21177466918877
-0.205744502528321	-1.20396847346211
-1.24816380314117	0.376021908195089
-0.145785746971861	0.108444805748788
-0.00883525320543215	-1.04259493358696
0.649900159784963	-0.975204032993366
0.0305360398912723	-0.468389676646796
-0.0883540681991973	-1.39524064061346
0.81441099748602	-1.30033765755428
0.262924085338432	-0.771012087346554
-0.398144962286259	-1.13847351067232
-0.156586892915391	-1.17822880978778
-0.524879545753586	-0.780538800357244
-0.691027352439019	0.109393386085038
0.762851793288835	-1.05782570512807
-0.722918334407848	-0.723147473716964
0.0584727861875231	-0.996944598018121
0.649900159784923	-0.975204032993367
0.0477805048105378	-0.774378651389142
0.477458931322967	-1.2107375783609
-0.0883540681995143	-1.39524064061354
0.102596571006482	-0.919187890103502
0.212424681490078	-0.474728165362767
0.202520917592604	-0.565580714653229
0.202520917592603	-0.56558071465323
0.4967154919542	-1.02762651943624
-0.80396509396474	-1.67013560674073
-0.164187922477751	-0.48935568892919
-0.399350525559792	-0.594848963439931
-0.429253324412067	0.119805140687256
0.136954354480522	-0.473249623896118
-0.397006298257233	-1.08354496192069
-0.0445471836733583	-1.38865788706146
-0.428045529939214	-0.833968392827376
0.750183426713197	-0.906560463826613
0.0257150482347629	-0.843726004780924
-0.365507102924067	-0.966378888106177
-0.0139169462609426	-0.990230883191791
1.49506650644515	-0.516783534789892
0.458545953434735	-0.996933989280677
-0.173753662747771	0.533919205309655
0.430256360755689	-0.841619660815035
0.640654762757251	-0.746087556907359
-0.165956483228598	-1.13698678232402
0.585813496885937	-0.514489315136493
0.51354050124939	-0.78232568985021
0.98792189509603	-1.72288008364789
0.274154933264577	-1.65444544875664
-0.282343405469002	-0.222413772931592
0.91692785408968	-0.782125994017744
0.10249378375633	-1.33526981675752
0.524357444690288	-1.29002244915168
0.91973020635992	-0.820400505948641
-0.183086232511146	-0.360825695827923
0.0476982041219298	-0.49299740728374
0.532441277741364	-0.0161603532680901
0.559507310217761	0.0858829482604351
-0.125922437422733	-0.570704129842994
0.936976020834841	-0.9422966424541
0.649479999736613	-0.340312411062366
-0.511085091769571	-0.33118856877203
-0.450036016984127	-1.46186583751443
0.325327407798652	-1.21238367832252
-0.363179073437481	-1.09357211385094
-0.218935462956364	-0.633732506262059
0.371202749628625	-0.797466014214234
-0.488684675960109	-0.963260802193122
0.0608805888298479	-1.65222735523535
0.435196733959175	-0.958468308722299
0.32691940882727	-1.03624533189278
0.676483837735931	-1.43203649272702
0.306044328793268	-1.60548377490844
1.05911171177909	-1.22288799279818
0.496715491954201	-1.02762651943624
-0.260781611812874	-1.15776488867803
0.960941763085535	-0.735360992964847
-0.937219747038487	-0.368884616566665
0.0331055036419973	-0.628636521008111
-0.0132463341989177	-1.37305170068162
0.524688563146149	-1.59143441698015
-0.0689950768015905	-1.26621129053799
1.34602809857986	-0.8280574147085
0.0331055036419974	-0.628636521008111
0.676483837735901	-1.43203649272703
0.326919409158955	-1.03624533199084
0.645214017967648	-1.36138347034787
0.847041779729244	-1.08513161590021
0.649297471781769	-0.887501698672158
-0.0618904894080088	-0.882844004408204
-0.245989959445991	-1.09760926538195
0.193132882280926	-0.894490103136312
-0.164187922477751	-0.48935568892919
-0.0169371633648207	-1.3006367218768
0.147229591473809	-1.33834850515909
0.266205823464634	0.17080837553078
0.454562622320418	-0.784204156397305
-0.254525881484142	-1.06365109012524
0.401038797191804	-1.76147715201748
-0.149835624186302	-0.188204685360077
0.502295266024287	-0.0793531909739366
0.266110982475642	-1.19004950342042
0.388008387767885	-1.14015974064982
0.163291202871123	-0.880510422335844
-0.262764850539762	-0.390886060823033
0.446582649554464	-1.02365635616352
-0.395511645477633	-0.759293548155437
1.23716616826463	-1.19786329711251
-0.104840075538005	-0.673626434107977
-0.191521425543328	-0.983611791038613
0.957882858711333	-1.03677563980659
0.495218779939975	-0.90804921919315
-0.339044395569923	-0.492685094748989
0.0227305537534969	-0.772039773004657
0.205278444582859	-0.694520960948254
-0.305314116726017	0.0588153145239695
0.169012167285995	-0.934342668842545
0.266226537995223	-0.966218285265084
1.37240504191624	-0.81727024909568
0.162570808372548	-0.736423039778006
0.674704811699778	-1.00926854109416
0.175296574794788	-0.729688343850193
0.22304082306908	-0.949344222208265
-0.030348211469362	-1.02542916706941
-0.0336716771501784	-1.58335131733272
-0.426713109808702	0.041346597032561
0.116836728276661	-1.21841104630567
0.262074631082189	-1.48076348074287
0.769706795848135	-0.953280406350086
0.425786655705072	-0.787702841901089
0.484872419646845	-1.62868074848459
-0.0353249345419926	-0.790624525005856
0.835374133734644	-0.676658804460309
-0.352779380745417	-1.1102642533778
-0.279226698311723	-1.09346298235366
-0.147754689638238	-1.37572871698516
1.09671112437595	-0.94412280987919
0.428655224893873	-1.87100366684248
-0.193271873877717	-0.557914589866836
0.490981285791107	-1.77229787222572
0.0276698181251347	-0.99678224685862
0.0610979991322409	-1.34913023477648
-0.638018407161413	0.109506214896933
0.517803151685705	-0.685371094142319
0.382503553663948	-1.24787288517458
0.624820869118042	-1.62023114473152
-0.0450597972980764	-1.23172117533637
-0.568824774594903	0.0431618250655751
-0.592558170844249	-1.34189378049552
0.228321867455203	-1.25829727606265
0.102596571006482	-0.919187890103502
0.0257150482347628	-0.843726004780923
0.762851793288834	-1.05782570512807
-0.691027352439592	0.109393386086557
0.847041779729245	-1.08513161590021
0.524357444690286	-1.29002244915168
0.162570808372548	-0.736423039778006
0.22304082306908	-0.949344222208265
0.425786655705368	-0.787702841901161
0.0539921955023936	-0.333356166905838
0.25051916810398	-0.835866915641315
-0.193271873877717	-0.557914589866835
0.950249452399587	-1.73902111808978
0.382503553663948	-1.24787288517458
-0.0434932013648308	-1.06541925516547
-0.593629774511294	-0.445331302558002
-0.1134459669181	-1.21467369492782
0.426091046229512	-1.08647275284901
-0.0523195206192331	-1.10025504281501
0.113985514001186	-0.728764406242023
-0.353299272964653	-0.848358982163866
0.0986376003993282	-0.478927952708931
-0.0449806093898833	-0.615295918013912
-0.205504279123447	-1.64999148189505
1.19089515470017	-1.10502294096482
0.333116291610806	-0.781525666052681
-0.0685410019067067	-0.858225465396986
0.627515873995231	-1.62310263287719
-0.25959382501604	-1.4758805539608
-0.319774856790786	-0.906634736876132
0.588940100582353	-1.61125030463077
-0.056865014118907	-1.11636950201896
0.762984781337867	-0.344480177745623
0.89034366735986	-0.595318074836259
-0.267658466251776	-1.60628501021571
0.513950588882583	-0.866850153120706
-0.279449166638194	-0.545110697406921
0.611736078416612	-0.997921379663804
0.243317675740757	-0.8696356800937
0.706870412405504	-0.938619121783911
-0.46852853753853	-0.776804188254416
0.321158071334702	-0.609394173739577
1.06366867099296	-1.22922137167279
0.818087355236491	-0.65326970043389
-0.0110133842899182	-1.60496933227363
-0.289606961150961	-1.17588159598065
-0.698045541503892	-0.669164304576682
-0.117529549024872	-1.2698460525472
0.110571349561339	-1.40398379073635
0.848528810104553	-0.996528581749766
-0.416720330737869	-0.857067918710466
0.473621900032889	-0.735537153379756
-0.492189509076353	-0.585533393099838
0.769825008077817	-0.376931041370795
-0.368544704547688	0.529818770804724
0.141531438650317	-1.17241816800367
0.098463782274199	-0.988454980506546
-0.339680607951949	-0.581862430684115
-0.0285631009322424	-0.725287360427838
0.477298995143873	-1.20442435195532
-0.0884007638601221	-0.63389110326208
-0.24106293272357	-1.2387366095098
0.437334685325659	-0.821377701455011
0.799367590556729	-1.39400085491972
0.415549334487451	-0.104014449884948
0.473088445895955	-0.979917117614261
0.218750806478106	-1.41450180802532
0.748826307570885	-0.954939009939536
-0.306140540555378	-1.62611190102996
0.225353414593923	-0.967511277452241
0.0298654637670965	-0.938248900034601
0.363224800482295	-1.34639142287612
-0.0957425088141403	-1.39423990637404
0.0495907678400085	-0.526371550814751
0.116092653881282	-0.427268276339308
-0.187779985709485	-0.891527097769333
0.229604828855543	-0.642470602865694
-0.0519689975530311	-1.22458825970859
0.357510931130999	-0.969847361969771
1.19008961787285	-0.714284893026058
0.126359310790783	-0.892605735547185
-0.0450597972980765	-1.23172117533637
0.860593537320695	-0.761766788843273
0.838749448517845	-1.06269411988904
0.182109345698095	-1.29060217931581
0.18868354031733	-0.970714374934561
0.0866112603382512	-0.890002535112856
0.00140723268175031	-1.24439640917413
0.0618743753084486	-1.09047795313672
0.240848402180978	-1.14271330390424
0.48810192384335	-1.1591612178216
0.230027655473276	-1.53867876758623
0.337610019336276	-0.470447079023048
-0.0228583380620612	-0.874023348683205
0.14812182462147	-0.625169339767346
-0.0969526182660793	-1.72603402014229
0.0328500754457457	-1.45520315562674
-0.196738899672836	-1.30160770836814
-0.161454761827665	-1.51878394372403
0.539486134423846	-1.0688335317007
0.624820869117938	-1.62023114473152
0.804108407600147	-1.36062645852433
0.406643462197161	-1.29713257869124
0.216064764948823	-0.93929346632144
-0.181153378139084	-0.771236479166898
0.353572459384495	-0.675969261709899
-0.0234623569693041	-0.93144228290033
0.645214017967647	-1.36138347034787
0.69394583644649	-0.420944448745968
0.528942159742138	-0.556774301817524
-0.180815304283451	-0.853231833515981
'''

gaussian_sv_phi_fulldf2 ='''
1.15683474839151	1.41447963364321
-0.331948669612474	0.355755573206987
-0.137489276804344	1.78822442450732
0.887740587562381	0.100471824799586
1.27348216118189	1.32406785478716
-2.07912056495386	-0.779843854200219
0.612568607078147	0.151940256302606
-0.636814064498371	-0.402879548629842
-1.68327650846625	0.532513147059687
0.231517116278802	-1.84715250575856
'''

In [ ]:
#@title full_nodf
gaussian_sv_x_nodf = '''
-0.500610351592751	0.873190821820748
-0.572054462350519	1.07338176389737
-1.39025754967044	1.39449109396387
-0.991207558789491	0.667619581745548
-0.861754843449569	0.675184984711284
-0.362456192130888	0.738200874280691
-0.338169137620666	0.728075934861134
-0.0856324763535983	0.611819907950118
-0.428305871361728	0.74114758249746
-0.438006718961559	1.14295887350146
-0.70560650010526	-0.170449356844364
-0.792689007087882	0.711488635081956
-0.751047471627368	0.11234050475663
-0.223157024182852	0.915093960982109
-0.618103057426005	0.0609134735151419
-0.769580081887699	0.640960540641538
-0.747489093008897	0.70654913069041
-0.523855285922805	0.791430818140339
-0.925344527079292	0.70810929246636
-0.62616495807989	0.327403823513176
-0.979633403360903	0.434552575156998
-0.592208654101705	0.458561210630672
-0.508947400426076	0.767317423604342
-0.749106862878798	0.840410581443016
-0.944224299895572	0.798907450984896
-0.48197797083613	0.555303517456864
-0.836342671235614	0.938856082897339
-1.19111412130552	1.26074446294536
-0.283061243572414	1.06425996281967
-1.42841340928759	0.811996513725233
-0.908377381675296	1.02604976640545
-0.774558637480227	0.602431544120423
-0.49231393157844	0.554958519709612
-1.04427963406007	0.575333309017104
-1.28560831588756	0.732224631910021
-0.595644657732748	1.54786750489853
-0.532501801922204	1.19451271587357
-0.876772041525661	0.776926113937278
-0.867662144981399	0.175458156779142
-0.959509671469567	0.436974026942231
-0.270294848113851	0.731892522038147
-0.745032441563915	0.788886741809027
0.0183547126595403	0.564913006913709
-0.552648682043651	0.665953642294392
-0.750526732618064	0.853698266766432
-0.754421936030182	1.05301123058312
-1.12476891214575	0.723484393911513
-0.904723318268758	1.10429008076944
-0.585467656736256	0.50704762708748
-0.970513242128368	0.309409852383812
-0.883225749298571	1.00321075022041
-0.245721138154611	0.812690685429635
-0.703432766420913	0.938865025922239
-0.48959415977254	0.29759195617928
-0.451360588567139	0.839103190016292
-0.635517776040063	0.612644408784267
-0.737318993928214	1.06886041933329
-0.801446085206081	1.07752980593389
-0.19123635292164	1.12574490345043
-0.248997633790444	1.90101922617709
0.265472682451608	0.42330750176922
-0.693053945212432	0.791036855239094
-0.991266082466411	0.633458593616494
-0.352167450915135	0.624635279779935
-0.467882949349743	1.06965295810799
-0.662444070735359	0.728893720213109
-1.14521258881496	0.684087873829084
-0.720783128746535	0.873931174353337
-0.868477380873275	0.256932047417467
-1.10550234074113	0.639505855900027
-0.696451521415833	0.0415043708641569
-0.791071840934657	0.882693767104436
-0.325879879864467	0.740319222488159
-0.924609887262136	0.356979072839712
-0.422810795360944	1.01656361183767
-0.810538655841016	0.997226376366473
-0.771424527135504	0.748767739573503
-0.300712627400203	0.786163412052052
-0.961569845471312	0.740062428421108
-0.521025771682711	0.810769547054848
-0.701008557992692	0.260325246697133
-0.942030556943281	0.944390616965116
-0.718995495777453	1.22623933404486
0.00197019167147266	0.165247535638129
-0.642633829365599	0.766994216947123
-0.433045086591639	0.746141575487506
-0.462836922887362	0.663840614559636
-1.48233123835896	0.606038229692133
-0.501420027790175	0.938850219239398
-0.791933622507743	0.64944074582821
-0.979467707704266	1.28500152852751
-0.873042634559876	0.59472243012676
-0.957637990736694	1.12160985398924
-1.26996318175053	0.79748409338648
-0.568416267744171	0.871696097640879
-1.01405788652986	0.70624576497599
-0.844271892598952	0.841594058681599
-0.813739850282996	1.13548495615713
-0.705771502482464	0.663509317838322
-0.569225470431013	0.580673580650434
-0.638966766421437	1.09684686815011
-0.807741314305551	1.01876465494407
-0.899565459948165	0.50373907219813
-0.365214073559932	1.33037848204515
-0.298133137787933	0.148021729772435
-0.490800955565576	1.0587621297663
-0.865873238713667	0.859907454655801
-0.679725878117249	1.00342419008394
-0.960172743554118	0.528683635565348
-0.383382661837955	0.830219637533549
-0.862348228800897	1.12004368635619
-1.12854203445001	0.259054853196331
-0.589104970446002	-0.0784733996784227
-0.579771026319836	1.2975933728279
-0.868433507805531	1.04057910628479
-0.961568084483332	1.02858810157789
-1.05970977454707	0.470017484072061
-0.87512403359354	0.442634524223389
-0.729622145598571	0.606179816351271
-0.414477245891022	0.691066846793753
-0.503379463189209	0.848661231135115
-1.01587522194782	1.28387222331424
-0.20914268391768	-0.0361743268294832
-0.90252099088965	0.274605017320062
-1.21721675826455	0.867450246483772
-0.742487727461627	0.582920625996629
-0.86412844460974	1.10399627934954
-0.951210191840269	0.635589663675056
-0.278995125098032	0.186177284216789
-0.784328506389268	0.970341453351062
-0.920928108006024	1.21026658878615
-0.772172768615575	0.629201362932475
-0.672719356392549	1.34991904283255
-0.42864582447419	0.342258464708111
-0.369134567508694	0.0422791170258508
-0.581145008891886	1.1037907369363
-0.802092832204026	0.951611054015665
-0.614327856042586	0.667563778106739
0.0449233374417874	0.234971138475236
-1.43473434584172	0.92235598350546
-0.817442041523448	0.358983185334634
-0.728952811529988	0.783601556474339
-1.16622425457126	0.761450142171634
-0.95442762512837	1.16896650928207
-0.49506376057832	0.577302396153506
-0.285080710108946	1.05991138665722
-0.632723588189074	0.89361984187625
-0.256672757259916	0.267063167840957
-0.581835487865737	0.354415634180068
-1.01606384763381	0.318700452066762
-0.657993112136901	0.928428948332835
-0.708848329544089	0.860441085592463
-0.797763887563721	0.854988687688794
-0.757926565965197	0.665705524166878
-1.01213052090077	1.31939790057912
-0.652731147393937	0.828148444421019
-0.151549485944831	1.11755537969147
-0.986676574137525	0.631779200321889
-0.163839030305626	1.33486275339004
-1.03595486006788	1.06010908758617
-0.124542322458033	1.16167826256998
-1.04063779627572	0.753550667836276
-0.845726533394854	1.08265354666632
-1.04322720709015	1.12384676659857
-1.10460050172643	1.19308977998159
-0.361899082073421	0.587114529340016
-0.653923719194977	1.08289932759649
-0.903335963116594	0.893069963220439
-0.896718381127497	0.621712321097305
-0.522059474427363	0.722576860178102
-0.862458319826671	0.849653966223259
-0.48965328310638	1.26666630439741
-1.09470308055755	0.753158671326421
-1.0145641805399	0.161110062762529
-0.820787476614915	1.0369080965874
-0.821966276316673	0.718258189509368
-0.934840137639966	1.0346385544307
-0.208251846772011	0.71532110957423
-0.0727801699508316	0.0785443657165781
-1.02609954009012	0.873034782263935
-0.714063656550342	0.947441801738887
-0.797481091934972	0.332190807587321
-0.514277582773046	0.950003095187391
-0.834037545051674	0.853182659079377
-1.08495281745819	0.208905846233511
-0.583202523873698	0.638559618673772
-0.776291019356156	0.915704676250995
-0.665508267924122	0.413504813643159
-0.458019380871312	0.847976203323406
-0.696691605701057	1.26438108719941
-0.813657128802075	0.274135616074163
-0.547455949544989	1.24883020350176
-0.271883269928432	0.00442406612767559
-0.785403444878395	0.92389399862531
-0.442429203288162	0.781008420269384
-0.448261448821876	1.12631555277422
-0.596506713121057	0.770019786771511
-1.00715427607941	0.745502450738859
-0.58827767705036	1.41738693422206
-0.827034198755001	0.641685519398297
-1.37293766398238	0.644439304981769
-1.25991007305749	0.945572952973905
-0.570388683329876	1.04195255375529
-0.427917143892885	0.917812908578404
-0.667262875836355	0.231972791923752
-0.917700455585451	0.192728290970642
-0.913048657721717	0.528990524375569
-0.906790982572435	0.235826394244816
-0.36275630120753	0.23691134835227
-0.959634826231792	0.535223169397215
-1.33564588850649	0.372514622596716
-0.405944774523766	0.0316644567684497
-0.125962249474058	0.554590557920857
-0.51860785298895	1.21771036923228
-0.697050689016758	1.14602715447948
0.00453171895782174	0.448947121568979
-0.386252055051013	1.27106914694963
-0.770843229647019	0.831584181307301
-0.178078357952585	0.194617007686733
-0.896130913740796	0.720818184819453
-1.50129300067401	1.02560184054818
-0.963354487420086	0.994755780250675
-0.664099203874816	0.446915425345656
-0.576424579373584	-0.087604476832991
-0.543074391153664	0.203427760115867
-0.486824338510964	-0.0275177716830087
-0.689439148857064	1.31128397984279
-0.140475877260554	0.597491218032681
-0.61597193261639	0.517478722302261
-0.922999693037429	1.2153091922176
-0.810232408240787	1.43689024232813
-0.420528634810027	0.848610622238948
-1.31236387425649	0.409641803671181
0.240557058886608	0.553375082850534
-0.707382362654101	0.31434375955826
-1.1148330287814	0.0399862972953594
-0.839630592425511	1.67579653601548
-0.642510930590327	1.1666055704347
-0.520959750668707	0.952112715001948
-0.51860785298895	1.21771036923229
-0.470442056973379	1.00928757712927
-0.573090401741192	0.214663029415835
-0.192914274867913	0.762008252900459
-0.615754377485388	-0.0291155928399654
-0.504660820319911	1.21588080509116
-0.351498614600971	0.987079686587729
-0.307525771465881	0.771251985304511
-0.839093956481402	1.21165885730785
-0.492933960203672	0.600210694246267
-0.695603687327474	0.477530984780145
-1.30570495725594	0.898086193201422
-0.857113899816433	0.400272801007232
-0.338169137620666	0.728075934861134
-0.483517909579701	0.272178346284289
-0.480060665124628	0.441965826408107
-0.495420141643096	0.724353858809347
-1.05448478027763	0.675497836580537
-0.547748764638239	0.903041542972646
0.12420933746456	0.272287151430853
-0.326095851758336	0.532354459268938
-0.692702584789657	0.889914418057201
-0.761443263913635	0.401827337916715
-0.910766237378432	1.12512526889881
-0.688974493014803	0.88148918806938
-1.48050485230257	0.680980489124044
-0.931393819742789	0.441392510345489
-1.36732802079492	1.45414140294056
-0.936813855591059	0.0303882322413868
-0.741706116003921	0.741570178214254
-0.31591450551232	0.969261533312932
-0.718778898893534	0.81466745407056
-0.894177269157495	0.68877515274622
-0.773962875543262	1.10149231219722
-0.828714230971048	0.837836225130633
0.0418579168549325	0.487175583701225
-0.694304987995255	0.671404537386342
-0.766332876928756	0.431283502513719
-1.08567212718491	0.486653138288239
-0.332069009683346	0.732271981925286
-0.645373318257113	1.09015158871002
-1.36579988073262	0.284792869419794
-0.402979509834748	0.793042978096213
-0.639215872776107	0.807439794901577
-0.41626760845236	1.09232434242251
-0.113540806228183	0.99373853465709
-0.999013185339004	0.162862838282593
-0.547395886733827	-0.0137626998805797
-0.953587161998385	0.795561070487147
-0.599365906122674	0.624840014980164
-1.1767936049648	0.782446219454058
-0.399111444107321	1.61415235536845
-0.498306622943796	0.835675897009165
-0.451650483091215	0.0187016492476519
-0.5653937870071	0.155136191524315
-0.920654136937657	0.631512079059869
-0.800580588648461	0.787533413180334
-0.837599365530482	1.05366717551531
-0.814527959341312	1.1127898814714
-0.731824440828954	0.624548713712409
-1.22640704142561	0.712730965947059
-0.961016334368527	0.684552710193469
-0.662023842990044	0.181906182075532
-0.100372020544909	0.249765248375888
-0.576741208992864	0.999171308466428
-0.629926592730516	1.17537597850999
-0.868710563402399	0.474764632354517
-1.2442167928191	0.812381475691629
0.431000206599535	0.675236119591475
-0.547765764849884	0.435679964865671
-0.851866261372755	0.266061192813342
-0.777618124557766	0.532242962163422
-0.889077346414847	0.876217782159931
-0.752984725945834	0.670328310502687
-1.53528339066844	1.02570299820106
-0.864208112902364	0.848480147275959
-0.224714457530825	0.701528082301835
-0.370561743357468	-0.0639119671853607
-0.87750224992447	0.731534458959753
-0.72619346616371	0.787180976393242
-0.00277674358380751	0.451305480183987
-1.59861047422043	0.437583744565278
-0.697643320216837	0.99441131009292
-1.05352919030699	0.657550978181482
-0.81485394832813	1.15904697490734
-0.849337922210364	0.976708455262434
-0.313255057369323	1.7521450700729
-0.890518999349475	0.341151992285476
-0.236947093183575	0.603322793786044
-0.859094366102942	0.849971687802407
0.165265145303161	0.820752327971757
-0.402510281327235	0.910511238229434
-0.896287291979128	1.27004507827208
-1.48050485230257	0.680980489124044
-0.610451744328099	0.574454535654218
-1.02057437018353	0.934729885845891
-0.708972354728906	0.802432939579082
-0.942432790303875	0.519062911963797
0.00497704846293883	1.3525264072073
-1.14686305399325	0.574743618394978
-0.533448669137327	0.485500752698453
-1.12799728580936	0.802337683749614
-0.200816000101462	-0.163532546809695
-0.502808235845653	0.703789395849082
-1.05145839844641	0.621104510226242
-0.613351082828892	0.829255400985745
-0.509931798977362	0.57632741623335
-0.525236014451063	0.975251555625519
-0.52840510963426	0.826548172852998
-1.10966130257628	0.967173884659135
-1.05330186049395	0.461897856240987
-0.755051507457406	0.718919247073179
-0.89748062966454	0.6465375910593
-0.726640879439407	1.06627581020647
-0.633010266781395	0.727011590646794
-0.565098897758583	0.939274594811483
-0.547748764638234	0.903041542972646
-0.122820824262932	1.10716360818958
-0.686012741717172	0.834176331007827
-0.471916371827924	0.747826737147389
-0.642224401256647	0.430957898369608
-0.614605341457408	1.28618913624742
-1.14190604627943	0.535069288355592
-1.0766368423364	0.831245616760486
-0.800846194691983	0.763808544800199
-1.36732802079511	1.4541414029394
-1.26364633717246	0.835108686102398
-0.628949739863376	0.750902497004383
-0.312740744588101	1.72790654608768
-1.10913972705035	0.571972043995948
-0.965514066561375	1.1167234946318
-0.495420141643115	0.724353858809405
-1.26104976228392	0.64445761034287
-1.38153185947517	0.703078859843019
-0.584155418504096	0.699140823491817
-1.06070579150517	0.567824262434802
-0.908953414744068	0.784039390070098
-0.236924799025949	0.249241716411673
-0.845787903239295	0.460815234284296
-0.823587958216994	0.782371408565512
-0.506933451637747	0.463379872888599
-0.687826287236638	0.658218457269993
-0.326297418039909	0.838280783873308
-0.778012629501423	0.924503715225769
-0.751554546967529	0.558637637904607
0.0413820206446387	0.271495236517075
-0.941090445862036	0.72289281948181
0.211907016080368	0.671757885685622
-0.33259948320488	0.525323159023456
-0.615346336350302	1.51375866603538
-0.837451773752615	0.730965894143199
-0.61871550939993	1.07048834112137
-0.501640030340525	1.04013881832589
-0.0949898624431281	0.958666320650716
-0.700683279472247	0.495216958969694
-0.789943074340677	0.249157148320589
-1.370802197005	0.607084931932822
-0.531647669197083	1.16355411736332
-0.532384278202252	0.877395826700106
-1.18327736282585	1.02667785339947
-0.824683237417028	0.954536246804227
-0.54407162695916	1.15639503664642
-0.624086758434383	1.07089696190925
-0.350268898740615	1.00862442246333
-1.3077241464412	0.985055579945886
-0.603080479741182	0.143389512624126
-0.8800096572145	0.81886550196486
-0.342965355551839	0.20572721753249
-0.647410524782668	0.51217371711547
-1.00950958239526	0.972635620791891
-0.757926086193713	0.585499354670371
-0.767144839910296	0.676082022553798
-0.434903898150513	1.06796403757662
-0.117646935422394	0.331375183202406
-0.649808824285648	1.47186200656924
-0.494942123286578	0.561474835912646
-0.498306622943796	0.835675897009165
-0.581535537686555	1.19948363738817
-0.70977418174595	0.381239338017668
-0.574711933520536	0.597868519265982
-1.05447329039314	0.663193979996637
-0.481188719992131	1.24907285085843
-1.12299242117197	0.901122772134394
-0.808297835240153	1.19740947734092
-0.689317577005327	0.558452752429829
-1.30158930986428	0.735546770614485
-0.704813836394459	1.0862590993193
-0.615691513155678	0.863543798933161
-0.71967053419025	0.63238672565608
-0.858350218901591	0.873846610289234
-0.785893871771499	0.155148001629905
-0.24227970275463	0.245274308270876
0.108788657649299	-0.118312306744524
-0.922907125907247	0.657122289408857
-0.547794797213777	0.159120206901318
-1.25667816049198	0.44680137975581
-0.781187008458953	0.739579694101682
-1.10830179880306	0.15558753438874
-0.991532721949481	0.690614054873304
-0.673413080548023	0.941901140137073
-0.956524974878172	0.783732916286287
-0.691083119417191	0.895144795131824
-0.48392027292107	1.806867001387
-0.526751151009697	-0.235362244943399
-1.37695811423306	0.92330177955599
-0.104250568991277	1.19613971082229
-0.883114555927235	0.728680944547501
-0.483517909579701	0.272178346284289
-0.863916949065013	0.837921160525549
-0.532198648209716	0.512476018952082
-0.78950657658446	0.781918283916112
-1.1173662465953	0.663627481275299
-0.28269792662147	-0.199844734030215
-0.76746705269354	0.949286379618369
-0.94342512380889	1.29710938179057
-0.566960748668938	1.22949359325288
-0.991829446243122	1.16601887658781
-0.0719686139126837	1.04519377037478
-0.716545981685913	0.954011737105438
-0.562721834974803	0.82611606158583
-0.637930908856723	0.604540858788163
-0.565536081223235	0.0260284120012075
-0.539462762603453	0.727167055010422
-0.591222742259274	0.557906274779208
-0.979324141476961	0.567945356678697
-0.435535266747747	0.831985834848919
-1.07081379528528	0.955640821665184
-0.577584742412286	0.860272256142425
-0.204616342504291	1.0491087281925
-0.804942861012403	0.672134927918226
-0.926886051273278	0.448284355490662
-0.493715167880373	1.03233587573108
-0.585019523506652	0.194394414189118
-0.742823246547233	0.888048857322237
-0.768553520678765	0.585298814977906
-0.534209288450922	0.925976524575037
-0.629808102096296	1.21695699639372
-0.639749013812163	0.38674586272392
-0.563465431874282	0.695479348234223
-0.71321696298716	0.664281016386256
-0.668166685195491	0.355289166757711
-0.561832251516791	0.641675571375437
-0.492526587569582	0.724933360072867
-1.19710386885497	0.365714294926257
-0.238816993128792	0.272118818521797
-0.992848439934696	0.9163111598097
-0.511205411271858	0.932680960181552
0.000480707452730157	1.19069604063679
0.00890482120844495	0.987767197190854
-0.598111203831554	1.51139872765684
-0.065094213000755	0.917006367271539
-0.668772373336502	0.810654274344862
-0.821632850643084	0.755839390367772
-0.990849461784358	0.183042088240788
-0.697050689016767	1.14602715447949
-0.366155688575439	1.34411557873168
-0.596145126282982	1.0117244655659
-0.671578234081033	0.966427957410038
-0.875661809400185	0.909808845089209
-1.11724909103074	0.831612781242008
-1.12755340449167	0.61506814487329
-0.388727000708486	0.593738493930633
-0.41633208509207	1.74764747320867
-1.01259978114737	0.933235723004381
-0.00813244971862909	0.895466825872858
-0.383351204749707	0.534644845596259
-0.193704963953218	0.678435349605847
-0.33710684208359	1.06599076667482
-0.762856746570394	1.39541809389533
-0.920916679408268	1.08836034262664
-1.01644092562961	0.566402748782003
-0.489556317201759	-0.499556077594283
0.472617115604709	-0.881115742899234
0.0615115786979104	-1.37711925893551
0.130077997977557	-0.204661126207828
-0.0442213134319905	-0.94797794360239
0.463556786279977	-0.6842617028907
0.432047660161543	-0.688234837595142
0.495937463165843	-0.756691715978066
-0.211418829184382	-0.31940164600145
0.175532791560521	-0.288013551580477
-0.719906389371412	-0.17995602438191
-0.023172844828439	-0.668076103748659
0.276482465567105	-0.863762570871943
-0.18471040309634	-1.06372646913433
-0.336575277145817	-0.963421908526138
-0.605860596850374	-0.335547680891774
0.23897859189484	-0.627658977409226
-0.437152177739451	0.0247330498837994
0.22327135936933	-1.09777630237152
-0.263059509899817	0.403973616681398
-0.291960777244192	-0.602413861226128
-0.0883023845116232	-1.00897944082289
0.141516906795023	-0.348104519125989
0.294711199762609	-1.17579350425504
-0.0580444392875057	-0.418280583240154
-0.507238560235461	-0.100176752893309
-0.135975301341437	-1.26651819267251
-0.299617759949901	0.0377572473876753
-0.401708700419367	-1.07460650935141
-0.438684430352106	-0.228841641575556
-0.44307480007623	-0.129932950586054
-0.271544278733751	-0.326779598900964
-0.0813829541357717	-0.229838274102312
0.632398305174208	-0.832857874308524
0.331278256688443	-0.776950714104693
0.275850314832016	-0.788638527427312
-0.0556892446741733	-0.731106352797098
-0.0711705906766802	-0.997464478211254
-0.0809207622590715	-0.644360536486071
-0.149463595285192	-1.29046166296828
0.172202869504727	-0.108612938613064
0.0695860433163577	-0.529653725361916
0.411677092430152	-0.861682775765965
-0.287287870350432	-1.15216628043364
0.610124463477599	-1.37432215291657
-0.527549166064294	0.392386334285832
0.278611936325343	-0.856678425156055
-0.25439149217038	-0.794039432029173
0.204732661504844	-0.782646109080029
0.158829943469708	-0.855749651186151
0.328165659963271	-1.04563438883945
-0.238966903873284	-0.165889493132413
-0.218314152814324	-0.40939154011306
-0.172128101582652	-0.389484307587001
-0.352986447438035	-0.526431940082695
0.00833581920643241	-0.730573236481975
-0.129893431837337	-0.866893495973347
0.362623724952197	-1.0116696229671
0.0548193584773312	-0.107070820991833
-0.302492054967488	-0.757431062389203
0.00156157801529907	-0.375605621364997
0.262384254652056	-0.671570648093619
0.376490169664349	-0.614394893215679
0.453040473106696	-0.26282489253975
-0.218650921485906	0.00572580386947075
-0.245430913133068	0.436529407499921
0.277668266116176	-0.750387994427309
0.525512508915971	-1.37092789126919
0.27651709858848	-1.12164664698871
0.345682483253896	-1.00917781268779
0.461815696119879	-1.19019958555001
-0.326676387828247	-0.215731286587581
-0.28995292697063	-0.399909742063027
0.0685264613561814	-0.52132858207862
0.452567927884395	-0.728795858763294
-0.0204767771631602	-0.457276887946389
-0.332992220811443	-0.80856669261174
-0.130836528174154	-0.74791339496565
0.547243484934298	-1.13766065351075
-0.286210208534285	-0.667753269779722
0.377294682177926	-0.309863228885587
-0.0809207622590718	-0.644360536486072
0.333516846150276	-0.75957461785613
0.207523223169462	0.37306106923345
-0.127102181495589	-0.772715737343127
0.190011643754225	-0.83765171146758
-0.377304720975765	-0.388809979285489
0.0685264613561809	-0.52132858207862
0.547293256058284	-0.591994615038959
0.0425960628872739	-0.984870314139006
0.677371317524052	-0.932624429004052
0.254883149080354	-0.722760312060244
0.0268970538022701	-1.30662647166366
0.0264671357133757	-1.04055759432862
0.283123941878072	-1.2070071994902
0.417062055913198	-1.52751241911321
0.0773010072391951	-0.832344787928029
0.524228141117639	-0.759155717731997
0.51892629482504	-1.38937730658352
0.0992483675646679	-0.666209052846271
0.350486253828444	-1.16600568429755
0.762656246568519	-1.02581978783376
-0.423287653615319	-0.528679545857853
0.0507367166425225	-0.919598040213389
-0.0750451128255039	-0.869646817419895
0.35240581846186	-0.962974230046295
-0.0684670535907964	-0.777832040578624
-0.218200160736938	-0.119289610787514
0.0990003249234716	-0.71225570536131
-0.188046847414256	-0.566065187290861
0.0950649863762761	-0.649737022719638
0.216732457742161	-0.610392861202481
-0.336301590906324	-0.335964896818097
0.252415563035492	0.568468137918737
-0.160056617954154	-0.819580505755293
0.595403754260589	-1.1286343565793
-0.0389340729792892	-0.638974177749962
0.00112463400407418	-0.907393990255859
-0.463896526049385	-0.636675870215536
0.152012142105273	-0.769642762140937
0.873988819451736	-0.184894907430315
-0.142883889044051	-0.970374393969378
-0.343319287300775	-0.760625009979543
0.250508983900882	-0.781739439378839
0.367644470595643	-1.03237839759231
-0.00409055016581796	-0.705801570710971
-0.423287653615314	-0.528679545857853
0.647898810897074	-1.32705900867539
0.213978208472116	-0.69241370361005
-0.402875547101285	-0.455033507706603
-0.160437786332733	-0.22327558068589
0.811728470203766	-1.09490283307443
0.272919578707872	-0.681560224140605
-0.0244501261949666	-0.90653611841251
0.191080699927515	-0.526101105303745
-0.327743951454436	-0.638837330274054
-0.4978151032608	-0.473375878862374
-0.0408161014741339	-0.693286161951341
0.775135820633365	-1.33373725386036
-0.0402819330648257	-0.512699191292094
-0.30451697782351	-0.428572438130145
0.20066449395802	-0.854601880859561
-0.263100522259652	-0.174994105232773
0.124834681981466	-0.78559224095352
0.272919578707872	-0.681560224140604
0.767201264751768	-1.1382356235227
-0.124611767646828	-0.895696205486524
-0.511640748590304	-0.466650692816922
-0.100689050923127	-0.441139693124086
0.0477868669602417	-0.465414645320068
0.268325232788673	-1.08038893304653
-0.749449040835338	0.28702521007148
0.99185233313212	-0.88072257433484
-0.212163339978489	-0.452507369762135
-0.749853612998418	0.125417071324218
0.271684687299098	-0.805905998323242
-0.381760774586097	-0.765880486176253
-0.0408161014741339	-0.693286161951342
-0.223789369465774	-1.02795777889524
-0.414682112699161	-0.531507581071355
0.239893645079798	-0.491152511473159
0.362984022774534	-1.15064137119484
-0.204317926748617	0.15160826770941
-0.0586053023415199	-0.990673409570974
0.0248748427172914	-0.94264281415094
-0.452860758912504	-0.27629331086948
0.122730126933276	-1.07627393193293
-0.463481141029476	-0.0638355591609186
-0.204120476030901	-0.73006955463748
-0.382032597181239	-0.306406053591621
-0.255332080931605	0.537308783088223
0.723306816041362	-0.896985505361784
-0.459817380781053	-0.344013848970237
0.441054115164376	-0.425283475465989
0.178608139485085	-0.676458114066633
0.50841548181818	-1.20289121005794
0.0277406367382042	-0.876040917167507
-0.164405179586559	0.29399414593388
-0.557129773522999	-0.151769750290227
-0.642868487076537	-0.466089285102787
-0.513666926857005	0.510162672301267
0.238661860567029	-0.281370957682227
0.0301833510320405	-0.452016637492022
0.470958228494423	-1.1144706430485
0.0614917336540292	-0.484249478183322
-0.135101609740136	-0.388959374336423
-0.302509714399672	-0.164329038220702
0.214284878454162	-0.910681819731677
0.449069002144245	-0.534449152390662
0.0741117312049742	-0.937025749037466
0.268226512010571	-0.486474254349055
-0.233075736454339	-0.967175816090944
-0.239730567148708	0.0795038272127352
-0.701238749792992	0.208646364498014
-0.432769401062368	-0.256023559530902
-0.0770203209432427	-0.637166523570323
-0.0236056350003484	-0.264259034282876
-0.37073634211534	-0.91610801276325
-0.969015324203319	0.21315496201378
-0.476244719167522	-0.457976667837827
0.555618558252794	-0.790406978224584
-0.247344661716541	0.622313284465802
-0.528229658712795	-0.417420704087124
-0.118195435684266	0.409286241681983
-0.565209055196218	0.177114291951077
-0.0638083234862756	-1.23516975877398
-0.461463762426917	-0.090078899169671
0.0957762939314323	-0.464582399233168
-0.572119800483025	-0.385702933365245
-0.0336597753855474	-0.413969547939537
-0.852102178026396	-0.126546764605418
0.162571365842927	-0.629087001013344
-0.035490732177122	0.223553756787264
-0.0538394417004588	-0.782100754130968
-0.528401555086918	-0.374055966695483
-0.557129773522999	-0.151769750290227
-0.171202335716166	0.183095439900049
-0.610574630613588	-0.283491848614526
0.250097371697585	-0.0491416533220268
-0.1196846494874	-0.283515820015784
-0.747333127521677	-0.324700876023637
0.0222700130950116	-0.487511609288638
0.300492747844152	-0.576682508998831
0.188603422032361	-0.84647230080781
-0.414328521097629	-0.889337735888524
-0.519428429858177	-0.109744809111309
0.115048030246561	-0.352976060589724
-0.0271998708213829	-1.07591568259655
0.20066449395802	-0.854601880859561
-0.498189974190349	-0.180528249599296
0.00535315629581497	0.0235959990582344
0.638369847287147	-1.53618788038609
-0.0776414541700493	-0.815772629287247
-0.0269208402290093	-0.563462870590991
-0.0750451128255047	-0.869646817419895
0.480667999269453	-1.03576164119267
-0.440114021693915	-1.18799771002887
0.552646427678942	-0.66461564888414
0.308770441513295	-0.671890938427024
0.0466512405221038	-0.231865994774278
0.312439180743843	-1.20924266971991
0.0450018892688846	-1.2880482892658
0.563877790807618	-1.12814809308929
0.00067894191253123	-0.845078205624266
0.0474730975770576	-0.945826120018909
0.0156670270336116	-0.705407678721256
-0.738997258659492	0.158547644432828
-0.626527184288773	-0.366245359399666
0.0798402844306399	-1.03880221865228
0.362984022774533	-1.15064137119484
0.794902519648254	-1.39125045100764
-0.160995712989039	-0.718516226129585
0.370277064956127	-0.813134135186532
0.870314367309788	-0.439599312397904
0.412030339851616	-1.2160599788806
0.266846731805829	-0.927594265754938
-0.188816609977773	0.0429344455750326
-0.0781896224960743	-0.568275987361994
0.234900429571078	-0.909909595201695
0.0584161488793777	-1.20576623529441
0.213978208472114	-0.692413703610054
0.27019168288306	-0.61597128886119
-0.00409843656230085	-1.23354340979663
0.424430803348771	-1.55872357684071
0.566722602601091	-1.47685397925822
-0.579042398568952	-0.487256621780225
0.108282767827142	-0.308134361899357
0.223309835764181	-1.11131603940912
-0.113283325011024	-0.735629532643849
-0.673009226183682	-0.545921082991108
-0.621091918356489	-0.119781267975564
0.0615115786979106	-1.37711925893551
0.141516906794315	-0.348104519124202
0.187087420188214	-0.418594553878751
0.321824829178312	-1.33924636692889
-0.0191032302726117	-1.44010613298837
-0.0683720674613509	-0.463127670145488
-0.283419669378893	-0.45327703788744
-0.239354396549677	-1.15527212964363
-0.0883355414795284	-0.517086081490959
-0.114764779772191	-0.447054499453782
-0.895200572847224	-0.506620478031108
0.218312184705937	-1.05788703848583
-0.0385628697727748	-0.643601444939574
0.032219960008369	-1.03132306048276
0.48883629201617	-1.22438293499868
0.298225285042811	-0.844419890183931
-0.447440704293161	0.0385214279274907
-0.142200709725248	-0.315484666656077
-0.22579459800172	-0.977589477591901
0.162763519452463	-1.42057883630635
0.209734822039041	-0.741888385305318
0.242777686042557	-0.684834924090911
0.270035396711908	-1.18798820942872
0.0739605087588414	-1.01598131668132
-0.256742297839895	-0.936893319875746
-0.0194647203485537	-0.68004556084777
-0.610191428676009	-0.179836002035315
-0.29842707483276	-1.01962119028253
0.0970999775475429	-1.15433079853667
0.0778204830722853	-0.583253957088446
0.0459256234706917	-1.17757667348752
-0.0727613438422238	-0.70257696647256
-0.216775333534896	-0.585788013110374
-0.221228303654619	-0.677904751955148
0.280412625934482	-0.843450766805529
-0.199707397019137	-0.911527798424455
0.525512508915971	-1.37092789126919
0.0513498048421512	-1.24578356731623
-0.077217517430793	-0.373772700525415
-0.0741875159902467	-0.56640489710366
-0.184110244620153	-0.442146229086435
-0.130699624296614	-0.615006295075365
0.111671752889731	-0.56893931132875
-0.202501520853917	-0.735213382908006
0.0459013565604582	-0.483704788857662
0.209914649436274	-0.793024679765687
0.0583766508648656	-1.13047166219333
-1.16406907891128	0.0347993555980695
-0.319583285300639	0.485121608253269
-0.62781647692929	0.115690232289956
0.173377652677124	-0.999655665084309
0.878786302042409	-0.182140397533382
0.325830741351311	-0.705658130442416
0.818071608709931	-1.48494137806595
0.0248341256496736	-0.149963894778507
-0.00487397181189726	-0.64262193820544
0.190011643754225	-0.83765171146758
-0.23235394221571	-0.994644981245527
0.0626005422585967	-0.948939360012939
0.547243484934295	-1.13766065351075
0.691490126388953	-1.35573934878263
-0.231976764913975	-0.590069343360344
0.417062055913198	-1.52751241911321
0.204732661504844	-0.782646109080028
0.294029433604455	-0.490248274867056
0.386832726324953	-1.27097533713997
-0.527549166064295	0.392386334285832
0.0563226406456936	-1.32592548319421
0.0809505734325216	-0.326014142847568
0.762656246568524	-1.02581978783375
0.511296010848627	-1.36709278184294
-0.736433017223761	-0.685667820634582
0.220990927715352	-1.12945298535603
-0.3287450784197	-0.6422982991236
0.0386012610333243	-0.753531997666001
0.14066713486655	-0.466019905360283
0.386870032883211	-1.01902760452623
0.133933308178403	-0.328307790233598
-0.289952926970629	-0.39990974206303
0.0437992966902474	-0.516750891655663
0.702759234364818	-0.832165047479619
-0.541588455188616	-0.246251182330584
0.0721811641207855	-0.802087344712579
0.228891020059059	-1.34318382471792
0.411677092430152	-0.861682775765965
0.406060887158142	-0.776844058476932
-0.027758057323874	-0.752762401805093
-0.142174999146652	-0.522090240034533
0.340120491367178	-0.89190065210099
0.107430821036898	-1.01505919868585
-0.208677996417061	-1.15961566331348
0.185377972399093	0.550827045137112
-0.0305614884984498	-0.438011437051657
-0.0377539633089503	-0.67073028626501
-0.0910478146393652	-0.247478107353869
0.088204258772451	-0.711373646077334
0.34371465546178	-0.669482013093015
0.275761696296726	-0.471577361637016
0.26749138336436	-0.664752829520677
0.2120460830401	-0.247028684697247
0.205516153373506	-0.528606689590805
0.332513406437451	-0.561612232273985
0.564029000975403	-1.29763094963625
-0.0678384414452473	-0.262961989657
0.0959611308456339	-0.171961103798405
0.519417782482594	0.27102310717888
0.322898862751694	-1.09264196705748
0.0525733916295307	-0.312960363200547
0.0546552176256636	-0.554258342365838
0.000164407957098535	-0.502915991602172
-0.15270516001186	-1.0584744192919
0.148102476731276	0.172994560882975
-0.147747078723691	-0.682950359248861
-0.0457633234113948	-0.397561815716457
-0.304929262034402	-0.95223334097856
-0.610009106645918	0.71768765706124
-0.114675185253892	0.895859750809641
-0.407223357481224	0.706737810513433
0.0207251614391482	0.417558623236012
-0.371417673715734	0.688716831666164
-0.779698801040095	0.736620931722357
-0.0485689028274633	0.295063725038933
-0.520904510303342	-0.205093427123091
-0.0340383355199826	0.12332799054233
-0.274623242577303	0.488110537330123
0.0720488235648187	0.614598093653754
-0.149899726989182	0.446213444597344
-0.108485985928446	0.9013389662473
-0.0617479248078368	0.870039153381451
-0.195343746532206	0.357185719105212
-0.437436836764407	0.392050066815167
-0.947941459162955	0.6851673828819
-0.453428854985463	0.395774977191099
-0.447615871426299	0.330878072958254
-0.108179860702746	0.826328501072199
-0.191714570927164	0.975262404350371
-0.0350745825585366	0.457077071926283
-0.344408834686721	0.578088979590434
-0.246445175786471	-0.0179572813461022
-0.330110040784211	0.752567510255697
0.182746842794921	0.225270944870869
-0.250005215687203	0.264764458199192
-0.0905419183064845	0.798219770261414
-0.418065492506143	0.77831998699742
-0.315383510143721	0.241395681829087
-0.180904214493916	0.940814501622561
-0.0329742202072033	1.14366708763218
-0.25097047032514	1.39198801069911
-0.115485516950847	0.133231444341073
-0.0315597758360891	0.386765245415073
-0.0591041240596724	0.988092978074783
-0.0652255524772856	0.979791578076262
-0.244205167027997	0.727649161671536
-0.252200452246332	0.1356340100353
-0.253397280644067	0.789024465472545
-0.116561419893169	1.01151059423411
-0.187427126651144	1.1915983303452
-0.187461466125536	1.07391039684423
-0.512375772289753	0.569236067639766
-0.264354275819285	0.906946364248539
0.459026962899349	0.238112527076179
-0.417850195250813	0.119845567213298
-0.528518962561438	0.134044507897658
-0.221820918564717	0.156245083336594
-0.460533091222986	-0.036442218007903
-0.523512050883435	-0.125125460796578
-0.150778858179786	0.779775250471756
-0.0982831621467875	-0.0291858673045287
-0.179288998782393	0.897951081647765
-0.372557198736615	0.223760316854237
-0.272262110466267	0.0951173843831199
-0.272256055274929	0.429860872584463
-0.334003623609245	0.485091552570633
-0.386380281731987	1.06240243043002
-0.429797694290784	0.72913108367447
-0.0743022521441691	1.71858759878947
0.208547954183833	1.24363724163335
0.387575674371908	0.628351913487625
-0.310945019655204	0.347712610426668
-0.315932627575987	-0.0374320316165369
-0.309432788236329	0.963639729208901
-0.355855441328981	0.631096643190849
0.00162496803633547	1.29410632167285
-0.186915584944609	0.932203348737622
-0.236197684891385	1.03032982742362
-0.0115600124069744	1.14999004059017
-0.336036387298797	0.562890610837061
-0.707114731147924	-0.126133141217996
0.138118282959262	0.742430074156083
0.113965261145076	0.448160609364017
-0.806851185176573	0.731204741350647
-0.479591126027097	0.420651390012248
-0.455616678005827	0.398337242678135
-0.382735533240846	0.532617642960931
-0.555046388206528	0.102354643292473
-0.245548871395073	0.211132116535345
0.0689941037309371	0.984065893441794
-0.120255529923622	1.05518817314185
-0.41059816560024	1.28363740979242
-0.752722737558235	0.445741505498372
-0.428895037844039	-0.238798269512863
-0.107743813515367	0.540749003242002
-0.573441006803089	0.56919162781161
-0.352250270208903	1.05244538266433
-0.142492075065457	0.924004658304787
-0.470181605855611	0.278511385304635
-0.249253161826655	0.071152392940407
0.129076973493323	0.449829728846855
-0.310375764058474	0.0931669284855806
0.049315991933741	0.397456238477836
-0.00287100638578121	0.538697958324567
-0.221458787488784	1.08731458975192
-0.474028536445796	0.65669128340692
0.0790843116695424	0.585718767195323
-0.0476121347081127	0.582464446193439
-0.536993727806164	1.36113036848983
0.760903619531662	0.293986800682274
-0.894881161328412	0.345007102628735
-0.140727402952868	0.2878014676559
-0.0859109733458261	0.598812071103908
-0.282138300591268	0.504699437367431
-0.423931793823982	0.0922339398909735
-0.503143339946786	1.18475153405874
-0.833976838237385	0.779461397325326
-0.165923603117987	-0.190876946924654
-0.10884487151005	0.27306046139176
-0.111309256514866	0.600270555119117
-0.368018057255548	-0.168597531822119
-0.0913497498501372	1.36184060458373
-0.390711869140579	0.0586432896914551
-0.312930079759433	0.746097907831753
-0.0318135774480833	0.449626729340142
-0.075799222122062	0.842424628718681
0.0740549941662438	0.674957094582604
-0.106271759943912	0.273996469384087
0.165733636368616	0.327325966461711
-0.0868639774634951	0.835265420827887
-0.258542662937628	0.136993025404806
0.0608168336222533	0.747731123350512
-0.187404504286065	1.2634160615505
0.105143553524987	0.805563146317572
0.137041296109095	0.31882990043573
-0.839216409602017	0.484721605235422
-0.387517218699584	-0.0397328494535347
-0.312060175370125	0.753108867722787
-0.0779223781296515	1.25103794790844
-0.61186230920802	0.354834915862403
-0.104919946043249	0.467326014229347
-0.395800546313536	-0.118914299222301
-0.158588757534597	0.266524606661311
-0.437919847315761	0.626141736982329
-0.130700693541066	0.304420844042627
-0.246757735419806	1.27974961982958
-0.235569510016924	0.263328898935229
-0.402908964206613	-0.112192225334979
-0.2372952122138	0.223478132370059
-0.18398790567742	0.866344764256405
-0.177474656071992	0.249160449615287
-0.395449254299328	-0.477798128387646
-0.0777403596999156	1.001392123386
-0.213866957533059	1.15733106697764
-0.519863189563919	0.280338642916839
-0.512238097997731	0.888492064119624
-0.106557124063798	0.935563814414393
0.0756934102687724	0.446591556574049
-0.203080336507063	0.389479191699598
-0.241644068762049	0.403438889616999
-0.253985295503119	1.03712375064695
-0.00618864844103354	0.502816391150415
-0.321998182090926	-0.173714038711506
0.465670576460524	0.376294727608182
-0.677375805806848	-0.239003901957268
-0.472981831033131	0.357701161817457
-0.0330780533107769	0.829320254685702
-0.0703380321782613	1.80768799561588
-0.36566367060265	0.548422954815679
-0.0977405417808809	0.924172676826864
-0.161152917405488	0.587909139038438
-0.16456558943812	0.150679221516637
-0.190961904960866	0.376058967694853
-0.226763729339008	-0.375564431950109
-0.173535306240825	0.742030122741396
-0.0393874867583199	1.23551108305533
-0.360521327869061	0.366668021891327
-0.0990737870035831	0.163316220422333
0.0672334552008021	0.191559915430689
-0.145806459098169	1.54163588814706
-0.399789273622142	0.697897022582974
-0.509381755090666	0.235689163417132
-0.388344726244579	1.04485392807047
-0.386538242336974	0.312516655337205
-0.304636925992928	0.0769142557065067
-0.38447775681181	1.98956624907058
0.0631337547568275	0.177794842003018
-0.312794588521294	0.731164824361805
-0.0168667110565362	-0.438435199064803
-0.201291262232648	0.895619583667383
0.0777388863713404	0.0959139840596229
-0.132875165430162	0.510749491247033
0.12596359411674	1.53281217970048
0.336806554211012	0.583639815328848
-0.474545682645299	0.0347147642354852
-0.00514741649137589	1.37422563395109
-0.029647154541579	-0.105487627229786
-0.0740504894921136	-0.218749651085591
-0.409857804915188	0.495112027929435
0.123856639833954	0.528958681898675
-0.121990346029667	0.609597817267183
-0.115789174491311	0.594881764370753
-0.124065250113559	0.0118956173940569
-0.0950324009406745	-0.169366049272924
-0.337695707852614	0.028891337362668
-0.361680960285554	0.93944598236251
-0.545574565593336	0.0736939859992172
-0.0415920630668803	0.721912117250543
0.124417024170569	0.590849905509598
-0.0646850920251012	0.325034825077148
-0.37430309044621	0.42461249982375
-0.250719492272287	0.858701025332214
-0.340665642486318	0.165344510596135
-0.0113091219646768	0.151105778518119
0.0276369039068362	0.651186472520711
-0.591378777893529	-0.0685299729711285
0.158334614026098	0.421367178776874
0.151645735561279	0.900330369438658
-0.0265631243207531	0.615022014210231
0.0168023588324084	1.10350776198948
-0.0296887812419563	0.959052764398481
-0.440858738659966	0.546338087917102
-0.321203757438211	0.574188268433021
-0.293414547462675	0.757693255069706
-0.0527662104958051	0.962857885312185
-0.104146888869014	0.634673550484447
-0.154943864286206	0.222848143587153
-0.0748211629621518	0.462276157940138
-0.211848734536782	1.28594422462841
0.0827611047693995	1.29408262502969
0.225947916322407	0.737207987538008
-0.0517210847108688	0.942414303991092
0.00772239547521779	0.36704659131352
-0.29932777019805	0.721181097449399
-0.101135217422921	0.692791725338468
-0.153803040948287	0.016741816499776
0.0842756219348791	0.88255469378436
-0.541254697461088	0.145515244046363
-0.285212412817831	0.906979172937759
0.0164704333340167	0.968194870481393
-0.0626710732185993	0.794889828503376
-0.129101573236543	0.320522535327183
0.174577710352234	1.27150798192198
-0.204502081491692	0.367949764710579
-0.287344060754032	0.401382103256347
-0.0260602383874559	1.91802283693415
-0.10152336569334	1.1036220151292
-0.271153528032602	-0.0285636676159744
-0.397443054620015	0.68543949486245
-0.153611817353283	0.842597657160553
-0.194441951537563	0.657144768203047
-0.0938260413262857	0.88457874810347
0.0946930912033273	1.41193795236236
-0.167902204658421	0.75471932508387
-0.0821711592071814	0.600530461382416
-0.318043892610687	0.143319881375226
0.165607404998755	0.799626779073542
-0.0194254271806334	1.1942620495325
0.0637767187290886	0.180791289186927
-0.0132080494613747	0.338823399611122
-0.104146888869014	0.634673550484448
0.098979513025518	-0.303595885956717
-0.263426475663795	0.763287592773267
-0.00708040332787164	0.540520544055598
0.0640362366701049	0.10416053756655
-0.179209977932835	0.619391446497909
0.188322387170593	-0.492212017518944
-0.0481999453914922	0.677991310120038
-0.323735090185106	0.948741185501115
-0.00637118232537194	0.294706876480212
-0.384752032090221	0.412780255355996
0.282495929545889	1.01053603437115
-0.299327770198052	0.721181097449401
-0.227755016876405	0.59235237072502
-0.0947014282946547	-0.488365734455782
-0.0404699782910494	0.693422063910613
-0.626141421113477	0.600178146077356
-0.315273671320292	0.777187395542811
-0.18615593380699	1.03804770063627
-0.124260123812258	1.08575704589035
0.155412076655993	0.495576853370394
-0.421114259099658	-0.0930969684077129
-0.477013678634423	0.522135919735711
-0.386982276853181	0.173949750583894
-0.406547882984959	0.66330915796796
-0.195261156191278	0.311721366855989
-0.187871669793923	1.60114723532494
-0.0903160881522444	0.172762138775083
-0.33760351129983	0.495315961971056
-0.345206207484424	0.440557638574037
-0.056632158907382	1.06122388415656
-0.0655581543767333	0.722419626249356
-0.120255529923527	1.05518817314101
-0.153212142601052	1.41494336375113
-0.0318135774480825	0.449626729340144
-0.00204537563332654	0.808323400105571
-0.137775889332294	-0.0702281919594669
-0.255825855202078	0.816958482061123
-0.503143339946787	1.18475153405873
0.0215763090426057	0.436181739410447
0.00494762393593303	0.707105873441205
-0.125036829536922	1.54191344131144
-0.094800101047497	0.342051818397134
-0.468713877857524	0.229155690392719
-0.0342985454609901	-0.170059822202239
-0.386380281731981	1.06240243043
-0.299836806851115	0.449951503015071
-0.2372952122138	0.223478132370059
-0.0932581636160825	0.703402873315189
0.217131636886997	0.666679747788756
-0.218173500119245	0.671177910725154
0.235363636182511	1.6300418912407
-0.0350605922028876	0.570239521340505
-0.132667149589357	0.557266855657971
-0.165453103051622	0.626179681231364
0.323558993198629	0.283989410210814
-0.264848426626713	1.04933146680721
-0.258542662937629	0.136993025404807
-0.18691558494461	0.932203348737621
-0.257693370583581	1.16673356907806
-0.187461466125538	1.07391039684424
0.258696816990861	1.66249078361944
-0.00730120598898131	1.12033187409093
0.122578426231409	0.298263765317369
-0.423564633767319	1.37201885581748
-0.0183810455538925	0.252241013355201
-0.0509543809569758	0.254246171871201
-0.535473296836167	0.894067227397714
-0.052766210495782	0.962857885313108
-0.32047984476301	0.974997111544359
-0.227622679219524	0.697386329240255
0.016812229966228	0.655543468919937
0.251561325447681	1.20159653230687
0.202117760803627	1.23731516791985
-0.132244336894404	0.928900701612363
-0.186007872006602	0.405390417949191
0.067570790217249	0.879758524707353
0.201248609288561	0.402221022122607
-0.154475536022159	0.228399968628757
-0.293915607556102	0.519792838284712
-0.0643263414495851	0.605732569775891
0.230477697520453	2.0709629942313
-0.0709969611188894	0.882615754710866
-0.339732194669982	0.85935627232731
-0.573441006803089	0.569191627811614
-0.722681737473887	0.68973060618628
-0.37301770427843	1.11548832633182
-0.227975729843995	0.695490463645028
0.225947916322408	0.737207987538009
-0.423931793823984	0.0922339398909736
-0.142239537433725	0.474180573872901
-0.542044309874966	0.319784432101017
-0.227776541559813	0.133981391754842
-0.570068505400124	0.232307401923573
-0.0192282424884177	1.68507340604301
0.175335317650361	1.82381401703284
-0.382208891201968	1.07863062419746
-0.255283877422878	0.817544563139154
-0.0485689028274633	0.295063725038933
-0.612523042294942	0.26591738748047
-0.360784706873365	0.082735365435018
-0.479591126027097	0.420651390012248
-0.241755765809869	0.685216269535672
-0.191714570927164	0.975262404350371
-0.507775878754694	1.3755592390831
-0.392360779330335	0.877503173843665
-0.371417673715734	0.688716831666164
0.0747765708379345	1.12707603996339
-0.480902689392039	0.805950552714133
-0.456190825419516	0.658493109490772
-0.495258318540447	0.239326927280498
-0.241289984746431	0.635177910641227
-0.486250046114208	0.563273890080088
-0.926068311076593	0.445987402315728
-0.165923603117988	-0.190876946924652
0.124815180696938	1.45780109203845
-0.276699810088822	1.13916427870733
0.0475434432603506	0.686651002825675
0.256389243377736	0.887367488023105
-0.0391573409955168	1.28789842561712
-0.330779924389235	0.657044393048783
-0.115684847726855	0.803579815143014
-0.846601323832962	0.36987783539624
-0.230254241668264	0.75134513294463
-0.595098008773661	-0.0171850463997604
-0.299215850391796	0.736892201151517
-0.0783689367056958	1.23469004697771
-0.496161954524709	0.379429004306399
-0.503301030223599	0.464813391349595
-0.0548220072149306	0.959408214470913
0.0755826821926116	1.09954823030922
-0.228333797068663	0.983279353700799
-0.363253027978689	0.437361734395438
-0.280732170412057	0.542320580005012
-0.450627655176062	0.727657189393714
-0.428042187181996	0.145306242118241
-0.29577783192375	0.227680094742952
-0.596314914802291	0.321177755493323
-0.273571602185105	1.33854696853376
-0.0916518841180881	-0.0761696947526155
-0.209187694502764	0.591962643120975
-0.548674496516725	0.783208128669135
0.399212804388803	1.0144651175592
-0.435597275616743	0.638594660870964
-0.242015981687349	0.680591132144114
0.111130048480339	0.736651658253689
-0.10867684253592	0.862196339199349
-0.0778109121251972	0.0554386917844873
0.158121278122449	0.79488117438485
-0.268837137329017	0.771695982152357
0.193431647897324	1.78996990770312
-0.00875088151777656	1.31566874413098
0.160718738719218	1.39951729215672
-0.0401250286294911	-0.0985739488984285
0.0577119170468192	1.38872564117413
-0.0758245966394754	0.25105309835163
0.193872329231817	0.710288249713291
0.130971832713346	0.325430710778901
-0.213730332265359	0.141059953482511
0.145558683290139	0.901610914877515
-0.303291935666522	1.30111520849128
0.114619150250286	1.05401331921405
-0.14212046562502	1.06172264649488
0.313132608843706	0.641612295034521
-0.0610384801721914	0.991130350546312
0.744980823250648	-0.551143537320491
0.228146240814191	0.0420554929490722
0.764771769937289	-0.473422836512563
-0.177372115126784	-0.127966574970149
0.640260361195728	-0.707044725944454
0.474742435180957	-0.570464609689126
0.441151588231646	-0.180684844857735
0.182225345858313	-0.110437044491203
0.0855386278397714	-0.266976727074511
0.785515143523145	-0.0360613443660372
0.0724666663940382	0.0361108127412772
0.0486164648221131	-0.301217136355509
0.0313438249794078	-0.0479536394917187
0.534115874631501	-0.340496224075528
-0.203215532099494	-0.165408240421708
0.164541521758705	-0.0161779240543036
-0.256902318229728	0.0899831198043699
0.909942576577458	-0.61191528350339
0.660900275433577	-0.262654083990295
0.0486164648221127	-0.301217136355509
0.403007704417722	-0.345285653698494
0.494930397274076	-0.280582316450041
0.0358126135720016	0.478251552976343
-0.081628809119715	0.340833829494355
0.390620973386176	0.214136854934035
0.0568389175136805	0.132539637583648
0.152284243786183	-0.113742015248592
0.0653793049056341	-0.267190625301661
0.146959619769586	-0.0376860953544093
0.927776246709567	-0.239521116649085
0.0908429099113653	-0.387534303856966
0.334002127515252	-0.126132446481224
0.257113930747406	-0.200754142601743
-0.0652279274278219	0.175532958467429
0.115334053046001	0.33084218573651
-0.131796913788976	0.476233628488396
0.471335589786302	-0.441483654811896
0.564724330647411	-0.827043397316271
0.315948798245716	0.500280249990273
0.374984671709955	-0.12198266508558
0.90925551469034	-0.743952231476095
0.824934073842737	-0.692945485099267
0.119347778105833	0.21135784222031
-0.138299450093568	-0.205622922796451
-0.570449899578353	0.739943145487211
-0.312541519631273	0.123398470955158
-0.276589972321688	-0.0834460103682654
0.285743135119673	0.98675025333213
0.404305838728295	0.520125210350992
-0.0347622814420333	0.813332373597537
0.288771597616865	-0.109387796797549
0.607602095046897	-0.360176418877634
0.896535225554751	-0.275761875730737
-0.302110994054041	0.284906756700311
0.627465855356079	-0.583280207331291
0.590084641940361	-0.321171353132467
0.213184151154054	-0.174658274391772
0.337995605007447	-0.309987621552601
0.395357671748359	-0.181083190445329
1.4045361894217	-0.710496247733665
0.252119086067659	0.0795490844754399
1.11110476769577	-0.422870062668386
0.236386077997749	-0.353128679088179
0.774360135546649	-0.241776295023108
0.615319616068817	-0.431333037664395
0.175283243504775	-0.263681643189934
0.0924566114134211	-0.478223409813654
0.089348029420943	-0.0611954380294445
0.843475519146088	-0.318195357995537
0.255352387139197	-0.304328637687853
-0.12730696624295	-0.14878361450129
0.262870994975491	0.56179389829974
0.365746328812143	-0.156596568890393
0.178386546067199	0.111822886852793
0.516981851923014	-0.630728795022413
0.327043574889886	-0.228302798438481
-0.0319242712487637	-0.290811484301015
0.661810724440125	-0.235795540341848
0.169341687612039	1.25894702430436
0.601312766150308	-0.499685056668846
0.498650741927874	0.849214762607369
0.140570634254445	0.299982062390518
0.313516762162743	-0.280684118931145
0.607602095047204	-0.360176418878094
0.134068363675226	-0.165248241427096
0.620554844508283	-0.405010241964168
0.402438385269869	-0.206710096481862
0.404305838728291	0.520125210350992
0.0356377885575243	0.269158066960929
0.578434332096648	-0.372033664768726
-0.605849558490718	0.675478418831024
0.411906160119158	0.0793229720617157
0.426701959197549	-0.26450386346094
0.246136223011633	-0.417529638122646
0.657858048525351	-0.474299150798165
0.740445220241588	-0.692030793992503
0.545456056313859	-0.0158754408577385
0.189279140745364	0.203827896583344
-0.0991296852108205	0.321680617117355
0.116292453457532	0.153664233870427
0.395332939710397	-0.1537781468257
0.663050357163333	-0.678748289698225
0.424005879205802	-0.499777774433918
1.05205259175771	-0.235896117451183
0.994089697777342	-0.513106892387802
0.331880384546065	-0.492530991603222
0.678828024610131	-0.427604582485631
0.475168889662583	-0.393806659664845
0.460924817111567	-0.491142667192699
0.414237098939799	-0.030675272104947
0.135918490660487	-0.32639442952662
-0.123738304859815	0.045879601456464
0.258874020376278	-0.225719119018945
-0.0559866167509672	-0.211795463447066
0.555367330234065	-0.0936896270023044
0.105269848899308	0.674287707138527
-0.161260824550622	0.151003410461885
0.0309739478126185	0.478316683183429
0.0811108555786223	0.501476379205475
0.419818847326161	-0.263329576807837
0.0544220978377433	-0.100781825696624
0.619297881968673	-0.587476267632378
0.266741123335184	-0.080702051536221
-0.430979072315076	0.057044592743596
0.0874017718817576	0.289912513468709
0.00927500152194714	0.365596976076512
0.122780762497258	-0.0645031639918642
0.286955530247561	0.211475902337863
0.677059699190704	-0.227633333119916
-0.238306805767344	-0.133945464091064
0.213355451960791	0.401230764050954
0.0343673658735026	0.342033687734145
0.554482097634431	-0.0861646279819362
0.698398965299716	-0.461482387101432
0.801961159503422	-0.582629607939516
1.0353133991375	-0.580888765437653
-0.0798589618932282	-0.0415740847121262
0.353313308236819	-0.321842530929376
0.170989200258355	0.149792631874388
0.240948032548906	0.380427732735414
0.398400862839633	-0.174658461478683
-0.0926167854002288	-0.0569254093385681
0.0465813438573996	0.209558598180943
-0.045531520986889	0.19173122623019
1.03703457451149	-0.558852163881413
0.289788483868752	-0.377187262840939
0.374122242527716	0.331311779918835
-0.354880352944692	-0.271333863389925
0.187771870276907	-0.753859726627796
0.901945479293246	-0.720581234303456
0.19606251589028	-0.272377963358552
0.269768736993571	0.364648376586249
-0.13290927819193	0.68359211252218
-0.363880106010557	-0.186745018248392
0.695271949150036	-0.371966214043452
0.527025025069318	0.0709067171451147
0.523715520033027	-0.299200611187638
0.204453093222483	0.16281690633853
0.350307522640005	-0.0692455442761559
0.553081861929808	-0.241209956896086
0.897448896258878	-0.323745066473582
0.517829755728985	-0.309135947207193
0.535229354655545	-0.696403475473846
0.682777779636873	-0.159348166311892
0.395047566290809	-0.143955662784123
0.351390236087829	0.169766060387799
0.235911416083187	-0.489856369580527
-0.217016001354325	-0.345616992090683
0.660036070295249	-0.0724348409660737
0.251593738805301	0.228665285857698
0.339484396718903	-0.135829055321355
0.166270847227877	-0.000633277486305038
0.272064962356831	0.0943547193184328
0.697857265023486	-0.64892842459691
-0.168680147171597	0.1280707480259
0.160866168014957	-0.114577937658552
0.138758986402147	-0.307115571488759
0.0220464157408501	-0.304068319372028
0.071564464967064	-0.241436906190716
0.0721262932368525	-0.241400849891032
0.429102087967445	-0.332260663119738
0.429102087952453	-0.332260663100456
0.619834606839921	-0.298575098150275
0.712698108643706	-0.30742271856978
0.416857991024133	-0.0619419278116023
0.387349653723829	-0.167378301570005
0.0842796239023317	0.131316932903575
-0.154287996330455	-0.0565553370811769
0.236363837498956	-0.462096507713065
0.41819927304546	-0.296615759595536
0.0324284945263335	-0.168648809888263
0.675179305168954	-0.508696581473068
-0.104440201590164	0.310420523521158
0.02117726802733	-0.213382940261769
-0.202679428401096	0.0573356072881011
-0.196180532879519	0.141722541845552
0.346833326149789	-0.0990678848206134
-0.427982113982059	-0.456140776098542
0.289190806001139	0.220837840864746
0.123799575421697	0.156924135317919
0.167452122187408	-0.0371528428261674
-0.203026435005777	0.678246547577685
0.632021356357946	-0.0301983144764644
0.255920128866576	-0.448823462359712
0.982281974287299	-0.377143555495844
0.27678659681828	-0.174711554259763
-0.118199991405006	-0.121902149328784
0.198212175631897	0.258470956879441
0.371609450468376	-0.2337197194636
0.672646857166821	-0.0893646368506714
-0.2231005191396	-0.206367758520103
0.352053244217646	0.281418994207523
0.235824316525097	0.129569427393765
0.929444888308939	-0.685138464860924
0.167830639799758	0.0775800637911876
0.313151821648146	0.440727695134242
0.616324304231693	-0.103449968588152
0.227385761197586	-0.236986402304887
0.147748882544501	-0.478239195392848
0.597458539234915	-0.555713138369327
0.0656819873824878	-0.149268535161263
0.427748226665346	-0.436496308962729
0.634479231126124	-0.0188234412599837
0.812229305999387	-0.449113097912484
1.18095066902412	-0.533497397070713
-0.125376110499167	0.525660742177449
0.11329558400715	0.0164797003005756
0.0459331998197161	0.430424512642344
1.19160994196479	-0.915736084768663
-0.112504009811991	0.134924287368917
0.590852903295678	0.185073470118173
-0.41579740084288	0.313221738252682
0.102785138837523	0.47219895794146
0.670227151774946	-0.145149502651555
0.189877069268135	0.255400397946477
1.10787903166877	-0.360520091007491
0.580690230887231	-0.16706915238287
0.422571923812957	-0.187768567861687
0.259961478623937	-0.180250818798621
0.693422738186631	-0.345209753398808
0.56675308283727	-1.11096543920406
0.116479534549973	-0.279029591516574
-0.000604488779997582	0.587783340531747
0.0465709468347115	0.515677592373816
0.156875155417652	-0.324475986998183
0.165203473441381	-0.253651401025355
0.498026511637045	-0.451809092705435
0.276408030086752	0.161515972723112
0.477408410240025	-0.235370551094565
-0.130960225559347	0.395218164904423
-0.444146587877206	0.488591841268321
0.308667886112974	0.132102367392015
-0.150993804030281	0.0787875308737038
0.149731754284949	0.0520836374454457
0.308667886112974	0.132102367392015
0.251182475225253	-0.581676427023065
0.174472316037641	0.421629767411483
0.174472316037641	0.421629767411483
0.0665010657064657	-0.0589253468460244
0.63609377619299	0.0102718815952591
1.14175176989955	-0.792448145313438
-0.104756370159166	-0.0936317751117024
-0.0152661971235877	0.223100491974622
0.19204360658983	0.113621356449899
-0.191689523043272	0.200983040018993
0.0737609205806185	0.507947715228658
0.00675552376705901	0.0816599119685564
0.488148926025757	-0.372241785810183
0.185068434714242	-0.59569568539492
0.243472454146809	0.0268812198229887
0.461106406953694	0.0581152615477166
0.354813568880296	-0.111532210430247
-0.200134087703005	0.140343489380857
0.42365034279714	-0.45428367720266
0.150867484678419	-0.0962528670845665
-0.144647283032305	-0.0801079898909077
-0.139499805323448	0.107361673713646
0.508787773921487	-0.244157816452967
0.0433608253043746	0.116740430868927
-0.0118867173634191	-0.15078009720267
-0.176129375862927	0.141928173058117
0.257329830891659	-0.45120177062406
0.463123355837048	0.0652397407690789
0.161634484295527	-0.0828057015119905
-0.206307250574227	0.373393623745152
0.897715511922899	-0.528071791213861
0.506220147716935	-0.327851750511233
-0.0741379649469206	0.0926116172223924
0.327253152493896	0.234154460258054
0.675718410191808	-0.293212839428818
0.0769968078743524	0.121478153664456
0.745862907765541	-0.54569991390295
0.437461564930085	-0.0195417929113727
0.089010205085505	-0.32561129295324
0.231046759944701	0.00335408588285386
0.350498509762413	-0.200132687482513
-0.0600144267299068	0.0952923143948641
0.27611188974287	0.0911156627269735
0.39296752204649	0.138369161103435
0.0162693199538078	-0.312676475380834
0.349861689160573	-0.250639400096669
0.45503102148261	-0.411070329333202
0.435387790856654	-0.0489929495624467
0.7456308261297	-0.25980623078407
0.212423683605894	-0.206045335504319
0.664928769130098	-0.326325750813224
0.176033721555856	-0.00895677694065912
-0.0647266954848532	0.316739357966654
0.412929245458394	-0.158781060319856
0.552150919309335	-0.495681395155975
0.267537489778485	0.031019391560006
1.27835022377703	-0.0474872178697312
-0.0162015508540166	-0.0176512118764306
0.228320643032901	-0.368079971812557
0.244921205483901	-0.106761639977381
0.416734268170527	-0.496207030102766
0.774098527972451	-0.491415520634087
0.344773706838996	-0.128346229434385
0.276236620537196	-0.114937574097469
0.966940823842833	0.142924175147504
-0.0672068253815928	-0.199382609985821
0.298274190061627	0.0098259107326197
-0.0284822060627902	-0.212777925010951
0.438657675325293	-0.041336832910668
-0.325850985968566	0.4732402665113
0.17722926224686	-0.108045799492342
0.827255057925699	-0.277373840253363
0.267671495839553	-0.269058696168569
0.223901007860464	-0.238821828434171
-0.0122561921228224	-0.187157380896049
0.511484648001068	-0.772665894128475
0.133137537587886	-0.142559790909518
0.321105209693354	-0.106515046402713
0.171317475586914	0.0323010545390688
0.606112315615867	-0.296581363898549
0.665542067859348	-0.414751791450533
0.0403791215919841	0.0149211921257056
0.245493823491175	-0.426651774859117
0.419647170808299	-0.280274727490117
0.115576896739142	-0.196713281176803
0.652541133239374	-0.258246833139418
0.580749306064006	0.149277763776617
0.663600833391435	-0.458005192419943
0.133137537587886	-0.142559790909519
0.381798036974233	0.398049969359322
0.311598139105765	0.173336385285853
0.449808032922429	-0.221038374686198
0.493597672137882	-0.0556264404836886
0.0668337751343619	-0.0667776550427293
0.274828549932775	-0.150899370975366
0.187392558489829	-0.264063410435765
0.520750872229063	-0.327251769842996
0.142236585602931	0.0106703908360878
-0.244425624977815	-0.23038435563904
0.770209971502554	-0.180472392169093
0.759163553983722	-0.148717790169686
1.03007693895186	-0.630470914952683
-0.163804995571427	-0.0828446800371052
-0.0259756345001628	-0.458552026488321
0.507794727664062	-0.607393599850043
0.0113398514541369	-0.292363488408347
0.0107355919273251	0.319619770012153
0.189687801642875	1.08844113212725
0.336046046495461	-0.0757266030235487
-0.130507871710606	0.137573658515059
0.324785166760374	-0.458168384465535
0.257136029798298	-0.167505602181904
0.0350534932475547	0.256566428253053
0.6944711100479	-0.320143944058126
0.127536276488913	-0.298791808725257
0.198941549288036	-0.721418422662132
0.0055580725046481	0.148206335263879
0.364153573155995	-0.133440470775753
0.963509833155491	-0.211864282003165
0.038056184727859	-0.0985488818743254
0.487995415026304	-0.816302113080923
0.590638470319896	-0.299915239679367
0.362133093898184	-0.0102074641151142
0.507362555206347	-0.107308912775352
0.526768034144399	-0.349860658391536
0.93226895619069	-0.665150496233409
0.0972647196147172	-0.424382354971724
0.482351691456961	-0.451158070478391
0.507595691055233	-0.00528234300372969
0.56449027768338	-0.479264017997404
-0.00409070313672306	0.537866025452265
0.641918068394898	-0.317687845685343
0.361774205481158	-0.131320296419154
0.202789687117001	-0.32373997003697
-0.0820322514180347	-0.101476525855862
0.256576847182202	-0.0829482915250592
-0.0324620506692864	-0.158835496306683
-0.345558897180922	-0.0778693297839738
0.135017553051485	0.179749687408643
0.118104767205903	-0.338950724392036
0.332656039499392	0.0163125122545119
0.0309642415911165	-0.541452757966856
0.0684870703552522	-0.18749597794256
0.186982552772499	0.151056190908036
1.22013897209674	-0.555893681044403
0.735951255004537	-0.819317436305111
0.295852211008072	-0.00417047087419933
0.121201735455222	0.145412831247167
0.283849408057376	-0.291532413306769
0.62053342338823	-0.297161141369525
0.382886458818216	0.214266189040651
0.559032973218271	-0.380933256428031
0.606112315615868	-0.296581363898551
-0.120455497491074	-0.075316940244359
0.284719311146753	-0.141219803598565
0.0231070194664893	0.450649013081262
0.177593627903065	0.254104663730643
0.466479852969338	0.116119181643025
0.628861419322217	-0.454278823084125
0.882089144650996	-0.296695113352512
1.21478046229371	0.122782137040353
0.317925057106875	-0.245423354546856
0.58776839926694	-0.323169273813073
0.0906778986821417	-0.411191958246735
0.283806274675207	0.176531432495603
0.691791682090436	-0.517003482108109
0.567039410838869	-0.498308862250423
0.0308586839273583	-0.075110363284147
0.88240916374015	-0.328176458211309
0.546372540538433	-0.286595360053589
0.549662711743591	-0.390363253640423
0.184071102472272	-0.233102514744976
0.372009832791702	-0.206394883629123
0.00661607339464839	-0.10548002779774
0.233694509364975	-0.0452268334449328
0.572809494041649	-0.653838608050732
-0.30368366584465	-0.202630517938723
0.480556393372871	-0.243156071845898
0.321540029598058	-0.284902894602862
0.473467577287979	-0.253299510581639
0.233694509364954	-0.0452268334448943
1.02294121003653	-0.612933353829905
-0.30368366584465	-0.202630517938724
1.30059910829929	-0.145875197910995
0.814351497731077	-0.063216191027086
0.334212366508904	-0.488789342754243
0.433144244415705	-0.294021965408695
0.556041441436576	-0.660907391297621
0.41116406233225	-0.198790995081221
-0.132385786027344	0.0253381339982329
0.243205487539902	-0.292613706891303
0.360071979997296	-0.511952802013205
-0.326197163999461	0.0469351036953797
0.263443240804871	-0.330499961099154
0.562022658326208	-0.0479388771965421
0.628528553834293	-0.457785032649792
0.235860457993435	-0.347629235505959
1.36328880723942	-1.03859425990705
0.42845424523426	-0.491798937017519
1.31846472267745	-0.843568442008563
1.38095108549995	-0.487170563051523
0.779768147375077	-0.645770933642609
0.430825219115535	-0.197901420013226
0.460982033420257	-0.508642668159188
-0.363014727865586	0.00893887752320247
-0.0950616406689792	-0.109369472264228
-0.166030238375688	-0.0882017436752928
0.565409137819698	-0.410304705979278
-0.0179013162307279	0.067306325463211
1.32495408055273	-0.829579773987822
1.27201289785073	-0.810692494514983
0.0534512316247159	-0.410674095770165
0.751756826324743	-0.611619566337956
0.73524296630194	-0.558494751291281
0.73524296630194	-0.558494751291281
0.00721482917095769	-0.342924805469659
0.31901960631839	2.67816056613358e-05
0.342520823932893	-0.65577402211413
1.35951112333445	-0.718166549781252
0.167111635254835	-0.0934082124056256
0.286099325872379	0.0389631778458644
0.355733840194866	0.0154914109024574
-0.206063372719446	-0.0624344240827078
-0.0625705709711486	-0.201055329666755
0.366317002959309	-0.554429332884301
0.358905759145433	-0.146068748253939
0.413180300111029	0.0830905153033395
0.140594763344468	-0.301004306172895
0.384603554267209	-0.11269050769762
-0.0956346455870244	0.204196654776043
0.51241976882716	-0.46162201151882
0.154437697964022	0.105981015672998
0.563705984748448	-0.472390261472353
0.158926913050019	-0.518068879798147
0.290649749493149	-0.533955281696262
0.364219409615173	-0.00341260518384862
0.681896535089723	-0.162997272784879
-0.223135086772121	-0.233921556197018
0.473987933202346	-0.0658723132493307
0.280332910339871	-0.244203522263706
0.308188945968655	-0.255956575892933
1.06508311193268	-0.697381545739636
0.551232931551728	-0.241765653541246
0.395599777905317	-0.462526937577048
0.288954015849947	-0.375320155749726
0.909893524720295	-0.750110836552116
0.242240115104153	0.289619268905357
0.0268512365163376	0.158434713032531
1.02666963233538	-0.583299120247851
0.674903499083645	-0.769589149908202
0.194023987939727	-0.0414374983800396
0.17704477478063	-0.0908581615945274
0.638174530028653	-0.171892895735243
0.359152387663652	-0.209397137053504
0.805384188038005	-0.594369275540345
0.679087563485517	-0.325069583551652
-0.335170189812015	0.617988587744923
-0.414689140351809	0.912686862668266
-0.618460952625299	0.0278378439331032
-0.309961019368958	0.251801768470093
-0.525209206128786	-0.327426800851932
-0.275955776122718	-0.116681653204707
-0.504918903545506	-0.22712433521694
-0.666035643406278	-0.114628567384449
-0.493780390578341	-0.454136511116901
-0.855413334766146	0.121853284488199
-0.573876769512109	0.0412527247636477
-0.0773628448631834	0.45505478707255
-0.109133403378724	-0.893299861975688
-0.504527804507046	0.326471042818333
-0.458923544485207	-0.214597848876584
-0.537697885915971	0.577644732905247
-0.863343095984802	-0.446359848006913
-0.0559924175760826	-0.178338621436062
-0.754795131086192	-0.353264146403751
-0.60087503583209	-0.300438799271214
-0.780464167224296	-0.0182456285019888
-0.738843375890754	0.0117705697453634
-0.831593706880518	0.0387189630277342
-0.183792910716642	-0.586915458828724
-0.202611404121253	-0.0647749864183599
-0.814001222964676	0.0654316014204857
-0.884890960119044	0.137191139021842
0.200841062968214	-0.358868713290025
-0.33230802589203	0.64725441894588
-0.389048727117528	0.881004060443309
-1.16098118867766	0.361630475281372
-0.552340339098899	-0.435330056222867
-0.751787166408953	0.572655500504854
-0.455078862607623	0.0753045673825221
-0.0978822840671772	0.496381307848157
-0.618460952625299	0.0278378439331031
-0.816164165220524	-0.529650320262422
-0.707961654357227	-0.477263919646665
-0.318043682175514	0.393031273549632
-0.544169372531868	-0.554279700707739
-0.526438895806365	-0.147377163255561
-1.12538695671805	0.0466780304975774
-0.577180305748143	0.120248917229123
-0.337032255945398	0.237661595394499
-0.0516952036778607	-0.496929008340261
0.55288796363773	-0.406185861247643
-0.394602640729576	-0.120586391644719
-0.253231850650438	1.03977255668848
-0.846372420529243	-0.276961735342593
-1.0524981271324	-0.366235171766589
-0.797571645431257	-0.77643447681135
-0.0328149385427853	-0.1673693892944
-0.375520244358504	-0.476087671432854
-0.634132436500803	-0.268738156420362
-0.819388132074846	-0.198016596199358
0.0300220185404251	-0.212597981896874
0.339826491131248	-0.927553693999277
-1.08414694266015	0.204726344643956
-0.259273614875042	-0.899457482095782
-0.483105092977217	-0.205696167166383
-0.984657636616313	-0.28567475156928
-1.15428050323938	-0.190643693192632
-0.309961019368958	0.251801768470093
-0.716671053759702	-0.113060807950672
-0.324614712462126	0.510897472510958
-0.543803781658647	-0.412455298321421
-0.916544291851889	0.252851078295617
-0.0509840414857586	-0.500846399514477
-0.534817592030509	-0.248613992695376
-0.730963514198851	-0.263689402162383
-0.320507412132431	-0.514248541925234
-1.27957160472342	0.261020861974323
-0.674976261639622	0.00462592856929194
-0.0211857423330175	0.911889344046158
-0.708943742696488	-0.260920659231126
-0.718234525966546	-0.0980014700073717
-0.879751442645777	-0.175015030279646
-0.47945003958529	-0.442181858382881
-0.622907042829561	-0.124739305506044
-0.458148379328741	0.50545983060048
-0.673134893022702	-0.431533812833419
-0.27620239390749	0.141974641149953
-0.625449976561155	0.00485484629577188
-0.758968453424769	0.0728268152051597
-0.793037453458537	-0.700095661995921
-0.78628708339021	0.651503200933786
-0.704258626983526	0.175304628999508
-0.243213613994448	-0.183054393728272
-0.581380081239571	-0.254021583514088
-0.488159774384282	-0.581359667172943
-0.562164105519142	-0.507931480682381
-0.28407249102538	0.588516863192485
-0.446460951776897	-0.382240098049188
-0.106428869120174	-0.306497211298112
-0.110644462216619	-0.644963903908676
-0.555941477398723	-0.0258028393228716
-0.576517707178032	0.177469732942059
-0.0836843775847644	-0.166439602859529
-0.792396802041458	-0.406525869124895
-0.0749699348219736	0.0305470101599288
-0.431243988345806	0.220304151993441
-0.692240645728898	0.0584024555981343
-0.287559676264928	0.290816624537797
-0.0955158617378035	-0.378027873607638
-0.645064827852669	-0.0553919324969521
-0.708529442833866	-0.428748306248106
-0.930158718360211	-0.231465571968856
-0.178774900267601	0.0985698306159257
-0.677892025746156	-0.172782228583738
-0.145287446849251	-0.816527729561486
0.267285620735519	-0.72653433178143
-0.135392260436238	0.00506019030434986
-0.397402969293092	-0.501791921237072
-0.614461826465713	0.409731359692392
-0.682035480465404	-0.371381060266972
-0.814783029665246	-0.060358445341809
-1.02326353149946	-0.395783509814054
-0.940898026285033	-0.0146986242734409
-0.467444583883592	0.809002651438773
-0.726018360004746	0.333269991623788
-0.72276802407092	-0.36056897959807
-0.962104793451964	0.00530557156633129
-0.485614386245124	0.401173601011309
-0.547286991984019	-0.457302468681441
-0.736092270889671	-0.422799801361796
0.195954201170824	-0.53359362765851
-0.401218519754551	-0.282252435240903
-1.15408169509003	0.152712379104559
-0.579075196656776	0.0820651107250545
-0.57991363925099	0.372351707921445
-0.650341554313186	-0.287524193116547
-0.55938923659624	-0.239030515406284
-1.06685828605253	-0.249953079857387
-0.558839366772932	0.000312211788150179
-0.227321903576651	-0.33548731773561
-0.790018036515045	0.187595930058267
-0.771362878831883	0.0335520618500657
-0.492498806633219	-0.376710087047524
0.119435598204785	-0.567049693740952
0.538060024987623	-0.571728841322373
-0.385628079567716	-0.720527049391983
-0.53875136091238	-0.327875205333877
-0.674318193831988	-0.16067439201088
0.530374858778539	-0.291698181544557
-0.763697957751913	-0.693050821364111
-0.430439706089337	-0.437118196541976
-0.361660762962231	0.404290050317421
-0.217750226405094	-0.619062966528523
-0.660748677884987	-0.161814264010007
-0.762898158559197	-0.200643983970756
-0.467809168638933	-0.464163118975804
0.232520616707158	-0.000689279913207526
-0.55749453941848	-0.348158664793089
-0.223907414892885	-0.0940570445592784
0.365589802541569	-0.376084000519175
-0.60541565723017	-0.136513432308697
0.264198743206925	-0.0974565951059159
-0.430147610716371	-0.388850271244377
-0.361660762962232	0.404290050317428
-0.590641217346309	0.0123729855763801
-0.428630774536157	-0.486109928047414
-0.660748677884181	-0.161814264011673
-0.595292376348182	0.352781181971004
-0.591801479174027	0.916428345232363
-0.550088927277211	-0.294781038466348
-0.550088927277211	-0.294781038466349
-0.423731814272764	-0.137037156080255
-0.125950354838194	-0.0380770976026512
-0.643464407560092	-0.385281955792083
-0.413582203108769	-0.141232367354274
0.195954201170824	-0.533593627658512
-0.468404829396821	-0.18032148054889
-0.308244966416897	-0.530016918993788
-0.368067558529244	-0.136333701873437
-0.579075196656777	0.0820651107250556
-0.559389236596253	-0.239030515406225
-0.144182707837083	-0.304042861248849
-0.581756476101594	0.369851086183987
-0.650341554313186	-0.287524193116547
-1.06685828605253	-0.249953079857386
-0.576880830669195	-0.41785212608183
-0.206566434412739	-0.990437347993176
-0.728745896561491	-0.216632315528974
-0.618988596566569	-0.0147375705651202
-0.51326038413878	-0.341337386338008
-0.460003163239228	0.0570369596845815
-0.244923261726927	0.947769181629301
-1.24428199236926	0.1158687713724
-0.902297283390663	-0.230607343740521
-0.246740854394353	-0.323962003867025
-0.162836899967194	1.33167718080054
-0.34804829744558	-0.274400386005078
-0.730288916205625	0.589901746816024
-0.655360346553056	-0.116491517599671
-0.239719246034373	-0.355186912138176
-0.831052838478284	-0.306621395063977
-0.710905212231455	-0.219857768115796
-0.520188216902716	-0.267554947119042
-0.0101594157293396	0.25064024332882
-0.593466913531856	-0.0271645244669831
-0.867447311091793	-0.305811040328648
-0.650305965748794	-0.171122458525991
-0.412898422400306	-0.826979506291601
-0.771982814223486	-0.43531632171351
-0.524689192615189	-0.141301228260774
-0.372752662165606	-0.781868144486782
-0.699829683928424	-0.119222542601291
-0.380103452274206	-0.279671802712007
-0.985150252081646	0.00704581289427178
-0.665085602032573	-0.596800341690173
-0.715178016735686	0.152222496270961
-0.982719322519658	0.00693524919203789
-0.798232077157492	-0.566812001304802
-0.703606811865078	0.00928050347610446
-0.423731814272764	-0.137037156080256
-0.366485845441109	-0.349954258984578
-0.527563597204057	0.200721012558105
0.227400422103943	-0.172326635643056
-0.271687215099492	-0.200478582851573
-0.809703983960899	-0.46547076609562
-1.05309558013904	0.0792047973746883
-0.70569519617411	-0.121183394257503
-0.518749389419506	-0.0377967006705404
-0.271687215099493	-0.200478582851572
-0.982719322519658	0.0069352491920379
-0.71517802251996	0.152222460457658
-0.826570961725424	0.189345446209318
-0.869737290556091	0.10992306730119
-0.810527515769942	-0.494010683134447
-0.522383718888858	-0.455806023844136
-0.426723711446817	-0.292621403395291
-0.390827520888104	-0.420988621869071
-0.643464407560088	-0.385281955792083
-0.813547593729372	0.0409445342293129
-0.790252098719061	-0.587978322500347
-0.266168970399068	-0.000336558635550633
-0.534378889277913	-0.212777885185174
-0.280717428340264	-0.100815832359589
-0.934423895890995	-0.151606068164837
-0.575749297352477	-0.34653106169241
-0.597564993288924	-0.361706329102558
-0.795548292677701	0.546223615640498
-0.825678817395414	0.0764516704387209
-0.412216952844571	-0.27948001391855
-0.411072163108241	-0.369103288708493
-0.983640768817518	-0.434748698936079
-0.0699314251005508	-0.809305443456427
-1.01995978556973	-0.00540498652628763
-0.466802831819036	-0.379478339444073
-0.372624150373836	-0.218787446113395
-0.753013007011971	-0.541307970725848
-0.666035643406283	-0.114628567384449
-0.307737043534789	-0.180461096600768
-0.21105578004427	-0.338698831651269
-0.354181284334522	-0.706277923361063
-0.276968392944105	-0.74918653555169
-0.657228840452949	0.158577668922913
-0.422232840085615	0.444016266196425
-0.866462868919181	-0.305359982182976
-0.727928209054216	-0.0236379635799791
-0.770467257470491	-0.131697480923767
-0.438848612306753	0.0770397768326028
-0.560218131379149	-0.569196952545617
-0.647009189166946	-0.363179947097554
-0.599950672375476	-0.370231906445747
-0.0695418866750826	-0.732626782685042
-0.417599807280423	-0.528329022410933
-0.856296445936738	0.429625425215606
-0.684904163881101	-0.272116455404169
-0.86405589381724	0.0870390455462861
-0.936058764630506	-0.297103226312092
-0.600575027757646	-0.419494087481648
-0.278803861762021	0.131100604479914
-0.18239901098282	-0.376386201957529
-0.248864762032422	-0.275031419558135
-0.507233267732306	-0.200394114069597
-0.9090049521706	-0.397436240568654
-1.18946171150206	-0.109947412509488
-0.355661166609019	-0.508364639030844
-1.05308232287786	-0.00402665098980739
-0.523960388375083	0.482644919746269
-0.420416899126599	-0.418651709303694
0.265116588582886	-0.452012897571759
-0.438084808548305	-0.454035870215656
-0.551741752064364	-0.385932734311163
-1.10504813372405	0.240058997333773
-0.609207180351853	-0.0522393297316468
0.0738384504525197	-0.362493454177926
-0.0836843775845547	-0.166439602860067
-0.814199845694323	-0.0590414879538716
-0.595292376348182	0.35278118197101
-0.144182707837082	-0.304042861248849
-0.60541565723017	-0.136513432308697
0.365589802541569	-0.376084000519174
-0.869737290556091	0.10992306730119
-0.730288916205625	0.589901746816023
-0.727928209054216	-0.0236379635799792
-0.560218131379144	-0.569196952545613
-0.864055893817237	0.0870390455462855
-0.671394547594066	-0.266519197805523
-0.834310717695519	-0.0573267907646059
-0.355661166609019	-0.508364639030842
-0.708943742696489	-0.260920659231127
-0.551741752064364	-0.385932734311164
-0.52610140698461	-0.301062342055536
-0.149144754546556	-0.519226424613582
-0.486361833228205	0.351384689817822
-0.625449976561155	0.00485484629577183
-0.434728338257784	-0.142967149286604
-0.581380081239572	-0.254021583514089
-0.0858669570578211	-0.564574096383887
-0.488159774384281	-0.581359667172943
-0.106428869120174	-0.306497211298113
-0.819388132074842	-0.198016596199356
-0.884890960119044	0.137191139021842
-0.739946704311364	0.196833897278663
-0.202611404121255	-0.0647749864183599
-1.15428050323938	-0.190643693192632
-0.719689798968286	-0.0926198832939643
-0.192368258809138	-0.206643601027725
-1.11698676984432	-0.107451996669966
-0.259273614875042	-0.899457482095782
-0.797571645431257	-0.776434476811348
-0.337032255945398	0.237661595394499
-0.651110047296176	-0.553901734688972
-0.577180305748143	0.120248917229122
-0.394602640729592	-0.120586391644654
-0.780464167224296	-0.0182456285019887
-0.952231479654637	-0.368945482951985
-0.504527804507045	0.326471042818335
-0.0773628448631816	0.455054787072539
-0.544169372531868	-0.554279700707739
-0.830270063869403	-0.552773909826252
-0.318043682175514	0.393031273549633
-0.552340339098898	-0.435330056222866
-0.58392642706674	-0.124663375044903
0.0439451559642215	-0.0246152686455839
-0.716671053759705	-0.113060807950671
-1.12538695671805	0.0466780304975773
-0.853384643283522	-0.303466554654552
-0.0070369974852224	-0.478862972211123
-0.253231850650422	1.03977255668842
-0.0245596675061258	0.0289089205006778
-0.537670247675495	-0.48251573694569
0.230594900426366	-0.791267102488755
-0.393531789152902	0.33466913938126
-0.850939541312688	-0.224797765039862
-0.400987177305294	-0.058788055824481
-0.366829950858668	-0.0541922184729821
-1.12778249020376	0.16438833854357
-0.408852670468092	-0.140450932590306
-0.831699656987916	0.11698791989386
-0.701792948640407	-0.342374002881106
-0.668548971208795	-0.102132248853569
-0.412961214444011	-0.330739341425097
-0.565227106654401	-0.198054856540771
-0.790846686337308	-0.136249864861798
-0.652955266037665	0.966277434239241
-0.628218778967403	-0.333157935512226
-0.429274946585544	-0.282873530248518
-0.292542262254523	-0.0721829577612189
-0.935051999905641	-0.20931094859863
-0.442967500211999	-0.232683843191226
-0.652277446896675	-0.230442389792184
-0.451009719153195	0.221445903844421
-0.840232856336209	0.486806168251563
-0.318189084111398	0.178444480312527
-1.10663253449757	0.326527752310226
-0.535699032270512	-0.245702980546781
-0.194148081685238	1.34884797331439
-0.386706317316137	-0.232787125503445
-0.609207180351847	-0.052239329731644
-0.622907042829561	-0.124739305506044
-1.15408169509002	0.15271237910456
-0.828200462505626	-0.135028344996406
-0.526438895806365	-0.14737716325556
-0.555941477398722	-0.0258028393228717
-0.310484699219694	-0.219285402699764
-0.493780390578341	-0.454136511116901
-0.738843375890754	0.0117705697453636
-0.543803781658647	-0.412455298321419
-0.916544291851889	0.252851078295617
-0.257197120468227	-0.557957208488677
-0.527102509402349	0.155497848369436
-0.746700977787647	-0.0602879462241335
-0.96124971589421	-0.399197095891153
-0.940898026281728	-0.0146986242822215
-0.287559676264926	0.290816624537797
-0.742231434278926	-0.437510493502575
-0.815932674566851	0.396566280461254
-1.10504813372405	0.240058997333773
-0.812648360849682	-0.313269694619835
-0.673134893022702	-0.431533812833419
-0.507854839404159	-0.336241873729793
-0.323473154153218	-0.455907821104338
-0.618856983457216	-0.0971665348543626
-0.578829555685187	0.0482038016387853
-0.826570961725424	0.189345446209318
-0.329593319703566	0.170448534284654
-0.422791821402632	0.343651806468435
-0.196711229211826	-0.630047055866555
'''
gaussian_sv_phi_nodf = '''
1.93693544828065	0.407779292117722
-0.121281831574311	0.171138582656352
0.841466474081402	0.0400233633413362
-1.96222283361484	0.934638136088843
-1.86641646959276	-1.26083262034282
0.12981486331844	-2.21284826881651
1.4562868937594	-0.639303812577775
0.695116978409615	1.31990370302676
-0.900329649895325	-0.795179555261594
1.97284897999099	0.692946173022281
'''

In [ ]:
#@title fulldf5
gaussian_sv_x_fulldf5 = '''
-1.59282423749759	-0.222307615431054
-1.18910025514939	-0.286378342169359
-0.561990098797348	0.269060441801178
-1.16548199940574	-0.667509647621743
-0.512778550506262	-0.16207312222852
-0.32625707015613	-0.720696373102389
-0.846419431732118	-0.630082903704785
-0.0902023738860001	-0.309448220301404
-1.10535454937131	-0.661270648097102
-0.161159929618046	0.423720780472643
-1.60300374145913	0.945833165390905
-1.11833871022487	-0.239166344270339
0.352352218366606	0.137580467501582
-0.526738687246403	0.0637302865083818
0.207447195082738	0.0292003799945601
-0.375765566304241	0.526914218923395
-0.787388934126118	-0.247200173687276
-0.894119523378861	-0.649196541808433
-0.737807252050014	0.333393257381205
-0.681757903977787	0.682872812876157
-0.106973432917697	0.404217816197022
-0.667481682373993	-0.438484269588905
-0.536509238824959	-0.324596660356988
-1.20788134590296	-0.895563049530321
-0.180512044755344	0.195860707220138
-0.286662942142547	0.485290736222809
-0.506818347198814	0.142352324285422
-1.2148709421476	-0.0710048829130281
-0.572277212854119	-0.11828193216407
-1.38679846562806	-0.15450887290224
-1.11406974108499	-0.828189747356221
-0.285302245158404	-0.114112595563319
-0.690629092318762	0.12319515812317
-0.617894692129271	0.0142619653197208
-1.10386310275833	0.25329007676175
-1.48549767772778	-0.15466988400303
-0.393330661390576	0.176544194384701
-1.2015204334308	-0.0543072753132411
0.313221407641839	0.235240383972772
-0.694121188639927	-1.32030304740625
-0.565766308056295	-0.101667152812488
-0.649469200649396	-0.672890600199828
-0.764587138486659	0.430735434193145
-0.774585047950708	-0.649684305950301
-1.55649554315932	-0.659788429926206
-0.646719525931568	-0.473305517742911
-0.763029127785922	0.0869323040183201
-0.238504289608277	-0.309497013517489
0.0288694263935873	0.141636962290087
-0.724443553159054	-0.275848580957703
-1.29293642673542	0.135542473214271
-0.196042887344124	-0.334674463600645
-0.910740074163159	0.105652329327898
0.0384552728227276	0.0495012848008406
-0.735088139825548	0.0398018460831059
-0.889100469690948	-1.17964435249894
-0.862555134873465	-0.659994038336585
-0.814395848776874	-0.499686667503787
-0.255749541318184	-0.0334817618660264
-0.476072193104324	0.46215952530446
-0.725974561413608	-0.853975509360806
-0.928821393778668	-0.526785164826059
-0.777499087791836	-0.303549241840796
-0.1379331062016	0.236236751008811
-0.495481832090953	-0.0963625791574249
-0.34695960198898	-0.521602965008116
-0.910733138221884	-0.299596366787129
-0.24539314773059	-0.253423347349554
-0.480924646782273	0.121135575256075
-0.609398333483734	0.387709344569658
0.259175106936475	0.469107487738421
-0.188643566245673	0.201661989357278
-0.546656426483159	-0.45584211367833
-0.528572276454262	0.556550909978326
-0.293528193140106	0.033280463861927
-0.976408928182598	-0.275144443309653
-0.548321413996449	-0.0203436625792271
-0.00284346002987155	0.403390121316488
-1.18516734590747	-0.250586644833257
-0.602571947267422	-0.138486706341408
0.433780395105142	0.246658151434692
-0.94205442772798	-0.467611864439182
0.0224096481601047	0.453463178184856
-1.41780905779412	-0.828117426615093
-0.897397207846454	-0.723699007681478
-0.783978517600906	-0.600606948429316
0.230502816322303	-0.190323548064603
-1.03419267050906	0.281974296117188
-0.460832315612488	-0.408061752126773
-0.472749373556838	0.0649620581451253
-0.927251267783338	-0.174080533560999
-0.555258878142717	0.340997916115792
-0.581580503011841	-0.382686398908131
-1.36393999785168	-0.21145859527153
-1.07456943111246	-1.27033971399449
-0.509692593981684	0.021152914968487
-0.658582060849932	-0.344452962198352
-0.982309933687334	-0.23833739230033
-0.560722475254369	0.0213218149630092
0.00766697392474931	0.0278103380103739
-1.22063048897055	-0.570129141561508
-0.264302123901525	0.213578973838119
-0.629724930990581	-0.15772212001604
-0.114556655973468	0.298452071665698
0.460714807491427	0.0603940484064592
-0.209876749750318	0.437159510747052
-0.910591596034993	0.0127838055249069
-0.814211680247892	-0.407959355302603
-0.812413690065304	-0.597525459392319
-0.637370552816889	-0.57805576865186
-0.708705479223752	-0.102598835458236
-1.20598357978675	0.525811838970713
-0.694071642102683	0.69807857731953
-0.497558992459017	0.482402399861632
-1.33543337558453	-1.14436422095023
-0.751559205874561	-0.250488099668565
-0.443651846852528	0.239385735284556
-1.35658764073707	-0.292929198683485
-0.674853632281365	-0.401535642333869
-0.941496945711583	-0.352754748583171
-1.09516462246764	-0.788204508785113
-0.653868228750776	-0.0617798390555192
-0.0852874704932504	0.327618989300156
-1.09786941734188	0.430866985085003
-0.852071906234381	-0.254943685952997
-1.20419316198303	-0.860926403875048
-0.658352382188931	-0.220262646535212
-1.03802269459744	-0.860280954180465
-0.429500736822006	-0.125300632393284
-0.32072825993947	0.140572981021287
-0.760268559373152	-0.152411685234343
-0.650019537512952	-0.326955043919972
-1.66862381895498	-0.173427753387052
-0.371301261489945	-0.0901391077684706
0.243945524670315	0.268760126202008
-0.626750321722636	-0.0690977011726489
-0.30437289356204	-0.274515887622105
-0.815322475730321	-0.0528767233986452
-0.142034187307419	-0.594519655370714
-1.31741097180601	-0.264757726495411
0.101819187851384	0.327698934206149
-0.229760451752129	-0.227277778394289
-1.27983677613399	-0.658607778300134
-1.04080092384441	-0.721349549540214
-0.407790060021709	-0.271011585494537
-0.527033957026211	-0.128126895690832
-0.924028231557771	0.0549819305819413
-0.123823511479717	0.554163183376447
-0.0699810107249355	0.473827775148842
-0.537146546632213	0.0621193418465653
-0.546106011162943	-0.16719123342784
-1.11128747421649	-0.11632175047445
-0.626309075847437	-0.384780687622268
-0.461897934214795	0.188704240353591
-1.02668536295161	-0.0102781671069997
-0.462541648096331	-0.368512736146361
0.023059157931173	0.136072146280411
-0.365891897883025	0.0507178567552767
-0.345062050491293	0.0706719308212192
-0.485608097825739	0.242284054402493
-0.48170050856771	0.429428160312681
-0.947624068550405	-0.0674290263350215
-1.24313519472072	0.0197734847187869
-0.863019473707134	0.0111986462364423
-1.29264086018094	-0.0422829603797875
-0.774794281077902	0.00935108533239893
-1.30931253464939	-0.523093095606867
-0.932666170245519	-0.285422113361425
-0.594866125137484	0.234390311498102
-0.480692516401947	-0.112139383393425
-0.332676125389999	-0.904674549768416
-0.527699112726152	-0.0920605442311417
-0.847143847316631	-0.229576652603788
-1.10105073655401	-0.198039827527329
-0.604388941404584	-0.198292702975773
-0.308084326126775	-0.174690778906976
-0.425417384175551	-0.25385663062165
-0.0689344636605956	-0.311937205184053
0.710352878366321	-0.190692873173052
-0.949878274809356	-0.122416061923713
-0.134465986597347	-0.308595107204117
-0.620039469253747	0.331811836794324
-0.162792410586554	-0.281868376790918
-1.05610956323828	-0.685802309859869
-0.786957658363439	0.727590979711037
-0.376493664820561	0.156131453737684
0.231906605570206	0.637748500586644
-0.635399073020976	0.326373856475246
-0.895525824259143	0.171467422212528
-0.961964438454578	-0.228505381233576
-0.248688231156434	0.267603772030131
-0.00455680701958753	0.452330881967197
0.521225268357904	-0.873810247388547
-0.154510381155163	-0.449399373298545
-0.271134072643414	0.0600200762484283
-0.333757324967372	0.421767389757123
0.132770134131614	0.653130993995517
-0.86423559139295	-0.215541919197409
-0.6385906332404	0.244347218507191
-0.783868366980818	0.0352516309703423
-0.210307163119911	-0.14287048748895
-0.757991007844793	0.488068380836313
-0.0555614752348021	-0.333551730000218
-0.728447063666857	0.22897125502215
-0.346611324559263	0.677767505749252
-0.220134621990508	0.357913546589184
-0.517455476712114	-0.257580166550149
-0.883659384358177	0.655998667392613
0.267496503205997	-0.123322073066294
-0.795327365478611	0.196428487561315
-0.945121106508216	1.16791760591281
-0.0762129259803871	-0.397820639850159
3.37019132089819e-05	-0.23338540697035
-1.05258309071014	-0.3778271880184
-1.07223492422848	0.0246179331080032
0.150563374871738	-0.272285426455414
-0.916179728109341	0.00977924631924608
-0.484085668861051	-0.0292457721232749
0.128017243153256	-1.01257004401443
-0.146910523842203	0.141682193077202
-1.54256673788371	-0.49531763492731
-0.66988952057738	0.168493348238589
-0.763610517900565	0.301384666758416
0.600186070403491	0.297578591927377
-0.199066703516531	-0.524385719550892
-0.0297267565847132	-0.150874993317281
-0.790401830010298	0.135062992973641
-0.827371911514112	-0.124879702898428
-0.0458297210110083	-0.260001728139511
-0.654973649363637	0.0491399207041632
-0.787023481032926	0.141449417731473
-0.702672618957669	0.205533970036181
-0.882780419766019	0.314080994442165
0.0878438620040139	-0.360799641418931
-0.391550214432146	0.342462354945721
-1.1091215534703	1.06534190327029
-1.42943456992172	-0.203123360990391
-0.852514745263952	-0.377169936431717
-0.850915724968113	-0.834067211882936
-1.05258309071014	-0.377827188018401
-0.514536276820414	0.0654249397194393
0.291805341330936	-0.0677430567842225
-0.600844634810076	-0.0454570804943147
-0.112579147347706	0.0997306645890621
-0.965883437298923	0.200076462862158
-0.198866632267423	0.113698837552662
-0.843718432156063	-0.581557084194007
-0.756788886069588	0.0384055746559811
-0.87724881963172	0.0927934202336852
-0.617559414431641	-0.0744116503623545
-1.15299836276321	0.157776616374596
-0.925872592644095	0.22623756265242
-0.846419431732118	-0.630082903704784
-0.158383472374698	0.232470969584706
-0.419918637457387	-0.115246581071199
-0.997469266948242	-0.280200019375981
-0.373688337496341	0.141053648770014
-0.579025897084655	-0.110468217311215
-0.849286000710715	-0.84212893201858
-0.514953814270674	-0.749434539813147
-0.437872903074703	-0.130688311933041
-0.42131995042973	-0.440417986767572
-0.682867984276257	-0.0663947830242306
-0.253595177042838	-0.450564247099752
-0.822330278271283	0.247427300756444
-0.272237598326015	0.0650409334381614
-1.51072612065987	0.0335525815312093
-0.0632995346826407	0.246032145063102
-0.0536062351768722	-0.149565500211728
-1.69007585471268	-0.909698187748928
-0.466523903461536	-0.720356015432749
-0.807509885780611	-0.299134769814427
-0.894022452503449	-0.208350073892407
-0.796192594562862	-0.209803349183696
-0.504532461138209	-0.662738362461181
-0.369781265166846	-0.968860567934024
-0.796849808947075	0.292342845846128
-1.2227091170128	0.395382495543949
-0.000313428384877336	-0.335311230761032
-0.536280904364708	-0.459546051770398
-0.348197220931311	0.600908106876265
-0.212925280965842	0.134396843633759
-0.717712659133963	-0.524155965288857
-0.479362783685985	0.0623154760971238
-0.588208303517049	-0.253904577926934
-0.302555358584344	0.378602866183646
-0.865260545959583	-0.317991245483106
-1.24026492930477	-0.441875979075075
-0.346903190549695	-0.31042141382615
-0.410818744689271	-0.187943298406986
0.0237824335760405	0.596081444589048
-0.583941495907171	-0.249810780291343
0.460632176020828	-0.283535854537718
0.157463523098016	-0.228957924394811
-0.781712617857425	-0.34239515863066
-0.444340370133463	0.121529022869359
-0.711881856857236	-0.547471768264467
-0.916801183260945	-0.0964620840131493
0.16691295760165	0.207834177168625
-0.351117521789303	0.202683058554899
-1.63642012612516	-0.0395369455278301
0.110071025766419	0.0256155187421331
0.0573558108690278	-0.647892415782045
-1.01212974442175	0.122201482952075
-0.761516121995382	-0.214916725093356
-0.303642612866936	-0.0511560506043506
-1.03329681588918	-0.241013892422922
0.0820216417692367	-0.390910152185741
-0.0708759370220229	0.236361928280427
-1.34713416822245	-0.783372851787683
-1.15088805265574	-0.392438157395338
-0.815277522797365	-0.0672949091953829
-0.5031567868733	-0.470253555801473
-0.940059708842379	0.0357589761902124
-0.0128009220441624	-0.415761908496704
-0.0898029022770466	-0.179919133289949
-0.134571039073952	-0.315125629908194
-0.600040994299499	-0.194196366303548
-0.244702378415454	-0.365607127358335
0.113804081102659	-0.240102519900341
-0.621944976633186	0.319793257353831
-0.892735060128441	-0.45989418650765
-0.306126202404577	0.247432613811342
-0.491306487202353	0.122487410868261
-0.649359578450401	-0.0367484972018457
-0.846168207149621	0.397138048153382
0.0557904202746147	0.354087237739939
-0.56953954007245	-0.060381655598481
-1.05014861705138	-0.268988882022258
-0.654104460090427	-0.506218617151044
-0.491471725648928	0.315977228320708
-0.78347441597778	0.0948479607092288
-0.822330278271283	0.247427300756444
-0.405504434196841	-0.155681704069427
-0.390353031788637	0.0942258603575897
-0.57309143439708	0.0410701623470255
-0.274527295925999	0.382609888411923
-0.259035340204283	0.0678700066505618
-0.593349954755246	-0.173531475091423
-0.492023688320957	0.586483619195416
-0.434453112541648	-0.282566139757421
0.79267666644828	0.0591134070891832
-0.418194378869977	0.379197403313785
-1.16029391643866	-0.102569394152583
-0.703978310866611	-0.0556119441541042
0.11437192850189	0.250514436052289
-0.614609295530691	-0.387489698146277
-0.548388879793268	0.220152458752648
-0.323216674473567	0.275526507738525
-1.12491125339065	-0.719239051175058
-1.65399357459322	-0.73571103711444
-0.681248694791215	-0.323631155366767
-0.362024471885549	0.223127985251374
-0.714260007993567	-0.141558477971752
-0.972555323505141	-0.652686661416584
-0.579025897084656	-0.110468217311216
-0.680908241360104	0.100288346370305
-0.0960659823187175	0.00968348351277963
-0.0412082444062852	0.301462634769669
-1.47507087987364	-0.534597719254666
-1.20453591169147	0.030950418450516
-0.900495238176704	0.124701050607877
-1.17403696776476	-0.449192805660024
-0.395342523998547	0.182913140057231
-1.51072612065986	0.0335525815312085
-0.901387718834584	-0.424401963140423
-0.470555881260516	-0.182592601843901
-0.612726085111471	0.231395196882983
-0.160317757279043	0.280083865419445
-0.842967239881259	-0.144588672480429
-0.997469266947572	-0.280200019376836
-0.486175225703562	0.0860718785914006
-0.934004178897268	-0.166889065724134
-0.867666080515291	-0.545464388064635
0.0752530065366917	0.109901064453156
-1.26794547523422	-0.694738129983204
0.501203543920706	-0.149568325057212
-0.22642968740058	0.275168164929782
-0.442617555646805	0.0794132421289495
-0.188570653312763	-0.617076531559693
-0.604322252153997	-0.388742280215809
0.063747636277684	0.23930550567495
-0.656416037345327	-0.41529261183086
-1.68262888097185	-1.22658855852758
-0.261956395130194	-0.317369263112204
-1.00722570297968	0.0237848652554708
-0.810575722799079	-0.86145381390626
-0.697958515083384	-0.0392345229983929
-0.889895915441262	0.17814896880113
-0.314640298215494	0.100615784165319
0.0377909025358854	0.24891097876919
-0.20171002785727	-0.426225803691279
-0.966447888830776	0.219249215341119
-0.296105904649411	0.194028309903795
-0.494046095101172	-0.000567654603401255
-1.09172593724477	-0.0153721724890721
-1.01723209358913	-0.051638133753829
-0.703610919841485	0.385593057307112
-0.222914576930866	0.137411956256332
-0.142631014222097	0.517140542658121
-0.28228836722461	0.577917197459662
-0.593111770298807	-0.578954457033164
-0.439756925271553	0.0997230772815554
-1.45235849117249	-0.244725424355198
-0.205150937386523	-0.354429524177766
-0.548098849835887	-0.638945178258207
0.494550811083955	0.621956347879379
0.0174586203099947	0.287636118534742
-0.304391639414968	0.0276930006234574
-1.12777051909137	-0.661862932584938
-1.45087766262057	-0.599278231683747
-0.655331480200236	0.0613795021519536
-0.068007123741265	-0.435967788861921
-0.709485418151232	0.197391323987505
-0.273382322045853	-0.305706893084058
-0.583941495907171	-0.249810780291343
-0.836715079336802	-0.31446312169075
-0.22325510720681	0.0404104503795986
-0.512618624036712	-0.25022071057057
0.0434060276649768	-0.0697037111965957
-0.484495001103351	-0.229050478752555
-0.97906992071342	0.428161557855674
-0.625259017360656	0.0921789429927843
0.269339408340958	-0.0649175519832709
-1.84319373994165	0.512617940486975
-0.878801465890357	-0.172434044426604
-0.870307825022779	-0.171732904088782
-1.38711032656977	-1.25277135671819
-0.806146235777763	-0.624552368509248
-0.38799972947848	-0.00882966504816327
-0.146366336887905	-0.0167831799535636
0.101608935841219	0.367894391613956
-0.361612538823125	-0.221811756325845
-0.0175162515875161	-0.117001061322341
-0.54296926464029	-0.32970764573096
-0.750984015014856	-0.403392971085686
-0.0492853538882249	0.629332923218948
-0.22923026469713	-0.303620182287057
-0.936957869056481	-0.611439064277126
-1.28691099791187	-0.457831775423917
-0.593790931345954	-0.526698838782051
-0.527272657330377	0.391252410917998
-0.631339952422465	0.86059168612386
-0.701370409170549	0.212687988188573
-0.57383155880091	-0.113390944387984
-0.398910046573865	-0.249874810431331
-0.158383472374698	0.232470969584706
-0.344216596679499	-0.207242161470866
-0.94891945583837	0.343058542326393
-1.02066826242633	-0.771413929254938
-0.739737922903402	-0.197672771010413
0.23296046310149	0.407088889153946
-1.00869046384403	-0.667323617624234
-0.81572743004544	-0.400963350023479
-0.55008465590878	0.00482672548730694
-0.690125241362353	0.0608221543774072
-0.273716355713912	-0.253815451789949
-0.470024699748361	-0.166612635779967
-0.836288489169793	0.202914189414702
-1.02951307626901	-0.265000883164976
-0.291783034002159	0.827343363799829
-0.920535158226057	-0.172973597479658
-0.186278850169108	-0.0844849584306312
-1.03235633419885	0.102615679182544
-0.101536410213679	0.348489675577887
-1.12567458869291	-0.672506456168173
-0.800830416992037	-0.886086727823785
-0.242953474965304	0.0510792157440268
-0.411059525487876	0.386038378142423
-0.733733091993627	0.330439794212717
-1.36919189709429	-0.0983444938597238
-0.204139211595787	0.23618821184193
-0.285368692649524	0.138164332632749
-1.01663848062816	-0.167474922004211
-0.659142587579892	0.0640975314664521
-1.67322545403301	-0.262641246647994
-0.437030052809379	0.130464127550816
-0.347876430157973	0.0121926090905279
-0.90390187881873	-0.134102875308843
-0.74972362566481	-0.528056773125261
-0.346099548915427	-0.691639695944937
-0.0163174603030179	-0.0606609538954822
-0.292566178669311	0.0250141770846197
-0.15845682826189	-0.615209405336781
-0.478149472809365	-0.147098838972458
-0.440831454255478	-0.0390580236813216
-0.5775236890567	-0.031572035920298
-0.798301842622016	-0.355742451374525
0.183197092826934	0.503359251249611
-0.763768885517234	-0.8983708981137
-0.334882986954944	-0.240897330741518
-0.505054026489884	0.0950250352761691
-0.302555358584348	0.378602866183648
-1.07223492422848	0.0246179331080031
-0.679413069221991	0.361522125093248
-0.366733491286073	-0.258848320879102
-1.35633045794678	0.417323176743522
-0.819991509159054	0.270162839959741
-0.516742841398797	0.327882205431162
-1.2353080495562	-0.222010843968655
-0.279843153898901	-0.290707760478825
-1.05720977754881	-0.0651606260583531
-1.00358009619166	-0.754259570204876
-0.585110375618376	-0.316074142112152
-0.210359810357247	-0.170465000588961
0.139159302057564	0.160578846354826
-1.08975784626445	-0.479049256195279
-0.948752390258731	0.254347126952668
-0.843162809910867	0.202570101570167
-0.896818123458841	0.165230321674577
0.136354488882966	0.300027111274438
1.02891662269227	-0.161360909788424
1.51150580784042	0.229168454894089
0.50660177197925	-0.555749209631
1.34038806973411	0.179607911604707
1.13264296940578	-0.516001479364046
1.12366217979101	-0.432246653378049
1.24607009191538	0.401245261639874
0.123772595682991	-0.258379959867754
-0.204005447304492	-0.531224796285017
-0.315568793570426	-0.25443622236884
0.762516363516462	-0.149958234437172
1.77753407310327	-0.37711373288784
1.63964217349938	0.0144384246805971
2.35916407600903	-0.592804643159718
-0.111758540102694	0.564049083486547
0.052918868981359	-0.348777402896885
0.312185753374613	-0.427648347218456
1.45592873790097	-0.306458130215866
-0.435555298321576	0.363700701760285
0.576997309955205	0.797851890016697
0.23931032477827	-0.348586761656268
0.491273956844901	0.235363765533379
1.94498575938887	-0.380117211584967
0.112341773046799	-0.496813955265228
-0.570270688996126	-0.750566317575846
0.82238560307846	0.301416367371366
0.00336863085063922	-0.0471319230288398
0.599782559192951	0.57731826229394
0.98549850072742	0.450763178068605
0.404455175676982	0.390667076292713
0.112548110310499	-0.460431508391567
0.212214949242521	-0.425999411936746
1.5185974457343	-0.180566065494233
0.583357663124677	-0.340025332686197
1.3022191827441	-0.328602559294682
0.505523237133409	0.105043690985266
1.18255326170437	0.407658883898107
1.28837011519948	0.0310856905955786
1.41505414454327	0.537282495191922
0.118675382185643	0.195796284833419
0.880122163255101	0.0177204378536371
1.39025397883907	-0.329975389461458
1.30674986663671	0.418847320495276
1.41805462315406	-0.562509896145771
-0.382680546082953	-0.346775215170938
0.963505435505244	-0.286743564045971
1.28259717187613	0.114548785134269
0.671098262821183	0.258159937856099
1.08361898878246	-0.194892500278482
1.14049687356198	-0.288936485373606
0.518618342723004	0.577818422004668
0.432228680552696	0.323572398279736
1.05675331368417	0.47669697355114
-0.234280273141669	0.858116950683178
0.755798563391036	-0.380448785350826
0.861179961792174	0.252199370907261
1.00784115186787	-0.0894436389119142
-0.101179738905095	-0.6078081069559
0.409030884977464	0.145712004827374
0.696289465620862	0.099356495420853
0.948501326073583	0.212030418047208
1.04360802990183	-0.291367941639206
0.914375019906229	-0.584011020425832
-0.131760442704968	-0.294669130559852
-0.835878033814477	-0.440190232548327
0.964891121740393	-0.666459997061941
1.39306591074837	-0.281950892604356
1.18685177750402	-0.139323356541725
0.938697586643044	-0.0308273268226738
1.51625803825025	-0.483762258643143
0.143198386020649	0.347293068209191
0.344059677588586	0.217038109854253
0.29313307533084	-0.0192989547156504
1.32707038911424	-0.198294275719107
0.151449415756241	0.0961416551568949
0.389795624482513	0.172523783185073
0.628115224666684	-0.476109069614034
1.57408535160596	-0.565529196742663
0.453682729579933	0.0783430790057147
0.936097444435508	-1.39187222196235
1.28837011519948	0.0310856905955787
-0.495763093840358	-0.375993753952339
0.0674324474917666	-0.468713606668481
1.0277731642119	0.0181832691955974
0.707257543725333	-0.359183589271581
0.360534610945875	-0.456202053107919
0.29313307533084	-0.0192989547156504
1.17366808052776	-0.0758491612105928
1.3081127690655	-0.247205706535052
1.18311962545564	-0.142267565019832
1.80517792517371	0.453167024161469
1.43080946804832	-0.138138065293296
1.26388019655183	-0.00662648626253726
0.857110391543597	0.0724025852373679
1.33289107432	-0.160834908078653
0.986709331554668	-0.0475600825386996
0.923252631260542	-0.103611384077997
1.38101092451968	-0.313580862586046
0.216406346044021	-0.278718903138411
0.9214900237109	-0.0173388746272567
1.07755511335204	-0.506528194217021
-0.688383705614402	-0.307061644559428
0.687697889023765	0.0687214357211453
0.6936718572138	-0.216650448345683
0.314945187731438	0.0598859063168546
0.399986452215634	0.411160038938177
0.610706360057114	0.120832403386163
-0.294481446185238	-0.530223937485347
1.17056473883653	0.497964564395169
0.907508919166756	0.00337272743654853
0.110745732550592	-0.568644753963513
-0.0361052842251059	0.366814667200347
-0.0435572740548426	0.369500093566832
0.649675454681341	-0.0524378467144053
0.828047458284044	-0.482228447523764
0.673718254486793	0.361846069053036
1.61269159286585	0.0134891041723589
0.607211761514342	0.583756554397393
0.378642833701483	0.222061396770487
-0.268086030496887	-0.532632575300117
1.30079764473178	0.523862916665743
0.583753709713647	0.499958862644783
0.830102124722051	0.254326453660638
1.16544691499608	-0.556762593950217
0.843678461216317	-0.179409354431266
-0.688383705614402	-0.307061644559428
1.11144464518644	-0.222288433345373
0.57212183475216	-0.50413401548322
0.219758010101986	0.281130026341707
0.354285767274358	-0.036440409330737
0.947347044193278	-0.805592679044847
1.09032451135239	0.301193944696867
0.898929295941588	0.100826259090046
0.23973753498964	-0.25938822891931
0.306487672093008	-0.442618698408416
0.673669326775221	0.0731318728713804
0.892147755265064	0.148984959731863
1.60163173022421	-0.502268285146546
0.138387000116671	-0.390330286686844
1.12850352448446	0.70234574229956
0.279062494140466	0.118869825646718
0.40452307355407	-0.136387759325536
-0.00721954342345835	-0.914095793865432
1.09032451135239	0.301193944696867
1.39417444185092	-0.192707655383242
1.03386691060155	-0.259908360922694
0.184828531292771	0.846207480332284
0.694592867488986	0.217259882893668
0.638428544339261	-0.340433805508392
1.43212175121951	-0.496172412380065
-0.312636800094352	-0.202975158263376
1.30303440829284	-0.454092807759105
0.317384862515308	-0.226598092737348
-0.843940980055015	0.141466531623869
0.149634867560027	-0.141166334253472
0.295966496471648	0.303829025228298
0.892147755265064	0.148984959731863
0.344983603460756	-0.307154053144841
0.544647218377499	0.147897348973898
0.68959739557039	-0.26825875588071
1.10140888233851	-0.0174851438634549
-0.194050604483324	-0.964011477556963
0.179284649662135	-0.0867891942982766
0.639440823592775	0.263355225955278
0.618037099184264	-0.0317043827487345
0.591226975701124	-0.290758608987989
0.213615304518052	0.204321044094969
1.30076582335748	0.964657922162786
0.0402034688963415	0.363708641858521
-0.28921805640801	-0.0919478886210905
0.518595516987051	-1.05542637197822
-0.337384945157001	1.20553671705902
-0.210408702589769	-1.01327317253198
0.572040083821443	0.0868311749931643
0.930074611682433	-0.275299904803289
1.03693160689359	0.377536088175398
0.0172589623711284	0.444388316779011
-0.578434544675824	0.123541849813224
0.307855771469398	-0.00697818890143007
-0.276931871584119	-0.354268852382628
0.221402934170052	-0.0562073046300047
0.687180194987663	-0.203115309722512
1.01757516142745	-0.461642526141267
0.771097579269401	-0.149573503578562
0.444336433198522	0.0334121832282247
0.502356643185678	0.17789816942998
1.09051562343006	-0.376850953759467
0.480212251794848	-0.826972419107725
1.03782355962474	0.338408277118941
0.78046523325016	-0.467580480820732
1.1495372290014	-0.0549205288847783
-0.0273815030625976	0.0513751917071517
0.202639083793684	0.275316944279814
0.203220448828937	0.592809243665271
0.248138853506782	-0.284365795635902
0.288979813235832	-0.191390539771466
1.37929125436773	0.648273257904212
0.236674209005078	1.0229381034539
1.26051425389824	0.0585439262915399
0.280062120445351	-0.808749262017737
0.0850682921538871	-0.274456532443978
0.467836587282053	0.182692471539762
0.00520965191456123	-0.00906526414943205
0.446679248991045	0.360472759482981
1.29984907460332	0.0644187425369331
-0.410699599414835	-0.36709220674729
0.484798076531412	-0.0843525284749814
-0.121651137541597	0.896980000541037
0.514640057204019	-0.1229237124048
0.150748041663391	0.469823574969289
0.575866938042184	-0.0586661009991003
0.121408347660576	0.258083811144645
0.662650489890462	-0.0379304433151064
0.750903233771818	0.197167453568236
-0.578434544675825	0.123541849813225
0.0830872314501185	-0.543013695478933
-0.327380231347849	0.600986185552342
0.242591123295823	0.0595494068231886
-0.254135226823003	0.463668350985936
0.111543653693546	0.542233821278883
0.676580582275916	-0.124574663823744
1.07587782330727	-0.330955269814501
1.22462273785667	0.0541379008236341
1.0455671958089	0.254132169145029
0.0397680943607468	0.301165101001564
0.43617965272799	-0.15039043454206
1.30705819978601	0.363437695125683
0.279062494140467	0.118869825646718
0.204090109265867	0.376450894626444
-0.085300338946002	-0.211298898644353
1.32255649072742	-0.438030868974781
1.44917943079497	-0.361787648573831
0.0284702455209376	-0.704993716374383
0.6936718572138	-0.216650448345683
1.52948444736237	-0.28655517079273
1.06820775523012	0.331272415332861
-0.400246224475652	-0.470450689137945
1.42063694211502	-0.155896322564051
-0.246803438336367	-0.945502006501851
1.79567265395978	-0.391320962704343
0.989729796739711	-0.314100475940557
1.282977674219	-0.358222297157546
0.141739162089832	-0.316265468727325
0.79743281272366	0.0723596650825043
1.45652154591614	0.100905309287138
0.592325437239856	0.0997304375267819
0.26053393816338	0.11572829607189
1.07242942950545	0.267429350939739
1.10140888233851	-0.0174851438634544
1.85220597947345	-0.317684699322557
0.791425832127044	0.0603287989290223
0.704524894585576	-0.289567292038193
0.241448594725256	-0.676858422347869
0.786782829849129	-0.198349718919754
1.11418153420516	-0.426718700053244
-0.507311544261758	-0.466386870655725
0.712684633823187	0.611342989865252
0.940293407134869	0.0795035993482722
1.43395292335024	0.145820947360551
0.57212183475216	-0.50413401548322
0.677029498922302	-0.679610196296801
1.44489194912446	-0.146591620356855
1.4859055236657	0.0256814596725282
1.98921782406712	0.0147322380862467
-0.232304196793256	0.453091478141135
-0.232438754545719	-0.471024768900775
0.478864862215777	-0.0606815197947122
0.83762592148985	-0.0213769479499999
-0.79255926894151	-0.657479047237418
-0.279655603163559	0.105299721431587
1.51150580784042	0.229168454894089
0.491273956844901	0.235363765533379
0.327745710520916	-0.347641986499078
2.06313484329502	-0.0813768681721155
1.25079824919765	0.0150094128007815
0.62713935346032	0.133852142519059
-0.24281244047087	-0.713166464226437
1.30236114155686	1.00357925751247
0.655895170279735	0.165343203009277
0.320034989512422	0.089881614472216
0.828559016385886	-0.104124424692745
0.448647386411823	-0.119844485139259
0.462004497822712	-0.244665189352484
0.344164464978325	-0.717184773958953
1.47210691601678	-0.362018634929714
1.33956067414722	-0.0848752750273754
0.440035036606341	-0.0801198894698963
0.554168740070815	0.118814037366326
0.997435609756994	-0.449860672135672
1.72782214226679	-0.252237651709682
0.770918077186792	-0.084931543750814
1.10857049131095	0.503621582559096
1.49765656368571	0.276651989174631
1.20141407351538	0.316591304860811
0.274456452762239	0.0369205878181687
0.066563563136586	0.0620675061279583
1.07653737103249	-0.0557972723118412
1.0468856808148	0.0594781260939529
1.17324477072826	-0.341182172430536
0.510980386108797	-0.308330235475328
1.11909326631022	0.226179913432402
0.754343320613946	-0.197809399821428
1.0767088918624	-0.25403159847556
0.993410599837847	-0.097063656925065
0.500411413249444	-0.244086025456991
1.42697027230926	0.35800286068211
1.39306591074837	-0.281950892604354
0.967230618876723	-0.27678043529339
0.377982808045252	-0.746001161391842
1.00670539509982	-0.0474978333058192
0.446387729763729	0.0777985919114062
1.17252393222488	-0.177830560311052
0.431487191769915	-0.166391080888251
0.829920006345424	-0.165505040946134
0.828772709293718	-0.275756812211525
0.892110585242607	-0.120496194425761
1.38561220713295	-0.0188647041023631
-0.141651332189302	0.333163699379247
0.0613694711555104	0.0703536179952175
0.12967306348229	-0.310553590289434
0.139602491035886	-0.174698991670984
0.436305140732022	-0.36535208115114
0.9045300080885	-0.0955161684182874
2.05565067958597	-0.291402650085077
0.482101498172066	-0.27879524473489
0.713425343768028	-0.653548319077935
0.707257543725338	-0.359183589271581
1.07874206065987	0.322498568683082
1.2081943504625	-0.241820474885944
1.57408535160596	-0.565529196742663
1.66534658841202	-0.667556029936879
0.833582824104436	-0.183832850980847
1.33289107432	-0.160834908078653
0.671098262821183	0.258159937856099
0.454335562906691	-0.217251519240468
1.6969305086543	-0.5408655355891
-0.382680546082953	-0.346775215170938
1.31302300922838	-0.228343261539532
0.130182203570763	0.0132761383557138
1.07755511335204	-0.506528194217021
0.933930656613986	-1.03348787891938
-0.182999650153992	-0.124341149212884
1.23791472938328	-0.228267243828752
1.13403588208435	0.588321000066294
0.942162345463469	0.362358776340464
0.84152207257379	0.171400021762325
1.03121036856048	-0.170624688059171
0.883558277419228	-0.0706573344265436
0.344059677588589	0.217038109854253
0.706669809481822	0.0805307173970417
1.4047114641342	-0.707080670025119
-0.0205939404095009	0.450646623555113
1.07957709257501	-0.351347651079327
1.50268007198481	0.318107152785955
1.39025397883907	-0.329975389461458
0.897824234162529	-0.606768687058434
0.892597883194832	0.1778909406037
-0.685967366306182	-0.46640763150986
1.29809129665395	0.144582090208376
1.7088045843636	-0.181428077943132
1.12841476349315	0.124294991697155
-0.609883435134516	-0.156326666300216
-0.00221034430022747	0.0730643957619504
0.178591675730619	0.636030961373511
-0.194492732882075	-0.264140621964013
1.62935854561252	0.153165452178596
1.13353685465737	-0.346841161589487
0.197812879501574	-0.854063609280587
0.971444662642411	0.12675781067443
0.764436856589115	-0.785299452720101
0.491943604754502	-0.598690858309666
0.92653608718367	-0.650835502681981
1.50870728801584	-0.381246537643843
0.778548641616745	0.262775936126627
1.06355402501264	-0.319453934105786
-0.141632587748621	-0.440194274081161
1.08334933078045	0.103216764086887
0.699791325534439	-0.119419174211448
0.105848323546931	0.0486034683197545
0.267485433548126	-0.124793138013712
1.42336812407225	0.403376778464298
0.459915502513923	-0.243474040498373
-0.199739704489032	0.385032047619292
0.0716320353075122	0.208253573648524
1.30490352416448	0.659110470012099
-0.448070646589281	-0.168265231721808
-1.10169485681883	0.524396883780341
-0.509313537454079	-0.270502375479089
0.0144593841441221	-0.21925308464309
-0.165234126036073	0.334325618527366
-0.979356203490733	0.0521487435457008
0.273264458383306	0.111432773794747
0.436531708387347	0.452811827280945
-0.440259504305336	0.264152403365096
0.08421651295832	0.275725755451687
-0.0377029618393111	-0.22724072412653
0.275960632907797	0.131440648889086
0.528532465406905	0.125508638715973
-0.00398782955876041	0.0294001636934463
-0.160755844107693	0.171508632647615
-0.509482292065837	0.0524995851982606
-0.372800192706267	0.0931067016645943
-0.239739493059592	0.637428688087275
-0.252851078772046	0.303422740692164
0.111929863406488	0.0440695501520459
-0.400899333425513	0.0126029640726971
-0.11999142821512	-0.544111078688998
-0.133514907056191	0.196871251736972
-0.0531300836663759	-0.229347821887098
-0.446332178413803	0.362747620473299
0.0864671894294721	-0.154918668366525
-0.21352199393214	0.489903948437149
0.233059899751606	-0.28508798037277
-0.320754167602744	-0.016668301295662
-0.228262885703423	-0.31744432035774
0.101249000881769	0.205748790091926
-0.424044358163396	-0.00216466247678624
-0.0393355581414137	0.018197667626987
0.355840163922859	-0.0776788329936391
0.0697867730055378	-0.10549412783441
-0.238356231282904	-0.0852214558400798
-0.499504996674377	-0.236767820031192
0.0161428893489808	0.387628254406793
0.443975125453142	-0.262647052755798
0.235117392039187	0.784285980184429
0.0311342896615718	-0.297771255861381
-0.504449268975905	0.247585515410218
-0.509102037244752	0.170230470853509
0.194158365138461	0.36437490396046
-0.256842629933923	-0.0972221768243172
0.311785096866615	-0.20900375500777
0.525525640746961	0.101349699321372
-0.0408493206495802	-0.0862177643513213
0.204352790421635	-0.1522008633094
0.122235313455789	0.314609103527921
-0.335383464251793	0.774178890210112
-0.299999404297116	-0.169168447708481
0.23754901777713	-0.132015421468193
-0.150896171373157	-0.0452086499769392
-0.307594909871047	-0.352472321470839
0.627021296992548	-0.144055852117272
0.725910779719691	0.282879097780337
-0.0990574784185104	0.314118756587033
-0.582726715804203	0.169219272771562
-0.189032880818709	0.102822849987948
-0.141308891407039	0.686363029333964
-0.22417187190854	-0.151135978005629
-1.04853919976037	-0.482072882126142
0.331988162273706	0.163275767529069
0.282608887415519	0.0913435078344145
-0.0858648554269344	0.517338495210097
-0.217466279148387	0.675790923259386
-0.0956601647997443	0.0768340421268949
-0.654898623537806	-0.0169666760096204
0.0505770959825381	0.668731659308096
-0.488562790792534	-0.231934045271912
0.288651577585219	-0.0277084766893596
0.143692958695413	1.13828140933548
0.235028329848776	0.399052343488864
0.515994298672953	-0.252788182927192
-0.974172070913201	-0.0500908553954499
-0.0435804759284897	0.446474252037134
0.267831760210327	0.556246802408267
-0.357645471355258	0.255024612526778
-0.683366468423467	-0.164535435416935
0.135814252910677	0.160291273447218
0.159054465459557	-0.330158237955321
-0.466429750251947	0.0163486151641182
-0.912669867338897	0.420524037988091
-0.242426855886734	-0.0354406325092962
0.499697920908375	-0.146164326100345
0.246670636134187	-0.197300730487963
-0.0161370571327277	0.509824283850537
-0.41048073231579	-0.0454499949601
-0.387736467871346	-0.114215754011601
-0.548560880988237	0.275229344717056
0.167930984208931	0.051704411854866
0.272779570961681	-0.357807150143303
-0.331777864482439	-0.0838409465289497
-0.171281040169858	0.0990461002502383
-0.214105765137613	-0.752146614626216
0.156769558116663	0.332869730868343
-0.776979312344707	0.227784452752988
-0.00130391894725459	-0.0191330620553349
-0.023326629879536	0.154959876001255
-0.903589510398426	-0.246958366504854
-0.105427846405663	-0.606755765667493
-0.752394376432849	-0.0901461938302222
0.176925533947857	0.171266611973894
0.216016505702743	0.569784166148981
0.341049491221173	0.0601098000527485
-0.336789928728783	0.124065494444826
-0.411003352220639	0.235102958267877
-0.501085431827115	0.402484298985354
0.46792961687069	-0.23010201647486
-0.0803706645710358	0.285166934830639
-0.557069329698063	-0.361205291015194
-0.00661045662449701	-0.235605476124431
-0.414351968842845	-0.0293219669205939
-0.358589673455486	0.0699856151955916
-0.343009050171289	0.00172133039777272
-0.208226982263695	0.855102545717862
0.288953136442905	0.395934224802515
0.566225899185247	0.882661510722616
0.277301764697433	-0.162905852106081
-0.742966609421303	-1.07457913894798
0.0417629957355397	-0.113048737528895
0.194832920688971	-0.301646844367969
-0.0699564412091908	-0.126862157130996
-0.189421693603804	0.528427513476018
0.0340960501895907	0.169791462882434
0.0247310560319819	-0.380040670823152
-0.485791638480081	0.0689603571633564
0.743968866838905	-0.0209748689395262
-0.397330635055278	-0.183877406114948
-0.350310139219482	0.0710840482211845
-0.409018782884949	0.389820227183247
-0.119747313946553	-0.223246262320313
0.406466422763487	0.432891231331383
-0.189312620810548	-0.0560826658994634
-0.0872497043176925	0.579780508367031
0.0476985194707882	0.133104898870656
-0.511352555264757	-0.081374580616132
0.2667587301001	0.00597310272506015
0.477040028416833	0.0767362846445678
0.0800866998130989	-0.0136634827114742
-0.170354040791751	-0.0852033673727488
0.112895505305005	-0.162988976815184
0.746057411844271	0.0454948722291726
-0.599630109439214	0.0330113143964303
-0.218477243730135	0.0988735917651204
-0.179747402905516	0.143882321597497
-0.199859311728098	0.37516134830454
0.154873923184383	0.249514529991944
0.341060499804634	-0.148962918278422
0.283733170915995	-0.0269972879323578
0.0778348932582219	0.18412936760423
-0.174214906531748	0.000830450474985712
0.476170687974183	-0.477307432405531
0.279500036603369	0.489414429654508
-0.389678860761801	-0.290376065151719
0.361563583838441	0.169663191292814
-0.187142075394076	0.252211085288344
-0.161191160882366	-0.214195591142211
-0.790440535969831	0.180948681759778
0.128096845993874	0.306594189814105
-0.481850634046773	0.12889413681074
-0.00861538091360424	0.590301157010469
-0.0592114670641158	0.145044073700693
-0.0166028578330105	-0.167709301309488
0.00445546796857219	0.235922237379737
0.18753721655776	0.100906526458049
0.475169781686357	0.409667811097599
0.111204092524179	0.279948053371344
-0.108212895005461	-0.587686288968219
0.0133579017869869	-0.417318790466946
-0.232212176899912	0.521128695501226
-0.241835475614895	-0.0225927432842074
-0.307264822310865	0.271742893737204
-0.465247222037845	0.136499777766966
-0.975664815210477	0.397728674969125
0.0697139451028886	-0.0337922409281317
-0.249828898435935	0.65538573343835
0.148338492612863	-0.186675854748095
-0.271629663065315	0.440835191680324
0.898493915331022	-0.0564787249288301
0.0439004609317813	0.351896904679852
0.451691465308304	0.178372615329927
0.115562078357759	-0.110890433900829
0.255611013099933	0.45011699676513
0.515242734939607	-0.284807937954514
0.010533211252768	-0.368077140274528
-0.414239922829724	0.335493696690379
0.189570070790182	-0.1775073388878
0.364727948056511	-0.441102277344015
-0.859574383337672	-0.132433092025037
-0.118589633783233	0.0133090130010909
0.24888304125517	0.0141007270245194
0.268212455744324	0.420895778702964
0.163429536325235	-0.159110211108765
0.580706379048911	-0.257139244980715
0.214545914104111	-0.250089640094402
-0.0538723067218659	0.509324451833023
0.133100161553545	-0.237670899244415
-0.262352605286372	0.140806658581492
-0.112767329400295	-0.234369158144071
-0.620861484790346	-0.524771977235136
0.0704992651824972	-0.188741916190791
-0.421189459453623	-0.2838365140056
0.397303952535432	0.225217245597493
0.283826372720273	-0.644853070225374
0.0667828714804492	-0.0265639985289637
0.223598163200849	0.214786495634648
0.250146020099163	0.380914029281277
0.00880758428552542	-0.289937223867357
-0.163951811056957	-0.0445352781005032
-0.287178284126095	-0.212122761783299
0.0531658478391651	0.0443035859910076
-0.108942096231963	0.264636608847618
-0.272887911773349	-0.122957407757138
-0.568689982532575	0.760743882539099
-0.382417371178656	0.185040009248777
-0.0605180297911512	0.114520976052765
0.224416939113028	-0.701821971985422
-0.0137246333018729	-0.339836584765921
-0.433773639109012	0.253915741929398
-0.290125915552388	-0.120498344180622
0.00911046901886715	-0.32022022787675
0.148748415527648	0.25309670225963
-0.0355925643609628	-0.147056647760545
-0.189553430650044	0.0466906677881755
0.108637240604354	-0.0255108395261051
0.330552822266772	0.314857543214661
0.524865355526104	0.146206630357704
0.186619865646355	0.0789920731151086
0.508295460536806	0.792192813272828
0.134686236437631	0.113830462954915
-0.327879366751915	0.0642616907702365
-0.170024569397837	0.344760245797105
-0.480057297398485	0.0532689837483764
-0.191686737219495	0.325143594995777
-0.0799770173055273	0.161761360768767
-0.869943811978629	0.237105497260359
-0.174570134243003	0.519282245632787
0.219459508006706	-0.429696261848987
-0.612245090804483	-0.526287731625305
-0.138229790987789	-0.277827580829881
-0.181093499487329	-0.341263142972956
-0.0630431337300302	-0.114166232047031
0.0318262008763486	-0.329317579996556
-0.0437638800791149	0.705727085373809
0.242269527168859	-0.0755107572966178
0.165688643464023	0.0306544395401607
-0.168147170408251	-0.211227939800488
-0.986383601470662	-0.11752081387901
0.0941237557928631	0.12304101542247
0.328467492336564	0.0865551571122245
-0.0605180297911512	0.114520976052765
0.281870669910229	0.256478870774877
-0.0516705365631748	0.185524064267846
-0.00669369575456151	-0.149936715707839
0.402058595387175	0.109827115915845
-0.154128591571415	-0.131264291576748
0.153387039267617	-0.0744224725689994
0.0663251552992104	0.201720294103508
-0.709285118290978	0.320221012594914
0.243012939462867	-0.298703020244367
-0.00872426213505193	0.329747708719933
-0.533338186276626	-0.0660994520882179
-0.189553430650044	0.0466906677881756
-0.521195371674101	0.372645241629221
0.616389453777363	-0.275307762143447
-0.498255691748678	-0.383591553361379
-0.424564638761112	0.385916663183841
-0.0998467934279876	-0.569665823498802
-0.27693065507734	-0.171913370250184
-0.609344539670393	-0.167958366918132
-0.346092663031978	0.0278058169806173
0.0794601965585991	0.437895173729284
-0.0923752892541021	-0.324857920483175
0.270454885595153	0.211963877008586
-0.11841676071073	-0.265950576784922
-0.339606718052957	-0.460982728254391
-0.811657851125663	0.306987209563609
0.064088503105772	0.244509019700028
-0.0850033559505173	0.502406961980074
-0.00539792893189181	0.0152283528436737
-0.333168319104816	-0.114202730161751
-0.0501620912105635	-0.0781179417204024
-0.466429750251947	0.0163486151641182
-0.197346097273686	0.515040890766432
-0.208226982263695	0.855102545717861
-0.28695391687787	-0.0978322461480563
0.425243167046627	1.0470019645017
0.0855816570648378	0.566802210352499
-0.411003352220639	0.235102958267877
0.265583779523707	-0.0751522534591769
-0.206212200485406	0.274640788416632
-0.617433696076895	0.339595432804193
-0.101538655581865	0.0081942890003544
-0.332008763215395	0.540042196934485
0.607029390350261	-0.322948138873581
-0.582726715804203	0.169219272771562
-0.0934142501685572	0.296746880076549
0.0800866998130997	-0.0136634827114742
-0.124333312460795	-0.313806084832959
0.0240650012695114	-0.136560757803472
-0.0841028186325778	0.0592835112735601
-0.452268949681249	-0.132578809429063
0.182750499246596	0.209164329565333
0.0137052933177115	-0.128636472163291
-0.297238730725603	-0.204833828963051
0.23982807773771	0.118730571397246
-0.256223750508139	0.215110977095018
0.194832920688971	-0.301646844367969
-0.654898623537806	-0.0169666760096205
-0.734353305211831	-0.0140756899797012
-0.509102037244753	0.170230470853509
-0.359979843615739	0.138727562044661
-0.635285206998832	0.0615792245596027
0.323876660687216	-0.313674268542854
-0.788159815950138	-0.0625981131576894
0.122254464072221	-0.338672320040254
0.214853674727289	0.295903825803636
-1.10110633884818	-0.362580247779167
-0.382417371178655	0.185040009248777
-0.533896348913532	-0.260173976817205
-0.333834358204959	-0.494706333690982
-0.351200595231575	-0.317639458891771
-0.302114777943685	0.0482460436876704
-0.472142834299332	-0.381097294887936
-0.149507673790305	0.0576908965332531
0.305238966653924	0.112208156856454
-0.136577142532099	-0.362291822949102
0.183541172223465	-0.482673552277165
0.520815477941228	0.296350414353909
-0.490879084426576	0.287110284781426
-0.170631072787045	0.238246905935809
-1.13741727489737	0.190661691966976
-0.473634961147581	-0.2808909703699
-0.599792542657386	0.108952179391098
-0.0161370571327277	0.509824283850537
-0.150667225610545	0.14155191752399
-0.438250941795747	-0.184750798194259
-0.0190768612461442	0.39795293269062
0.00911046901886715	-0.32022022787675
-0.336789928728782	0.124065494444826
-0.27554467737684	0.178049831606623
-0.19706853680082	-0.478645846835567
0.299537462465307	0.197125648348532
-0.191854723304373	0.575705609345287
-0.0307219168622407	0.388062885610407
-0.909889274988957	-0.0415582656798175
-0.496737323652011	0.141581807487762
-0.691397730147451	0.195880614853832
0.273264458383306	0.111432773794747
-0.0629519326128707	0.407979488819002
0.0665894360696277	-0.0874657241963995
-0.0435804759284897	0.446474252037133
0.100182160621332	-0.190541868465803
-0.400899333425513	0.0126029640726971
-1.04390837186352	0.26183012635178
0.0112300590353009	-0.142947067830504
-0.165234126036073	0.334325618527366
-0.468188795473523	-0.297334412862521
-0.468981462091895	0.0164662734438316
-0.175397747019814	0.667348627856735
-0.062141200907706	-0.0775073286955989
-0.0409644085259678	-0.0965197653791397
-0.878476141875679	-0.406540845932472
-0.390509142223571	0.309579602221983
0.46792961687069	-0.23010201647486
-0.0660488554980744	0.362531597729505
-0.281198661132936	0.1500821801903
-0.124895820177544	0.0655148255269709
0.217190686424868	-0.109761605020326
-0.455604335391444	-0.0233965324921825
0.123594459403791	-0.426048691213718
-0.383211946739407	-0.485806737774284
-0.186695312023078	0.42945783214362
-0.512705677959276	-0.01738166191629
0.230673009573324	0.0550270765341248
0.0450716401989433	0.3634787902588
-0.332916783077074	0.0655859995108794
-0.315862958917777	-0.140384709985807
-0.473377810056876	-0.103547749424441
-0.222759469437879	-0.235402354958116
0.275999267766128	-0.094292177592852
0.373122583871554	0.710713987978772
-0.331775959466634	-0.213177386581879
0.00523225911857169	0.394482887650947
-0.398625859383978	-0.292859388361183
-0.313437385345777	0.455672620506477
0.275759214434661	-0.478471732740008
-0.317048421521511	0.57088899622277
-0.392428414356305	-0.345893821625691
0.121543366918948	-0.299007697361068
-0.77673126077916	0.266187680863524
-0.364241540643762	0.253980660931141
-0.325828551875322	-0.0788301405168597
-0.29424345017472	0.348050289317748
-0.390258120856364	-0.138132630787467
-0.420632051836159	-0.319385030327386
-0.262691458158218	-0.138221892725023
0.287054074011521	0.697188096261634
-0.426505327331482	-0.170644404633298
-0.228107353605852	0.220920674421741
-0.694924822551943	0.130332093346812
-0.992417145142071	0.276219573524898
-0.043594336998238	0.238067348641017
-0.0477444404398213	0.165386985914644
-0.407895972938146	-0.0345725853830049
0.131870111673098	-0.432551862203397
-0.208803704745364	-0.356438487908339
0.237950875881506	-0.383004679886525
-0.136744178349281	0.0712239732087985
-0.00443773647835193	-0.0321380502832036
-0.835300787886463	0.418314834617914
-0.43209712491239	0.276642332244179
-0.430043565325023	-0.0434274701739652
-0.353978112234483	-0.595449394763565
-0.316604383298823	-0.202459176711264
0.317290238566233	-1.47505776086824
-0.494034591799712	-1.01327972737346
0.269289158812978	-1.63454350621737
0.63652943615079	-0.278880036733418
0.268027953773571	-1.07004320313006
-0.0414956427004578	-0.943655555041291
0.545980269262617	-1.10572883013127
-0.166751045462678	-0.924949235697646
0.555631269062415	-0.973880053756943
0.76519092621013	-0.968709802645453
-0.0286794934364981	-0.916554538432407
0.279981295400457	-0.548847812384873
0.44169211769199	-0.558897419535557
0.676951141049229	-0.953276146289578
0.322294986030032	-0.777639710136079
0.360638269341515	-0.736555359652923
0.159963930888122	-0.328564271400882
-0.313946377372376	-0.865609450874531
0.426396399752981	-1.10345369963686
0.279981295400458	-0.548847812384872
0.0483717088719585	-1.08643713092165
0.580823157204316	-1.0138183056168
0.349575890986883	-0.221421755945499
-0.476676095781641	-0.420066254825465
-0.0154165904048144	-0.890921754404127
0.553321778960335	-0.224761290852404
0.401751541460349	-0.713880352916225
0.413905522984632	-0.671396286862334
0.432130507424214	-0.519976035777409
-0.0828385743984951	-1.79448504566276
0.29993881356318	-1.28179221510668
0.230802381028323	-0.724581407787014
0.728200684928385	-0.649558044220428
-0.392635144477974	-0.815027038344555
0.571362309676348	-0.197412501415201
0.177425568209465	0.00948983481594066
0.743007683612703	-0.97958148727412
0.621511279512501	-1.33085104219687
0.52853016944052	-0.12043056943235
0.591086184602786	-0.814884765275235
1.18679846522146	-0.724399068638584
0.778255852911828	-1.35387357184229
0.178456236872392	-0.901164432973552
0.478898566129675	0.425384047989229
0.790941017332894	0.668486953900232
0.665577812253913	0.391411281609026
0.1664403798485	-0.319410413779043
-0.580642344552658	-0.317946909896571
0.310696451713551	-0.192507524355682
-0.100383562929414	-0.339852794575113
0.17459840803275	-0.43921850116013
0.445922046329056	-1.07636042633061
0.0116711026485523	-1.50457800058139
0.645686777723595	-0.0192789752136222
0.441967499405891	-1.45316314511438
0.481561816196729	-1.16423285365836
0.550916818320239	-0.618920708999959
0.866953992322028	-0.715503426478856
0.606554422740145	-0.990642316689636
0.235189121402613	-1.89623240370433
0.338413254913352	-0.760153682235529
0.532840699511538	-1.37577990921959
0.474696666378795	-0.917621021844287
0.249951432540849	-0.784981052190765
0.455089000235483	-1.49721068153771
0.301006971872433	-0.827282111913866
0.445064465909581	-0.409544974129279
0.158688418551203	-1.09883480868437
0.46209464660285	-0.969048711906962
-0.390976125121412	-0.5760010753354
0.290020566917649	-0.928896016862109
0.0255224225749791	-0.538999125495727
0.221706494985593	-1.45637013057858
0.177819283655913	-0.860910430133273
0.405433175497357	-1.01405255273701
0.194681883019027	-1.25396139628879
0.553130604295834	-0.264656476942669
0.275191128538118	-0.498134937435312
-0.0654563102750099	0.491165761495084
0.829510480647843	-0.966465685465821
0.321627801310713	-0.890267370224291
0.645071889392508	0.0205499019178114
0.24522974758577	-0.939068459340563
0.445922046329056	-1.07636042633061
-0.0120699367929899	-0.912929824514561
0.436414937673567	-1.36543447489908
0.155819686041598	-1.13301814448149
0.310696451713551	-0.192507524355682
0.414391264727733	-0.439115033901439
0.126014831658431	-1.32156879641479
0.744427819443755	1.29670375316242
0.451562239318854	-0.817840287887518
0.559589167164211	-0.810526042656155
0.342237744051439	-0.836135740539446
0.729901118263205	-1.15451545845476
1.06372791352441	-0.935291395299154
0.588726090085561	-0.97291670684592
0.207341895283918	-0.669740023518446
-0.111388952208837	-0.38298855519492
0.112556282954996	-0.21419694761224
0.693889069916766	-0.312779540146168
0.90691620813106	-1.30132877898897
0.853451907278019	-1.22787019237263
-0.421869115432552	-1.76525694902352
0.859527609579082	-1.36066626681496
0.363449779601082	-0.278023484379224
0.902173915007996	-0.962472736229486
0.483840158071649	-0.743848797056205
0.565451297182181	-1.09247466234625
0.427142544551483	-0.962992144874858
-0.114307965977575	-1.54529485826079
0.196898357302845	-0.631224216107719
0.329422062280374	-0.78241021381465
-0.337169694221	-1.32443854329369
0.434170654876578	-0.857165456817249
-0.221648072863206	-0.234968486844132
0.157891167430073	-0.114370526828615
0.147105827600697	-0.523348957738734
-0.39061137397677	-0.461496655759759
0.313919307951422	-1.25679067318499
0.255114217371138	-0.633461267747591
0.84762394048124	-0.864377150570226
0.438498694084743	-0.402145245566094
0.311269331777637	-0.0773584279760965
0.563259744255599	-0.557747297660443
0.119812594493197	-0.284087154485655
0.390500674751647	-0.793737688664396
0.386154985628261	-0.884599960033741
0.613675558701962	-0.925712485779354
0.487684197476517	0.0674703610788805
0.774049935509277	-0.517240465515705
0.202915165231334	0.240691446256599
0.492766078418535	-1.05006850637965
0.571200508731729	-1.19231211894532
0.657083830778789	-1.5208841953073
0.958860399432389	-1.39646254780416
0.498433338836481	-0.863598595285889
0.0388436628092482	-1.58023942397966
0.305069611733062	-0.265504271494605
0.387674335817918	-0.836981449841706
0.549008787041093	-0.999053246617788
0.411703876533058	-0.586143143222505
-0.190595760112983	-0.751519545604062
0.413483620558352	-0.730911793506177
0.935936598726554	-1.42788777018159
0.487174796479523	-0.552287944447591
0.477945438263206	-0.677229696892451
0.628250974901554	-0.0845938464507609
1.08485984096047	-0.314403466983095
0.856318326065167	-1.57886923617578
0.265542600818469	-0.890746058201104
0.223853574131146	-0.169752315956154
-0.251581517185433	0.00593920494223459
-0.0849005911376107	0.143113524715783
0.572690059531687	-1.02670416278711
0.178727423536798	-0.773083999704926
0.478352699722439	-0.80078320335474
0.455228189479503	-0.597980727812537
0.5758166238793	-0.495102966582535
0.295178709569752	-1.30409684616521
-0.112998389109719	-1.94257225992425
0.220848902513484	-1.27505109216566
0.468691536704641	-0.504599806812795
0.692568709516653	-0.947179631889181
0.553117211101459	-1.14197211364661
0.382738860353016	-0.744633144619705
0.697700018299157	-1.09234162228229
0.388514290269985	-0.530784122419244
0.719805078265248	-1.11341316305272
0.538228372162427	-0.677073445732948
0.167327128906505	-0.914122914174805
0.444036442838719	-0.280217950591897
-0.544840110330329	0.143711228902332
0.934364866423021	-1.18172160940455
0.333954775479513	-0.503360218898888
0.753817861645538	-0.333610878811107
0.773015068160231	-0.546577717871328
0.708306357786941	-0.310689189534455
0.539884247958748	0.222259485561919
0.539884248838352	0.222259511810239
0.317533238851329	-0.254366176179566
0.317533238851328	-0.254366176179566
0.651397796804446	-1.02972426588234
0.808837088468407	-1.0310538601603
0.0369995590722791	-0.711308798245455
0.679476086262623	-0.966911450784637
0.229054939548288	-0.922651871285913
0.0866849556118137	-0.233206482267508
0.409457254855164	-0.731938340203882
0.320467745156714	-0.403559993726194
0.598382359519522	-0.399105058251521
0.502946125516111	-1.2279598667851
0.114430681526815	-0.331973316665728
0.544739331107844	-0.780529011503372
0.29120774364647	-0.701582223054467
0.292496522667102	-0.429701110115297
0.887514105886863	-0.583311786605406
0.703901826313006	-0.291537115222033
0.432613462527175	-0.0380570791935904
0.489563969562682	-0.44390700054179
0.594053286101043	-0.401575230248081
-0.554151590861746	0.169024454482996
0.594648675622008	-0.855568488988737
0.707071865694524	-1.20003636621964
0.568159108873925	-1.26837974255179
1.04763133829202	-0.304705915061374
0.490812898259296	-0.476442834026164
0.266848400064177	-0.439112360856259
0.193356692539792	-1.02061143078673
-0.13432836333054	-0.880300766652957
0.347251995521702	-0.704318483613944
0.310896418020806	-0.600470579394504
0.00742981555908321	-0.741142697234553
0.309193065885285	-1.59048879174994
0.229217014041513	-0.932614596578555
0.196597508576428	-0.749348835079699
0.567989276936965	-1.03408985140054
0.363836154512643	-0.904449384015949
0.970065329265194	-0.653557559052628
0.901222337528612	-0.939375029029659
0.712334832866599	-0.432509397777074
0.893674472319175	-0.770777195819926
0.606851395480789	-1.15767303291611
1.04108997521807	-0.652787472460927
0.625628912852654	-1.0550990177423
-0.282517011473077	-0.0912356338110808
0.214066004155161	-0.743886396632061
0.304535475374207	-0.448221811394331
0.47019712135155	-1.50219687256585
0.367050914794648	-0.665502312382562
0.590482404536225	-0.869373553391708
-0.547143509115202	0.116974453801273
0.0776212619186328	-0.288467920660627
0.772184403481809	-1.19302357190725
0.427759858308228	-0.367737135485649
0.928943353726348	-1.38674872466513
0.181720053986902	-1.24034379280333
0.255141056666145	-0.823092692156428
0.268560082811864	-1.07555231150192
0.642822089995784	-0.457731589089357
1.63289154999772	-1.04666435121738
0.543797660850476	-0.4174839466979
0.156771626281155	-0.530569789112626
0.318566029621303	-0.0756274239629051
0.183102993432271	-0.813321912789285
0.313664066755153	-0.650368870076859
0.089823567246446	-1.85667571532742
0.401696779093221	-0.863755646637693
0.67176270515483	-1.02767870308209
0.397096143305644	0.0767297272724941
-0.166734839657867	-0.0125231641634515
0.178913914715175	-0.608149204032239
0.277806483967942	-0.540804841430422
0.627773979115792	-0.492760253726841
0.178913914715175	-0.608149204032239
0.600662102220975	-1.15048670655667
0.392084885036327	-0.818514998869955
0.392084885036327	-0.818514998869955
0.434907459147919	-0.91475836289632
-0.223184276088432	-0.886513137877282
0.57787342081944	-1.30946678514015
0.546478914524396	0.416510001745318
0.458735016199247	-0.179497798381152
-0.132783885967758	-0.83785971410811
0.389436871111478	-0.128315206575349
-0.198973090026702	-0.448702389648966
0.463157476970091	-0.541597878019485
0.542819781391332	-0.956095257347755
0.909284790428659	-0.918273370840683
0.250239168648949	-0.426785932469789
0.314000056789784	-0.519200970920191
0.454107920623037	-1.01142751216481
0.230104630642214	-0.202908991077213
0.908147741277454	-1.29783680285081
0.407622989613016	-0.978690184608306
0.381561249530287	-0.28505847320408
0.429112660609406	-0.0054477308300642
0.458983019759307	-1.25227849743481
0.484811834560118	-0.446562588401844
0.419381169427511	-0.750151670833183
-0.0190931529554238	-0.615174883395615
0.706197229927702	-0.638105599680532
0.595509458563144	-0.766536280289357
0.30899419918321	-1.07028909778137
-0.404164952087446	0.0268003206245757
0.577853502556251	-1.44244039158574
0.588084373951252	-1.25512461987463
-0.116157340328528	-0.349897962372087
0.316093437249141	-0.633831560677118
0.599117488316834	-1.15941309234368
0.00127109853816506	-0.758634490051145
0.958370044611638	-0.69041312468596
1.15970476485621	-0.971991945714253
0.137698441242103	-0.233059854316217
0.156577106987987	-0.869350810372758
0.653772960453617	-0.883667944586177
0.0934164356058324	-0.205595500854059
0.492667764360268	-0.847919994640346
0.464212034043513	-1.00206709925375
0.690623886329401	0.00236914579169467
0.827715463868362	-0.761272076904584
0.737592919566758	-0.823186116403079
0.58433558920039	-0.751403290587311
0.463292389801157	-1.17200147458205
0.348696192848244	-0.995805599471574
0.663438646819312	-0.870702424948693
0.625105174002541	-0.461711717395728
0.100844616865961	-0.0223967425349866
0.311854333581451	-1.12273496302423
0.828927279928883	-0.877561172982297
0.261943260700586	-0.617172558074672
0.473184663286497	-1.18601461648624
0.259662439932086	-0.0479641662412939
0.642973915087389	-0.722957333310746
0.604188050281215	-0.684928656275774
0.57731447494832	-0.526242841387216
0.464331281360481	-1.65979844224199
0.417432534127066	-1.0352919847532
0.103160865512079	-0.865506072208462
0.608177871712976	-0.520256493489265
0.71272810128548	0.293906577256312
0.518282760153376	-0.624864301955444
0.654388759865555	0.0388758401441376
1.27130574068275e-06	-6.22462291801768e-08
-0.967599721699805	-0.424408355010488
0.818613788031389	-1.17567485691572
0.634983179865816	-0.745074652749036
0.366059934515712	-0.134375584136438
0.366059934515712	-0.134375584136438
0.614932927731245	-0.789094246127814
0.68260735463404	-0.63321728632881
0.273911013598956	-0.941848571965024
0.602230954240465	-0.853217382843543
0.448144248793847	-0.305203989348728
0.698005613159814	-1.2311967612795
0.616803754632107	-0.915129927643072
0.230844257460323	-0.401182488419213
0.410350849908645	-0.631000854576701
0.816766656004982	-0.925029737122081
0.475520848541849	-0.671154164688591
0.437735838258301	-1.05722616658504
0.287553076611635	-1.03053948429246
0.741506676916646	-0.957055077883362
0.273911013598956	-0.941848571965024
-0.110357834981406	0.351915393046724
0.129372117911844	-0.584346981773071
0.313931393154499	-0.787687634341722
0.427161062377286	-1.04726142101138
0.480509083008693	-0.603513792926186
0.550193248745645	-0.694058517476267
0.513542864759834	-0.512032641253633
0.320033507101529	-0.84748912449361
0.0590748141310849	-0.712134901842559
0.241875490277452	-0.641128914882845
0.470091806582349	-0.697781917376241
0.205419827895603	-1.10544576201997
0.702153384804984	-1.42082502912191
0.380900865492013	-0.896025511233534
0.511892728553684	-0.436592781501468
0.94235710890717	-1.3840046024406
0.304012485654155	-1.05725884615768
0.37054087279601	-0.573339054407096
-0.187977599954372	-0.113210720622705
-0.209664885071638	-1.11002539070517
0.244601615879865	-0.824377950380552
0.79744826973366	-0.562830606920091
0.598445998070617	-0.959053434412659
0.146532549259807	-0.411755942947649
0.464585420525785	-0.516322526171376
0.548782151523798	-0.726950069710606
0.953687444122452	-0.803596187881815
0.876979628800712	-0.461999930256637
0.0793213465634497	-0.970991423409538
0.710652014999606	-0.914385350348212
0.349963888611425	-0.0927378268535074
1.09021627057191	-0.846224712991725
0.443483280656754	-0.491599757609635
0.571694378170875	-0.682470612565386
0.555993200468304	-1.02080346615007
0.380792036299024	-1.26482849444297
0.777683213854331	-1.55317262564928
0.663171982389716	-0.56943861124215
0.627046420723362	-1.12045582669736
0.636408489720026	-1.13914664512918
0.735510102187384	-0.578530768908343
0.227012840586124	0.563767315585504
0.641159512208433	-0.809512107954294
0.493181976057948	-0.916622260811671
0.905488999612271	-0.256755062146285
0.462687585456334	-0.122858587042643
0.574245193823385	-0.720651730725982
0.262137023325471	-0.549520659367892
0.0761520457325783	0.0183897040479042
0.00518227457862486	-0.444171978585126
0.617888344512762	-0.53508863480738
0.205220219690971	-0.661493398403535
0.718343513517178	-0.141313681307956
0.510898177777402	-0.850282921585995
0.454325921649407	-0.620383377358674
0.304299021002823	-1.75036315775784
0.920453129217706	-0.634177946900898
0.869089994346431	-0.227927836981474
-0.00976878575804684	-0.351780643712725
0.611150160023392	-1.25975554714884
0.294880662273348	-1.02391225581352
0.329840851088196	-0.6847412077334
0.806059815462321	-1.08033091347685
0.698005613159815	-1.2311967612795
0.248741530056306	0.11316793273443
0.520160921540155	-0.854683270542632
0.0244217480427065	-0.0915030616906009
0.269135087547777	-0.611267284726813
0.111172311530502	-0.471190655589241
0.514110929837041	-1.22516778917933
0.70013936056055	-1.18591455159816
0.386026365835439	-0.977452502516632
0.594212825319531	-1.06294552775765
0.738069388959667	-0.967113746967364
0.374708055510361	-0.14412030050887
0.394233008034808	-0.348296661856342
0.646875698970363	-1.58429902899515
0.123072308292864	-1.36341335033151
-0.379779390620809	-0.432256084916857
0.814616580193949	-1.00357424014639
0.372313924420583	-1.15235284679689
0.582093509150898	-0.567150692915239
0.0241951536553723	-1.30837675650787
0.357664570683778	-0.98918075166148
0.301709960338262	-0.16004711137816
0.390752865637636	-0.973236337765051
0.347424454208062	-1.37495855494058
0.272755744483598	-0.0160244763978803
0.372313924420584	-1.15235284679689
0.0663807754325832	-1.26356156063047
0.350760718876577	-1.27453068679145
0.390752865637636	-0.973236337765051
0.550811206964852	-1.42112779235122
0.272755744483598	-0.0160244763978807
-0.0144884982691963	-1.69517138708111
0.474420543670714	-0.64571406561544
0.20645128819759	-0.881803329781694
0.381559646689813	-0.629217099141534
1.01093655945035	-0.979413084894056
0.0656751217325946	-1.40412050867809
0.517089734221616	-0.336163334023697
0.444443083523742	-0.289820515035664
0.574143256131978	-0.597970365459344
0.24203500371512	0.0399692177107121
0.723302751015408	-1.20872290338149
0.326114210450193	-1.05356961432543
0.567441040788969	-0.913321743542798
0.0122346314414951	-0.908798182668755
0.380568484563057	-2.06559526960607
0.348346018058779	-1.27178306364541
0.230675331373503	-1.6682045668286
0.910332145260973	-1.25938758919545
0.535786703270367	-1.38231773247339
0.0656751217325947	-1.40412050867808
0.804967039032242	-0.893650607432655
0.231386611959047	0.0343568516727221
0.321376884802351	-0.288471323640132
0.649686574658625	-0.524660652024477
0.459982605631193	-1.02981225349217
-0.00269994870638494	-0.961570181690999
0.101257414814149	-2.11297004681516
0.27422931986411	-1.52787443549209
-0.00390023598645623	-1.05027091646253
0.667849819274467	-0.859870529196001
0.98250692914163	-1.40441111642611
0.98250692914163	-1.40441111642611
0.106919541116677	-0.899194051682021
0.166275624130162	-1.1782753510242
0.470847265218835	-1.14985832010099
1.0201743632156	-1.1860274104198
0.473781682661321	-0.172023557610816
0.134323888527231	-0.769569428735124
0.0605458617442274	-0.839220695237731
0.257092582594928	-0.539835952820419
0.394036227592824	-0.791853757985827
0.233769879680992	-1.01041458981816
0.0559859612821492	-1.31137096567007
0.641589826482751	-1.09249783459155
0.596229852373343	-0.509571782694325
0.514406913492634	-0.811741291197625
-0.282295101145673	-0.264194260676495
0.629694195350883	-0.964032902164679
0.343446061973775	-0.307514167207635
0.670596608785033	-1.02819860996102
0.609663130479815	-0.845841626007995
0.656996263774283	-0.930299228283393
0.274603017799947	-0.506393961244366
0.487284565309427	-0.803864443513873
0.578143831113864	-0.44153160305057
0.223559499004446	-0.84278093562633
0.167201885147482	-0.76117733376821
0.252139049820941	-0.813227178217052
0.691777371771547	-1.67040084599261
0.369288853753266	-1.1120683786505
0.852110680169398	-1.19996414779972
0.626583518074093	-1.24845739772043
0.747865866060857	-1.08402711183191
-0.257760833367524	-0.331223157266248
0.1276150712515	-0.526124153826886
0.709188486400459	-1.33151444843744
0.753717890893868	-2.01918218550184
0.421481855760179	-0.575969836721146
0.294111750453356	-0.416542184276788
0.101636765662539	-0.772866503989168
0.169952896487548	-1.18926789487196
0.191336887943861	-1.06753897643021
0.765938952743554	-0.949570298157009
-0.57513153619357	0.0686464663167701
-0.798600540729827	-0.11069524747719
-0.413184010237457	0.683385376317786
-0.517016435289131	0.382385996259475
-0.441134645545182	0.838591062573658
-0.623852939812222	1.03577377524059
-0.21136499905647	0.155564460188278
-0.228998297341513	0.868484125523307
-0.853467507809182	0.85341717670437
-0.412921791864173	0.655727535067824
-0.575003685407116	0.0888466812139626
-0.895533405786069	-0.19551474266809
-0.208721707470156	0.808470122075658
-0.526250236447194	0.499086450085775
0.102997708986491	0.139395249005473
-0.762858292961921	0.481065229538402
-1.18756333234166	1.14869292278825
0.25499994324043	0.118808908851174
-0.349201290268382	0.669761375426862
-0.833621804593139	1.05608046600483
-0.601107134777639	0.670996924133798
-0.734238256764194	0.509311299848666
-0.710021744898223	0.57372458811356
0.173416340662538	-0.246022554326202
-0.0449680672998342	0.174557916287413
-1.33216948240819	0.868738216518189
-0.783316777953408	0.745123448705335
0.706972774432243	-0.604079341018942
-0.628614196393195	0.341431828803593
-0.375060475882524	0.187577647166775
-0.994051959069212	0.568329415603731
-1.27024692060448	0.839629212088432
-1.12303128741588	0.69003698496736
-0.818555414902255	0.946377332728047
-0.461140118971969	-0.135146456680254
-0.413184010237458	0.683385376317787
0.419744839932321	0.65636349116059
-0.0270657702044818	0.466291498244903
-0.408768048733046	0.998835139022758
0.513709679213683	0.418161894601551
-0.856029305032251	0.60944153280786
-0.804004220280744	0.589973307822363
-0.203701176887584	0.138775334237864
-0.666839974344827	0.666901643564602
0.222618216489686	-0.0860258154090772
-0.217069480945152	-0.154181044292349
0.500392649835883	-0.508980534824248
-0.393418336842337	0.192021626987036
-0.465465650179691	0.880607690620075
-0.3808585056304	2.0404246005193
0.474773208835486	0.970863615011271
-0.167859115826818	-0.302190452677721
0.327400288955393	0.218027608334771
-0.656876420945523	0.955319957567521
-1.63688670931286	0.988151127160909
0.0954556188816324	-1.39595451334719
0.527119455922486	-0.486948626129158
-1.11957589110219	0.78049075575576
-0.404519783506493	0.881213852561981
-0.21136499905647	0.155564460188278
-0.866118252693629	0.751324814541756
-1.4058204712631	1.03774069731769
-0.517016435289133	0.382385996259476
-1.26211115880309	0.943291546786601
-1.66460745005128	0.0872256261281835
0.051799682625099	1.02416136493542
-1.08894118066112	0.409587772005166
0.669361500581456	0.202694102875637
-0.63351973276161	0.985407708643557
-0.530669205580753	0.565233461106548
0.465585198466094	-0.228014533931626
-1.16730821828693	1.07019495169823
-0.62976683729522	0.345543080850522
-0.539795163540596	-0.331459065393283
-0.860796922125088	1.92092467761288
0.000707739649672556	0.371997432358511
-0.58540507408101	0.948427868917948
-0.306955237907222	0.615185202569563
-0.109124442009908	0.718683396075831
-0.598200365866004	0.185930978920948
-0.609542584977859	1.20562216097634
-0.0734742203278434	-0.483483202321343
-0.253807326491913	0.918395065407953
-0.46813560243065	0.527401261784784
0.890138944901995	0.717804206302858
-0.14386770480412	0.469396977058542
-0.529685892062425	0.193891137610096
-1.0571416457244	0.966239022689536
-0.392000476183792	0.524284516998387
0.104451610177571	0.727997916833769
0.429577495527106	0.431939423787872
-0.344653922401356	0.907850786839713
-0.276658680783485	0.617825972833994
-0.308327618658854	0.377186186256256
-0.0538107807366729	0.215120038928835
-0.0483656644763766	0.97655244142414
-0.751719896353298	0.596768089246899
-1.42621333150925	0.597710581934656
-1.19769027663145	1.09864304494454
0.145640914668569	-0.473382987927214
-0.552610055147217	0.837487184473304
-1.0290318416921	0.839857824006762
-1.79586437290053	0.858019889974741
0.129635470000082	-0.462258467975402
-0.510157423843004	1.04329076957922
-0.680859863256458	1.10049023764524
-0.892857897726206	1.1618650914636
-0.00165862395340813	-0.380158049359225
-0.681960042912709	0.990733652478234
0.352515087034463	0.129615890619833
0.84221466908025	-0.495478123383636
-0.0413797852982006	-0.111885799879102
0.214315756402646	0.495152946351076
0.151052559086617	-0.904004443225766
-0.177892269374059	1.43763049997898
-0.622763917234688	0.855845923902667
-0.480501928118191	1.26840492372496
-1.23898398493047	0.864843295148979
-0.495665236933793	-0.407278167887499
-0.467502724117713	0.862880042889124
-0.179171844041549	1.19808015942252
-0.671157094189307	0.944205185418829
-0.568205451179819	0.937668652015625
0.225613511700427	0.825198932883809
-0.0691489422435192	0.150021965852064
1.27134459662467	-0.562890078595003
-0.582892882062415	0.79921273528921
-0.0522943762952605	1.01723931211599
-0.618296586814193	-0.453913866985857
-0.676614337181514	-0.461570772778387
-0.222772360398414	0.677029971666809
0.387448686527765	1.48082392161644
-0.8444096368103	1.00700408410808
-0.655548795815887	0.815339560916008
0.158944380425477	-0.106055379002144
-0.530088110617966	0.655670304027721
-0.369612274704138	0.380312275076345
-0.70339940823003	0.727431074998403
0.371223254551917	-0.666214136397749
0.252552883105945	-0.72957740775516
-0.223306051898105	-0.328185159928233
-1.03767070717869	1.09224240231853
-0.772745966891092	0.870715003714134
-0.267708339466683	-1.93955205070779
0.616838381984752	0.226474073292764
-0.546005180870972	0.492928958049977
-0.794544821050873	0.736140992204734
0.304345236569745	0.50785767935744
0.193294844665998	0.637387133069106
-0.870831451226862	1.19953680701654
-0.76956991461665	0.643978063313645
-1.67709495847169	0.521114723208986
-0.355139915029838	0.577656559189654
-0.470389484835978	-0.0524541978015121
0.563988126479209	-0.418654641818481
-0.358753307311282	0.771468923519843
-0.205637080044162	-0.413287807403583
-0.503733950002437	0.458876605627429
-0.794544821050872	0.736140992204734
-0.0523233127693602	-0.603404050746753
-0.63330088595514	1.62334688353898
0.193294844666548	0.63738713306872
-0.858232001835755	0.789128072135351
-0.811738465814451	0.0895642272449164
-0.410984402176428	0.264218163299234
-0.410984402176428	0.264218163299234
-0.231452851117991	0.573195872956681
-1.93251940618833	0.323523911700776
-0.39851111117368	0.429107857842388
0.0111836685576531	-0.298669609422828
1.27134459662467	-0.562890078595005
-0.696554977371082	0.938366597228491
-0.239510677915454	-0.304550987116594
-1.06913447586689	0.96557755723722
-0.61829658681419	-0.453913866985857
0.387448686527765	1.48082392161645
-0.0521547338374735	0.608266152009853
-0.676614337181514	-0.461570772778387
-0.22277236039839	0.677029971666797
-0.844409636810361	1.0070040841081
-0.723439301578406	0.726836108506472
0.939341271504829	0.181211344478399
-0.210680065576179	0.898990117621704
-0.621260586662325	0.970760517783415
-0.29631433292942	0.514667007011747
-0.00942434838898801	0.974561344355318
-1.24515112731503	0.60592598579391
-1.13385351222627	1.2367733744111
-1.13036970014664	1.18945754790128
0.446798705737324	0.0948681590189924
-0.909518281955436	0.471586482262646
-0.916903222635523	0.795062352126679
-0.935392001973529	0.684323076706936
-0.128309604426231	0.812966721774404
0.301101593467053	-0.174015169410086
-0.720630439003721	0.361000768114216
-0.288173418184172	0.612679774002341
0.00413533612711185	0.441265927710913
-0.278479162689725	-0.254784310952278
-0.814409288892571	0.867697116403772
-0.26773884849242	0.547886685667711
0.331520056000283	0.0457456193052435
-0.624605492405882	0.961313819171467
-0.701580162184879	1.01022792412125
-0.644469028674464	0.705508966457289
0.271742319432016	0.796929820347888
-1.08530002468332	0.455092704905908
-0.448334300127408	1.04624198924091
-1.11116265767064	0.889196084327831
-0.100968491834868	1.09953124357567
-0.846373936814949	0.610137578120382
-0.905874935210655	0.788285619885167
-1.09868770953063	1.33496740641527
-1.30943036190553	0.953492491012451
-0.231452851117991	0.573195872956681
-0.552301333739402	0.540813619685058
-0.0253811373326205	0.575299751458636
-0.51719484675671	-0.159957793737434
-0.339555006519	-0.0493712402540882
-0.911769223242697	1.0950102642582
-1.37340964128214	1.01208984226918
-0.688694395862418	0.94248567571783
-0.0639428812010672	0.862941641783683
-0.339555006518999	-0.0493712402540884
-0.905874935210654	0.788285619885165
-0.846373936813624	0.610137578119959
-1.30016467771716	1.05960265309448
-0.312639660178841	1.13281384080884
0.584589833722078	0.657671007687204
0.151631904054005	0.988660071245289
0.133471826498421	0.0596813185201951
-0.447810683203293	0.456999658803797
-0.398511111168086	0.429107857839818
-1.28574618124183	0.883685342613825
0.0741860839140064	0.911461687711599
0.555218339870566	-0.0634067422899136
-0.143332675042577	0.260709241692267
-0.17577442101379	0.00930278755790671
-1.32736061614115	1.20772761146376
-0.218644806867089	0.616616138732882
0.398408577833127	0.376187110565634
-0.928350368171925	0.226910882277938
0.0612103812181645	0.313486353681351
-0.433848436232978	0.692992949910116
-0.114376098463702	0.431803376763769
-0.256022488363929	1.24024593928751
0.339451674069135	0.180104810077081
-0.616412504206686	1.09235062459282
-0.143814124851305	-0.576893333792467
-0.204661889310656	0.663869349366845
-0.326013881405018	1.20641469490216
-0.228998297341515	0.868484125523307
-0.609668372640992	-0.750994883634085
-0.483635173252802	0.0957154049243309
-0.140165621189276	0.779566243806462
0.474688416143415	-0.130201853483485
-0.408866357606627	0.373106057800164
-0.557338606894772	0.4254728307797
-0.431927319508749	0.981714925361971
-0.0963671642037701	0.449245711346492
-0.313785645962851	0.660796646869352
-0.753045473720174	0.354241735333005
-0.229901517016244	1.05853275823099
-0.875732516416284	0.647636017148118
-1.1396221681795	1.20709643383104
0.870659263287014	-0.0575603713290519
-0.844343705035479	0.964442574213012
-0.30345956970482	0.901393455489776
-0.31181232194369	0.953973967204464
-0.00178192330138097	1.12368444454183
-0.856463582887671	1.26358102652867
0.251565918880463	0.627509630698028
-0.367558331625163	0.214055820793051
0.106710098092453	0.102125562718902
-0.443004907692918	0.657578289836026
0.165157046131111	0.60041575895795
-0.385165651293911	0.958685574073923
-0.636339606039411	1.14293636623103
0.701932886817924	0.0524423904922078
-0.819972486491693	0.746537565521752
-0.557507131119953	-0.0710016957329549
-0.0454081528854386	0.460028561521913
0.797493097931036	-0.906366901768711
-0.087765340184176	0.820112156837727
-0.445914148394922	1.20852315345294
-1.2305063540392	1.08071149766082
-1.16490701891649	0.381598457326178
0.489614819764887	-1.06716439451243
-1.42621333150914	0.597710581934622
-0.646593836371102	0.851485683844094
-0.858232001835755	0.789128072135352
-0.0521547338374732	0.608266152009852
-0.358753307311282	0.771468923519843
0.563988126479209	-0.418654641818481
-0.312639660178842	1.13281384080884
-0.935392001973529	0.684323076706938
-0.09636716420377	0.449245711346493
-0.229901517016244	1.05853275823099
-0.00178192330138087	1.12368444454183
0.256156943423988	0.579369363808153
0.173554666471119	0.597307115971622
0.701932886817924	0.0524423904922076
-0.860796922125087	1.92092467761288
-0.445914148394922	1.20852315345294
-0.917118859351725	0.870200781965094
-0.0483484382983832	0.0358331423557742
-1.12819891001116	0.481289481769737
-0.253807326491913	0.918395065407951
-0.00433424451195728	0.0299008738186698
-0.392000476183794	0.524284516998387
0.0175192383741122	0.130432646800858
0.10445161017757	0.72799791683377
-0.308327618658856	0.377186186256255
-1.63688670932952	0.988151127163684
-0.783316777953408	0.745123448705335
-0.575750084455798	0.649041558436253
-0.0449680672998341	0.174557916287413
-1.40582047126288	1.03774069731767
-1.23281013347055	-0.7122008307158
-0.213414256259943	-0.747013831951985
-1.01324457355299	0.830567256297338
-0.404519783506493	0.881213852561981
0.474773208835486	0.970863615011271
-0.666839974344827	0.666901643564601
-0.462060440882499	0.93770026637401
-0.203701176887584	0.138775334237863
0.500392649835883	-0.508980534824249
-0.601107134777639	0.670996924133798
-0.513595239846119	0.48728977331981
-0.526250236447202	0.499086450085775
-0.89553340578604	-0.195514742668115
0.513709679213683	0.418161894601551
0.419744839943242	0.656363491152325
-0.408768048733046	0.998835139022758
-1.27024692060448	0.839629212088431
-0.128885274092352	0.550078523424509
-1.07433915824902	0.280432029665006
-1.26211115880309	0.9432915467866
-0.804004220280744	0.589973307822368
-0.383430266507593	0.933891498102052
0.279825484828039	-0.158639712188049
-0.393418336842337	0.192021626987036
-0.226335331007361	-0.346552753504726
-0.419213259036635	0.932579483500104
0.810186475153009	-0.276632957199087
-0.661053699109826	0.692949531969501
-0.500470974200779	0.753285372051467
0.640942864994106	-0.298878023014793
-0.405395200674771	0.543312968743115
-0.318745936057834	0.954995166694161
0.0387589245546192	-0.131485322586245
-0.874192425987466	0.630378239188128
0.0715834008403008	0.624483000430313
-0.481199200217074	1.17771176725279
-0.314018811932908	0.548928428207824
-0.549236706387202	0.62224371744918
-0.658266152869894	1.00629025690223
-0.840822384971665	0.272975422996973
-0.776529250167034	0.700093826420676
-0.16355250057469	0.639050238659709
-0.518246559479193	0.185830551236278
-1.11872313362193	1.16467830987587
-0.839079901686951	0.636007037577721
-0.22516795433253	0.853975770430104
-0.712771160574923	0.521131017797772
-0.969960801775109	-0.182554946457271
-0.454378009446689	0.413477193096688
-1.06754433692805	0.32598275307725
-0.28575340016231	0.67189490887775
0.252133442201758	1.0737940171081
-0.407144246813582	0.312868762974021
-1.16490701891649	0.381598457326178
-0.109124442009908	0.718683396075831
-0.0522943762952604	1.01723931211599
-0.492092893477252	1.15420946197885
-0.856029305032256	0.60944153280786
-0.0483656644763768	0.97655244142414
-0.559950000786593	0.798721652258446
-0.853467507809182	0.85341717670437
-0.734238256764195	0.509311299848664
0.0517996826250988	1.02416136493542
-1.08894118066112	0.409587772005166
0.121255775795404	0.592824513030142
-0.16392545787249	0.240517429261419
0.0260949381737169	0.473377634125113
-1.27094200970138	1.33110844111286
-1.23898398493385	0.864843295149872
-1.79586437290053	0.85801988997474
-0.804533248811307	1.44844164298486
-0.769501786860203	0.592032605578916
-1.2305063540392	1.08071149766082
-0.925361059796131	1.29314938443307
-0.609542584977859	1.20562216097634
-0.521959575779935	0.736186095755584
-0.630880293596153	0.941140130025624
-0.184626276788159	0.192428595913486
-0.105418955688606	-0.289010609865504
-1.30016467771716	1.05960265309448
-0.198835822717642	0.482953959818925
-0.844604835951902	0.580149367919978
-0.164059986354656	0.402046824614528
'''

gaussian_sv_phi_fulldf5 = '''
-0.878394645987724	-0.804214483949313
0.363940482968383	-0.545438224664453
0.886256308437429	-1.66671586994716
1.75050400107845	0.468112146370155
-0.749987409949064	1.70365960412754
-0.776626594687817	0.042688561682037
0.564197833538098	1.22649156591549
0.86962303070899	1.35478564944047
-0.0457670771434133	0.0349295519874349
-1.56085071769235	-1.01973307866096
'''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/Colab\ Notebooks/GaussianSVPerl/SavedPickles/BBC/labels2_250_GaussianSV_BBC.pkl /content/
!cp /content/drive/MyDrive/Colab\ Notebooks/GaussianSVPerl/SavedPickles/BBC/labels_full_df2_GaussianSV_BBC.pkl /content/
!cp /content/drive/MyDrive/Colab\ Notebooks/GaussianSVPerl/SavedPickles/BBC/labels_full_df5_GaussianSV_BBC.pkl /content/
!cp /content/drive/MyDrive/Colab\ Notebooks/GaussianSVPerl/SavedPickles/BBC/labels_full_GaussianSV_BBC.pkl /content/

In [ ]:
labels = load_obj('labels2_250_GaussianSV_BBC')
labels_df2 = load_obj('labels_full_df2_GaussianSV_BBC')
labels_df5 = load_obj('labels_full_df2_GaussianSV_BBC')
labels_full = load_obj('labels_full_GaussianSV_BBC')
labels_original_set = ['business', 'entertainment', 'politics', 'sport', 'tech']
labels_original=[]
for l in labels_original_set:
  for _ in range(50):
    labels_original.append(l)

In [ ]:
def text_to_coord(x): return np.array(x.split()).reshape(-1,2).astype(float)
# x_coord = text_to_coord(gaussian_sv_x)
# phi_coord = text_to_coord(gaussian_sv_phi)
# x_coord_norm = text_to_coord(gaussian_sv_x_norm)
# phi_coord_norm = text_to_coord(gaussian_sv_phi_norm)
x_coord_df2 = text_to_coord(gaussian_sv_x_fulldf2)
phi_coord_df2 = text_to_coord(gaussian_sv_phi_fulldf2)
x_coord_df5 = text_to_coord(gaussian_sv_x_fulldf5)
phi_coord_df5 = text_to_coord(gaussian_sv_phi_fulldf5)
x_coord_nodf = text_to_coord(gaussian_sv_x_nodf)
phi_coord_nodf = text_to_coord(gaussian_sv_phi_nodf)

In [ ]:
import seaborn as sb
def plotVis(x_coord,phi_coord,labels):
  lim = 5
  fig, ax = plt.subplots( figsize=(20, 20))
  sb.scatterplot(ax=ax,x=x_coord[:,0],y=x_coord[:,1],hue=labels,alpha=0.8,palette='deep')
  ax.set(ylim=(-lim,lim))
  ax.set(xlim=(-lim,lim))
  ax.text(0,0, 'X' ,c='black')
  ax.scatter(phi_coord[:, 0], phi_coord[:, 1], alpha=1.0,  edgecolors='black', facecolors='none', s=30)
  for indx, topic in enumerate(phi_coord):
        ax.text(phi_coord[indx, 0], phi_coord[indx, 1], 'topic'+str(indx))

In [ ]:
plotVis(x_coord,phi_coord,labels_original)

In [ ]:
# plotVis(x_coord_norm,phi_coord_norm,labels)

In [ ]:
plotVis(x_coord_df2,phi_coord_df2,labels_df2)

In [ ]:
plotVis(x_coord_df5,phi_coord_df5,labels_df5)

In [ ]:
plotVis(x_coord_nodf,phi_coord_nodf,labels_full)

In [ ]:
#cal_knn --> Calcs KNN dist > Cluster quality
cal_knn(x_coord, labels_original)

In [ ]:
cal_knn(x_coord_norm, labels)

In [ ]:
cal_knn(x_coord_df2, labels_df2)

In [ ]:
cal_knn(x_coord_df5, labels_df5)

In [ ]:
cal_knn(x_coord_nodf, labels_full)

# Comments :

<5/6/21>
* Topic quality is good. We can see keywords and other words that are closely related to the keywords in the topics
* Visualization has some issues though. It can't form and all the documents seems to converge at a single coordinate.

<5/7/21>
* Introduced dimension 'C' as new irrelevant center (we are not visualizing that)
* WAF to have two diff sigmas , 1 for relv i.e query(Q) , 1 for irrelv for C
* First Q dim should have relv sigma, Last dim (for Center C) should have irrelv sigma

<5/15/21>
* Tried to implement QxV as decoder weight without query > result - Cluster with Q centers (Centered at Keyword)
** K topics essentially grouped into Q different topics , Works Like PLSV.
* **ToDo** - Implement GaussianSV with VAE (Short Paper)

<5/27/21>
* Find the baselines (2016-2021)

 - PLSV, GaussianSV [2017], LDA, ETM[2020], GPU-DMM (author source) (can run), CRFTM [2019]
Topic Modeling of Short Texts: A Pseudo-Document View, KDD 16,  https://github.com/qiang2100/STTM

skipped: [GaussianLDA], GPU-PDMM [2017]

Incorporating word embeddings into topic modeling of short text, Knowledge and Information Systems 2019, https://github.com/nonobody/CRFTM



Topic modeling for short texts with auxiliary word embeddings: GPUDMM SIGIR 16
Enhancing topic modeling for short texts with auxiliary word embeddings: GPU-PDMM TOIS 2017
Topic modeling over short texts by incorporating word embeddings: PAKDD 2017 (https://github.com/qiang2100/ETM)
(https://github.com/soberqian/TopicModel4J)
Topic Discovery for Short Texts Using Word Embeddings, ICDM-16 (no source)
Improving Topic Models with Latent Feature Word Representations

datasets: https://github.com/qiang2100/STTM
* Design the tasks: kNN, NMPI, Visualization examples, topic examples, what else? (check the baselines papers)
* Check the baseline codes, and run them. (omit the baselines without code published)
* Omit the results >24hours
* Finalize the datasets: N datasets, N*3, N=3,4: normal, small (50docs per label), short (all docs and length<21), short and small.
BBC-headlines&title,
BBC
Search Snippet:
--------------------------------------
BBC (normal) (~)2k
BBC (small)
BBC-title (short)

WOS (normal) (~46k)
WOS (small)
WOS-title (short)

Search Snippet (short) ( ~12k)
Search Snippet (short and small)

--------------------------------------

If time then run --->

Twitter (short) {~5k docs} - Sanders Twitter corpus
Twitter (short and small)
labels?


Agnews (https://www.kaggle.com/amananandrai/ag-news-classification-dataset) (normal) (>100k, 4 classes)
Agnews (small)
AgNews-title (short)




# Save/Download Results

In [ ]:
doc_ids_model = [i.item() for i in doc_ids]

In [ ]:
save_obj(get_topwords(beta, id_vocab),"top100_docmodel_"+d_data)

In [ ]:
save_obj(x_list,"x_list")
save_obj(indices,"doc_id_desm")
save_obj(doc_ids_model,"doc_ids_model")
save_obj(labels_list,"labels_list")
save_obj(all_rscores,"all_rscores")
save_obj(keywords,'keywords_'+d_data)

In [ ]:
from google.colab import files
files.download("doc_ids_model.pkl")
files.download("doc_id_desm.pkl")
files.download("x_list.pkl")
files.download("labels_list.pkl")
files.download("all_rscores.pkl")
files.download("keywords_"+d_data+".pkl")
files.download("top100_docmodel_"+d_data+".pkl")

In [ ]:
rm -r *.pkl*